# Hccb Warehouse Location


In [1]:
import pandas as pd
from pulp import *
import time
import pandas as pd
import plotly as py

In [2]:
scenario_name = 'Warehouses location' 


### optimal number of warehouse we need to obtaion

In [3]:
number_of_whs = 5 # that should be placed in East Region

### Percentage of demands met in different band zone

In [4]:
distance_band_1 = 50 
distance_band_2 = 100
distance_band_3 = 150
distance_band_4 = 200


### Potential warehouse that can be selected and what is their status

In [5]:
#This is where we can change the status of a warehouse
# meaning we can force it to be open or force it closed.
#(0,1 to let the solver pick, 1,1 to force it to be used, or 0,0 to force it closed.)


wh_status = pd.read_excel('wareh.xlsx', engine = 'openpyxl')
wh_status = wh_status.to_dict();
wh_status

{1: {0: 'W1', 1: 0, 2: 1},
 2: {0: 'W2', 1: 0, 2: 1},
 3: {0: 'W3', 1: 0, 2: 1},
 4: {0: 'W4', 1: 0, 2: 1},
 5: {0: 'W5', 1: 0, 2: 1},
 6: {0: 'W6', 1: 0, 2: 1},
 7: {0: 'W7', 1: 0, 2: 1},
 8: {0: 'W8', 1: 0, 2: 1},
 9: {0: 'W9', 1: 0, 2: 1},
 10: {0: 'W10', 1: 0, 2: 1},
 11: {0: 'W11', 1: 0, 2: 1},
 12: {0: 'W12', 1: 0, 2: 1},
 13: {0: 'W13', 1: 0, 2: 1},
 14: {0: 'W14', 1: 0, 2: 1},
 15: {0: 'W15', 1: 0, 2: 1},
 16: {0: 'W16', 1: 0, 2: 1},
 17: {0: 'W17', 1: 0, 2: 1},
 18: {0: 'W18', 1: 0, 2: 1},
 19: {0: 'W19', 1: 0, 2: 1},
 20: {0: 'W20', 1: 0, 2: 1}}

In [6]:
# name of supplier area , demand, lat, long 
supplier = {
    1:('S1',100000,23,72),
    2:('S2',200000,29,76),
    3:('S3',230000,17,74),
    4:('S4',100000,21,80),
    5:('S5',250000,24,80),
}

demand_supp = {
    1:100000,
    2:200000,
    3:230000,
    4:200000,
    5:100000,
}

# distance between supplier to potential warehouse (w,s)
dist_supply = {
    (1,1) : 528,	(1,2) : 316,	(1,3) : 938,	(1,4) : 628,	(1,5) : 426,
(2,1) : 364,	(2,2) : 917,	(2,3) : 326,	(2,4) : 597,	(2,5) : 708,
(3,1) : 689,	(3,2) : 643,	(3,3) : 806,	(3,4) : 269,	(3,5) : 145,
(4,1) : 547,	(4,2) : 322,	(4,3) : 942,	(4,4) : 612,	(4,5) : 404,
(5,1) : 679,	(5,2) : 611,	(5,3) : 823,	(5,4) : 302,	(5,5) : 146,
(6,1) : 597,	(6,2) : 217,	(6,3) : 1038,	(6,4) : 707,	(6,5) : 474,
(7,1) : 145,	(7,2) : 772,	(7,3) : 523,	(7,4) : 718,	(7,5) : 738,
(8,1) : 312,	(8,2) : 446,	(8,3) : 811,	(8,4) : 702,	(8,5) : 583,
(9,1) : 396,	(9,2) : 732,	(9,3) : 522,	(9,4) : 446,	(9,5) : 491,
(10,1) : 635,	(10,2) : 1117,	(10,3) : 211,	(10,4) : 525,	(10,5) : 743,
(11,1) : 407,	(11,2) : 596,	(11,3) : 660,	(11,4) : 465,	(11,5) : 418,
(12,1) : 734,	(12,2) : 379,	(12,3) : 1042,	(12,4) : 554,	(12,5) : 275,
(13,1) : 534,	(13,2) : 280,	(13,3) : 968,	(13,4) : 668,	(13,5) : 460,
(14,1) : 310,	(14,2) : 935,	(14,3) : 339,	(14,4) : 689,	(14,5) : 785,
(15,1) : 463,	(15,2) : 1043,	(15,3) : 200,	(15,4) : 639,	(15,5) : 793,
(16,1) : 518,	(16,2) : 340,	(16,3) : 917,	(16,4) : 608,	(16,5) : 414,
(17,1) : 636,	(17,2) : 1018,	(17,3) : 356,	(17,4) : 380,	(17,5) : 600,
(18,1) : 761,	(18,2) : 603,	(18,3) : 912,	(18,4) : 327,	(18,5) : 66,
(19,1) : 511,	(19,2) : 844,	(19,3) : 458,	(19,4) : 360,	(19,5) : 489,
(20,1) : 643,	(20,2) : 952,	(20,3) : 449,	(20,4) : 292,	(20,5) : 505,

}

### Generate the map for warehouse and custumers

In [7]:

input_map = True
output_map = True


## Our Linear Programming

In [8]:
#This is code to start running the model. Linear programming
# to find the time for code
start_time=time.time()

def optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name,supplier,dist_supply,demand_supp):
    
    hccb_min = LpProblem("Als", LpMinimize)


    # Y(i,j) is assign_vars.
    assign_vars = LpVariable.dicts("Asssignment",[(w, c) for w in warehouses for c in customers],0,1,LpInteger)
      
    Yij_supp = LpVariable.dicts("Supply",[(w,s) for w in warehouses for s in supplier],0,1,LpInteger)

    # Another dicionary called 'facility_vars' is created to contain 
    #the decision variables Xi - the decision of a facility at city i to be opened or not
    
    facility_vars = LpVariable.dicts("Open", [w for w in warehouses],0,1,LpInteger)
    # The objective function is added to 'prob' first
    
  
    # 1.
    total_weighted_demand_objective = lpSum([customer_demands[c]*distance[w,c]*assign_vars[w,c]
                       for w in warehouses for c in customers ])
    total_weighted_demand_objective += lpSum([demand_supp[s]* dist_supply[w,s]* Yij_supp[w,s] 
                    for w in warehouses for s in supplier])
    
    #
    # Every customer must be served
    """
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
     """
    #2
    for customer in customers:
        hccb_min += LpConstraint(e = lpSum([assign_vars[w, customer] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name=str(customer)+"_Served", 
                                    rhs=1)
    #3
    # every supplier is fully used
    for supplys in supplier:
        hccb_min += LpConstraint(e= lpSum([Yij_supp[w, supplys] for w in warehouses]),
                                    sense = LpConstraintGE,
                                    name = str(supplys)+"Done",
                                    rhs = 1)
    #4
    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    hccb_min += LpConstraint(e = lpSum([facility_vars[w] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name="FacilityCount", 
                                    rhs=number_of_whs)
    
    # Use the user's status to fix some warehouses in the solution
    #5
    #constrint to link supplier to customers
    for ss in supplier:
        for w in warehouses:
            hccb_min+= LpConstraint(e = Yij_supp[w,ss] -facility_vars[w] ,
                                   sense = LpConstraintLE,
                                   name = str(ss)+str(w)+"complete",
                                   rhs = 0)
    for w in warehouses:
        hccb_min += LpConstraint(e = lpSum([Yij_supp[w,ss] for ss in supplier]) - facility_vars[w],
                                    sense = LpConstraintGE,
                                    name = str(w)+"serve each",
                                    rhs = 0)
        
    for facility in warehouses:
        w = facility 
        hccb_min += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintGE, 
                                    name=str(facility) + "_" + "Lower Bound",
                                    rhs=wh_status[w][1])
    for facility in warehouses:
        w = facility
        hccb_min += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintLE, 
                                    name=str(facility) + "_" + "Upper Bound",
                                    rhs=wh_status[w][2])

    #9
    for facility in warehouses:
        bigM = 10000
        hccb_min += LpConstraint(e = lpSum(assign_vars[facility, customer] for customer in customers)-bigM*facility_vars[facility] , 
                                    sense=LpConstraintLE, 
                                    name=str(facility) + "_Route-BigM Version",
                                    rhs=0)        
    

    # Setting problem objective
    hccb_min.setObjective(total_weighted_demand_objective)

    # The problem is solved using PuLP's choice of Solver
    _solver = pulp.PULP_CBC_CMD(keepFiles=False,fracGap=0.00,maxSeconds=3000, msg=True)
    hccb_min.solve(solver=_solver)

    # The status of the solution is printed to the screen
    #print ("Status:", LpStatus[als_problem.status])
    file.write('\nstatus:'+ LpStatus[hccb_min.status])
    print("Optimization Status",LpStatus[hccb_min.status] ) #print in Jupyter Notebook
    if LpStatus[hccb_min.status] == "Infeasible" :
        print("********* ERROR: Model not feasible, don't use results. One thing to check: see that the number of available warehouses is more than the fixed status.")
    #print objective
    total_supply = sum(demand_supp.values())
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c]*assign_vars[w,c].varValue for c in customers)
                                    for w in warehouses
                                    if facility_vars[w].varValue>0}
    
    #print(total_demand_to_warehouse)
    #print("Total Demand",total_demand)
    file.write("\nTotal Demand:"+ str(total_demand))
    file.write("\nTotal Supply:"+ str(total_supply))
    #print("Objective: ", value(als_problem.objective))
    file.write("\nObjective: "+ str(value(hccb_min.objective)))
    wgt_avg_dist = value(hccb_min.objective)/(total_demand+total_supply)
    
    file.write("\nWeighted Average Distance: {:.2f} km" .format(wgt_avg_dist))
    print("Weighted Average Distance: {:.2f} km" .format(wgt_avg_dist))
    
    end_time = time.time()
    time_diff = end_time - start_time
    #print("Run Time in seconds {:.1f}" .format(time_diff))
    file.write("\nRun Time in seconds {:.1f}" .format(time_diff))
    print("Run Time in seconds {:.1f}" .format(time_diff))
 
    
    #preparing data to write in excel sheets
    opened_warehouses = []
      
    for w in facility_vars.keys():
        if(facility_vars[w].varValue > 0):
            #print(warehouses[w])
            wh = {
                'Warehouse Key': w,
                'Warehouse City':warehouses[w][1],
                'State':warehouses[w][2],
                'ZipCode':warehouses[w][3],
                'Lat':warehouses[w][4],
                'Long':warehouses[w][5],
                'Total Demand to Warehouse':total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)
            
            
    #converting the list to dataframe        
    df_wh = pd.DataFrame.from_records(opened_warehouses)
    
    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'State', 'Total Demand to Warehouse','Lat','Long']]
    list_warehouses_open = list(df_wh['Warehouse Key'])
    
    # writing detailed files
    writer = pd.ExcelWriter(scenario_name+'_depand for n no of warehouse.xlsx')
    df_wh.to_excel(writer,'Opened Warehouses',index=False)
    
#     print("Customers Assigned to warehouses")
    customers_assignment = []   
    for (w,c) in assign_vars.keys():
        if assign_vars[(w,c)].varValue > 0:
            cust = {
                'Warehouse':str(warehouses[w][0]+','+warehouses[w][1]),
                'Customer':str(customers[c][0]+','+customers[c][1]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w,c],
                'Warehouse Latitude' : warehouses[w][4],
                'Warehouse Longitude' : warehouses[w][5],
                'Customers Latitude' : customers[c][4],
                'Customers Longitude': customers[c][5]
            }
            customers_assignment.append(cust)
                  
    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse','Warehouse Latitude', 'Warehouse Longitude' ,'Customer','Customers Latitude', 'Customers Longitude','Distance', 'Customer Demand']]
    df_cu.to_excel(writer,'Customer Distribution',index=False)
    
    supplier_assignment = []
    for (w,s) in Yij_supp.keys():
        if Yij_supp[(w,s)].varValue > 0:
            suppl = {
                'Supplier' : str(supplier[s][0]),
                'Warehouse' : str(warehouses[w][0]+','+warehouses[w][1]),
                'Supplier supply' : demand_supp[s],
                'Warehouse Latitude' : warehouses[w][4],
                'Warehouse Longitude' : warehouses[w][5],
                'Supplier Latitude' : supplier[s][2],
                'Supplier Longitude' : supplier[s][3]
            }
            supplier_assignment.append(suppl)
    df_su = pd.DataFrame.from_records(supplier_assignment)
    df_su_copy = df_su.copy()
    df_su = df_su[['Warehouse','Warehouse Latitude','Warehouse Longitude','Supplier','Supplier Latitude','Supplier Longitude','Supplier supply']]
    
    df_su.to_excel(writer,'Supplier Distribution',index = False)
    writer.close()

    #writing percent demand within each distance bands
    total_demand = sum(df_cu['Customer Demand'])
    percent_demand_distance_band_1 = sum(df_cu[df_cu['Distance']<distance_band_1]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_2 = sum(df_cu[df_cu['Distance']<distance_band_2]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_3 = sum(df_cu[df_cu['Distance']<distance_band_3]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_4 = sum(df_cu[df_cu['Distance']<distance_band_4]['Customer Demand'])*100/total_demand
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[0], percent_demand_distance_band_1))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[1], percent_demand_distance_band_2))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[2], percent_demand_distance_band_3))
    file.write("\nPercent Demand served within {} kms : {:.1f}" .format(distance_band[3], percent_demand_distance_band_4))
   
    return df_cu_copy,list_warehouses_open,df_su_copy
         
            


## Lat Long (location) for Potantial warehouse

In [9]:
def get_data():
    # Warehouses:: 
    # id:(name, city, state, pincode, lat, long)
    
    warehouses= pd.read_excel('warehou.xlsx', engine = 'openpyxl')
    warehouses = warehouses.to_dict()
    warehouses

# Customers/distributors details in form of their location
# customers:: 
    # id:(name, city, state, pincode, lat, long)
    customers={
    1:('A1' ,'B1' , 'India',' ', 25.444 , 77.4430000000002) ,
2:('A2' ,'B2' , 'India',' ', 25.9111 , 76.2820000000001) ,
3:('A3' ,'B3' , 'India',' ', 23.6251 , 77.6530000000002) ,
4:('A4' ,'B4' , 'India',' ', 19.6112 , 76.3000000000001) ,
5:('A5' ,'B5' , 'India',' ', 26.1311 , 73.615) ,
6:('A6' ,'B6' , 'India',' ', 25.6513 , 78.5980000000002) ,
7:('A7' ,'B7' , 'India',' ', 25.404 , 76.7740000000001) ,
8:('A8' ,'B8' , 'India',' ', 24.6498 , 79.1380000000002) ,
9:('A9' ,'B9' , 'India',' ', 26.546 , 73.648) ,
10:('A10' ,'B10' , 'India',' ', 25.4115 , 74.6050000000001) ,
11:('A11' ,'B11' , 'India',' ', 25.9192 , 76.7290000000001) ,
12:('A12' ,'B12' , 'India',' ', 20.523 , 76.4500000000001) ,
13:('A13' ,'B13' , 'India',' ', 23.7897 , 75.6280000000001) ,
14:('A14' ,'B14' , 'India',' ', 26.519 , 73.93) ,
15:('A15' ,'B15' , 'India',' ', 21.1465 , 77.2150000000002) ,
16:('A16' ,'B16' , 'India',' ', 19.1162 , 79.0090000000002) ,
17:('A17' ,'B17' , 'India',' ', 20.1024 , 78.6520000000002) ,
18:('A18' ,'B18' , 'India',' ', 25.9559 , 73.75) ,
19:('A19' ,'B19' , 'India',' ', 21.1288 , 76.1320000000001) ,
20:('A20' ,'B20' , 'India',' ', 25.3981 , 77.5540000000002) ,
21:('A21' ,'B21' , 'India',' ', 22.5567 , 73.414) ,
22:('A22' ,'B22' , 'India',' ', 25.6678 , 79.8880000000003) ,
23:('A23' ,'B23' , 'India',' ', 25.9012 , 77.2630000000002) ,
24:('A24' ,'B24' , 'India',' ', 20.0786 , 76.0300000000001) ,
25:('A25' ,'B25' , 'India',' ', 26.1536 , 73.57) ,
26:('A26' ,'B26' , 'India',' ', 23.1615 , 78.2560000000002) ,
27:('A27' ,'B27' , 'India',' ', 23.7336 , 77.6950000000002) ,
28:('A28' ,'B28' , 'India',' ', 25.3925 , 77.2780000000002) ,
29:('A29' ,'B29' , 'India',' ', 21.0904 , 73.381) ,
30:('A30' ,'B30' , 'India',' ', 25.4144 , 76.4320000000001) ,
31:('A31' ,'B31' , 'India',' ', 25.6838 , 76.3090000000001) ,
32:('A32' ,'B32' , 'India',' ', 24.7063 , 78.6610000000002) ,
33:('A33' ,'B33' , 'India',' ', 24.5449 , 75.6970000000001) ,
34:('A34' ,'B34' , 'India',' ', 25.5536 , 79.5430000000002) ,
35:('A35' ,'B35' , 'India',' ', 26.6018 , 73.438) ,
36:('A36' ,'B36' , 'India',' ', 27.1234 , 74.125) ,
37:('A37' ,'B37' , 'India',' ', 26.124 , 76.1020000000001) ,
38:('A38' ,'B38' , 'India',' ', 25.9177 , 73.234) ,
39:('A39' ,'B39' , 'India',' ', 25.9175 , 73.945) ,
40:('A40' ,'B40' , 'India',' ', 26.431 , 75.8350000000001) ,
41:('A41' ,'B41' , 'India',' ', 20.744 , 76.4770000000001) ,
42:('A42' ,'B42' , 'India',' ', 26.2738 , 79.4710000000002) ,
43:('A43' ,'B43' , 'India',' ', 19.6664 , 78.6130000000002) ,
44:('A44' ,'B44' , 'India',' ', 25.6773 , 79.5640000000002) ,
45:('A45' ,'B45' , 'India',' ', 23.4698 , 76.4920000000001) ,
46:('A46' ,'B46' , 'India',' ', 26.7761 , 73.714) ,
47:('A47' ,'B47' , 'India',' ', 26.5687 , 78.4810000000002) ,
48:('A48' ,'B48' , 'India',' ', 26.901 , 76.3660000000001) ,
49:('A49' ,'B49' , 'India',' ', 26.1604 , 77.9530000000002) ,
50:('A50' ,'B50' , 'India',' ', 21.6794 , 77.6620000000002) ,
51:('A51' ,'B51' , 'India',' ', 24.7054 , 78.9370000000002) ,
52:('A52' ,'B52' , 'India',' ', 25.9323 , 74.5180000000001) ,
53:('A53' ,'B53' , 'India',' ', 25.9491 , 78.3280000000002) ,
54:('A54' ,'B54' , 'India',' ', 26.558 , 75.5740000000001) ,
55:('A55' ,'B55' , 'India',' ', 20.62 , 78.2650000000002) ,
56:('A56' ,'B56' , 'India',' ', 20.7063 , 74.6800000000001) ,
57:('A57' ,'B57' , 'India',' ', 23.1877 , 75.3880000000001) ,
58:('A58' ,'B58' , 'India',' ', 24.752 , 75.0670000000001) ,
59:('A59' ,'B59' , 'India',' ', 25.9068 , 79.6690000000003) ,
60:('A60' ,'B60' , 'India',' ', 25.3334 , 75.6940000000001) ,
61:('A61' ,'B61' , 'India',' ', 19.9976 , 75.2500000000001) ,
62:('A62' ,'B62' , 'India',' ', 25.9016 , 77.3950000000002) ,
63:('A63' ,'B63' , 'India',' ', 25.368 , 76.9000000000001) ,
64:('A64' ,'B64' , 'India',' ', 26.6014 , 73.768) ,
65:('A65' ,'B65' , 'India',' ', 27.2813 , 77.4520000000002) ,
66:('A66' ,'B66' , 'India',' ', 21.5532 , 75.3340000000001) ,
67:('A67' ,'B67' , 'India',' ', 20.711 , 75.2980000000001) ,
68:('A68' ,'B68' , 'India',' ', 26.0379 , 77.5900000000002) ,
69:('A69' ,'B69' , 'India',' ', 25.581 , 78.8590000000002) ,
70:('A70' ,'B70' , 'India',' ', 25.8219 , 75.1420000000001) ,
71:('A71' ,'B71' , 'India',' ', 25.0395 , 79.7890000000003) ,
72:('A72' ,'B72' , 'India',' ', 27.0351 , 76.1830000000001) ,
73:('A73' ,'B73' , 'India',' ', 26.3796 , 76.5310000000001) ,
74:('A74' ,'B74' , 'India',' ', 25.7088 , 74.6560000000001) ,
75:('A75' ,'B75' , 'India',' ', 19.6878 , 76.8970000000001) ,
76:('A76' ,'B76' , 'India',' ', 20.0568 , 74.4490000000001) ,
77:('A77' ,'B77' , 'India',' ', 20.984 , 76.7710000000001) ,
78:('A78' ,'B78' , 'India',' ', 21.2965 , 73.549) ,
79:('A79' ,'B79' , 'India',' ', 20.2197 , 73.399) ,
80:('A80' ,'B80' , 'India',' ', 23.2061 , 75.9640000000001) ,
81:('A81' ,'B81' , 'India',' ', 19.9078 , 73.816) ,
82:('A82' ,'B82' , 'India',' ', 26.249 , 75.6460000000001) ,
83:('A83' ,'B83' , 'India',' ', 20.0771 , 78.7060000000002) ,
84:('A84' ,'B84' , 'India',' ', 20.6415 , 75.2140000000001) ,
85:('A85' ,'B85' , 'India',' ', 23.8414 , 75.4900000000001) ,
86:('A86' ,'B86' , 'India',' ', 25.681 , 75.9040000000001) ,
87:('A87' ,'B87' , 'India',' ', 21.1661 , 79.5610000000002) ,
88:('A88' ,'B88' , 'India',' ', 25.1785 , 76.2490000000001) ,
89:('A89' ,'B89' , 'India',' ', 25.4985 , 75.0160000000001) ,
90:('A90' ,'B90' , 'India',' ', 25.9707 , 76.5070000000001) ,
91:('A91' ,'B91' , 'India',' ', 20.0968 , 74.8690000000001) ,
92:('A92' ,'B92' , 'India',' ', 25.4017 , 77.1490000000002) ,
93:('A93' ,'B93' , 'India',' ', 23.4516 , 73.669) ,
94:('A94' ,'B94' , 'India',' ', 20.2784 , 78.3820000000002) ,
95:('A95' ,'B95' , 'India',' ', 19.3097 , 77.7370000000002) ,
96:('A96' ,'B96' , 'India',' ', 25.5928 , 76.1260000000001) ,
97:('A97' ,'B97' , 'India',' ', 26.842 , 74.086) ,
98:('A98' ,'B98' , 'India',' ', 25.0235 , 79.6960000000003) ,
99:('A99' ,'B99' , 'India',' ', 20.5895 , 76.9030000000001) ,
100:('A100' ,'B100' , 'India',' ', 20.2102 , 73.915) ,
101:('A101' ,'B101' , 'India',' ', 22.035 , 73.039) ,
102:('A102' ,'B102' , 'India',' ', 21.6064 , 73.894) ,
103:('A103' ,'B103' , 'India',' ', 20.876 , 78.9130000000002) ,
104:('A104' ,'B104' , 'India',' ', 26.1014 , 75.0430000000001) ,
105:('A105' ,'B105' , 'India',' ', 23.1521 , 79.6840000000003) ,
106:('A106' ,'B106' , 'India',' ', 24.5651 , 78.4330000000002) ,
107:('A107' ,'B107' , 'India',' ', 20.2718 , 75.0190000000001) ,
108:('A108' ,'B108' , 'India',' ', 20.1064 , 77.4670000000002) ,
109:('A109' ,'B109' , 'India',' ', 21.7203 , 75.2620000000001) ,
110:('A110' ,'B110' , 'India',' ', 20.5968 , 75.2770000000001) ,
111:('A111' ,'B111' , 'India',' ', 20.0689 , 75.7000000000001) ,
112:('A112' ,'B112' , 'India',' ', 25.1165 , 74.6080000000001) ,
113:('A113' ,'B113' , 'India',' ', 25.6883 , 77.5300000000002) ,
114:('A114' ,'B114' , 'India',' ', 19.7461 , 74.218) ,
115:('A115' ,'B115' , 'India',' ', 19.9581 , 75.8530000000001) ,
116:('A116' ,'B116' , 'India',' ', 20.7556 , 78.5950000000002) ,
117:('A117' ,'B117' , 'India',' ', 18.9712 , 78.9730000000002) ,
118:('A118' ,'B118' , 'India',' ', 24.9749 , 73.054) ,
119:('A119' ,'B119' , 'India',' ', 26.9634 , 74.5000000000001) ,
120:('A120' ,'B120' , 'India',' ', 20.0511 , 78.4150000000002) ,
121:('A121' ,'B121' , 'India',' ', 23.4718 , 78.9850000000002) ,
122:('A122' ,'B122' , 'India',' ', 25.3813 , 74.059) ,
123:('A123' ,'B123' , 'India',' ', 26.1125 , 76.3150000000001) ,
124:('A124' ,'B124' , 'India',' ', 18.164 , 77.4730000000002) ,
125:('A125' ,'B125' , 'India',' ', 20.6616 , 78.3160000000002) ,
126:('A126' ,'B126' , 'India',' ', 21.2939 , 77.9560000000002) ,
127:('A127' ,'B127' , 'India',' ', 23.3354 , 73.321) ,
128:('A128' ,'B128' , 'India',' ', 20.789 , 77.2540000000002) ,
129:('A129' ,'B129' , 'India',' ', 24.2936 , 79.6270000000003) ,
130:('A130' ,'B130' , 'India',' ', 20.0378 , 74.5300000000001) ,
131:('A131' ,'B131' , 'India',' ', 25.038 , 77.0140000000002) ,
132:('A132' ,'B132' , 'India',' ', 20.0711 , 75.0550000000001) ,
133:('A133' ,'B133' , 'India',' ', 24.7705 , 73.861) ,
134:('A134' ,'B134' , 'India',' ', 25.2071 , 74.2) ,
135:('A135' ,'B135' , 'India',' ', 25.7205 , 73.078) ,
136:('A136' ,'B136' , 'India',' ', 26.3909 , 79.6900000000003) ,
137:('A137' ,'B137' , 'India',' ', 25.4754 , 76.3630000000001) ,
138:('A138' ,'B138' , 'India',' ', 24.9871 , 74.308) ,
139:('A139' ,'B139' , 'India',' ', 23.5529 , 76.2670000000001) ,
140:('A140' ,'B140' , 'India',' ', 25.9451 , 77.8990000000002) ,
141:('A141' ,'B141' , 'India',' ', 25.3792 , 76.3120000000001) ,
142:('A142' ,'B142' , 'India',' ', 26.4485 , 73.756) ,
143:('A143' ,'B143' , 'India',' ', 25.4931 , 74.3350000000001) ,
144:('A144' ,'B144' , 'India',' ', 21.287 , 79.6780000000003) ,
145:('A145' ,'B145' , 'India',' ', 25.3755 , 78.2020000000002) ,
146:('A146' ,'B146' , 'India',' ', 22.5914 , 78.4870000000002) ,
147:('A147' ,'B147' , 'India',' ', 25.3952 , 78.3190000000002) ,
148:('A148' ,'B148' , 'India',' ', 25.261 , 73.006) ,
149:('A149' ,'B149' , 'India',' ', 22.0265 , 77.6230000000002) ,
150:('A150' ,'B150' , 'India',' ', 20.0879 , 78.7210000000002) ,
151:('A151' ,'B151' , 'India',' ', 23.4222 , 75.8290000000001) ,
152:('A152' ,'B152' , 'India',' ', 23.2209 , 74.7250000000001) ,
153:('A153' ,'B153' , 'India',' ', 19.0385 , 75.1480000000001) ,
154:('A154' ,'B154' , 'India',' ', 24.3705 , 77.3860000000002) ,
155:('A155' ,'B155' , 'India',' ', 25.3996 , 78.6430000000002) ,
156:('A156' ,'B156' , 'India',' ', 20.417 , 74.4880000000001) ,
157:('A157' ,'B157' , 'India',' ', 22.6218 , 76.1440000000001) ,
158:('A158' ,'B158' , 'India',' ', 25.0429 , 73.633) ,
159:('A159' ,'B159' , 'India',' ', 26.4397 , 73.921) ,
160:('A160' ,'B160' , 'India',' ', 25.8324 , 74.053) ,
161:('A161' ,'B161' , 'India',' ', 20.7279 , 77.8060000000002) ,
162:('A162' ,'B162' , 'India',' ', 22.0314 , 73.402) ,
163:('A163' ,'B163' , 'India',' ', 20.402 , 76.7050000000001) ,
164:('A164' ,'B164' , 'India',' ', 20.6375 , 73.09) ,
165:('A165' ,'B165' , 'India',' ', 25.3598 , 78.2080000000002) ,
166:('A166' ,'B166' , 'India',' ', 19.9454 , 73.24) ,
167:('A167' ,'B167' , 'India',' ', 20.7144 , 75.4150000000001) ,
168:('A168' ,'B168' , 'India',' ', 21.282 , 77.1040000000002) ,
169:('A169' ,'B169' , 'India',' ', 24.9827 , 75.7120000000001) ,
170:('A170' ,'B170' , 'India',' ', 25.6705 , 76.6600000000001) ,
171:('A171' ,'B171' , 'India',' ', 20.1348 , 77.0650000000002) ,
172:('A172' ,'B172' , 'India',' ', 25.6956 , 73.003) ,
173:('A173' ,'B173' , 'India',' ', 20.2746 , 78.8620000000002) ,
174:('A174' ,'B174' , 'India',' ', 25.3615 , 77.0050000000002) ,
175:('A175' ,'B175' , 'India',' ', 25.9914 , 76.9720000000002) ,
176:('A176' ,'B176' , 'India',' ', 25.8945 , 75.2440000000001) ,
177:('A177' ,'B177' , 'India',' ', 25.0193 , 74.5510000000001) ,
178:('A178' ,'B178' , 'India',' ', 24.8778 , 74.7910000000001) ,
179:('A179' ,'B179' , 'India',' ', 25.8763 , 76.5730000000001) ,
180:('A180' ,'B180' , 'India',' ', 19.6763 , 73.027) ,
181:('A181' ,'B181' , 'India',' ', 25.4531 , 77.1880000000002) ,
182:('A182' ,'B182' , 'India',' ', 25.1764 , 77.0890000000002) ,
183:('A183' ,'B183' , 'India',' ', 26.2984 , 78.0130000000002) ,
184:('A184' ,'B184' , 'India',' ', 24.6723 , 74.8270000000001) ,
185:('A185' ,'B185' , 'India',' ', 25.8183 , 73.198) ,
186:('A186' ,'B186' , 'India',' ', 26.249 , 79.4500000000002) ,
187:('A187' ,'B187' , 'India',' ', 25.7289 , 77.1700000000002) ,
188:('A188' ,'B188' , 'India',' ', 26.0132 , 79.8190000000003) ,
189:('A189' ,'B189' , 'India',' ', 19.4675 , 75.4780000000001) ,
190:('A190' ,'B190' , 'India',' ', 19.9497 , 77.7460000000002) ,
191:('A191' ,'B191' , 'India',' ', 26.1358 , 73.429) ,
192:('A192' ,'B192' , 'India',' ', 25.3063 , 73.891) ,
193:('A193' ,'B193' , 'India',' ', 20.2755 , 74.6920000000001) ,
194:('A194' ,'B194' , 'India',' ', 20.2474 , 77.2120000000002) ,
195:('A195' ,'B195' , 'India',' ', 25.5846 , 79.0630000000002) ,
196:('A196' ,'B196' , 'India',' ', 25.9333 , 76.8370000000001) ,
197:('A197' ,'B197' , 'India',' ', 25.2651 , 78.0070000000002) ,
198:('A198' ,'B198' , 'India',' ', 24.9423 , 74.3620000000001) ,
199:('A199' ,'B199' , 'India',' ', 20.3029 , 78.5770000000002) ,
200:('A200' ,'B200' , 'India',' ', 24.6965 , 74.5420000000001) ,
201:('A201' ,'B201' , 'India',' ', 25.9017 , 78.3790000000002) ,
202:('A202' ,'B202' , 'India',' ', 24.7421 , 77.3290000000002) ,
203:('A203' ,'B203' , 'India',' ', 19.8586 , 74.3800000000001) ,
204:('A204' ,'B204' , 'India',' ', 24.9969 , 74.074) ,
205:('A205' ,'B205' , 'India',' ', 20.3216 , 78.3880000000002) ,
206:('A206' ,'B206' , 'India',' ', 24.2341 , 75.6610000000001) ,
207:('A207' ,'B207' , 'India',' ', 24.8312 , 78.0970000000002) ,
208:('A208' ,'B208' , 'India',' ', 25.9316 , 73.309) ,
209:('A209' ,'B209' , 'India',' ', 23.9806 , 79.4140000000002) ,
210:('A210' ,'B210' , 'India',' ', 23.3737 , 74.3950000000001) ,
211:('A211' ,'B211' , 'India',' ', 20.719 , 74.197) ,
212:('A212' ,'B212' , 'India',' ', 19.8791 , 74.4040000000001) ,
213:('A213' ,'B213' , 'India',' ', 19.9907 , 75.3370000000001) ,
214:('A214' ,'B214' , 'India',' ', 24.6823 , 77.6710000000002) ,
215:('A215' ,'B215' , 'India',' ', 25.8349 , 78.8950000000002) ,
216:('A216' ,'B216' , 'India',' ', 21.2915 , 78.5170000000002) ,
217:('A217' ,'B217' , 'India',' ', 25.421 , 75.4210000000001) ,
218:('A218' ,'B218' , 'India',' ', 26.0588 , 74.7160000000001) ,
219:('A219' ,'B219' , 'India',' ', 24.8009 , 75.5080000000001) ,
220:('A220' ,'B220' , 'India',' ', 25.4054 , 74.287) ,
221:('A221' ,'B221' , 'India',' ', 25.8906 , 74.149) ,
222:('A222' ,'B222' , 'India',' ', 21.6184 , 73.957) ,
223:('A223' ,'B223' , 'India',' ', 21.8275 , 75.7030000000001) ,
224:('A224' ,'B224' , 'India',' ', 20.4835 , 79.8790000000003) ,
225:('A225' ,'B225' , 'India',' ', 19.1196 , 77.3560000000002) ,
226:('A226' ,'B226' , 'India',' ', 20.0029 , 77.0380000000002) ,
227:('A227' ,'B227' , 'India',' ', 20.409 , 75.6730000000001) ,
228:('A228' ,'B228' , 'India',' ', 21.627 , 77.2060000000002) ,
229:('A229' ,'B229' , 'India',' ', 24.5992 , 73.06) ,
230:('A230' ,'B230' , 'India',' ', 22.0565 , 79.8520000000003) ,
231:('A231' ,'B231' , 'India',' ', 19.4175 , 77.3320000000002) ,
232:('A232' ,'B232' , 'India',' ', 26.0085 , 77.6110000000002) ,
233:('A233' ,'B233' , 'India',' ', 20.499 , 79.2880000000002) ,
234:('A234' ,'B234' , 'India',' ', 19.9742 , 76.6210000000001) ,
235:('A235' ,'B235' , 'India',' ', 25.9012 , 79.8430000000003) ,
236:('A236' ,'B236' , 'India',' ', 19.8304 , 76.0630000000001) ,
237:('A237' ,'B237' , 'India',' ', 18.5877 , 75.3640000000001) ,
238:('A238' ,'B238' , 'India',' ', 22.7352 , 73.126) ,
239:('A239' ,'B239' , 'India',' ', 20.3092 , 79.7590000000003) ,
240:('A240' ,'B240' , 'India',' ', 26.9099 , 75.0460000000001) ,
241:('A241' ,'B241' , 'India',' ', 20.271 , 75.7090000000001) ,
242:('A242' ,'B242' , 'India',' ', 20.069 , 74.014) ,
243:('A243' ,'B243' , 'India',' ', 25.4124 , 73.093) ,
244:('A244' ,'B244' , 'India',' ', 24.5047 , 75.6370000000001) ,
245:('A245' ,'B245' , 'India',' ', 23.0774 , 76.3210000000001) ,
246:('A246' ,'B246' , 'India',' ', 23.1561 , 75.9880000000001) ,
247:('A247' ,'B247' , 'India',' ', 24.8067 , 74.3440000000001) ,
248:('A248' ,'B248' , 'India',' ', 21.448 , 79.3780000000002) ,
249:('A249' ,'B249' , 'India',' ', 20.754 , 73.537) ,
250:('A250' ,'B250' , 'India',' ', 25.043 , 75.6760000000001) ,
251:('A251' ,'B251' , 'India',' ', 25.9324689 , 75.6790000000001) ,
252:('A252' ,'B252' , 'India',' ', 23.0305 , 76.5850000000001) ,
253:('A253' ,'B253' , 'India',' ', 20.2694 , 78.7270000000002) ,
254:('A254' ,'B254' , 'India',' ', 26.4405 , 76.8700000000001) ,
255:('A255' ,'B255' , 'India',' ', 25.6039 , 76.9990000000002) ,
256:('A256' ,'B256' , 'India',' ', 20.897 , 78.7180000000002) ,
257:('A257' ,'B257' , 'India',' ', 20.2515 , 74.7280000000001) ,
258:('A258' ,'B258' , 'India',' ', 21.9965 , 76.4080000000001) ,
259:('A259' ,'B259' , 'India',' ', 25.306 , 77.1760000000002) ,
260:('A260' ,'B260' , 'India',' ', 25.3703 , 78.6580000000002) ,
261:('A261' ,'B261' , 'India',' ', 25.6654 , 76.1860000000001) ,
262:('A262' ,'B262' , 'India',' ', 19.4032 , 75.7390000000001) ,
263:('A263' ,'B263' , 'India',' ', 25.0063 , 73.903) ,
264:('A264' ,'B264' , 'India',' ', 17.8737 , 75.7810000000001) ,
265:('A265' ,'B265' , 'India',' ', 25.513 , 74.164) ,
266:('A266' ,'B266' , 'India',' ', 20.2892 , 74.8360000000001) ,
267:('A267' ,'B267' , 'India',' ', 25.5857 , 78.7240000000002) ,
268:('A268' ,'B268' , 'India',' ', 24.6311 , 78.3940000000002) ,
269:('A269' ,'B269' , 'India',' ', 26.2016 , 78.4420000000002) ,
270:('A270' ,'B270' , 'India',' ', 26.8672 , 76.8340000000001) ,
271:('A271' ,'B271' , 'India',' ', 25.3692 , 73.606) ,
272:('A272' ,'B272' , 'India',' ', 25.7217 , 79.0930000000002) ,
273:('A273' ,'B273' , 'India',' ', 25.2739 , 73.552) ,
274:('A274' ,'B274' , 'India',' ', 24.8148 , 75.4060000000001) ,
275:('A275' ,'B275' , 'India',' ', 25.8851 , 76.5190000000001) ,
276:('A276' ,'B276' , 'India',' ', 20.8405 , 73.972) ,
277:('A277' ,'B277' , 'India',' ', 23.469 , 79.4290000000002) ,
278:('A278' ,'B278' , 'India',' ', 20.388 , 77.5990000000002) ,
279:('A279' ,'B279' , 'India',' ', 19.7131 , 73.837) ,
280:('A280' ,'B280' , 'India',' ', 20.789 , 75.1930000000001) ,
281:('A281' ,'B281' , 'India',' ', 23.985 , 77.4550000000002) ,
282:('A282' ,'B282' , 'India',' ', 19.9196 , 77.6020000000002) ,
283:('A283' ,'B283' , 'India',' ', 25.6739 , 79.5580000000002) ,
284:('A284' ,'B284' , 'India',' ', 20.9248 , 77.4280000000002) ,
285:('A285' ,'B285' , 'India',' ', 24.6051 , 78.7660000000002) ,
286:('A286' ,'B286' , 'India',' ', 25.0533 , 79.7140000000003) ,
287:('A287' ,'B287' , 'India',' ', 25.3706 , 73.834) ,
288:('A288' ,'B288' , 'India',' ', 21.3348 , 77.1190000000002) ,
289:('A289' ,'B289' , 'India',' ', 25.3794 , 73.423) ,
290:('A290' ,'B290' , 'India',' ', 24.5528 , 74.8060000000001) ,
291:('A291' ,'B291' , 'India',' ', 21.2951 , 73.3) ,
292:('A292' ,'B292' , 'India',' ', 20.6059 , 75.0070000000001) ,
293:('A293' ,'B293' , 'India',' ', 21.014 , 75.1450000000001) ,
294:('A294' ,'B294' , 'India',' ', 21.606 , 79.2520000000002) ,
295:('A295' ,'B295' , 'India',' ', 25.0471 , 74.161) ,
296:('A296' ,'B296' , 'India',' ', 26.7836 , 78.8680000000002) ,
297:('A297' ,'B297' , 'India',' ', 19.0749 , 74.239) ,
298:('A298' ,'B298' , 'India',' ', 20.313 , 76.3540000000001) ,
299:('A299' ,'B299' , 'India',' ', 25.0632 , 73.165) ,
300:('A300' ,'B300' , 'India',' ', 25.9731 , 73.102) ,
301:('A301' ,'B301' , 'India',' ', 25.8911 , 76.5640000000001) ,
302:('A302' ,'B302' , 'India',' ', 20.848 , 74.005) ,
303:('A303' ,'B303' , 'India',' ', 25.987 , 73.255) ,
304:('A304' ,'B304' , 'India',' ', 20.8349 , 79.3600000000002) ,
305:('A305' ,'B305' , 'India',' ', 24.6303 , 75.6910000000001) ,
306:('A306' ,'B306' , 'India',' ', 20.7998 , 75.5710000000001) ,
307:('A307' ,'B307' , 'India',' ', 23.0273 , 74.9110000000001) ,
308:('A308' ,'B308' , 'India',' ', 27.0334 , 73.144) ,
309:('A309' ,'B309' , 'India',' ', 18.8788 , 76.8400000000001) ,
310:('A310' ,'B310' , 'India',' ', 20.654 , 78.3400000000002) ,
311:('A311' ,'B311' , 'India',' ', 24.4386 , 78.4390000000002) ,
312:('A312' ,'B312' , 'India',' ', 26.5402 , 73.075) ,
313:('A313' ,'B313' , 'India',' ', 20.1075 , 73.759) ,
314:('A314' ,'B314' , 'India',' ', 20.7326 , 76.0420000000001) ,
315:('A315' ,'B315' , 'India',' ', 21.1211 , 74.7100000000001) ,
316:('A316' ,'B316' , 'India',' ', 26.6687 , 75.9070000000001) ,
317:('A317' ,'B317' , 'India',' ', 20.2479 , 75.7180000000001) ,
318:('A318' ,'B318' , 'India',' ', 19.8167 , 74.266) ,
319:('A319' ,'B319' , 'India',' ', 26.7503 , 74.116) ,
320:('A320' ,'B320' , 'India',' ', 22.3225 , 76.5370000000001) ,
321:('A321' ,'B321' , 'India',' ', 26.06 , 75.4540000000001) ,
322:('A322' ,'B322' , 'India',' ', 25.0185 , 74.152) ,
323:('A323' ,'B323' , 'India',' ', 20.0032 , 73.252) ,
324:('A324' ,'B324' , 'India',' ', 25.4937 , 73.897) ,
325:('A325' ,'B325' , 'India',' ', 21.4428 , 73.522) ,
326:('A326' ,'B326' , 'India',' ', 18.6545 , 79.0240000000002) ,
327:('A327' ,'B327' , 'India',' ', 19.0342 , 74.4760000000001) ,
328:('A328' ,'B328' , 'India',' ', 25.4417 , 79.3900000000002) ,
329:('A329' ,'B329' , 'India',' ', 24.4476 , 78.1000000000002) ,
330:('A330' ,'B330' , 'India',' ', 26.6488 , 78.0790000000002) ,
331:('A331' ,'B331' , 'India',' ', 27.2715 , 77.6050000000002) ,
332:('A332' ,'B332' , 'India',' ', 23.9588 , 79.0330000000002) ,
333:('A333' ,'B333' , 'India',' ', 23.5149 , 77.6410000000002) ,
334:('A334' ,'B334' , 'India',' ', 25.9042 , 78.4690000000002) ,
335:('A335' ,'B335' , 'India',' ', 25.2558 , 78.2440000000002) ,
336:('A336' ,'B336' , 'India',' ', 25.93 , 75.5470000000001) ,
337:('A337' ,'B337' , 'India',' ', 20.233 , 73.084) ,
338:('A338' ,'B338' , 'India',' ', 26.345 , 76.8910000000001) ,
339:('A339' ,'B339' , 'India',' ', 25.3 , 75.8200000000001) ,
340:('A340' ,'B340' , 'India',' ', 24.9156 , 74.233) ,
341:('A341' ,'B341' , 'India',' ', 18.5379 , 73.573) ,
342:('A342' ,'B342' , 'India',' ', 25.3813 , 75.9820000000001) ,
343:('A343' ,'B343' , 'India',' ', 21.4834 , 73.471) ,
344:('A344' ,'B344' , 'India',' ', 25.3966 , 77.8570000000002) ,
345:('A345' ,'B345' , 'India',' ', 20.489 , 75.2170000000001) ,
346:('A346' ,'B346' , 'India',' ', 26.7826098 , 78.0160000000002) ,
347:('A347' ,'B347' , 'India',' ', 26.2009 , 75.6190000000001) ,
348:('A348' ,'B348' , 'India',' ', 25.4171 , 78.5800000000002) ,
349:('A349' ,'B349' , 'India',' ', 18.6145 , 76.2280000000001) ,
350:('A350' ,'B350' , 'India',' ', 25.4835 , 75.4330000000001) ,
351:('A351' ,'B351' , 'India',' ', 21.6275 , 78.3250000000002) ,
352:('A352' ,'B352' , 'India',' ', 26.1046 , 74.9050000000001) ,
353:('A353' ,'B353' , 'India',' ', 24.5849 , 73.843) ,
354:('A354' ,'B354' , 'India',' ', 21.1893 , 77.5360000000002) ,
355:('A355' ,'B355' , 'India',' ', 20.871 , 79.5940000000002) ,
356:('A356' ,'B356' , 'India',' ', 26.5398 , 76.6420000000001) ,
357:('A357' ,'B357' , 'India',' ', 21.3255 , 79.0150000000002) ,
358:('A358' ,'B358' , 'India',' ', 20.316 , 75.2800000000001) ,
359:('A359' ,'B359' , 'India',' ', 20.105 , 75.5440000000001) ,
360:('A360' ,'B360' , 'India',' ', 19.9788 , 78.1540000000002) ,
361:('A361' ,'B361' , 'India',' ', 25.1441 , 75.3610000000001) ,
362:('A362' ,'B362' , 'India',' ', 26.0636 , 78.2620000000002) ,
363:('A363' ,'B363' , 'India',' ', 26.4727 , 76.6510000000001) ,
364:('A364' ,'B364' , 'India',' ', 25.7632 , 76.4380000000001) ,
365:('A365' ,'B365' , 'India',' ', 26.3142 , 74.5450000000001) ,
366:('A366' ,'B366' , 'India',' ', 23.6325 , 76.1110000000001) ,
367:('A367' ,'B367' , 'India',' ', 25.7069 , 78.5230000000002) ,
368:('A368' ,'B368' , 'India',' ', 23.1561 , 79.0210000000002) ,
369:('A369' ,'B369' , 'India',' ', 21.2709 , 73.738) ,
370:('A370' ,'B370' , 'India',' ', 24.2812 , 78.0610000000002) ,
371:('A371' ,'B371' , 'India',' ', 24.8836 , 78.1690000000002) ,
372:('A372' ,'B372' , 'India',' ', 25.8686 , 76.3450000000001) ,
373:('A373' ,'B373' , 'India',' ', 19.9924 , 74.9770000000001) ,
374:('A374' ,'B374' , 'India',' ', 24.7675 , 76.9210000000001) ,
375:('A375' ,'B375' , 'India',' ', 21.4523 , 78.9760000000002) ,
376:('A376' ,'B376' , 'India',' ', 27.8579 , 73.159) ,
377:('A377' ,'B377' , 'India',' ', 24.0586 , 79.1260000000002) ,
378:('A378' ,'B378' , 'India',' ', 25.1532 , 79.0270000000002) ,
379:('A379' ,'B379' , 'India',' ', 19.4085 , 75.2890000000001) ,
380:('A380' ,'B380' , 'India',' ', 25.9288 , 79.3660000000002) ,
381:('A381' ,'B381' , 'India',' ', 23.2264 , 77.4970000000002) ,
382:('A382' ,'B382' , 'India',' ', 20.369 , 73.975) ,
383:('A383' ,'B383' , 'India',' ', 20.663 , 79.7950000000003) ,
384:('A384' ,'B384' , 'India',' ', 21.7374 , 76.1920000000001) ,
385:('A385' ,'B385' , 'India',' ', 25.6915 , 78.2800000000002) ,
386:('A386' ,'B386' , 'India',' ', 24.0054 , 73.531) ,
387:('A387' ,'B387' , 'India',' ', 20.6949 , 73.678) ,
388:('A388' ,'B388' , 'India',' ', 24.6844 , 78.6700000000002) ,
389:('A389' ,'B389' , 'India',' ', 24.6055 , 75.4600000000001) ,
390:('A390' ,'B390' , 'India',' ', 19.1012 , 76.8280000000001) ,
391:('A391' ,'B391' , 'India',' ', 25.5803 , 77.9170000000002) ,
392:('A392' ,'B392' , 'India',' ', 24.2981 , 76.6930000000001) ,
393:('A393' ,'B393' , 'India',' ', 21.5738 , 78.5920000000002) ,
394:('A394' ,'B394' , 'India',' ', 20.9841 , 78.6310000000002) ,
395:('A395' ,'B395' , 'India',' ', 25.3689 , 78.4720000000002) ,
396:('A396' ,'B396' , 'India',' ', 25.2135 , 73.153) ,
397:('A397' ,'B397' , 'India',' ', 23.5161 , 73.276) ,
398:('A398' ,'B398' , 'India',' ', 25.3935 , 76.7920000000001) ,
399:('A399' ,'B399' , 'India',' ', 26.5008 , 75.7300000000001) ,
400:('A400' ,'B400' , 'India',' ', 25.0399 , 75.0850000000001) ,
401:('A401' ,'B401' , 'India',' ', 26.4395 , 78.8290000000002) ,
402:('A402' ,'B402' , 'India',' ', 23.6506 , 77.2600000000002) ,
403:('A403' ,'B403' , 'India',' ', 25.3578 , 79.0690000000002) ,
404:('A404' ,'B404' , 'India',' ', 20.681 , 76.9420000000001) ,
405:('A405' ,'B405' , 'India',' ', 25.4951 , 79.5910000000002) ,
406:('A406' ,'B406' , 'India',' ', 19.8099 , 74.9830000000001) ,
407:('A407' ,'B407' , 'India',' ', 20.6142 , 76.8550000000001) ,
408:('A408' ,'B408' , 'India',' ', 24.6377 , 74.9530000000001) ,
409:('A409' ,'B409' , 'India',' ', 20.62 , 76.2010000000001) ,
410:('A410' ,'B410' , 'India',' ', 20.36 , 74.6980000000001) ,
411:('A411' ,'B411' , 'India',' ', 24.1328 , 76.3990000000001) ,
412:('A412' ,'B412' , 'India',' ', 25.1243 , 73.702) ,
413:('A413' ,'B413' , 'India',' ', 26.2131 , 78.2860000000002) ,
414:('A414' ,'B414' , 'India',' ', 24.8523 , 76.0870000000001) ,
415:('A415' ,'B415' , 'India',' ', 19.8867 , 79.2070000000002) ,
416:('A416' ,'B416' , 'India',' ', 26.295 , 73.888) ,
417:('A417' ,'B417' , 'India',' ', 21.3165 , 76.7830000000001) ,
418:('A418' ,'B418' , 'India',' ', 25.2206 , 79.5340000000002) ,
419:('A419' ,'B419' , 'India',' ', 26.1264 , 77.0830000000002) ,
420:('A420' ,'B420' , 'India',' ', 23.4324 , 77.9950000000002) ,
421:('A421' ,'B421' , 'India',' ', 23.5311 , 76.5970000000001) ,
422:('A422' ,'B422' , 'India',' ', 26.0618 , 75.4750000000001) ,
423:('A423' ,'B423' , 'India',' ', 25.0536 , 73.792) ,
424:('A424' ,'B424' , 'India',' ', 26.6792 , 79.8580000000003) ,
425:('A425' ,'B425' , 'India',' ', 25.7668 , 74.5570000000001) ,
426:('A426' ,'B426' , 'India',' ', 25.6957 , 74.314) ,
427:('A427' ,'B427' , 'India',' ', 26.6377 , 77.7160000000002) ,
428:('A428' ,'B428' , 'India',' ', 26.1378 , 77.0860000000002) ,
429:('A429' ,'B429' , 'India',' ', 20.1375 , 74.7820000000001) ,
430:('A430' ,'B430' , 'India',' ', 25.2582 , 73.954) ,
431:('A431' ,'B431' , 'India',' ', 24.4557 , 78.5650000000002) ,
432:('A432' ,'B432' , 'India',' ', 19.9171 , 78.4540000000002) ,
433:('A433' ,'B433' , 'India',' ', 25.3758 , 78.6400000000002) ,
434:('A434' ,'B434' , 'India',' ', 19.2176 , 78.9400000000002) ,
435:('A435' ,'B435' , 'India',' ', 20.1037 , 76.2460000000001) ,
436:('A436' ,'B436' , 'India',' ', 26.2507 , 79.7350000000003) ,
437:('A437' ,'B437' , 'India',' ', 20.6194 , 77.7070000000002) ,
438:('A438' ,'B438' , 'India',' ', 25.3749 , 77.6140000000002) ,
439:('A439' ,'B439' , 'India',' ', 21.2141 , 76.1980000000001) ,
440:('A440' ,'B440' , 'India',' ', 24.8141 , 73.036) ,
441:('A441' ,'B441' , 'India',' ', 25.2167 , 73.456) ,
442:('A442' ,'B442' , 'India',' ', 20.1578 , 78.9550000000002) ,
443:('A443' ,'B443' , 'India',' ', 25.4216 , 73.777) ,
444:('A444' ,'B444' , 'India',' ', 26.7247 , 75.0640000000001) ,
445:('A445' ,'B445' , 'India',' ', 26.3283 , 76.9360000000001) ,
446:('A446' ,'B446' , 'India',' ', 24.9223 , 77.6320000000002) ,
447:('A447' ,'B447' , 'India',' ', 24.0746 , 78.1390000000002) ,
448:('A448' ,'B448' , 'India',' ', 20.567 , 74.9560000000001) ,
449:('A449' ,'B449' , 'India',' ', 20.1262 , 78.2380000000002) ,
450:('A450' ,'B450' , 'India',' ', 26.464 , 77.0470000000002) ,
451:('A451' ,'B451' , 'India',' ', 25.1246 , 76.2190000000001) ,
452:('A452' ,'B452' , 'India',' ', 24.9414 , 79.1500000000002) ,
453:('A453' ,'B453' , 'India',' ', 25.3906 , 75.8920000000001) ,
454:('A454' ,'B454' , 'India',' ', 20.5459 , 79.8040000000003) ,
455:('A455' ,'B455' , 'India',' ', 24.6611 , 73.558) ,
456:('A456' ,'B456' , 'India',' ', 20.2635 , 78.4900000000002) ,
457:('A457' ,'B457' , 'India',' ', 25.0426 , 79.6870000000003) ,
458:('A458' ,'B458' , 'India',' ', 20.5401 , 77.8420000000002) ,
459:('A459' ,'B459' , 'India',' ', 25.8686 , 76.8670000000001) ,
460:('A460' ,'B460' , 'India',' ', 25.3937 , 75.7720000000001) ,
461:('A461' ,'B461' , 'India',' ', 24.3298 , 74.05) ,
462:('A462' ,'B462' , 'India',' ', 25.4141 , 74.5480000000001) ,
463:('A463' ,'B463' , 'India',' ', 24.3358 , 73.513) ,
464:('A464' ,'B464' , 'India',' ', 25.9072 , 77.9590000000002) ,
465:('A465' ,'B465' , 'India',' ', 26.2853 , 77.9920000000002) ,
466:('A466' ,'B466' , 'India',' ', 19.7239 , 75.9250000000001) ,
467:('A467' ,'B467' , 'India',' ', 21.4192 , 75.2710000000001) ,
468:('A468' ,'B468' , 'India',' ', 21.8538 , 74.8030000000001) ,
469:('A469' ,'B469' , 'India',' ', 25.7527 , 76.8790000000001) ,
470:('A470' ,'B470' , 'India',' ', 25.4249 , 75.7330000000001) ,
471:('A471' ,'B471' , 'India',' ', 26.3012 , 78.7750000000002) ,
472:('A472' ,'B472' , 'India',' ', 25.6597 , 79.6300000000003) ,
473:('A473' ,'B473' , 'India',' ', 20.189 , 74.203) ,
474:('A474' ,'B474' , 'India',' ', 23.8299 , 77.2210000000002) ,
475:('A475' ,'B475' , 'India',' ', 24.1903 , 73.483) ,
476:('A476' ,'B476' , 'India',' ', 25.048 , 77.0350000000002) ,
477:('A477' ,'B477' , 'India',' ', 20.5977 , 74.3470000000001) ,
478:('A478' ,'B478' , 'India',' ', 25.3651 , 76.8310000000001) ,
479:('A479' ,'B479' , 'India',' ', 25.2527 , 79.4170000000002) ,
480:('A480' ,'B480' , 'India',' ', 23.1758 , 76.1650000000001) ,
481:('A481' ,'B481' , 'India',' ', 26.6658 , 77.5960000000002) ,
482:('A482' ,'B482' , 'India',' ', 25.2647 , 79.4380000000002) ,
483:('A483' ,'B483' , 'India',' ', 25.4352 , 74.6380000000001) ,
484:('A484' ,'B484' , 'India',' ', 25.0193 , 73.612) ,
485:('A485' ,'B485' , 'India',' ', 25.2615 , 79.4980000000002) ,
486:('A486' ,'B486' , 'India',' ', 24.1461 , 79.3690000000002) ,
487:('A487' ,'B487' , 'India',' ', 23.2297 , 78.7720000000002) ,
488:('A488' ,'B488' , 'India',' ', 21.2938 , 73.876) ,
489:('A489' ,'B489' , 'India',' ', 24.7553 , 78.1780000000002) ,
490:('A490' ,'B490' , 'India',' ', 26.2255 , 76.6570000000001) ,
491:('A491' ,'B491' , 'India',' ', 25.9264 , 73.933) ,
492:('A492' ,'B492' , 'India',' ', 20.0588 , 73.291) ,
493:('A493' ,'B493' , 'India',' ', 19.5579 , 75.9970000000001) ,
494:('A494' ,'B494' , 'India',' ', 27.2897 , 74.9950000000001) ,
495:('A495' ,'B495' , 'India',' ', 21.264 , 78.9670000000002) ,
496:('A496' ,'B496' , 'India',' ', 24.8836 , 75.9580000000001) ,
497:('A497' ,'B497' , 'India',' ', 25.6846 , 77.0710000000002) ,
498:('A498' ,'B498' , 'India',' ', 20.3551 , 76.9330000000001) ,
499:('A499' ,'B499' , 'India',' ', 25.7159 , 78.4840000000002) ,
500:('A500' ,'B500' , 'India',' ', 25.0414 , 75.2560000000001) ,
501:('A501' ,'B501' , 'India',' ', 22.4994 , 76.2910000000001) ,
502:('A502' ,'B502' , 'India',' ', 25.4687 , 77.0320000000002) ,
503:('A503' ,'B503' , 'India',' ', 25.3913 , 77.4910000000002) ,
504:('A504' ,'B504' , 'India',' ', 20.6747 , 73.579) ,
505:('A505' ,'B505' , 'India',' ', 19.1145 , 74.23) ,
506:('A506' ,'B506' , 'India',' ', 25.5859 , 75.3910000000001) ,
507:('A507' ,'B507' , 'India',' ', 25.2058 , 77.3530000000002) ,
508:('A508' ,'B508' , 'India',' ', 25.0008 , 75.5860000000001) ,
509:('A509' ,'B509' , 'India',' ', 25.4553 , 79.2160000000002) ,
510:('A510' ,'B510' , 'India',' ', 26.2784 , 79.6510000000003) ,
511:('A511' ,'B511' , 'India',' ', 23.1633 , 77.2330000000002) ,
512:('A512' ,'B512' , 'India',' ', 25.8814 , 76.5490000000001) ,
513:('A513' ,'B513' , 'India',' ', 25.6581 , 75.1330000000001) ,
514:('A514' ,'B514' , 'India',' ', 19.3878 , 79.6090000000003) ,
515:('A515' ,'B515' , 'India',' ', 19.9354 , 76.0270000000001) ,
516:('A516' ,'B516' , 'India',' ', 25.4646 , 75.8260000000001) ,
517:('A517' ,'B517' , 'India',' ', 25.3782 , 79.8370000000003) ,
518:('A518' ,'B518' , 'India',' ', 23.2975 , 77.8750000000002) ,
519:('A519' ,'B519' , 'India',' ', 20.0401 , 77.8240000000002) ,
520:('A520' ,'B520' , 'India',' ', 20.5168 , 75.9850000000001) ,
521:('A521' ,'B521' , 'India',' ', 23.9841 , 78.1870000000002) ,
522:('A522' ,'B522' , 'India',' ', 24.6652 , 76.3510000000001) ,
523:('A523' ,'B523' , 'India',' ', 25.2093 , 76.0330000000001) ,
524:('A524' ,'B524' , 'India',' ', 24.5958 , 75.0040000000001) ,
525:('A525' ,'B525' , 'India',' ', 22.359 , 75.2860000000001) ,
526:('A526' ,'B526' , 'India',' ', 26.257 , 77.2930000000002) ,
527:('A527' ,'B527' , 'India',' ', 26.214 , 74.9020000000001) ,
528:('A528' ,'B528' , 'India',' ', 23.7362 , 78.3730000000002) ,
529:('A529' ,'B529' , 'India',' ', 20.994 , 76.0060000000001) ,
530:('A530' ,'B530' , 'India',' ', 21.3754 , 73.672) ,
531:('A531' ,'B531' , 'India',' ', 23.6168 , 76.5610000000001) ,
532:('A532' ,'B532' , 'India',' ', 24.653 , 74.8210000000001) ,
533:('A533' ,'B533' , 'India',' ', 24.1699 , 74.5660000000001) ,
534:('A534' ,'B534' , 'India',' ', 26.485 , 74.4250000000001) ,
535:('A535' ,'B535' , 'India',' ', 27.34 , 77.4850000000002) ,
536:('A536' ,'B536' , 'India',' ', 21.9636 , 74.5540000000001) ,
537:('A537' ,'B537' , 'India',' ', 25.3216 , 73.723) ,
538:('A538' ,'B538' , 'India',' ', 24.9441 , 78.7900000000002) ,
539:('A539' ,'B539' , 'India',' ', 25.3232 , 75.1270000000001) ,
540:('A540' ,'B540' , 'India',' ', 25.6574 , 77.8360000000002) ,
541:('A541' ,'B541' , 'India',' ', 26.3319 , 78.2140000000002) ,
542:('A542' ,'B542' , 'India',' ', 26.8206 , 78.1630000000002) ,
543:('A543' ,'B543' , 'India',' ', 18.9664 , 76.4890000000001) ,
544:('A544' ,'B544' , 'India',' ', 25.9021 , 75.7660000000001) ,
545:('A545' ,'B545' , 'India',' ', 25.5789 , 75.7600000000001) ,
546:('A546' ,'B546' , 'India',' ', 26.2399 , 73.219) ,
547:('A547' ,'B547' , 'India',' ', 25.839 , 74.8630000000001) ,
548:('A548' ,'B548' , 'India',' ', 25.0205 , 78.7930000000002) ,
549:('A549' ,'B549' , 'India',' ', 23.4538 , 77.6080000000002) ,
550:('A550' ,'B550' , 'India',' ', 24.587 , 73.066) ,
551:('A551' ,'B551' , 'India',' ', 25.878 , 77.6560000000002) ,
552:('A552' ,'B552' , 'India',' ', 21.3152 , 75.1060000000001) ,
553:('A553' ,'B553' , 'India',' ', 25.4094 , 78.3490000000002) ,
554:('A554' ,'B554' , 'India',' ', 19.6114 , 77.8540000000002) ,
555:('A555' ,'B555' , 'India',' ', 25.9025 , 75.2650000000001) ,
556:('A556' ,'B556' , 'India',' ', 22.5665 , 76.8010000000001) ,
557:('A557' ,'B557' , 'India',' ', 22.8665 , 73.741) ,
558:('A558' ,'B558' , 'India',' ', 20.13 , 78.0370000000002) ,
559:('A559' ,'B559' , 'India',' ', 26.6173 , 79.3180000000002) ,
560:('A560' ,'B560' , 'India',' ', 23.4529 , 75.0340000000001) ,
561:('A561' ,'B561' , 'India',' ', 23.1827 , 75.6670000000001) ,
562:('A562' ,'B562' , 'India',' ', 25.0741 , 73.828) ,
563:('A563' ,'B563' , 'India',' ', 22.9616 , 78.4300000000002) ,
564:('A564' ,'B564' , 'India',' ', 24.313 , 77.5690000000002) ,
565:('A565' ,'B565' , 'India',' ', 26.4512 , 78.4090000000002) ,
566:('A566' ,'B566' , 'India',' ', 22.3593 , 74.5210000000001) ,
567:('A567' ,'B567' , 'India',' ', 24.7442 , 79.1950000000002) ,
568:('A568' ,'B568' , 'India',' ', 24.672 , 74.023) ,
569:('A569' ,'B569' , 'India',' ', 22.0288 , 77.4940000000002) ,
570:('A570' ,'B570' , 'India',' ', 24.7135 , 74.4670000000001) ,
571:('A571' ,'B571' , 'India',' ', 20.711 , 79.3450000000002) ,
572:('A572' ,'B572' , 'India',' ', 26.203 , 78.3040000000002) ,
573:('A573' ,'B573' , 'India',' ', 25.0912 , 78.8770000000002) ,
574:('A574' ,'B574' , 'India',' ', 24.6012 , 77.2870000000002) ,
575:('A575' ,'B575' , 'India',' ', 26.0324 , 76.2310000000001) ,
576:('A576' ,'B576' , 'India',' ', 20.5 , 79.3330000000002) ,
577:('A577' ,'B577' , 'India',' ', 26.9782 , 74.263) ,
578:('A578' ,'B578' , 'India',' ', 23.7889 , 74.3830000000001) ,
579:('A579' ,'B579' , 'India',' ', 21.016 , 77.7550000000002) ,
580:('A580' ,'B580' , 'India',' ', 17.5406 , 79.8760000000003) ,
581:('A581' ,'B581' , 'India',' ', 25.0026 , 73.627) ,
582:('A582' ,'B582' , 'India',' ', 25.9078 , 74.254) ,
583:('A583' ,'B583' , 'India',' ', 25.0179 , 76.2760000000001) ,
584:('A584' ,'B584' , 'India',' ', 26.3566 , 75.2080000000001) ,
585:('A585' ,'B585' , 'India',' ', 26.9135 , 78.8890000000002) ,
586:('A586' ,'B586' , 'India',' ', 25.7118 , 75.3940000000001) ,
587:('A587' ,'B587' , 'India',' ', 26.6402 , 76.7500000000001) ,
588:('A588' ,'B588' , 'India',' ', 19.1084 , 77.5030000000002) ,
589:('A589' ,'B589' , 'India',' ', 21.0826 , 78.4960000000002) ,
590:('A590' ,'B590' , 'India',' ', 20.1101 , 79.0510000000002) ,
591:('A591' ,'B591' , 'India',' ', 18.4194 , 76.2610000000001) ,
592:('A592' ,'B592' , 'India',' ', 19.9346 , 78.7780000000002) ,
593:('A593' ,'B593' , 'India',' ', 20.2156 , 78.8140000000002) ,
594:('A594' ,'B594' , 'India',' ', 20.0042 , 75.0820000000001) ,
595:('A595' ,'B595' , 'India',' ', 24.6149 , 75.7570000000001) ,
596:('A596' ,'B596' , 'India',' ', 26.3642 , 75.5380000000001) ,
597:('A597' ,'B597' , 'India',' ', 23.4522 , 73.264) ,
598:('A598' ,'B598' , 'India',' ', 25.4118 , 76.4440000000001) ,
599:('A599' ,'B599' , 'India',' ', 20.243 , 73.288) ,
600:('A600' ,'B600' , 'India',' ', 20.0884 , 74.6260000000001) ,
601:('A601' ,'B601' , 'India',' ', 19.9755 , 78.8830000000002) ,
602:('A602' ,'B602' , 'India',' ', 22.6451 , 77.6170000000002) ,
603:('A603' ,'B603' , 'India',' ', 20.4294 , 75.8500000000001) ,
604:('A604' ,'B604' , 'India',' ', 24.8411 , 77.4460000000002) ,
605:('A605' ,'B605' , 'India',' ', 19.3299 , 79.2730000000002) ,
606:('A606' ,'B606' , 'India',' ', 20.832 , 79.4800000000002) ,
607:('A607' ,'B607' , 'India',' ', 26.9523 , 74.7850000000001) ,
608:('A608' ,'B608' , 'India',' ', 19.8974 , 77.6470000000002) ,
609:('A609' ,'B609' , 'India',' ', 22.4811 , 76.5130000000001) ,
610:('A610' ,'B610' , 'India',' ', 25.9948 , 76.5790000000001) ,
611:('A611' ,'B611' , 'India',' ', 25.3815 , 77.1940000000002) ,
612:('A612' ,'B612' , 'India',' ', 27.0910662 , 79.2370000000002) ,
613:('A613' ,'B613' , 'India',' ', 25.9851 , 78.4210000000002) ,
614:('A614' ,'B614' , 'India',' ', 20.76 , 75.6400000000001) ,
615:('A615' ,'B615' , 'India',' ', 22.3942 , 73.942) ,
616:('A616' ,'B616' , 'India',' ', 23.6506 , 77.7520000000002) ,
617:('A617' ,'B617' , 'India',' ', 26.6488 , 78.6640000000002) ,
618:('A618' ,'B618' , 'India',' ', 25.857 , 77.9740000000002) ,
619:('A619' ,'B619' , 'India',' ', 25.9291 , 77.9020000000002) ,
620:('A620' ,'B620' , 'India',' ', 24.6095 , 73.267) ,
621:('A621' ,'B621' , 'India',' ', 20.8043 , 76.1170000000001) ,
622:('A622' ,'B622' , 'India',' ', 19.6607 , 79.1290000000002) ,
623:('A623' ,'B623' , 'India',' ', 20.49 , 73.459) ,
624:('A624' ,'B624' , 'India',' ', 25.0256 , 76.6330000000001) ,
625:('A625' ,'B625' , 'India',' ', 26.4915 , 78.5530000000002) ,
626:('A626' ,'B626' , 'India',' ', 23.6288 , 76.9750000000002) ,
627:('A627' ,'B627' , 'India',' ', 26.6248 , 79.2640000000002) ,
628:('A628' ,'B628' , 'India',' ', 20.419 , 78.7480000000002) ,
629:('A629' ,'B629' , 'India',' ', 20.2221 , 75.0370000000001) ,
630:('A630' ,'B630' , 'India',' ', 19.0497 , 73.117) ,
631:('A631' ,'B631' , 'India',' ', 26.5136 , 74.7550000000001) ,
632:('A632' ,'B632' , 'India',' ', 19.8375 , 79.0660000000002) ,
633:('A633' ,'B633' , 'India',' ', 26.3465 , 75.8950000000001) ,
634:('A634' ,'B634' , 'India',' ', 24.7331 , 76.9150000000001) ,
635:('A635' ,'B635' , 'India',' ', 26.391 , 73.069) ,
636:('A636' ,'B636' , 'India',' ', 25.0099 , 76.2850000000001) ,
637:('A637' ,'B637' , 'India',' ', 25.9112 , 77.2030000000002) ,
638:('A638' ,'B638' , 'India',' ', 24.1749 , 73.108) ,
639:('A639' ,'B639' , 'India',' ', 26.3261 , 73.507) ,
640:('A640' ,'B640' , 'India',' ', 25.7581 , 79.4530000000002) ,
641:('A641' ,'B641' , 'India',' ', 26.1139 , 78.1480000000002) ,
642:('A642' ,'B642' , 'India',' ', 25.2675 , 74.11) ,
643:('A643' ,'B643' , 'India',' ', 23.4101 , 79.5040000000002) ,
644:('A644' ,'B644' , 'India',' ', 19.6047 , 77.5150000000002) ,
645:('A645' ,'B645' , 'India',' ', 20.582 , 78.3130000000002) ,
646:('A646' ,'B646' , 'India',' ', 23.22 , 73.111) ,
647:('A647' ,'B647' , 'India',' ', 24.661 , 78.0520000000002) ,
648:('A648' ,'B648' , 'India',' ', 19.4393 , 75.9550000000001) ,
649:('A649' ,'B649' , 'India',' ', 25.8683 , 78.0280000000002) ,
650:('A650' ,'B650' , 'India',' ', 26.9107 , 77.7190000000002) ,
651:('A651' ,'B651' , 'India',' ', 23.0269 , 73.396) ,
652:('A652' ,'B652' , 'India',' ', 27.9392 , 73.705) ,
653:('A653' ,'B653' , 'India',' ', 26.3909 , 74.5150000000001) ,
654:('A654' ,'B654' , 'India',' ', 26.4603 , 74.4340000000001) ,
655:('A655' ,'B655' , 'India',' ', 18.993 , 74.6350000000001) ,
656:('A656' ,'B656' , 'India',' ', 19.7712 , 77.4160000000002) ,
657:('A657' ,'B657' , 'India',' ', 24.0663 , 74.3560000000001) ,
658:('A658' ,'B658' , 'India',' ', 25.0458 , 79.4740000000002) ,
659:('A659' ,'B659' , 'India',' ', 25.0038 , 73.033) ,
660:('A660' ,'B660' , 'India',' ', 19.8463 , 77.5330000000002) ,
661:('A661' ,'B661' , 'India',' ', 23.1885 , 78.5320000000002) ,
662:('A662' ,'B662' , 'India',' ', 23.231 , 78.0460000000002) ,
663:('A663' ,'B663' , 'India',' ', 22.6492 , 76.4650000000001) ,
664:('A664' ,'B664' , 'India',' ', 24.7549 , 78.0850000000002) ,
665:('A665' ,'B665' , 'India',' ', 24.9167 , 78.4630000000002) ,
666:('A666' ,'B666' , 'India',' ', 26.2557 , 76.0990000000001) ,
667:('A667' ,'B667' , 'India',' ', 25.3343 , 79.3420000000002) ,
668:('A668' ,'B668' , 'India',' ', 20.525 , 76.8580000000001) ,
669:('A669' ,'B669' , 'India',' ', 25.5631 , 73.354) ,
670:('A670' ,'B670' , 'India',' ', 18.9913 , 77.3920000000002) ,
671:('A671' ,'B671' , 'India',' ', 24.8363 , 74.9140000000001) ,
672:('A672' ,'B672' , 'India',' ', 19.1753 , 77.4610000000002) ,
673:('A673' ,'B673' , 'India',' ', 18.7638 , 74.9710000000001) ,
674:('A674' ,'B674' , 'India',' ', 20.289 , 77.9470000000002) ,
675:('A675' ,'B675' , 'India',' ', 21.5283 , 77.7580000000002) ,
676:('A676' ,'B676' , 'India',' ', 24.1189 , 79.4950000000002) ,
677:('A677' ,'B677' , 'India',' ', 27.3716 , 76.6360000000001) ,
678:('A678' ,'B678' , 'India',' ', 22.5662 , 76.9570000000001) ,
679:('A679' ,'B679' , 'India',' ', 23.1105 , 78.0040000000002) ,
680:('A680' ,'B680' , 'India',' ', 25.1779 , 73.936) ,
681:('A681' ,'B681' , 'India',' ', 23.5423 , 78.1720000000002) ,
682:('A682' ,'B682' , 'India',' ', 24.3582 , 79.6750000000003) ,
683:('A683' ,'B683' , 'India',' ', 23.1146 , 74.5240000000001) ,
684:('A684' ,'B684' , 'India',' ', 21.5497 , 78.9340000000002) ,
685:('A685' ,'B685' , 'India',' ', 26.1425 , 76.3720000000001) ,
686:('A686' ,'B686' , 'India',' ', 19.1725 , 77.8270000000002) ,
687:('A687' ,'B687' , 'India',' ', 19.8492 , 73.357) ,
688:('A688' ,'B688' , 'India',' ', 20.1614 , 79.2550000000002) ,
689:('A689' ,'B689' , 'India',' ', 20.669 , 75.1990000000001) ,
690:('A690' ,'B690' , 'India',' ', 25.5127 , 75.7540000000001) ,
691:('A691' ,'B691' , 'India',' ', 23.3236 , 78.7450000000002) ,
692:('A692' ,'B692' , 'India',' ', 22.0765 , 74.5930000000001) ,
693:('A693' ,'B693' , 'India',' ', 20.3149 , 75.3130000000001) ,
694:('A694' ,'B694' , 'India',' ', 24.6096 , 74.6230000000001) ,
695:('A695' ,'B695' , 'India',' ', 20.6075 , 76.8820000000001) ,
696:('A696' ,'B696' , 'India',' ', 24.8495 , 73.51) ,
697:('A697' ,'B697' , 'India',' ', 19.7947 , 76.6450000000001) ,
698:('A698' ,'B698' , 'India',' ', 25.9868 , 76.1890000000001) ,
699:('A699' ,'B699' , 'India',' ', 26.2489 , 77.1220000000002) ,
700:('A700' ,'B700' , 'India',' ', 20.5887 , 78.6790000000002) ,
701:('A701' ,'B701' , 'India',' ', 24.5694 , 76.2100000000001) ,
702:('A702' ,'B702' , 'India',' ', 22.3884 , 79.7680000000003) ,
703:('A703' ,'B703' , 'India',' ', 23.8573 , 78.7390000000002) ,
704:('A704' ,'B704' , 'India',' ', 20.3269 , 76.0840000000001) ,
705:('A705' ,'B705' , 'India',' ', 26.4392 , 78.5440000000002) ,
706:('A706' ,'B706' , 'India',' ', 25.2559 , 73.924) ,
707:('A707' ,'B707' , 'India',' ', 24.2081 , 74.5990000000001) ,
708:('A708' ,'B708' , 'India',' ', 26.5426 , 76.6750000000001) ,
709:('A709' ,'B709' , 'India',' ', 26.118 , 76.4710000000001) ,
710:('A710' ,'B710' , 'India',' ', 22.4161 , 78.3100000000002) ,
711:('A711' ,'B711' , 'India',' ', 20.5097 , 74.155) ,
712:('A712' ,'B712' , 'India',' ', 25.2298 , 76.6720000000001) ,
713:('A713' ,'B713' , 'India',' ', 19.4111 , 77.9800000000002) ,
714:('A714' ,'B714' , 'India',' ', 19.8029 , 75.9790000000001) ,
715:('A715' ,'B715' , 'India',' ', 20.1718 , 75.6580000000001) ,
716:('A716' ,'B716' , 'India',' ', 25.1326 , 73.885) ,
717:('A717' ,'B717' , 'India',' ', 22.8783 , 73.486) ,
718:('A718' ,'B718' , 'India',' ', 19.0512 , 76.8250000000001) ,
719:('A719' ,'B719' , 'India',' ', 27.4671 , 79.1320000000002) ,
720:('A720' ,'B720' , 'India',' ', 26.3123 , 73.48) ,
721:('A721' ,'B721' , 'India',' ', 19.417 , 75.4930000000001) ,
722:('A722' ,'B722' , 'India',' ', 21.2303 , 75.3730000000001) ,
723:('A723' ,'B723' , 'India',' ', 25.4083 , 75.1120000000001) ,
724:('A724' ,'B724' , 'India',' ', 25.1769 , 78.7090000000002) ,
725:('A725' ,'B725' , 'India',' ', 25.9481 , 79.7830000000003) ,
726:('A726' ,'B726' , 'India',' ', 24.972 , 74.7970000000001) ,
727:('A727' ,'B727' , 'India',' ', 24.1039 , 77.7820000000002) ,
728:('A728' ,'B728' , 'India',' ', 22.9472 , 79.3390000000002) ,
729:('A729' ,'B729' , 'India',' ', 26.0517 , 77.2390000000002) ,
730:('A730' ,'B730' , 'India',' ', 23.5622641 , 73.978) ,
731:('A731' ,'B731' , 'India',' ', 24.9288 , 75.7270000000001) ,
732:('A732' ,'B732' , 'India',' ', 26.8111 , 78.1810000000002) ,
733:('A733' ,'B733' , 'India',' ', 24.5034 , 76.7440000000001) ,
734:('A734' ,'B734' , 'India',' ', 23.6019 , 73.651) ,
735:('A735' ,'B735' , 'India',' ', 24.4352 , 76.7620000000001) ,
736:('A736' ,'B736' , 'India',' ', 24.1793 , 77.4190000000002) ,
737:('A737' ,'B737' , 'India',' ', 24.8969 , 77.7730000000002) ,
738:('A738' ,'B738' , 'India',' ', 20.4738 , 77.2090000000002) ,
739:('A739' ,'B739' , 'India',' ', 26.676 , 78.2290000000002) ,
740:('A740' ,'B740' , 'India',' ', 25.2553 , 76.7260000000001) ,
741:('A741' ,'B741' , 'India',' ', 20.5036 , 78.8920000000002) ,
742:('A742' ,'B742' , 'India',' ', 21.0647 , 77.8300000000002) ,
743:('A743' ,'B743' , 'India',' ', 19.9941 , 78.9970000000002) ,
744:('A744' ,'B744' , 'India',' ', 26.3176 , 79.7770000000003) ,
745:('A745' ,'B745' , 'India',' ', 23.5956531 , 76.8130000000001) ,
746:('A746' ,'B746' , 'India',' ', 19.9399 , 77.0560000000002) ,
747:('A747' ,'B747' , 'India',' ', 26.5143 , 73.495) ,
748:('A748' ,'B748' , 'India',' ', 24.1846 , 77.4340000000002) ,
749:('A749' ,'B749' , 'India',' ', 26.2695 , 75.6640000000001) ,
750:('A750' ,'B750' , 'India',' ', 18.9808 , 78.2920000000002) ,
751:('A751' ,'B751' , 'India',' ', 23.3344 , 75.1960000000001) ,
752:('A752' ,'B752' , 'India',' ', 18.851 , 78.2500000000002) ,
753:('A753' ,'B753' , 'India',' ', 26.125 , 74.293) ,
754:('A754' ,'B754' , 'India',' ', 19.243 , 76.5700000000001) ,
755:('A755' ,'B755' , 'India',' ', 20.836 , 75.7450000000001) ,
756:('A756' ,'B756' , 'India',' ', 26.4055 , 79.3870000000002) ,
757:('A757' ,'B757' , 'India',' ', 25.9686 , 77.7250000000002) ,
758:('A758' ,'B758' , 'India',' ', 27.0109 , 73.204) ,
759:('A759' ,'B759' , 'India',' ', 26.6907 , 79.5850000000002) ,
760:('A760' ,'B760' , 'India',' ', 19.526 , 73.174) ,
761:('A761' ,'B761' , 'India',' ', 22.2086 , 78.4990000000002) ,
762:('A762' ,'B762' , 'India',' ', 20.1441 , 76.3930000000001) ,
763:('A763' ,'B763' , 'India',' ', 26.551 , 77.6590000000002) ,
764:('A764' ,'B764' , 'India',' ', 20.0148 , 73.282) ,
765:('A765' ,'B765' , 'India',' ', 23.6082 , 76.5040000000001) ,
766:('A766' ,'B766' , 'India',' ', 26.385 , 74.9170000000001) ,
767:('A767' ,'B767' , 'India',' ', 27.7407 , 75.3580000000001) ,
768:('A768' ,'B768' , 'India',' ', 19.6323 , 74.5900000000001) ,
769:('A769' ,'B769' , 'India',' ', 26.2378 , 73.183) ,
770:('A770' ,'B770' , 'India',' ', 25.5694 , 79.7260000000003) ,
771:('A771' ,'B771' , 'India',' ', 25.8581 , 78.7360000000002) ,
772:('A772' ,'B772' , 'India',' ', 23.6238 , 77.0530000000002) ,
773:('A773' ,'B773' , 'India',' ', 27.1658 , 75.2740000000001) ,
774:('A774' ,'B774' , 'India',' ', 26.2078 , 73.699) ,
775:('A775' ,'B775' , 'India',' ', 20.4484 , 76.2520000000001) ,
776:('A776' ,'B776' , 'India',' ', 21.7068 , 74.008) ,
777:('A777' ,'B777' , 'India',' ', 18.6045 , 77.4790000000002) ,
778:('A778' ,'B778' , 'India',' ', 25.9472 , 77.6290000000002) ,
779:('A779' ,'B779' , 'India',' ', 24.67 , 74.7010000000001) ,
780:('A780' ,'B780' , 'India',' ', 26.7888 , 74.4730000000001) ,
781:('A781' ,'B781' , 'India',' ', 19.7874 , 79.8340000000003) ,
782:('A782' ,'B782' , 'India',' ', 20.1616 , 77.5510000000002) ,
783:('A783' ,'B783' , 'India',' ', 24.485 , 77.8210000000002) ,
784:('A784' ,'B784' , 'India',' ', 19.7502 , 78.4510000000002) ,
785:('A785' ,'B785' , 'India',' ', 24.2958 , 78.3610000000002) ,
786:('A786' ,'B786' , 'India',' ', 23.6292 , 73.231) ,
787:('A787' ,'B787' , 'India',' ', 19.1383 , 73.717) ,
788:('A788' ,'B788' , 'India',' ', 19.077 , 77.3260000000002) ,
789:('A789' ,'B789' , 'India',' ', 27.4747 , 77.5630000000002) ,
790:('A790' ,'B790' , 'India',' ', 25.0709 , 73.213) ,
791:('A791' ,'B791' , 'India',' ', 18.959 , 74.077) ,
792:('A792' ,'B792' , 'India',' ', 26.2476 , 78.6010000000002) ,
793:('A793' ,'B793' , 'India',' ', 25.8283 , 77.7490000000002) ,
794:('A794' ,'B794' , 'India',' ', 19.3196 , 76.0150000000001) ,
795:('A795' ,'B795' , 'India',' ', 19.4584 , 73.696) ,
796:('A796' ,'B796' , 'India',' ', 20.1156 , 74.9650000000001) ,
797:('A797' ,'B797' , 'India',' ', 21.1005 , 77.7040000000002) ,
798:('A798' ,'B798' , 'India',' ', 19.5437 , 73.453) ,
799:('A799' ,'B799' , 'India',' ', 25.3363 , 75.3820000000001) ,
800:('A800' ,'B800' , 'India',' ', 26.1821 , 79.0600000000002) ,
801:('A801' ,'B801' , 'India',' ', 23.4482 , 74.089) ,
802:('A802' ,'B802' , 'India',' ', 20.51 , 74.275) ,
803:('A803' ,'B803' , 'India',' ', 21.5668 , 77.0980000000002) ,
804:('A804' ,'B804' , 'India',' ', 26.5713 , 76.1560000000001) ,
805:('A805' ,'B805' , 'India',' ', 20.668 , 75.5170000000001) ,
806:('A806' ,'B806' , 'India',' ', 19.6739 , 79.5220000000002) ,
807:('A807' ,'B807' , 'India',' ', 24.2824 , 78.6730000000002) ,
808:('A808' ,'B808' , 'India',' ', 23.9614 , 79.8550000000003) ,
809:('A809' ,'B809' , 'India',' ', 25.1632 , 79.0540000000002) ,
810:('A810' ,'B810' , 'India',' ', 26.4626 , 77.9260000000002) ,
811:('A811' ,'B811' , 'India',' ', 26.3868 , 74.7580000000001) ,
812:('A812' ,'B812' , 'India',' ', 24.2642 , 74.5840000000001) ,
813:('A813' ,'B813' , 'India',' ', 20.0888 , 75.3310000000001) ,
814:('A814' ,'B814' , 'India',' ', 23.0511 , 78.1210000000002) ,
815:('A815' ,'B815' , 'India',' ', 20.668 , 73.78) ,
816:('A816' ,'B816' , 'India',' ', 25.608 , 76.1710000000001) ,
817:('A817' ,'B817' , 'India',' ', 20.5334 , 76.5520000000001) ,
818:('A818' ,'B818' , 'India',' ', 26.4429 , 74.3710000000001) ,
819:('A819' ,'B819' , 'India',' ', 22.6836 , 77.8960000000002) ,
820:('A820' ,'B820' , 'India',' ', 25.5647 , 78.3550000000002) ,
821:('A821' ,'B821' , 'India',' ', 23.4839 , 73.849) ,
822:('A822' ,'B822' , 'India',' ', 19.2424 , 77.0260000000002) ,
823:('A823' ,'B823' , 'India',' ', 19.1231 , 73.393) ,
824:('A824' ,'B824' , 'India',' ', 25.0888 , 73.693) ,
825:('A825' ,'B825' , 'India',' ', 25.6449 , 74.092) ,
826:('A826' ,'B826' , 'India',' ', 19.0385 , 77.9140000000002) ,
827:('A827' ,'B827' , 'India',' ', 20.0255 , 76.5460000000001) ,
828:('A828' ,'B828' , 'India',' ', 22.7588 , 76.8610000000001) ,
829:('A829' ,'B829' , 'India',' ', 18.6446 , 79.7020000000003) ,
830:('A830' ,'B830' , 'India',' ', 23.547 , 73.744) ,
831:('A831' ,'B831' , 'India',' ', 26.9253 , 78.9280000000002) ,
832:('A832' ,'B832' , 'India',' ', 24.7947 , 78.0400000000002) ,
833:('A833' ,'B833' , 'India',' ', 26.5408 , 76.7650000000001) ,
834:('A834' ,'B834' , 'India',' ', 20.1243 , 74.9080000000001) ,
835:('A835' ,'B835' , 'India',' ', 27.1391 , 77.3230000000002) ,
836:('A836' ,'B836' , 'India',' ', 27.9763 , 78.6160000000002) ,
837:('A837' ,'B837' , 'India',' ', 26.8143 , 78.0880000000002) ,
838:('A838' ,'B838' , 'India',' ', 19.8244 , 79.3150000000002) ,
839:('A839' ,'B839' , 'India',' ', 19.606 , 75.7990000000001) ,
840:('A840' ,'B840' , 'India',' ', 19.3082 , 73.177) ,
841:('A841' ,'B841' , 'India',' ', 27.3946 , 73.873) ,
842:('A842' ,'B842' , 'India',' ', 19.7681 , 77.8450000000002) ,
843:('A843' ,'B843' , 'India',' ', 27.3012 , 74.056) ,
844:('A844' ,'B844' , 'India',' ', 26.9105 , 78.5890000000002) ,
845:('A845' ,'B845' , 'India',' ', 19.8379 , 77.6350000000002) ,
846:('A846' ,'B846' , 'India',' ', 19.6225 , 78.0340000000002) ,
847:('A847' ,'B847' , 'India',' ', 25.9576 , 73.342) ,
848:('A848' ,'B848' , 'India',' ', 27.7043 , 78.9610000000002) ,
849:('A849' ,'B849' , 'India',' ', 25.2718 , 76.6090000000001) ,
850:('A850' ,'B850' , 'India',' ', 20.2347 , 75.1300000000001) ,
851:('A851' ,'B851' , 'India',' ', 20.6483 , 77.5000000000002) ,
852:('A852' ,'B852' , 'India',' ', 26.6137 , 79.5550000000002) ,
853:('A853' ,'B853' , 'India',' ', 22.8118 , 74.3650000000001) ,
854:('A854' ,'B854' , 'India',' ', 25.5536 , 76.4950000000001) ,
855:('A855' ,'B855' , 'India',' ', 23.7636 , 78.0010000000002) ,
856:('A856' ,'B856' , 'India',' ', 23.6446 , 78.7330000000002) ,
857:('A857' ,'B857' , 'India',' ', 26.5072 , 74.6440000000001) ,
858:('A858' ,'B858' , 'India',' ', 22.8335 , 79.6540000000003) ,
859:('A859' ,'B859' , 'India',' ', 23.4525 , 77.9680000000002) ,
860:('A860' ,'B860' , 'India',' ', 25.4241 , 74.296) ,
861:('A861' ,'B861' , 'India',' ', 26.6317 , 74.248) ,
862:('A862' ,'B862' , 'India',' ', 26.1617 , 74.4370000000001) ,
863:('A863' ,'B863' , 'India',' ', 24.9306 , 77.2180000000002) ,
864:('A864' ,'B864' , 'India',' ', 23.7974 , 76.0360000000001) ,
865:('A865' ,'B865' , 'India',' ', 23.5833 , 73.963) ,
866:('A866' ,'B866' , 'India',' ', 20.2794 , 73.189) ,
867:('A867' ,'B867' , 'India',' ', 18.9005 , 73.225) ,
868:('A868' ,'B868' , 'India',' ', 25.3131 , 79.1440000000002) ,
869:('A869' ,'B869' , 'India',' ', 25.3582 , 75.4480000000001) ,
870:('A870' ,'B870' , 'India',' ', 25.375 , 78.6340000000002) ,
871:('A871' ,'B871' , 'India',' ', 25.44 , 75.3970000000001) ,
872:('A872' ,'B872' , 'India',' ', 26.0774 , 77.9200000000002) ,
873:('A873' ,'B873' , 'India',' ', 19.659 , 74.3980000000001) ,
874:('A874' ,'B874' , 'India',' ', 27.7862879 , 79.6990000000003) ,
875:('A875' ,'B875' , 'India',' ', 26.1249 , 74.4520000000001) ,
876:('A876' ,'B876' , 'India',' ', 26.6937 , 79.8310000000003) ,
877:('A877' ,'B877' , 'India',' ', 22.0338 , 74.8660000000001) ,
878:('A878' ,'B878' , 'India',' ', 26.9017 , 74.8300000000001) ,
879:('A879' ,'B879' , 'India',' ', 25.7273 , 73.072) ,
880:('A880' ,'B880' , 'India',' ', 18.678 , 75.3400000000001) ,
881:('A881' ,'B881' , 'India',' ', 21.7257 , 75.3850000000001) ,
882:('A882' ,'B882' , 'India',' ', 20.1392 , 77.1520000000002) ,
883:('A883' ,'B883' , 'India',' ', 25.4737 , 76.6810000000001) ,
884:('A884' ,'B884' , 'India',' ', 18.9422 , 73.462) ,
885:('A885' ,'B885' , 'India',' ', 19.3583 , 73.96) ,
886:('A886' ,'B886' , 'India',' ', 24.3529 , 75.7750000000001) ,
887:('A887' ,'B887' , 'India',' ', 20.1122 , 77.5390000000002) ,
888:('A888' ,'B888' , 'India',' ', 27.1576 , 75.5200000000001) ,
889:('A889' ,'B889' , 'India',' ', 24.2376 , 77.8030000000002) ,
890:('A890' ,'B890' , 'India',' ', 25.3144 , 79.5100000000002) ,
891:('A891' ,'B891' , 'India',' ', 26.3012 , 76.1470000000001) ,
892:('A892' ,'B892' , 'India',' ', 22.0059 , 79.7980000000003) ,
893:('A893' ,'B893' , 'India',' ', 18.5539 , 73.42) ,
894:('A894' ,'B894' , 'India',' ', 23.2264 , 78.5350000000002) ,
895:('A895' ,'B895' , 'India',' ', 27.3946 , 79.6420000000003) ,
896:('A896' ,'B896' , 'India',' ', 20.3067 , 73.6) ,
897:('A897' ,'B897' , 'India',' ', 21.0561 , 75.6040000000001) ,
898:('A898' ,'B898' , 'India',' ', 19.1404 , 78.1990000000002) ,
899:('A899' ,'B899' , 'India',' ', 25.7119 , 74.7760000000001) ,
900:('A900' ,'B900' , 'India',' ', 26.8239 , 79.0840000000002) ,
901:('A901' ,'B901' , 'India',' ', 20.0458 , 74.7310000000001) ,
902:('A902' ,'B902' , 'India',' ', 26.746 , 75.0940000000001) ,
903:('A903' ,'B903' , 'India',' ', 26.3076 , 75.2290000000001) ,
904:('A904' ,'B904' , 'India',' ', 26.2766 , 79.8250000000003) ,
905:('A905' ,'B905' , 'India',' ', 26.2361 , 73.789) ,
906:('A906' ,'B906' , 'India',' ', 24.0209 , 78.8020000000002) ,
907:('A907' ,'B907' , 'India',' ', 26.5271012 , 76.8220000000001) ,
908:('A908' ,'B908' , 'India',' ', 26.6787 , 75.8620000000001) ,
909:('A909' ,'B909' , 'India',' ', 27.2865 , 73.384) ,
910:('A910' ,'B910' , 'India',' ', 25.3789 , 79.0000000000002) ,
911:('A911' ,'B911' , 'India',' ', 22.0332 , 77.0770000000002) ,
912:('A912' ,'B912' , 'India',' ', 24.3414 , 74.9470000000001) ,
913:('A913' ,'B913' , 'India',' ', 26.7979 , 76.5820000000001) ,
914:('A914' ,'B914' , 'India',' ', 20.742 , 77.1100000000002) ,
915:('A915' ,'B915' , 'India',' ', 27.6215058 , 73.057) ,
916:('A916' ,'B916' , 'India',' ', 27.0477 , 74.9230000000001) ,
917:('A917' ,'B917' , 'India',' ', 25.5375 , 73.39) ,
918:('A918' ,'B918' , 'India',' ', 25.004 , 76.7320000000001) ,
919:('A919' ,'B919' , 'India',' ', 26.3104 , 74.9740000000001) ,
920:('A920' ,'B920' , 'India',' ', 25.8967 , 78.1120000000002) ,
921:('A921' ,'B921' , 'India',' ', 25.4931 , 74.299) ,
922:('A922' ,'B922' , 'India',' ', 21.41 , 79.1980000000002) ,
923:('A923' ,'B923' , 'India',' ', 23.5107 , 73.246) ,
924:('A924' ,'B924' , 'India',' ', 24.4113 , 74.9800000000001) ,
925:('A925' ,'B925' , 'India',' ', 23.0687 , 77.9080000000002) ,
926:('A926' ,'B926' , 'India',' ', 24.4323 , 79.2700000000002) ,
927:('A927' ,'B927' , 'India',' ', 23.277 , 73.222) ,
928:('A928' ,'B928' , 'India',' ', 24.8847 , 73.609) ,
929:('A929' ,'B929' , 'India',' ', 27.2945 , 76.7530000000001) ,
930:('A930' ,'B930' , 'India',' ', 19.9483 , 79.2760000000002) ,
931:('A931' ,'B931' , 'India',' ', 24.2676805 , 78.8230000000002) ,
932:('A932' ,'B932' , 'India',' ', 23.1969 , 74.221) ,
933:('A933' ,'B933' , 'India',' ', 19.3358 , 73.336) ,
934:('A934' ,'B934' , 'India',' ', 24.9673 , 78.5290000000002) ,
935:('A935' ,'B935' , 'India',' ', 23.565 , 73.03) ,
936:('A936' ,'B936' , 'India',' ', 28.4266 , 74.128) ,
937:('A937' ,'B937' , 'India',' ', 20.0602 , 76.2160000000001) ,
938:('A938' ,'B938' , 'India',' ', 20.2421 , 75.2920000000001) ,
939:('A939' ,'B939' , 'India',' ', 20.6531 , 78.3640000000002) ,
940:('A940' ,'B940' , 'India',' ', 26.487 , 79.3030000000002) ,
941:('A941' ,'B941' , 'India',' ', 24.7453 , 79.4890000000002) ,
942:('A942' ,'B942' , 'India',' ', 23.617 , 78.6250000000002) ,
943:('A943' ,'B943' , 'India',' ', 19.4224 , 77.4010000000002) ,
944:('A944' ,'B944' , 'India',' ', 21.2998 , 79.1860000000002) ,
945:('A945' ,'B945' , 'India',' ', 20.6183 , 79.3300000000002) ,
946:('A946' ,'B946' , 'India',' ', 21.912 , 79.3360000000002) ,
947:('A947' ,'B947' , 'India',' ', 26.8868 , 79.7620000000003) ,
948:('A948' ,'B948' , 'India',' ', 18.8988 , 73.243) ,
949:('A949' ,'B949' , 'India',' ', 19.6045 , 73.348) ,
950:('A950' ,'B950' , 'India',' ', 26.3076 , 75.4420000000001) ,
951:('A951' ,'B951' , 'India',' ', 26.5713 , 74.245) ,
952:('A952' ,'B952' , 'India',' ', 28.5883 , 76.7140000000001) ,
953:('A953' ,'B953' , 'India',' ', 19.9231332 , 74.4220000000001) ,
954:('A954' ,'B954' , 'India',' ', 25.366 , 74.3380000000001) ,
955:('A955' ,'B955' , 'India',' ', 21.2788 , 73.765) ,
956:('A956' ,'B956' , 'India',' ', 23.461 , 73.969) ,
957:('A957' ,'B957' , 'India',' ', 20.653 , 74.4280000000001) ,
958:('A958' ,'B958' , 'India',' ', 19.8342 , 75.0580000000001) ,
959:('A959' ,'B959' , 'India',' ', 20.0053 , 75.4630000000001) ,
960:('A960' ,'B960' , 'India',' ', 24.2407 , 73.168) ,
961:('A961' ,'B961' , 'India',' ', 18.9632 , 74.209) ,
962:('A962' ,'B962' , 'India',' ', 26.9017 , 74.6410000000001) ,
963:('A963' ,'B963' , 'India',' ', 21.2939 , 76.7170000000001) ,
964:('A964' ,'B964' , 'India',' ', 25.3334 , 78.9580000000002) ,
965:('A965' ,'B965' , 'India',' ', 18.0544 , 75.7840000000001) ,
966:('A966' ,'B966' , 'India',' ', 19.0488 , 79.5460000000002) ,
967:('A967' ,'B967' , 'India',' ', 25.017 , 75.5830000000001) ,
968:('A968' ,'B968' , 'India',' ', 20.8742 , 78.4060000000002) ,
969:('A969' ,'B969' , 'India',' ', 20.452 , 78.2470000000002) ,
970:('A970' ,'B970' , 'India',' ', 22.9628 , 73.12) ,
971:('A971' ,'B971' , 'India',' ', 25.3784 , 73.114) ,
972:('A972' ,'B972' , 'India',' ', 19.6841 , 76.9690000000002) ,
973:('A973' ,'B973' , 'India',' ', 18.164 , 75.5290000000001) ,
974:('A974' ,'B974' , 'India',' ', 19.4111 , 77.4640000000002) ,
975:('A975' ,'B975' , 'India',' ', 20.0022439 , 74.6650000000001) ,
976:('A976' ,'B976' , 'India',' ', 26.1160819 , 76.0780000000001) ,
977:('A977' ,'B977' , 'India',' ', 23.201255 , 79.6720000000003) ,
978:('A978' ,'B978' , 'India',' ', 26.4892 , 74.137) ,
979:('A979' ,'B979' , 'India',' ', 20.0960587 , 79.4560000000002) ,
980:('A980' ,'B980' , 'India',' ', 23.201255 , 76.0930000000001) ,
981:('A981' ,'B981' , 'India',' ', 19.9231332 , 73.009) ,
982:('A982' ,'B982' , 'India',' ', 22.6670691 , 77.5450000000002) ,
983:('A983' ,'B983' , 'India',' ', 26.6014 , 76.6150000000001) ,
984:('A984' ,'B984' , 'India',' ', 23.4469641 , 74.4070000000001) ,
985:('A985' ,'B985' , 'India',' ', 21.2933578 , 77.9830000000002) ,
986:('A986' ,'B986' , 'India',' ', 19.9231332 , 75.1360000000001) ,
987:('A987' ,'B987' , 'India',' ', 25.6915 , 76.3240000000001) ,
988:('A988' ,'B988' , 'India',' ', 25.1533 , 73.018) ,
989:('A989' ,'B989' , 'India',' ', 25.9584678 , 77.8600000000002) ,
990:('A990' ,'B990' , 'India',' ', 23.2406 , 73.273) ,
991:('A991' ,'B991' , 'India',' ', 25.9396 , 77.6680000000002) ,
992:('A992' ,'B992' , 'India',' ', 20.7619 , 74.3590000000001) ,
993:('A993' ,'B993' , 'India',' ', 22.8414 , 74.032) ,
994:('A994' ,'B994' , 'India',' ', 26.9342 , 77.4400000000002) ,
995:('A995' ,'B995' , 'India',' ', 27.0316 , 76.3480000000001) ,
996:('A996' ,'B996' , 'India',' ', 19.1559 , 76.8850000000001) ,
997:('A997' ,'B997' , 'India',' ', 19.6134554 , 79.4260000000002) ,
998:('A998' ,'B998' , 'India',' ', 25.3201 , 78.1930000000002) ,
999:('A999' ,'B999' , 'India',' ', 18.0654 , 76.7110000000001) ,
1000:('A1000' ,'B1000' , 'India',' ', 24.6149 , 74.9620000000001) ,
1001:('A1001' ,'B1001' , 'India',' ', 22.6375 , 79.1050000000002) ,
1002:('A1002' ,'B1002' , 'India',' ', 25.9445169 , 77.3830000000002) ,
1003:('A1003' ,'B1003' , 'India',' ', 25.1681768 , 74.062) ,
1004:('A1004' ,'B1004' , 'India',' ', 21.428933 , 74.6680000000001) ,
1005:('A1005' ,'B1005' , 'India',' ', 25.9744 , 74.9890000000001) ,
1006:('A1006' ,'B1006' , 'India',' ', 21.9009 , 78.6100000000002) ,
1007:('A1007' ,'B1007' , 'India',' ', 23.0775 , 75.8800000000001) ,
1008:('A1008' ,'B1008' , 'India',' ', 20.2759 , 73.774) ,
1009:('A1009' ,'B1009' , 'India',' ', 26.3868 , 74.17) ,
1010:('A1010' ,'B1010' , 'India',' ', 19.0531 , 74.5390000000001) ,
1011:('A1011' ,'B1011' , 'India',' ', 21.2373 , 76.3330000000001) ,
1012:('A1012' ,'B1012' , 'India',' ', 26.9086309 , 75.4300000000001) ,
1013:('A1013' ,'B1013' , 'India',' ', 21.7599 , 76.6000000000001) ,
1014:('A1014' ,'B1014' , 'India',' ', 19.7246059 , 78.3760000000002) ,
1015:('A1015' ,'B1015' , 'India',' ', 19.2246059 , 75.2230000000001) ,
1016:('A1016' ,'B1016' , 'India',' ', 19.46059 , 78.3370000000002) ,


    }
    
   # Demands of Customer/distributor  
    

     #  demand::
    # id: demand_value
    customer_demands ={

1: 154,
2: 252,
3: 1,
4: 881,
5: 86,
6: 95,
7: 305,
8: 288,
9: 48,
10: 295,
11: 416,
12: 85,
13: 237,
14: 534,
15: 59,
16: 575,
17: 8,
18: 206,
19: 76,
20: 1,
21: 609,
22: 488,
23: 180,
24: 888,
25: 218,
26: 459,
27: 200,
28: 294,
29: 193,
30: 339,
31: 88,
32: 88,
33: 287,
34: 509,
35: 321,
36: 547,
37: 202,
38: 538,
39: 439,
40: 190,
41: 199,
42: 139,
43: 455,
44: 379,
45: 125,
46: 746,
47: 444,
48: 346,
49: 389,
50: 404,
51: 170,
52: 241,
53: 193,
54: 546,
55: 691,
56: 325,
57: 66,
58: 467,
59: 606,
60: 703,
61: 109,
62: 319,
63: 417,
64: 114,
65: 129,
66: 415,
67: 255,
68: 392,
69: 194,
70: 483,
71: 112,
72: 540,
73: 261,
74: 431,
75: 509,
76: 532,
77: 361,
78: 248,
79: 373,
80: 1587,
81: 175,
82: 440,
83: 311,
84: 143,
85: 509,
86: 154,
87: 597,
88: 274,
89: 241,
90: 604,
91: 331,
92: 202,
93: 212,
94: 199,
95: 110,
96: 364,
97: 216,
98: 484,
99: 1,
100: 321,
101: 667,
102: 421,
103: 327,
104: 82,
105: 186,
106: 161,
107: 258,
108: 776,
109: 460,
110: 153,
111: 288,
112: 396,
113: 207,
114: 127,
115: 214,
116: 500,
117: 292,
118: 115,
119: 152,
120: 463,
121: 258,
122: 265,
123: 185,
124: 258,
125: 40,
126: 232,
127: 181,
128: 320,
129: 381,
130: 133,
131: 799,
132: 17,
133: 1135,
134: 220,
135: 217,
136: 324,
137: 421,
138: 521,
139: 228,
140: 269,
141: 361,
142: 329,
143: 357,
144: 307,
145: 134,
146: 30,
147: 103,
148: 408,
149: 331,
150: 636,
151: 468,
152: 240,
153: 444,
154: 340,
155: 318,
156: 244,
157: 16,
158: 160,
159: 576,
160: 804,
161: 389,
162: 612,
163: 527,
164: 183,
165: 415,
166: 478,
167: 477,
168: 771,
169: 207,
170: 327,
171: 203,
172: 484,
173: 192,
174: 272,
175: 206,
176: 255,
177: 557,
178: 361,
179: 261,
180: 733,
181: 239,
182: 709,
183: 567,
184: 523,
185: 480,
186: 394,
187: 201,
188: 222,
189: 71,
190: 308,
191: 39,
192: 16,
193: 701,
194: 177,
195: 499,
196: 667,
197: 363,
198: 322,
199: 148,
200: 413,
201: 291,
202: 544,
203: 440,
204: 500,
205: 598,
206: 53,
207: 294,
208: 246,
209: 318,
210: 353,
211: 391,
212: 223,
213: 367,
214: 154,
215: 243,
216: 337,
217: 480,
218: 104,
219: 417,
220: 357,
221: 289,
222: 99,
223: 255,
224: 187,
225: 98,
226: 458,
227: 382,
228: 402,
229: 547,
230: 668,
231: 34,
232: 239,
233: 177,
234: 247,
235: 348,
236: 452,
237: 244,
238: 231,
239: 263,
240: 420,
241: 261,
242: 99,
243: 287,
244: 165,
245: 208,
246: 93,
247: 260,
248: 362,
249: 1,
250: 855,
251: 484,
252: 158,
253: 209,
254: 186,
255: 181,
256: 129,
257: 209,
258: 341,
259: 794,
260: 183,
261: 454,
262: 492,
263: 511,
264: 228,
265: 124,
266: 408,
267: 386,
268: 165,
269: 867,
270: 184,
271: 531,
272: 147,
273: 337,
274: 162,
275: 215,
276: 640,
277: 255,
278: 262,
279: 1168,
280: 318,
281: 9,
282: 341,
283: 645,
284: 429,
285: 40,
286: 142,
287: 256,
288: 315,
289: 220,
290: 377,
291: 1,
292: 1319,
293: 375,
294: 490,
295: 342,
296: 464,
297: 463,
298: 750,
299: 88,
300: 1,
301: 196,
302: 549,
303: 639,
304: 167,
305: 136,
306: 246,
307: 136,
308: 349,
309: 71,
310: 501,
311: 286,
312: 201,
313: 168,
314: 512,
315: 431,
316: 513,
317: 163,
318: 386,
319: 118,
320: 244,
321: 419,
322: 414,
323: 1000,
324: 356,
325: 515,
326: 312,
327: 372,
328: 206,
329: 306,
330: 355,
331: 460,
332: 1,
333: 232,
334: 287,
335: 244,
336: 578,
337: 431,
338: 209,
339: 365,
340: 15,
341: 160,
342: 546,
343: 87,
344: 735,
345: 1,
346: 1,
347: 405,
348: 1,
349: 777,
350: 273,
351: 451,
352: 93,
353: 104,
354: 294,
355: 1,
356: 702,
357: 677,
358: 354,
359: 315,
360: 216,
361: 871,
362: 156,
363: 248,
364: 255,
365: 477,
366: 308,
367: 339,
368: 52,
369: 204,
370: 160,
371: 734,
372: 243,
373: 284,
374: 603,
375: 96,
376: 491,
377: 118,
378: 525,
379: 301,
380: 309,
381: 264,
382: 685,
383: 99,
384: 517,
385: 329,
386: 1051,
387: 289,
388: 424,
389: 392,
390: 189,
391: 367,
392: 1,
393: 152,
394: 513,
395: 1,
396: 185,
397: 151,
398: 3087,
399: 388,
400: 341,
401: 387,
402: 80,
403: 605,
404: 193,
405: 1167,
406: 202,
407: 119,
408: 177,
409: 91,
410: 65,
411: 100,
412: 250,
413: 106,
414: 39,
415: 137,
416: 38,
417: 63,
418: 242,
419: 50,
420: 412,
421: 235,
422: 1,
423: 126,
424: 509,
425: 194,
426: 194,
427: 225,
428: 278,
429: 165,
430: 345,
431: 755,
432: 40,
433: 1,
434: 281,
435: 387,
436: 7,
437: 164,
438: 76,
439: 16,
440: 774,
441: 139,
442: 149,
443: 159,
444: 497,
445: 37,
446: 271,
447: 408,
448: 211,
449: 265,
450: 367,
451: 102,
452: 1718,
453: 450,
454: 409,
455: 373,
456: 386,
457: 240,
458: 539,
459: 297,
460: 233,
461: 228,
462: 47,
463: 178,
464: 184,
465: 344,
466: 147,
467: 329,
468: 267,
469: 243,
470: 349,
471: 304,
472: 232,
473: 175,
474: 846,
475: 125,
476: 1,
477: 272,
478: 531,
479: 172,
480: 1011,
481: 497,
482: 352,
483: 486,
484: 352,
485: 432,
486: 306,
487: 163,
488: 571,
489: 260,
490: 440,
491: 290,
492: 481,
493: 15,
494: 363,
495: 261,
496: 159,
497: 176,
498: 449,
499: 613,
500: 804,
501: 59,
502: 582,
503: 289,
504: 325,
505: 308,
506: 748,
507: 29,
508: 338,
509: 223,
510: 203,
511: 652,
512: 227,
513: 202,
514: 292,
515: 555,
516: 320,
517: 88,
518: 238,
519: 1195,
520: 356,
521: 903,
522: 116,
523: 219,
524: 1,
525: 461,
526: 391,
527: 114,
528: 557,
529: 1,
530: 79,
531: 261,
532: 398,
533: 823,
534: 275,
535: 212,
536: 548,
537: 194,
538: 651,
539: 251,
540: 415,
541: 34,
542: 232,
543: 84,
544: 288,
545: 380,
546: 470,
547: 67,
548: 310,
549: 326,
550: 239,
551: 636,
552: 349,
553: 58,
554: 89,
555: 1255,
556: 137,
557: 206,
558: 88,
559: 250,
560: 667,
561: 445,
562: 35,
563: 599,
564: 298,
565: 105,
566: 376,
567: 380,
568: 218,
569: 1,
570: 244,
571: 317,
572: 1422,
573: 201,
574: 685,
575: 420,
576: 433,
577: 194,
578: 825,
579: 363,
580: 525,
581: 132,
582: 804,
583: 113,
584: 216,
585: 243,
586: 502,
587: 521,
588: 201,
589: 338,
590: 294,
591: 518,
592: 386,
593: 826,
594: 38,
595: 1048,
596: 378,
597: 63,
598: 315,
599: 458,
600: 206,
601: 553,
602: 171,
603: 915,
604: 533,
605: 608,
606: 419,
607: 156,
608: 186,
609: 1,
610: 159,
611: 407,
612: 422,
613: 341,
614: 159,
615: 680,
616: 200,
617: 215,
618: 131,
619: 336,
620: 270,
621: 759,
622: 431,
623: 159,
624: 685,
625: 666,
626: 1,
627: 386,
628: 307,
629: 141,
630: 257,
631: 1,
632: 544,
633: 142,
634: 30,
635: 153,
636: 18,
637: 265,
638: 1,
639: 400,
640: 103,
641: 24,
642: 513,
643: 508,
644: 277,
645: 1,
646: 124,
647: 308,
648: 88,
649: 441,
650: 266,
651: 46,
652: 85,
653: 114,
654: 116,
655: 305,
656: 154,
657: 449,
658: 354,
659: 485,
660: 288,
661: 4,
662: 436,
663: 197,
664: 1,
665: 392,
666: 126,
667: 205,
668: 257,
669: 280,
670: 123,
671: 338,
672: 279,
673: 185,
674: 27,
675: 278,
676: 701,
677: 667,
678: 388,
679: 197,
680: 277,
681: 232,
682: 287,
683: 98,
684: 277,
685: 403,
686: 146,
687: 664,
688: 361,
689: 409,
690: 155,
691: 1,
692: 428,
693: 279,
694: 361,
695: 290,
696: 579,
697: 508,
698: 90,
699: 509,
700: 183,
701: 116,
702: 373,
703: 576,
704: 331,
705: 450,
706: 186,
707: 25,
708: 758,
709: 331,
710: 381,
711: 294,
712: 773,
713: 114,
714: 166,
715: 482,
716: 291,
717: 227,
718: 301,
719: 14,
720: 262,
721: 627,
722: 137,
723: 480,
724: 265,
725: 520,
726: 531,
727: 209,
728: 63,
729: 471,
730: 136,
731: 62,
732: 26,
733: 102,
734: 268,
735: 299,
736: 268,
737: 285,
738: 40,
739: 915,
740: 246,
741: 636,
742: 226,
743: 231,
744: 111,
745: 231,
746: 342,
747: 1,
748: 217,
749: 345,
750: 239,
751: 438,
752: 43,
753: 1,
754: 267,
755: 185,
756: 1,
757: 532,
758: 91,
759: 244,
760: 340,
761: 369,
762: 474,
763: 316,
764: 321,
765: 589,
766: 283,
767: 300,
768: 849,
769: 148,
770: 291,
771: 105,
772: 317,
773: 1258,
774: 168,
775: 311,
776: 196,
777: 205,
778: 439,
779: 445,
780: 410,
781: 75,
782: 247,
783: 329,
784: 474,
785: 226,
786: 229,
787: 8,
788: 213,
789: 311,
790: 1,
791: 1247,
792: 297,
793: 336,
794: 60,
795: 265,
796: 692,
797: 26,
798: 77,
799: 249,
800: 43,
801: 153,
802: 229,
803: 435,
804: 390,
805: 413,
806: 373,
807: 1226,
808: 700,
809: 213,
810: 465,
811: 362,
812: 603,
813: 141,
814: 630,
815: 150,
816: 523,
817: 164,
818: 1,
819: 376,
820: 316,
821: 426,
822: 537,
823: 560,
824: 19,
825: 406,
826: 233,
827: 1,
828: 863,
829: 120,
830: 450,
831: 609,
832: 421,
833: 298,
834: 744,
835: 363,
836: 274,
837: 527,
838: 178,
839: 377,
840: 335,
841: 1,
842: 596,
843: 142,
844: 34,
845: 459,
846: 281,
847: 363,
848: 188,
849: 663,
850: 341,
851: 1597,
852: 384,
853: 215,
854: 28,
855: 469,
856: 344,
857: 272,
858: 264,
859: 317,
860: 396,
861: 76,
862: 1350,
863: 567,
864: 1,
865: 121,
866: 465,
867: 460,
868: 351,
869: 453,
870: 160,
871: 657,
872: 419,
873: 376,
874: 111,
875: 600,
876: 94,
877: 277,
878: 334,
879: 127,
880: 327,
881: 328,
882: 413,
883: 524,
884: 299,
885: 314,
886: 860,
887: 327,
888: 185,
889: 278,
890: 416,
891: 399,
892: 251,
893: 262,
894: 279,
895: 604,
896: 249,
897: 1,
898: 352,
899: 164,
900: 279,
901: 131,
902: 731,
903: 814,
904: 403,
905: 276,
906: 14,
907: 1,
908: 94,
909: 221,
910: 1,
911: 298,
912: 367,
913: 180,
914: 355,
915: 25,
916: 212,
917: 389,
918: 259,
919: 416,
920: 225,
921: 98,
922: 175,
923: 134,
924: 328,
925: 409,
926: 1,
927: 359,
928: 1225,
929: 934,
930: 299,
931: 476,
932: 102,
933: 663,
934: 770,
935: 766,
936: 589,
937: 614,
938: 213,
939: 26,
940: 316,
941: 134,
942: 283,
943: 352,
944: 987,
945: 542,
946: 468,
947: 437,
948: 225,
949: 177,
950: 445,
951: 377,
952: 380,
953: 149,
954: 5,
955: 145,
956: 139,
957: 466,
958: 1,
959: 788,
960: 431,
961: 134,
962: 390,
963: 1848,
964: 213,
965: 674,
966: 259,
967: 270,
968: 403,
969: 525,
970: 352,
971: 124,
972: 633,
973: 380,
974: 53,
975: 78,
976: 551,
977: 104,
978: 239,
979: 206,
980: 175,
981: 145,
982: 291,
983: 296,
984: 415,
985: 110,
986: 1052,
987: 215,
988: 554,
989: 537,
990: 142,
991: 326,
992: 721,
993: 1,
994: 285,
995: 1,
996: 409,
997: 421,
998: 209,
999: 542,
1000: 384,
1001: 777,
1002: 306,
1003: 310,
1004: 1034,
1005: 712,
1006: 550,
1007: 293,
1008: 292,
1009: 103,
1010: 1293,
1011: 231,
1012: 272,
1013: 219,
1014: 552,
1015: 221,
1016: 114,


    }
    
    # Distance between each warehouse to each customer
    # (from, to): distance
    distance = {
(1,1) : 128,	(2,1) : 630,	(3,1) : 254,	(4,1) : 104,	(5,1) : 222,	(6,1) : 185,	(7,1) : 568,	(8,1) : 318,	(9,1) : 409,	(10,1) : 773,	(11,1) : 277,	(12,1) : 128,	(13,1) : 160,	(14,1) : 679,	(15,1) : 745,	(16,1) : 118,	(17,1) : 655,	(18,1) : 237,	(19,1) : 489,	(20,1) : 579,
(1,2) : 0,	(2,2) : 621,	(3,2) : 364,	(4,2) : 24,	(5,2) : 334,	(6,2) : 101,	(7,2) : 515,	(8,2) : 219,	(9,2) : 421,	(10,2) : 803,	(11,2) : 283,	(12,2) : 222,	(13,2) : 40,	(14,2) : 654,	(15,2) : 744,	(16,2) : 23,	(17,2) : 702,	(18,2) : 363,	(19,2) : 527,	(20,2) : 637,
(1,3) : 272,	(2,3) : 497,	(3,3) : 116,	(4,3) : 258,	(5,3) : 99,	(6,3) : 359,	(7,3) : 498,	(8,3) : 362,	(9,3) : 271,	(10,3) : 599,	(11,3) : 176,	(12,3) : 275,	(13,3) : 312,	(14,3) : 563,	(15,3) : 600,	(16,3) : 251,	(17,3) : 474,	(18,3) : 183,	(19,3) : 317,	(20,3) : 394,
(1,4) : 643,	(2,4) : 222,	(3,4) : 453,	(4,4) : 640,	(5,4) : 472,	(6,4) : 744,	(7,4) : 414,	(8,4) : 579,	(9,4) : 234,	(10,4) : 167,	(11,4) : 363,	(12,4) : 707,	(13,4) : 678,	(14,4) : 314,	(15,4) : 236,	(16,4) : 619,	(17,4) : 78,	(18,4) : 559,	(19,4) : 117,	(20,4) : 115,
(1,5) : 273,	(2,5) : 609,	(3,5) : 596,	(4,5) : 297,	(5,5) : 572,	(6,5) : 291,	(7,5) : 422,	(8,5) : 143,	(9,5) : 488,	(10,5) : 853,	(11,5) : 388,	(12,5) : 492,	(13,5) : 254,	(14,5) : 602,	(15,5) : 733,	(16,5) : 279,	(17,5) : 793,	(18,5) : 621,	(19,5) : 622,	(20,5) : 756,
(1,6) : 238,	(2,6) : 715,	(3,6) : 238,	(4,6) : 213,	(5,6) : 208,	(6,6) : 261,	(7,6) : 676,	(8,6) : 438,	(9,6) : 489,	(10,6) : 826,	(11,6) : 368,	(12,6) : 58,	(13,6) : 262,	(14,6) : 773,	(15,6) : 824,	(16,6) : 234,	(17,6) : 697,	(18,6) : 172,	(19,6) : 545,	(20,6) : 612,
(1,7) : 72,	(2,7) : 593,	(3,7) : 292,	(4,7) : 56,	(5,7) : 262,	(6,7) : 158,	(7,7) : 513,	(8,7) : 250,	(9,7) : 381,	(10,7) : 756,	(11,7) : 243,	(12,7) : 189,	(13,7) : 111,	(14,7) : 635,	(15,7) : 713,	(16,7) : 55,	(17,7) : 648,	(18,7) : 294,	(19,7) : 476,	(20,7) : 578,
(1,8) : 318,	(2,8) : 679,	(3,8) : 141,	(4,8) : 295,	(5,8) : 118,	(6,8) : 364,	(7,8) : 677,	(8,8) : 488,	(9,8) : 453,	(10,8) : 755,	(11,8) : 356,	(12,8) : 173,	(13,8) : 351,	(14,8) : 747,	(15,8) : 777,	(16,8) : 307,	(17,8) : 618,	(18,8) : 57,	(19,8) : 482,	(20,8) : 528,
(1,9) : 276,	(2,9) : 651,	(3,9) : 615,	(4,9) : 301,	(5,9) : 589,	(6,9) : 280,	(7,9) : 464,	(8,9) : 179,	(9,9) : 525,	(10,9) : 893,	(11,9) : 420,	(12,9) : 490,	(13,9) : 252,	(14,9) : 644,	(15,9) : 775,	(16,9) : 285,	(17,9) : 830,	(18,9) : 633,	(19,9) : 658,	(20,9) : 791,
(1,10) : 178,	(2,10) : 534,	(3,10) : 473,	(4,10) : 201,	(5,10) : 449,	(6,10) : 235,	(7,10) : 380,	(8,10) : 54,	(9,10) : 382,	(10,10) : 760,	(11,10) : 270,	(12,10) : 400,	(13,10) : 175,	(14,10) : 544,	(15,10) : 660,	(16,10) : 176,	(17,10) : 687,	(18,10) : 503,	(19,10) : 513,	(20,10) : 643,
(1,11) : 46,	(2,11) : 639,	(3,11) : 335,	(4,11) : 22,	(5,11) : 304,	(6,11) : 107,	(7,11) : 546,	(8,11) : 261,	(9,11) : 430,	(10,11) : 808,	(11,11) : 292,	(12,11) : 177,	(13,11) : 73,	(14,11) : 677,	(15,11) : 760,	(16,11) : 49,	(17,11) : 701,	(18,11) : 324,	(19,11) : 528,	(20,11) : 631,
(1,12) : 550,	(2,12) : 232,	(3,12) : 369,	(4,12) : 547,	(5,12) : 386,	(6,12) : 651,	(7,12) : 381,	(8,12) : 499,	(9,12) : 153,	(10,12) : 261,	(11,12) : 273,	(12,12) : 614,	(13,12) : 586,	(14,12) : 324,	(15,12) : 290,	(16,12) : 527,	(17,12) : 155,	(18,12) : 475,	(19,12) : 23,	(20,12) : 125,
(1,13) : 226,	(2,13) : 395,	(3,13) : 321,	(4,13) : 232,	(5,13) : 306,	(6,13) : 326,	(7,13) : 319,	(8,13) : 175,	(9,13) : 200,	(10,13) : 585,	(11,13) : 74,	(12,13) : 378,	(13,13) : 256,	(14,13) : 433,	(15,13) : 518,	(16,13) : 205,	(17,13) : 499,	(18,13) : 383,	(19,13) : 323,	(20,13) : 449,
(1,14) : 248,	(2,14) : 646,	(3,14) : 589,	(4,14) : 272,	(5,14) : 563,	(6,14) : 252,	(7,14) : 466,	(8,14) : 167,	(9,14) : 512,	(10,14) : 884,	(11,14) : 402,	(12,14) : 461,	(13,14) : 223,	(14,14) : 643,	(15,14) : 771,	(16,14) : 257,	(17,14) : 817,	(18,14) : 605,	(19,14) : 644,	(20,14) : 775,
(1,15) : 495,	(2,15) : 322,	(3,15) : 271,	(4,15) : 489,	(5,15) : 290,	(6,15) : 593,	(7,15) : 435,	(8,15) : 484,	(9,15) : 169,	(10,15) : 348,	(11,15) : 240,	(12,15) : 531,	(13,15) : 533,	(14,15) : 411,	(15,15) : 390,	(16,15) : 472,	(17,15) : 217,	(18,15) : 377,	(19,15) : 86,	(20,15) : 142,
(1,16) : 747,	(2,16) : 502,	(3,16) : 430,	(4,16) : 736,	(5,16) : 461,	(6,16) : 839,	(7,16) : 678,	(8,16) : 760,	(9,16) : 436,	(10,16) : 346,	(11,16) : 515,	(12,16) : 725,	(13,16) : 786,	(14,16) : 595,	(15,16) : 503,	(16,16) : 724,	(17,16) : 223,	(18,16) : 514,	(19,16) : 307,	(20,16) : 176,
(1,17) : 640,	(2,17) : 454,	(3,17) : 329,	(4,17) : 629,	(5,17) : 358,	(6,17) : 732,	(7,17) : 606,	(8,17) : 660,	(9,17) : 348,	(10,17) : 359,	(11,17) : 413,	(12,17) : 622,	(13,17) : 680,	(14,17) : 547,	(15,17) : 481,	(16,17) : 617,	(17,17) : 215,	(18,17) : 419,	(19,17) : 231,	(20,17) : 131,
(1,18) : 258,	(2,18) : 590,	(3,18) : 576,	(4,18) : 283,	(5,18) : 552,	(6,18) : 284,	(7,18) : 406,	(8,18) : 121,	(9,18) : 466,	(10,18) : 832,	(11,18) : 366,	(12,18) : 479,	(13,18) : 242,	(14,18) : 585,	(15,18) : 714,	(16,18) : 262,	(17,18) : 771,	(18,18) : 602,	(19,18) : 600,	(20,18) : 733,
(1,19) : 488,	(2,19) : 219,	(3,19) : 348,	(4,19) : 487,	(5,19) : 359,	(6,19) : 589,	(7,19) : 329,	(8,19) : 429,	(9,19) : 84,	(10,19) : 316,	(11,19) : 207,	(12,19) : 569,	(13,19) : 523,	(14,19) : 304,	(15,19) : 305,	(16,19) : 465,	(17,19) : 223,	(18,19) : 451,	(19,19) : 53,	(20,19) : 191,
(1,20) : 140,	(2,20) : 632,	(3,20) : 244,	(4,20) : 117,	(5,20) : 212,	(6,20) : 195,	(7,20) : 574,	(8,20) : 329,	(9,20) : 410,	(10,20) : 771,	(11,20) : 279,	(12,20) : 122,	(13,20) : 172,	(14,20) : 683,	(15,20) : 746,	(16,20) : 130,	(17,20) : 652,	(18,20) : 225,	(19,20) : 487,	(20,20) : 574,
(1,21) : 450,	(2,21) : 255,	(3,21) : 549,	(4,21) : 464,	(5,21) : 542,	(6,21) : 536,	(7,21) : 66,	(8,21) : 262,	(9,21) : 245,	(10,21) : 519,	(11,21) : 270,	(12,21) : 633,	(13,21) : 466,	(14,21) : 236,	(15,21) : 373,	(16,21) : 435,	(17,21) : 499,	(18,21) : 629,	(19,21) : 363,	(20,21) : 497,
(1,22) : 369,	(2,22) : 805,	(3,22) : 266,	(4,22) : 344,	(5,22) : 246,	(6,22) : 381,	(7,22) : 791,	(8,22) : 568,	(9,22) : 579,	(10,22) : 882,	(11,22) : 473,	(12,22) : 157,	(13,22) : 391,	(14,22) : 871,	(15,22) : 905,	(16,22) : 365,	(17,22) : 744,	(18,22) : 164,	(19,22) : 611,	(20,22) : 652,
(1,23) : 100,	(2,23) : 661,	(3,23) : 303,	(4,23) : 76,	(5,23) : 271,	(6,23) : 137,	(7,23) : 584,	(8,23) : 312,	(9,23) : 445,	(10,23) : 814,	(11,23) : 309,	(12,23) : 124,	(13,23) : 124,	(14,23) : 705,	(15,23) : 779,	(16,23) : 100,	(17,23) : 700,	(18,23) : 280,	(19,23) : 532,	(20,23) : 625,
(1,24) : 595,	(2,24) : 187,	(3,24) : 432,	(4,24) : 594,	(5,24) : 448,	(6,24) : 697,	(7,24) : 365,	(8,24) : 524,	(9,24) : 181,	(10,24) : 208,	(11,24) : 314,	(12,24) : 671,	(13,24) : 630,	(14,24) : 280,	(15,24) : 230,	(16,24) : 572,	(17,24) : 133,	(18,24) : 537,	(19,24) : 82,	(20,24) : 145,
(1,25) : 278,	(2,25) : 612,	(3,25) : 602,	(4,25) : 302,	(5,25) : 577,	(6,25) : 295,	(7,25) : 424,	(8,25) : 148,	(9,25) : 492,	(10,25) : 856,	(11,25) : 393,	(12,25) : 497,	(13,25) : 258,	(14,25) : 604,	(15,25) : 736,	(16,25) : 283,	(17,25) : 797,	(18,25) : 626,	(19,25) : 626,	(20,25) : 760,
(1,26) : 345,	(2,26) : 513,	(3,26) : 53,	(4,26) : 329,	(5,26) : 60,	(6,26) : 427,	(7,26) : 544,	(8,26) : 437,	(9,26) : 294,	(10,26) : 579,	(11,26) : 231,	(12,26) : 311,	(13,26) : 385,	(14,26) : 588,	(15,26) : 605,	(16,26) : 325,	(17,26) : 444,	(18,26) : 152,	(19,26) : 306,	(20,26) : 356,
(1,27) : 265,	(2,27) : 508,	(3,27) : 115,	(4,27) : 250,	(5,27) : 96,	(6,27) : 350,	(7,27) : 506,	(8,27) : 362,	(9,27) : 281,	(10,27) : 611,	(11,27) : 184,	(12,27) : 263,	(13,27) : 304,	(14,27) : 573,	(15,27) : 611,	(16,27) : 244,	(17,27) : 486,	(18,27) : 176,	(19,27) : 329,	(20,27) : 405,
(1,28) : 115,	(2,28) : 617,	(3,28) : 259,	(4,28) : 92,	(5,28) : 227,	(6,28) : 180,	(7,28) : 552,	(8,28) : 301,	(9,28) : 398,	(10,28) : 764,	(11,28) : 263,	(12,28) : 145,	(13,28) : 149,	(14,28) : 664,	(15,28) : 733,	(16,28) : 103,	(17,28) : 648,	(18,28) : 248,	(19,28) : 481,	(20,28) : 574,
(1,29) : 574,	(2,29) : 124,	(3,29) : 593,	(4,29) : 584,	(5,29) : 594,	(6,29) : 666,	(7,29) : 102,	(8,29) : 405,	(9,29) : 248,	(10,29) : 395,	(11,29) : 339,	(12,29) : 734,	(13,29) : 595,	(14,29) : 87,	(15,29) : 229,	(16,29) : 556,	(17,29) : 408,	(18,29) : 686,	(19,29) : 318,	(20,29) : 433,
(1,30) : 53,	(2,30) : 579,	(3,30) : 318,	(4,30) : 49,	(5,30) : 289,	(6,30) : 152,	(7,30) : 488,	(8,30) : 217,	(9,30) : 373,	(10,30) : 754,	(11,30) : 235,	(12,30) : 220,	(13,30) : 92,	(14,30) : 617,	(15,30) : 701,	(16,30) : 30,	(17,30) : 650,	(18,30) : 326,	(19,30) : 476,	(20,30) : 584,
(1,31) : 23,	(2,31) : 600,	(3,31) : 346,	(4,31) : 30,	(5,31) : 317,	(6,31) : 124,	(7,31) : 500,	(8,31) : 213,	(9,31) : 398,	(10,31) : 780,	(11,31) : 260,	(12,31) : 224,	(13,31) : 62,	(14,31) : 635,	(15,31) : 723,	(16,31) : 0,	(17,31) : 679,	(18,31) : 349,	(19,31) : 504,	(20,31) : 614,
(1,32) : 272,	(2,32) : 647,	(3,32) : 141,	(4,32) : 249,	(5,32) : 111,	(6,32) : 324,	(7,32) : 635,	(8,32) : 439,	(9,32) : 421,	(10,32) : 739,	(11,32) : 315,	(12,32) : 154,	(13,32) : 305,	(14,32) : 712,	(15,32) : 750,	(16,32) : 260,	(17,32) : 606,	(18,32) : 92,	(19,32) : 461,	(20,32) : 519,
(1,33) : 152,	(2,33) : 470,	(3,33) : 335,	(4,33) : 160,	(5,33) : 313,	(6,33) : 250,	(7,33) : 373,	(8,33) : 142,	(9,33) : 277,	(10,33) : 662,	(11,33) : 145,	(12,33) : 327,	(13,33) : 179,	(14,33) : 503,	(15,33) : 593,	(16,33) : 132,	(17,33) : 572,	(18,33) : 377,	(19,33) : 396,	(20,33) : 518,
(1,34) : 335,	(2,34) : 772,	(3,34) : 242,	(4,34) : 310,	(5,34) : 219,	(6,34) : 352,	(7,34) : 754,	(8,34) : 532,	(9,34) : 546,	(10,34) : 856,	(11,34) : 437,	(12,34) : 130,	(13,34) : 359,	(14,34) : 837,	(15,34) : 874,	(16,34) : 330,	(17,34) : 719,	(18,34) : 145,	(19,34) : 582,	(20,34) : 628,
(1,35) : 299,	(2,35) : 659,	(3,35) : 636,	(4,35) : 323,	(5,35) : 610,	(6,35) : 300,	(7,35) : 468,	(8,35) : 194,	(9,35) : 539,	(10,35) : 904,	(11,35) : 437,	(12,35) : 512,	(13,35) : 274,	(14,35) : 649,	(15,35) : 782,	(16,35) : 307,	(17,35) : 844,	(18,35) : 655,	(19,35) : 673,	(20,35) : 806,
(1,36) : 252,	(2,36) : 707,	(3,36) : 610,	(4,36) : 275,	(5,36) : 582,	(6,36) : 230,	(7,36) : 531,	(8,36) : 224,	(9,36) : 564,	(10,36) : 940,	(11,36) : 447,	(12,36) : 450,	(13,36) : 220,	(14,36) : 707,	(15,36) : 833,	(16,36) : 267,	(17,36) : 868,	(18,36) : 615,	(19,36) : 694,	(20,36) : 822,
(1,37) : 28,	(2,37) : 636,	(3,37) : 393,	(4,37) : 49,	(5,37) : 363,	(6,37) : 84,	(7,37) : 521,	(8,37) : 215,	(9,37) : 440,	(10,37) : 824,	(11,37) : 303,	(12,37) : 239,	(13,37) : 13,	(14,37) : 665,	(15,37) : 759,	(16,37) : 50,	(17,37) : 725,	(18,37) : 390,	(19,37) : 550,	(20,37) : 662,
(1,38) : 311,	(2,38) : 593,	(3,38) : 621,	(4,38) : 335,	(5,38) : 598,	(6,38) : 335,	(7,38) : 397,	(8,38) : 152,	(9,38) : 487,	(10,38) : 843,	(11,38) : 398,	(12,38) : 532,	(13,38) : 294,	(14,38) : 579,	(15,38) : 715,	(16,38) : 315,	(17,38) : 790,	(18,38) : 651,	(19,38) : 623,	(20,38) : 758,
(1,39) : 238,	(2,39) : 585,	(3,39) : 557,	(4,39) : 263,	(5,39) : 532,	(6,39) : 267,	(7,39) : 407,	(8,39) : 108,	(9,39) : 454,	(10,39) : 824,	(11,39) : 350,	(12,39) : 460,	(13,39) : 222,	(14,39) : 583,	(15,39) : 710,	(16,39) : 242,	(17,39) : 759,	(18,39) : 582,	(19,39) : 588,	(20,39) : 720,
(1,40) : 70,	(2,40) : 658,	(3,40) : 434,	(4,40) : 89,	(5,40) : 404,	(6,40) : 72,	(7,40) : 531,	(8,40) : 214,	(9,40) : 470,	(10,40) : 854,	(11,40) : 334,	(12,40) : 267,	(13,40) : 32,	(14,40) : 683,	(15,40) : 783,	(16,40) : 90,	(17,40) : 759,	(18,40) : 429,	(19,40) : 584,	(20,40) : 698,
(1,41) : 527,	(2,41) : 239,	(3,41) : 350,	(4,41) : 524,	(5,41) : 366,	(6,41) : 628,	(7,41) : 375,	(8,41) : 479,	(9,41) : 136,	(10,41) : 283,	(11,41) : 251,	(12,41) : 592,	(13,41) : 564,	(14,41) : 329,	(15,41) : 305,	(16,41) : 504,	(17,41) : 176,	(18,41) : 455,	(19,41) : 0,	(20,41) : 138,
(1,42) : 327,	(2,42) : 821,	(3,42) : 310,	(4,42) : 304,	(5,42) : 284,	(6,42) : 323,	(7,42) : 786,	(8,42) : 539,	(9,42) : 594,	(10,42) : 919,	(11,42) : 477,	(12,42) : 105,	(13,42) : 344,	(14,42) : 881,	(15,42) : 927,	(16,42) : 328,	(17,42) : 785,	(18,42) : 217,	(19,42) : 641,	(20,42) : 696,
(1,43) : 680,	(2,43) : 453,	(3,43) : 373,	(4,43) : 669,	(5,43) : 403,	(6,43) : 773,	(7,43) : 618,	(8,43) : 691,	(9,43) : 370,	(10,43) : 332,	(11,43) : 445,	(12,43) : 667,	(13,43) : 719,	(14,43) : 546,	(15,43) : 468,	(16,43) : 657,	(17,43) : 193,	(18,43) : 463,	(19,43) : 244,	(20,43) : 123,
(1,44) : 336,	(2,44) : 783,	(3,44) : 254,	(4,44) : 311,	(5,44) : 231,	(6,44) : 349,	(7,44) : 762,	(8,44) : 536,	(9,44) : 556,	(10,44) : 868,	(11,44) : 446,	(12,44) : 126,	(13,44) : 358,	(14,44) : 847,	(15,44) : 885,	(16,44) : 332,	(17,44) : 731,	(18,44) : 157,	(19,44) : 594,	(20,44) : 641,
(1,45) : 250,	(2,45) : 408,	(3,45) : 230,	(4,45) : 246,	(5,45) : 218,	(6,45) : 350,	(7,45) : 381,	(8,45) : 264,	(9,45) : 186,	(10,45) : 557,	(11,45) : 61,	(12,45) : 342,	(13,45) : 287,	(14,45) : 464,	(15,45) : 522,	(16,45) : 227,	(17,45) : 452,	(18,45) : 301,	(19,45) : 278,	(20,45) : 389,
(1,46) : 276,	(2,46) : 674,	(3,46) : 622,	(4,46) : 300,	(5,46) : 596,	(6,46) : 271,	(7,46) : 488,	(8,46) : 198,	(9,46) : 544,	(10,46) : 914,	(11,46) : 436,	(12,46) : 486,	(13,46) : 249,	(14,46) : 668,	(15,46) : 798,	(16,46) : 287,	(17,46) : 849,	(18,46) : 636,	(19,46) : 677,	(20,46) : 808,
(1,47) : 234,	(2,47) : 783,	(3,47) : 332,	(4,47) : 212,	(5,47) : 301,	(6,47) : 218,	(7,47) : 722,	(8,47) : 452,	(9,47) : 561,	(10,47) : 911,	(11,47) : 431,	(12,47) : 38,	(13,47) : 245,	(14,47) : 835,	(15,47) : 897,	(16,47) : 239,	(17,47) : 785,	(18,47) : 263,	(19,47) : 628,	(20,47) : 702,
(1,48) : 101,	(2,48) : 719,	(3,48) : 438,	(4,48) : 105,	(5,48) : 407,	(6,48) : 0,	(7,48) : 601,	(8,48) : 286,	(9,48) : 522,	(10,48) : 904,	(11,48) : 384,	(12,48) : 223,	(13,48) : 71,	(14,48) : 748,	(15,48) : 843,	(16,48) : 124,	(17,48) : 802,	(18,48) : 416,	(19,48) : 628,	(20,48) : 735,
(1,49) : 172,	(2,49) : 719,	(3,49) : 300,	(4,49) : 149,	(5,49) : 268,	(6,49) : 179,	(7,49) : 654,	(8,49) : 387,	(9,49) : 498,	(10,49) : 856,	(11,49) : 366,	(12,49) : 50,	(13,49) : 189,	(14,49) : 769,	(15,49) : 834,	(16,49) : 175,	(17,49) : 734,	(18,49) : 251,	(19,49) : 573,	(20,49) : 654,
(1,50) : 454,	(2,50) : 384,	(3,50) : 201,	(4,50) : 445,	(5,50) : 221,	(6,50) : 549,	(7,50) : 473,	(8,50) : 473,	(9,50) : 200,	(10,50) : 416,	(11,50) : 228,	(12,50) : 468,	(13,50) : 493,	(14,50) : 470,	(15,50) : 459,	(16,50) : 431,	(17,50) : 281,	(18,50) : 306,	(19,50) : 154,	(20,50) : 196,
(1,51) : 297,	(2,51) : 667,	(3,51) : 142,	(4,51) : 274,	(5,51) : 115,	(6,51) : 345,	(7,51) : 660,	(8,51) : 467,	(9,51) : 442,	(10,51) : 751,	(11,51) : 340,	(12,51) : 161,	(13,51) : 330,	(14,51) : 734,	(15,51) : 768,	(16,51) : 286,	(17,51) : 616,	(18,51) : 72,	(19,51) : 476,	(20,51) : 527,
(1,52) : 180,	(2,52) : 587,	(3,52) : 507,	(4,52) : 204,	(5,52) : 481,	(6,52) : 213,	(7,52) : 425,	(8,52) : 102,	(9,52) : 436,	(10,52) : 814,	(11,52) : 320,	(12,52) : 401,	(13,52) : 165,	(14,52) : 594,	(15,52) : 713,	(16,52) : 184,	(17,52) : 740,	(18,52) : 527,	(19,52) : 566,	(20,52) : 694,
(1,53) : 209,	(2,53) : 723,	(3,53) : 271,	(4,53) : 185,	(5,53) : 240,	(6,53) : 222,	(7,53) : 671,	(8,53) : 417,	(9,53) : 498,	(10,53) : 846,	(11,53) : 372,	(12,53) : 28,	(13,53) : 229,	(14,53) : 777,	(15,53) : 834,	(16,53) : 208,	(17,53) : 720,	(18,53) : 212,	(19,53) : 563,	(20,53) : 637,
(1,54) : 98,	(2,54) : 665,	(3,54) : 462,	(4,54) : 118,	(5,54) : 432,	(6,54) : 88,	(7,54) : 529,	(8,54) : 206,	(9,54) : 483,	(10,54) : 868,	(11,54) : 350,	(12,54) : 295,	(13,54) : 62,	(14,54) : 685,	(15,54) : 790,	(16,54) : 116,	(17,54) : 776,	(18,54) : 459,	(19,54) : 600,	(20,54) : 717,
(1,55) : 576,	(2,55) : 417,	(3,55) : 280,	(4,55) : 566,	(5,55) : 308,	(6,55) : 669,	(7,55) : 552,	(8,55) : 594,	(9,55) : 289,	(10,55) : 364,	(11,55) : 348,	(12,55) : 570,	(13,55) : 616,	(14,55) : 509,	(15,55) : 460,	(16,55) : 554,	(17,55) : 217,	(18,55) : 377,	(19,55) : 183,	(20,55) : 122,
(1,56) : 555,	(2,56) : 72,	(3,56) : 493,	(4,56) : 561,	(5,56) : 500,	(6,56) : 655,	(7,56) : 215,	(8,56) : 433,	(9,56) : 155,	(10,56) : 293,	(11,56) : 283,	(12,56) : 679,	(13,56) : 584,	(14,56) : 150,	(15,56) : 189,	(16,56) : 534,	(17,56) : 276,	(18,56) : 593,	(19,56) : 183,	(20,56) : 295,
(1,57) : 292,	(2,57) : 329,	(3,57) : 343,	(4,57) : 298,	(5,57) : 333,	(6,57) : 392,	(7,57) : 267,	(8,57) : 207,	(9,57) : 142,	(10,57) : 525,	(11,57) : 61,	(12,57) : 438,	(13,57) : 321,	(14,57) : 369,	(15,57) : 452,	(16,57) : 271,	(17,57) : 447,	(18,57) : 418,	(19,57) : 273,	(20,57) : 405,
(1,58) : 171,	(2,58) : 474,	(3,58) : 402,	(4,58) : 188,	(5,58) : 381,	(6,58) : 256,	(7,58) : 346,	(8,58) : 74,	(9,58) : 305,	(10,58) : 687,	(11,58) : 188,	(12,58) : 375,	(13,58) : 186,	(14,58) : 494,	(15,58) : 599,	(16,58) : 158,	(17,58) : 609,	(18,58) : 443,	(19,58) : 433,	(20,58) : 562,
(1,59) : 345,	(2,59) : 807,	(3,59) : 280,	(4,59) : 321,	(5,59) : 257,	(6,59) : 352,	(7,59) : 783,	(8,59) : 550,	(9,59) : 580,	(10,59) : 894,	(11,59) : 469,	(12,59) : 129,	(13,59) : 366,	(14,59) : 870,	(15,59) : 910,	(16,59) : 343,	(17,59) : 757,	(18,59) : 182,	(19,59) : 619,	(20,59) : 667,
(1,60) : 84,	(2,60) : 546,	(3,60) : 373,	(4,60) : 101,	(5,60) : 346,	(6,60) : 174,	(7,60) : 432,	(8,60) : 142,	(9,60) : 358,	(10,60) : 743,	(11,60) : 224,	(12,60) : 294,	(13,60) : 103,	(14,60) : 574,	(15,60) : 671,	(16,60) : 72,	(17,60) : 651,	(18,60) : 394,	(19,60) : 475,	(20,60) : 594,
(1,61) : 612,	(2,61) : 109,	(3,61) : 492,	(4,61) : 614,	(5,61) : 505,	(6,61) : 713,	(7,61) : 307,	(8,61) : 512,	(9,61) : 192,	(10,61) : 206,	(11,61) : 331,	(12,61) : 712,	(13,61) : 644,	(14,61) : 202,	(15,61) : 160,	(16,61) : 590,	(17,61) : 187,	(18,61) : 596,	(19,61) : 146,	(20,61) : 222,
(1,62) : 114,	(2,62) : 667,	(3,62) : 297,	(4,62) : 89,	(5,62) : 265,	(6,62) : 146,	(7,62) : 594,	(8,62) : 324,	(9,62) : 450,	(10,62) : 817,	(11,62) : 315,	(12,62) : 111,	(13,62) : 137,	(14,62) : 713,	(15,62) : 785,	(16,62) : 113,	(17,62) : 701,	(18,62) : 269,	(19,62) : 534,	(20,62) : 625,
(1,63) : 84,	(2,63) : 596,	(3,63) : 281,	(4,63) : 66,	(5,63) : 251,	(6,63) : 166,	(7,63) : 520,	(8,63) : 262,	(9,63) : 381,	(10,63) : 755,	(11,63) : 244,	(12,63) : 179,	(13,63) : 122,	(14,63) : 640,	(15,63) : 714,	(16,63) : 68,	(17,63) : 644,	(18,63) : 281,	(19,63) : 474,	(20,63) : 573,
(1,64) : 266,	(2,64) : 656,	(3,64) : 608,	(4,64) : 290,	(5,64) : 581,	(6,64) : 267,	(7,64) : 472,	(8,64) : 180,	(9,64) : 525,	(10,64) : 896,	(11,64) : 418,	(12,64) : 478,	(13,64) : 240,	(14,64) : 650,	(15,64) : 780,	(16,64) : 276,	(17,64) : 831,	(18,64) : 624,	(19,64) : 658,	(20,64) : 790,
(1,65) : 184,	(2,65) : 796,	(3,65) : 425,	(4,65) : 171,	(5,65) : 392,	(6,65) : 117,	(7,65) : 699,	(8,65) : 395,	(9,65) : 584,	(10,65) : 956,	(11,65) : 446,	(12,65) : 149,	(13,65) : 173,	(14,65) : 834,	(15,65) : 916,	(16,65) : 200,	(17,65) : 842,	(18,65) : 375,	(19,65) : 674,	(20,65) : 766,
(1,66) : 455,	(2,66) : 181,	(3,66) : 392,	(4,66) : 458,	(5,66) : 396,	(6,66) : 556,	(7,66) : 239,	(8,66) : 360,	(9,66) : 47,	(10,66) : 360,	(11,66) : 177,	(12,66) : 570,	(13,66) : 486,	(14,66) : 248,	(15,66) : 294,	(16,66) : 433,	(17,66) : 297,	(18,66) : 489,	(19,66) : 143,	(20,66) : 280,
(1,67) : 540,	(2,67) : 124,	(3,67) : 441,	(4,67) : 542,	(5,67) : 451,	(6,67) : 641,	(7,67) : 267,	(8,67) : 442,	(9,67) : 121,	(10,67) : 275,	(11,67) : 259,	(12,67) : 645,	(13,67) : 571,	(14,67) : 211,	(15,67) : 218,	(16,67) : 518,	(17,67) : 230,	(18,67) : 543,	(19,67) : 120,	(20,67) : 236,
(1,68) : 134,	(2,68) : 689,	(3,68) : 301,	(4,68) : 110,	(5,68) : 269,	(6,68) : 153,	(7,68) : 618,	(8,68) : 348,	(9,68) : 471,	(10,68) : 835,	(11,68) : 336,	(12,68) : 88,	(13,68) : 154,	(14,68) : 736,	(15,68) : 806,	(16,68) : 136,	(17,68) : 717,	(18,68) : 265,	(19,68) : 552,	(20,68) : 639,
(1,69) : 265,	(2,69) : 727,	(3,69) : 231,	(4,69) : 241,	(5,69) : 202,	(6,69) : 288,	(7,69) : 695,	(8,69) : 463,	(9,69) : 500,	(10,69) : 829,	(11,69) : 383,	(12,69) : 76,	(13,69) : 290,	(14,69) : 787,	(15,69) : 833,	(16,69) : 260,	(17,69) : 698,	(18,69) : 155,	(19,69) : 550,	(20,69) : 611,
(1,70) : 117,	(2,70) : 582,	(3,70) : 447,	(4,70) : 141,	(5,70) : 420,	(6,70) : 166,	(7,70) : 442,	(8,70) : 120,	(9,70) : 411,	(10,70) : 795,	(11,70) : 285,	(12,70) : 339,	(13,70) : 107,	(14,70) : 600,	(15,70) : 708,	(16,70) : 120,	(17,70) : 712,	(18,70) : 464,	(19,70) : 536,	(20,70) : 659,
(1,71) : 369,	(2,71) : 755,	(3,71) : 205,	(4,71) : 345,	(5,71) : 187,	(6,71) : 397,	(7,71) : 754,	(8,71) : 554,	(9,71) : 530,	(10,71) : 821,	(11,71) : 433,	(12,71) : 181,	(13,71) : 396,	(14,71) : 824,	(15,71) : 851,	(16,71) : 361,	(17,71) : 682,	(18,71) : 101,	(19,71) : 553,	(20,71) : 589,
(1,72) : 115,	(2,72) : 727,	(3,72) : 460,	(4,72) : 122,	(5,72) : 428,	(6,72) : 23,	(7,72) : 602,	(8,72) : 283,	(9,72) : 533,	(10,72) : 917,	(11,72) : 396,	(12,72) : 246,	(13,72) : 80,	(14,72) : 753,	(15,72) : 851,	(16,72) : 138,	(17,72) : 817,	(18,72) : 439,	(19,72) : 642,	(20,72) : 752,
(1,73) : 54,	(2,73) : 675,	(3,73) : 385,	(4,73) : 50,	(5,73) : 354,	(6,73) : 56,	(7,73) : 569,	(8,73) : 265,	(9,73) : 472,	(10,73) : 853,	(11,73) : 333,	(12,73) : 196,	(13,73) : 46,	(14,73) : 708,	(15,73) : 797,	(16,73) : 74,	(17,73) : 748,	(18,73) : 368,	(19,73) : 575,	(20,73) : 680,
(1,74) : 167,	(2,74) : 565,	(3,74) : 483,	(4,74) : 191,	(5,74) : 458,	(6,74) : 213,	(7,74) : 409,	(8,74) : 84,	(9,74) : 410,	(10,74) : 789,	(11,74) : 294,	(12,74) : 389,	(13,74) : 157,	(14,74) : 575,	(15,74) : 691,	(16,74) : 169,	(17,74) : 714,	(18,74) : 506,	(19,74) : 539,	(20,74) : 668,
(1,75) : 638,	(2,75) : 280,	(3,75) : 416,	(4,75) : 633,	(5,75) : 438,	(6,75) : 738,	(7,75) : 461,	(8,75) : 595,	(9,75) : 250,	(10,75) : 202,	(11,75) : 366,	(12,75) : 683,	(13,75) : 675,	(14,75) : 372,	(15,75) : 297,	(16,75) : 615,	(17,75) : 65,	(18,75) : 520,	(19,75) : 116,	(20,75) : 54,
(1,76) : 626,	(2,76) : 28,	(3,76) : 550,	(4,76) : 631,	(5,76) : 560,	(6,76) : 725,	(7,76) : 248,	(8,76) : 498,	(9,76) : 221,	(10,76) : 245,	(11,76) : 353,	(12,76) : 747,	(13,76) : 654,	(14,76) : 120,	(15,76) : 118,	(16,76) : 604,	(17,76) : 263,	(18,76) : 653,	(19,76) : 218,	(20,76) : 304,
(1,77) : 505,	(2,77) : 274,	(3,77) : 312,	(4,77) : 501,	(5,77) : 328,	(6,77) : 605,	(7,77) : 395,	(8,77) : 473,	(9,77) : 139,	(10,77) : 315,	(11,77) : 236,	(12,77) : 559,	(13,77) : 542,	(14,77) : 363,	(15,77) : 344,	(16,77) : 482,	(17,77) : 197,	(18,77) : 417,	(19,77) : 39,	(20,77) : 140,
(1,78) : 547,	(2,78) : 131,	(3,78) : 569,	(4,78) : 558,	(5,78) : 569,	(6,78) : 640,	(7,78) : 92,	(8,78) : 381,	(9,78) : 226,	(10,78) : 402,	(11,78) : 313,	(12,78) : 707,	(13,78) : 569,	(14,78) : 111,	(15,78) : 244,	(16,78) : 529,	(17,78) : 405,	(18,78) : 661,	(19,78) : 304,	(20,78) : 424,
(1,79) : 651,	(2,79) : 82,	(3,79) : 631,	(4,79) : 660,	(5,79) : 636,	(6,79) : 746,	(7,79) : 189,	(8,79) : 492,	(9,79) : 287,	(10,79) : 329,	(11,79) : 399,	(12,79) : 798,	(13,79) : 674,	(14,79) : 12,	(15,79) : 146,	(16,79) : 631,	(17,79) : 369,	(18,79) : 729,	(19,79) : 318,	(20,79) : 412,
(1,80) : 278,	(2,80) : 356,	(3,80) : 284,	(4,80) : 279,	(5,80) : 275,	(6,80) : 379,	(7,80) : 321,	(8,80) : 241,	(9,80) : 143,	(10,80) : 526,	(11,80) : 6,	(12,80) : 397,	(13,80) : 311,	(14,80) : 407,	(15,80) : 474,	(16,80) : 255,	(17,80) : 433,	(18,80) : 360,	(19,80) : 257,	(20,80) : 380,
(1,81) : 662,	(2,81) : 49,	(3,81) : 612,	(4,81) : 669,	(5,81) : 619,	(6,81) : 759,	(7,81) : 231,	(8,81) : 517,	(9,81) : 275,	(10,81) : 276,	(11,81) : 399,	(12,81) : 797,	(13,81) : 688,	(14,81) : 64,	(15,81) : 100,	(16,81) : 642,	(17,81) : 319,	(18,81) : 713,	(19,81) : 285,	(20,81) : 367,
(1,82) : 73,	(2,82) : 635,	(3,82) : 435,	(4,82) : 96,	(5,82) : 405,	(6,82) : 99,	(7,82) : 505,	(8,82) : 187,	(9,82) : 451,	(10,82) : 836,	(11,82) : 317,	(12,82) : 285,	(13,82) : 46,	(14,82) : 658,	(15,82) : 761,	(16,82) : 89,	(17,82) : 744,	(18,82) : 436,	(19,82) : 568,	(20,82) : 685,
(1,83) : 644,	(2,83) : 459,	(3,83) : 331,	(4,83) : 633,	(5,83) : 361,	(6,83) : 736,	(7,83) : 612,	(8,83) : 665,	(9,83) : 355,	(10,83) : 362,	(11,83) : 419,	(12,83) : 625,	(13,83) : 684,	(14,83) : 553,	(15,83) : 486,	(16,83) : 622,	(17,83) : 219,	(18,83) : 421,	(19,83) : 237,	(20,83) : 135,
(1,84) : 548,	(2,84) : 113,	(3,84) : 452,	(4,84) : 551,	(5,84) : 462,	(6,84) : 649,	(7,84) : 263,	(8,84) : 447,	(9,84) : 131,	(10,84) : 270,	(11,84) : 268,	(12,84) : 656,	(13,84) : 580,	(14,84) : 201,	(15,84) : 207,	(16,84) : 526,	(17,84) : 231,	(18,84) : 554,	(19,84) : 129,	(20,84) : 242,
(1,85) : 226,	(2,85) : 395,	(3,85) : 336,	(4,85) : 233,	(5,85) : 321,	(6,85) : 325,	(7,85) : 311,	(8,85) : 161,	(9,85) : 207,	(10,85) : 591,	(11,85) : 86,	(12,85) : 386,	(13,85) : 254,	(14,85) : 430,	(15,85) : 518,	(16,85) : 206,	(17,85) : 507,	(18,85) : 397,	(19,85) : 332,	(20,85) : 459,
(1,86) : 45,	(2,86) : 586,	(3,86) : 377,	(4,86) : 66,	(5,86) : 348,	(6,86) : 133,	(7,86) : 473,	(8,86) : 174,	(9,86) : 394,	(10,86) : 778,	(11,86) : 258,	(12,86) : 264,	(13,86) : 62,	(14,86) : 616,	(15,86) : 711,	(16,86) : 41,	(17,86) : 683,	(18,86) : 387,	(19,86) : 507,	(20,86) : 622,
(1,87) : 588,	(2,87) : 556,	(3,87) : 235,	(4,87) : 573,	(5,87) : 268,	(6,87) : 670,	(7,87) : 671,	(8,87) : 655,	(9,87) : 399,	(10,87) : 499,	(11,87) : 417,	(12,87) : 526,	(13,87) : 628,	(14,87) : 647,	(15,87) : 603,	(16,87) : 568,	(17,87) : 353,	(18,87) : 305,	(19,87) : 318,	(20,87) : 261,
(1,88) : 75,	(2,88) : 550,	(3,88) : 317,	(4,88) : 77,	(5,88) : 290,	(6,88) : 176,	(7,88) : 458,	(8,88) : 194,	(9,88) : 346,	(10,88) : 728,	(11,88) : 208,	(12,88) : 247,	(13,88) : 111,	(14,88) : 587,	(15,88) : 672,	(16,88) : 52,	(17,88) : 628,	(18,88) : 335,	(19,88) : 453,	(20,88) : 564,
(1,89) : 136,	(2,89) : 548,	(3,89) : 440,	(4,89) : 158,	(5,89) : 415,	(6,89) : 199,	(7,89) : 407,	(8,89) : 88,	(9,89) : 381,	(10,89) : 763,	(11,89) : 259,	(12,89) : 357,	(13,89) : 135,	(14,89) : 564,	(15,89) : 674,	(16,89) : 133,	(17,89) : 683,	(18,89) : 465,	(19,89) : 507,	(20,89) : 633,
(1,90) : 24,	(2,90) : 635,	(3,90) : 354,	(4,90) : 9,	(5,90) : 323,	(6,90) : 96,	(7,90) : 535,	(8,90) : 243,	(9,90) : 430,	(10,90) : 811,	(11,90) : 292,	(12,90) : 199,	(13,90) : 51,	(14,90) : 670,	(15,90) : 757,	(16,90) : 36,	(17,90) : 707,	(18,90) : 346,	(19,90) : 533,	(20,90) : 639,
(1,91) : 610,	(2,91) : 69,	(3,91) : 514,	(4,91) : 614,	(5,91) : 525,	(6,91) : 711,	(7,91) : 272,	(8,91) : 497,	(9,91) : 196,	(10,91) : 228,	(11,91) : 332,	(12,91) : 722,	(13,91) : 641,	(14,91) : 162,	(15,91) : 141,	(16,91) : 588,	(17,91) : 226,	(18,91) : 617,	(19,91) : 177,	(20,91) : 262,
(1,92) : 103,	(2,92) : 611,	(3,92) : 267,	(4,92) : 81,	(5,92) : 236,	(6,92) : 173,	(7,92) : 542,	(8,92) : 288,	(9,92) : 393,	(10,92) : 762,	(11,92) : 258,	(12,92) : 155,	(13,92) : 138,	(14,92) : 657,	(15,92) : 728,	(16,92) : 90,	(17,92) : 649,	(18,92) : 260,	(19,92) : 480,	(20,92) : 575,
(1,93) : 366,	(2,93) : 337,	(3,93) : 518,	(4,93) : 382,	(5,93) : 506,	(6,93) : 447,	(7,93) : 158,	(8,93) : 168,	(9,93) : 265,	(10,93) : 591,	(11,93) : 238,	(12,93) : 562,	(13,93) : 378,	(14,93) : 330,	(15,93) : 460,	(16,93) : 353,	(17,93) : 552,	(18,93) : 586,	(19,93) : 398,	(20,93) : 536,
(1,94) : 613,	(2,94) : 426,	(3,94) : 313,	(4,94) : 603,	(5,94) : 341,	(6,94) : 706,	(7,94) : 574,	(8,94) : 628,	(9,94) : 316,	(10,94) : 349,	(11,94) : 382,	(12,94) : 604,	(13,94) : 653,	(14,94) : 520,	(15,94) : 459,	(16,94) : 591,	(17,94) : 202,	(18,94) : 407,	(19,94) : 200,	(20,94) : 111,
(1,95) : 690,	(2,95) : 371,	(3,95) : 422,	(4,95) : 682,	(5,95) : 449,	(6,95) : 787,	(7,95) : 554,	(8,95) : 670,	(9,95) : 330,	(10,95) : 237,	(11,95) : 431,	(12,95) : 707,	(13,95) : 728,	(14,95) : 464,	(15,95) : 375,	(16,95) : 666,	(17,95) : 97,	(18,95) : 521,	(19,95) : 195,	(20,95) : 57,
(1,96) : 36,	(2,96) : 585,	(3,96) : 353,	(4,96) : 50,	(5,96) : 325,	(6,96) : 136,	(7,96) : 480,	(8,96) : 192,	(9,96) : 386,	(10,96) : 770,	(11,96) : 249,	(12,96) : 244,	(13,96) : 68,	(14,96) : 618,	(15,96) : 708,	(16,96) : 21,	(17,96) : 671,	(18,96) : 362,	(19,96) : 496,	(20,96) : 608,
(1,97) : 243,	(2,97) : 679,	(3,97) : 596,	(4,97) : 267,	(5,97) : 568,	(6,97) : 233,	(7,97) : 502,	(8,97) : 196,	(9,97) : 537,	(10,97) : 913,	(11,97) : 423,	(12,97) : 449,	(13,97) : 214,	(14,97) : 678,	(15,97) : 804,	(16,97) : 256,	(17,97) : 842,	(18,97) : 605,	(19,97) : 668,	(20,97) : 797,
(1,98) : 360,	(2,98) : 746,	(3,98) : 199,	(4,98) : 336,	(5,98) : 180,	(6,98) : 390,	(7,98) : 744,	(8,98) : 544,	(9,98) : 522,	(10,98) : 815,	(11,98) : 423,	(12,98) : 176,	(13,98) : 388,	(14,98) : 815,	(15,98) : 843,	(16,98) : 352,	(17,98) : 676,	(18,98) : 96,	(19,98) : 546,	(20,98) : 584,
(1,99) : 546,	(2,99) : 278,	(3,99) : 336,	(4,99) : 542,	(5,99) : 355,	(6,99) : 646,	(7,99) : 421,	(8,99) : 514,	(9,99) : 176,	(10,99) : 283,	(11,99) : 278,	(12,99) : 594,	(13,99) : 584,	(14,99) : 370,	(15,99) : 333,	(16,99) : 523,	(17,99) : 157,	(18,99) : 441,	(19,99) : 46,	(20,99) : 99,
(1,100) : 630,	(2,100) : 29,	(3,100) : 586,	(4,100) : 637,	(5,100) : 593,	(6,100) : 727,	(7,100) : 206,	(8,100) : 485,	(9,100) : 246,	(10,100) : 291,	(11,100) : 367,	(12,100) : 766,	(13,100) : 656,	(14,100) : 64,	(15,100) : 129,	(16,100) : 609,	(17,100) : 319,	(18,100) : 686,	(19,100) : 267,	(20,100) : 360,
(1,101) : 515,	(2,101) : 223,	(3,101) : 597,	(4,101) : 529,	(5,101) : 592,	(6,101) : 601,	(7,101) : 0,	(8,101) : 326,	(9,101) : 273,	(10,101) : 493,	(11,101) : 322,	(12,101) : 696,	(13,101) : 532,	(14,101) : 185,	(15,101) : 331,	(16,101) : 500,	(17,101) : 492,	(18,101) : 681,	(19,101) : 375,	(20,101) : 503,
(1,102) : 502,	(2,102) : 148,	(3,102) : 525,	(4,102) : 512,	(5,102) : 524,	(6,102) : 596,	(7,102) : 98,	(8,102) : 344,	(9,102) : 186,	(10,102) : 410,	(11,102) : 266,	(12,102) : 659,	(13,102) : 525,	(14,102) : 152,	(15,102) : 271,	(16,102) : 483,	(17,102) : 397,	(18,102) : 616,	(19,102) : 278,	(20,102) : 405,
(1,103) : 579,	(2,103) : 486,	(3,103) : 250,	(4,103) : 566,	(5,103) : 281,	(6,103) : 667,	(7,103) : 611,	(8,103) : 623,	(9,103) : 342,	(10,103) : 430,	(11,103) : 378,	(12,103) : 545,	(13,103) : 619,	(14,103) : 577,	(15,103) : 531,	(16,103) : 558,	(17,103) : 283,	(18,103) : 337,	(19,103) : 249,	(20,103) : 190,
(1,104) : 128,	(2,104) : 609,	(3,104) : 472,	(4,104) : 152,	(5,104) : 445,	(6,104) : 158,	(7,104) : 462,	(8,104) : 137,	(9,104) : 441,	(10,104) : 824,	(11,104) : 316,	(12,104) : 347,	(13,104) : 109,	(14,104) : 624,	(15,104) : 735,	(16,104) : 136,	(17,104) : 742,	(18,104) : 484,	(19,104) : 566,	(20,104) : 690,
(1,105) : 447,	(2,105) : 636,	(3,105) : 97,	(4,105) : 427,	(5,105) : 118,	(6,105) : 511,	(7,105) : 687,	(8,105) : 573,	(9,105) : 428,	(10,105) : 655,	(11,105) : 377,	(12,105) : 336,	(13,105) : 484,	(14,105) : 717,	(15,105) : 714,	(16,105) : 430,	(17,105) : 510,	(18,105) : 106,	(19,105) : 409,	(20,105) : 415,
(1,106) : 259,	(2,106) : 621,	(3,106) : 131,	(4,106) : 237,	(5,106) : 99,	(6,106) : 318,	(7,106) : 608,	(8,106) : 417,	(9,106) : 394,	(10,106) : 717,	(11,106) : 288,	(12,106) : 167,	(13,106) : 294,	(14,106) : 685,	(15,106) : 724,	(16,106) : 245,	(17,106) : 586,	(18,106) : 105,	(19,106) : 438,	(20,106) : 500,
(1,107) : 589,	(2,107) : 84,	(3,107) : 491,	(4,107) : 593,	(5,107) : 502,	(6,107) : 690,	(7,107) : 270,	(8,107) : 481,	(9,107) : 173,	(10,107) : 239,	(11,107) : 310,	(12,107) : 698,	(13,107) : 620,	(14,107) : 177,	(15,107) : 165,	(16,107) : 567,	(17,107) : 222,	(18,107) : 594,	(19,107) : 156,	(20,107) : 250,
(1,108) : 604,	(2,108) : 333,	(3,108) : 353,	(4,108) : 597,	(5,108) : 377,	(6,108) : 702,	(7,108) : 493,	(8,108) : 586,	(9,108) : 251,	(10,108) : 271,	(11,108) : 346,	(12,108) : 630,	(13,108) : 643,	(14,108) : 426,	(15,108) : 365,	(16,108) : 581,	(17,108) : 126,	(18,108) : 455,	(19,108) : 120,	(20,108) : 35,
(1,109) : 440,	(2,109) : 190,	(3,109) : 391,	(4,109) : 444,	(5,109) : 394,	(6,109) : 540,	(7,109) : 229,	(8,109) : 341,	(9,109) : 46,	(10,109) : 378,	(11,109) : 164,	(12,109) : 560,	(13,109) : 470,	(14,109) : 251,	(15,109) : 306,	(16,109) : 418,	(17,109) : 315,	(18,109) : 487,	(19,109) : 159,	(20,109) : 296,
(1,110) : 552,	(2,110) : 117,	(3,110) : 450,	(4,110) : 554,	(5,110) : 460,	(6,110) : 653,	(7,110) : 271,	(8,110) : 453,	(9,110) : 133,	(10,110) : 264,	(11,110) : 271,	(12,110) : 657,	(13,110) : 583,	(14,110) : 206,	(15,110) : 207,	(16,110) : 529,	(17,110) : 223,	(18,110) : 553,	(19,110) : 123,	(20,110) : 234,
(1,111) : 599,	(2,111) : 153,	(3,111) : 455,	(4,111) : 599,	(5,111) : 469,	(6,111) : 700,	(7,111) : 337,	(8,111) : 515,	(9,111) : 179,	(10,111) : 206,	(11,111) : 316,	(12,111) : 685,	(13,111) : 632,	(14,111) : 247,	(15,111) : 201,	(16,111) : 576,	(17,111) : 155,	(18,111) : 559,	(19,111) : 105,	(20,111) : 178,
(1,112) : 189,	(2,112) : 504,	(3,112) : 460,	(4,112) : 210,	(5,112) : 437,	(6,112) : 256,	(7,112) : 353,	(8,112) : 31,	(9,112) : 354,	(10,112) : 730,	(11,112) : 245,	(12,112) : 406,	(13,112) : 191,	(14,112) : 515,	(15,112) : 630,	(16,112) : 183,	(17,112) : 659,	(18,112) : 496,	(19,112) : 485,	(20,112) : 616,
(1,113) : 129,	(2,113) : 656,	(3,113) : 271,	(4,113) : 105,	(5,113) : 239,	(6,113) : 171,	(7,113) : 591,	(8,113) : 332,	(9,113) : 436,	(10,113) : 799,	(11,113) : 302,	(12,113) : 107,	(13,113) : 157,	(14,113) : 704,	(15,113) : 771,	(16,113) : 125,	(17,113) : 681,	(18,113) : 245,	(19,113) : 516,	(20,113) : 604,
(1,114) : 663,	(2,114) : 45,	(3,114) : 589,	(4,114) : 669,	(5,114) : 598,	(6,114) : 762,	(7,114) : 263,	(8,114) : 530,	(9,114) : 261,	(10,114) : 235,	(11,114) : 392,	(12,114) : 787,	(13,114) : 691,	(14,114) : 108,	(15,114) : 82,	(16,114) : 642,	(17,114) : 275,	(18,114) : 691,	(19,114) : 252,	(20,114) : 326,
(1,115) : 609,	(2,115) : 170,	(3,115) : 453,	(4,115) : 608,	(5,115) : 468,	(6,115) : 710,	(7,115) : 357,	(8,115) : 531,	(9,115) : 191,	(10,115) : 194,	(11,115) : 326,	(12,115) : 689,	(13,115) : 643,	(14,115) : 263,	(15,115) : 208,	(16,115) : 586,	(17,115) : 136,	(18,115) : 558,	(19,115) : 102,	(20,115) : 161,
(1,116) : 576,	(2,116) : 452,	(3,116) : 262,	(4,116) : 564,	(5,116) : 292,	(6,116) : 667,	(7,116) : 582,	(8,116) : 607,	(9,116) : 315,	(10,116) : 398,	(11,116) : 361,	(12,116) : 556,	(13,116) : 616,	(14,116) : 544,	(15,116) : 496,	(16,116) : 554,	(17,116) : 251,	(18,116) : 355,	(19,116) : 216,	(20,116) : 157,
(1,117) : 759,	(2,117) : 502,	(3,117) : 445,	(4,117) : 748,	(5,117) : 475,	(6,117) : 851,	(7,117) : 681,	(8,117) : 770,	(9,117) : 443,	(10,117) : 338,	(11,117) : 524,	(12,117) : 739,	(13,117) : 799,	(14,117) : 594,	(15,117) : 499,	(16,117) : 737,	(17,117) : 219,	(18,117) : 529,	(19,117) : 312,	(20,117) : 179,
(1,118) : 343,	(2,118) : 502,	(3,118) : 604,	(4,118) : 365,	(5,118) : 585,	(6,118) : 391,	(7,118) : 300,	(8,118) : 133,	(9,118) : 420,	(10,118) : 759,	(11,118) : 352,	(12,118) : 564,	(13,118) : 337,	(14,118) : 483,	(15,118) : 622,	(16,118) : 340,	(17,118) : 716,	(18,118) : 650,	(19,118) : 555,	(20,118) : 693,
(1,119) : 211,	(2,119) : 692,	(3,119) : 570,	(4,119) : 233,	(5,119) : 542,	(6,119) : 190,	(7,119) : 524,	(8,119) : 207,	(9,119) : 538,	(10,119) : 918,	(11,119) : 417,	(12,119) : 410,	(13,119) : 178,	(14,119) : 697,	(15,119) : 818,	(16,119) : 226,	(17,119) : 841,	(18,119) : 574,	(19,119) : 666,	(20,119) : 792,
(1,120) : 636,	(2,120) : 430,	(3,120) : 335,	(4,120) : 626,	(5,120) : 364,	(6,120) : 729,	(7,120) : 584,	(8,120) : 648,	(9,120) : 331,	(10,120) : 337,	(11,120) : 402,	(12,120) : 627,	(13,120) : 676,	(14,120) : 523,	(15,120) : 456,	(16,120) : 613,	(17,120) : 192,	(18,120) : 429,	(19,120) : 210,	(20,120) : 106,
(1,121) : 371,	(2,121) : 592,	(3,121) : 29,	(4,121) : 352,	(5,121) : 40,	(6,121) : 440,	(7,121) : 624,	(8,121) : 495,	(9,121) : 374,	(10,121) : 642,	(11,121) : 307,	(12,121) : 284,	(13,121) : 409,	(14,121) : 668,	(15,121) : 680,	(16,121) : 354,	(17,121) : 502,	(18,121) : 79,	(19,121) : 378,	(20,121) : 410,
(1,122) : 233,	(2,122) : 530,	(3,122) : 522,	(4,122) : 256,	(5,122) : 499,	(6,122) : 282,	(7,122) : 357,	(8,122) : 54,	(9,122) : 399,	(10,122) : 768,	(11,122) : 300,	(12,122) : 455,	(13,122) : 226,	(14,122) : 530,	(15,122) : 655,	(16,122) : 232,	(17,122) : 705,	(18,122) : 556,	(19,122) : 533,	(20,122) : 667,
(1,123) : 21,	(2,123) : 641,	(3,123) : 377,	(4,123) : 31,	(5,123) : 347,	(6,123) : 81,	(7,123) : 534,	(8,123) : 232,	(9,123) : 442,	(10,123) : 824,	(11,123) : 303,	(12,123) : 217,	(13,123) : 26,	(14,123) : 674,	(15,123) : 765,	(16,123) : 44,	(17,123) : 722,	(18,123) : 370,	(19,123) : 548,	(20,123) : 656,
(1,124) : 799,	(2,124) : 392,	(3,124) : 542,	(4,124) : 794,	(5,124) : 568,	(6,124) : 898,	(7,124) : 600,	(8,124) : 761,	(9,124) : 415,	(10,124) : 173,	(11,124) : 531,	(12,124) : 826,	(13,124) : 837,	(14,124) : 477,	(15,124) : 356,	(16,124) : 776,	(17,124) : 112,	(18,124) : 640,	(19,124) : 282,	(20,124) : 164,
(1,125) : 574,	(2,125) : 422,	(3,125) : 275,	(4,125) : 563,	(5,125) : 303,	(6,125) : 667,	(7,125) : 556,	(8,125) : 594,	(9,125) : 292,	(10,125) : 371,	(11,125) : 348,	(12,125) : 565,	(13,125) : 614,	(14,125) : 515,	(15,125) : 466,	(16,125) : 552,	(17,125) : 224,	(18,125) : 371,	(19,125) : 188,	(20,125) : 129,
(1,126) : 501,	(2,126) : 399,	(3,126) : 222,	(4,126) : 491,	(5,126) : 247,	(6,126) : 594,	(7,126) : 507,	(8,126) : 522,	(9,126) : 236,	(10,126) : 398,	(11,126) : 276,	(12,126) : 503,	(13,126) : 540,	(14,126) : 488,	(15,126) : 462,	(16,126) : 478,	(17,126) : 256,	(18,126) : 325,	(19,126) : 161,	(20,126) : 165,
(1,127) : 400,	(2,127) : 333,	(3,127) : 553,	(4,127) : 417,	(5,127) : 542,	(6,127) : 478,	(7,127) : 136,	(8,127) : 195,	(9,127) : 288,	(10,127) : 594,	(11,127) : 273,	(12,127) : 599,	(13,127) : 411,	(14,127) : 315,	(15,127) : 453,	(16,127) : 388,	(17,127) : 565,	(18,127) : 622,	(19,127) : 417,	(20,127) : 554,
(1,128) : 532,	(2,128) : 317,	(3,128) : 300,	(4,128) : 525,	(5,128) : 321,	(6,128) : 630,	(7,128) : 448,	(8,128) : 516,	(9,128) : 190,	(10,128) : 317,	(11,128) : 274,	(12,128) : 565,	(13,128) : 570,	(14,128) : 408,	(15,128) : 374,	(16,128) : 508,	(17,128) : 183,	(18,128) : 405,	(19,128) : 79,	(20,128) : 105,
(1,129) : 379,	(2,129) : 694,	(3,129) : 134,	(4,129) : 356,	(5,129) : 123,	(6,129) : 426,	(7,129) : 710,	(8,129) : 541,	(9,129) : 473,	(10,129) : 747,	(11,129) : 389,	(12,129) : 229,	(13,129) : 412,	(14,129) : 767,	(15,129) : 785,	(16,129) : 367,	(17,129) : 606,	(18,129) : 30,	(19,129) : 484,	(20,129) : 513,
(1,130) : 625,	(2,130) : 37,	(3,130) : 545,	(4,130) : 630,	(5,130) : 555,	(6,130) : 725,	(7,130) : 254,	(8,130) : 501,	(9,130) : 218,	(10,130) : 239,	(11,130) : 351,	(12,130) : 745,	(13,130) : 654,	(14,130) : 128,	(15,130) : 119,	(16,130) : 604,	(17,130) : 255,	(18,130) : 647,	(19,130) : 211,	(20,130) : 296,
(1,131) : 116,	(2,131) : 572,	(3,131) : 249,	(4,131) : 100,	(5,131) : 219,	(6,131) : 201,	(7,131) : 508,	(8,131) : 271,	(9,131) : 354,	(10,131) : 723,	(11,131) : 219,	(12,131) : 188,	(13,131) : 155,	(14,131) : 619,	(15,131) : 689,	(16,131) : 98,	(17,131) : 611,	(18,131) : 257,	(19,131) : 441,	(20,131) : 539,
(1,132) : 609,	(2,132) : 88,	(3,132) : 502,	(4,132) : 612,	(5,132) : 513,	(6,132) : 709,	(7,132) : 287,	(8,132) : 502,	(9,132) : 191,	(10,132) : 219,	(11,132) : 329,	(12,132) : 714,	(13,132) : 640,	(14,132) : 181,	(15,132) : 151,	(16,132) : 587,	(17,132) : 208,	(18,132) : 605,	(19,132) : 160,	(20,132) : 243,
(1,133) : 273,	(2,133) : 469,	(3,133) : 520,	(4,133) : 294,	(5,133) : 500,	(6,133) : 335,	(7,133) : 291,	(8,133) : 54,	(9,133) : 354,	(10,133) : 713,	(11,133) : 272,	(12,133) : 490,	(13,133) : 274,	(14,133) : 465,	(15,133) : 593,	(16,133) : 267,	(17,133) : 657,	(18,133) : 565,	(19,133) : 490,	(20,133) : 626,
(1,134) : 224,	(2,134) : 512,	(3,134) : 502,	(4,134) : 246,	(5,134) : 480,	(6,134) : 280,	(7,134) : 345,	(8,134) : 32,	(9,134) : 377,	(10,134) : 747,	(11,134) : 278,	(12,134) : 444,	(13,134) : 221,	(14,134) : 515,	(15,134) : 637,	(16,134) : 221,	(17,134) : 683,	(18,134) : 538,	(19,134) : 511,	(20,134) : 644,
(1,135) : 327,	(2,135) : 576,	(3,135) : 628,	(4,135) : 351,	(5,135) : 605,	(6,135) : 356,	(7,135) : 376,	(8,135) : 153,	(9,135) : 479,	(10,135) : 829,	(11,135) : 396,	(12,135) : 549,	(13,135) : 313,	(14,135) : 559,	(15,135) : 697,	(16,135) : 330,	(17,135) : 780,	(18,135) : 661,	(19,135) : 615,	(20,135) : 751,
(1,136) : 351,	(2,136) : 845,	(3,136) : 327,	(4,136) : 328,	(5,136) : 303,	(6,136) : 343,	(7,136) : 811,	(8,136) : 563,	(9,136) : 618,	(10,136) : 939,	(11,136) : 501,	(12,136) : 129,	(13,136) : 367,	(14,136) : 905,	(15,136) : 950,	(16,136) : 352,	(17,136) : 804,	(18,136) : 231,	(19,136) : 663,	(20,136) : 714,
(1,137) : 45,	(2,137) : 582,	(3,137) : 327,	(4,137) : 45,	(5,137) : 299,	(6,137) : 145,	(7,137) : 488,	(8,137) : 211,	(9,137) : 378,	(10,137) : 759,	(11,137) : 240,	(12,137) : 225,	(13,137) : 84,	(14,137) : 619,	(15,137) : 704,	(16,137) : 22,	(17,137) : 657,	(18,137) : 335,	(19,137) : 483,	(20,137) : 592,
(1,138) : 222,	(2,138) : 490,	(3,138) : 483,	(4,138) : 243,	(5,138) : 462,	(6,138) : 287,	(7,138) : 328,	(8,138) : 7,	(9,138) : 352,	(10,138) : 723,	(11,138) : 253,	(12,138) : 439,	(13,138) : 224,	(14,138) : 495,	(15,138) : 615,	(16,138) : 216,	(17,138) : 658,	(18,138) : 523,	(19,138) : 486,	(20,138) : 619,
(1,139) : 241,	(2,139) : 403,	(3,139) : 254,	(4,139) : 239,	(5,139) : 241,	(6,139) : 342,	(7,139) : 364,	(8,139) : 241,	(9,139) : 185,	(10,139) : 563,	(11,139) : 49,	(12,139) : 350,	(13,139) : 276,	(14,139) : 454,	(15,139) : 519,	(16,139) : 217,	(17,139) : 463,	(18,139) : 322,	(19,139) : 287,	(20,139) : 403,
(1,140) : 165,	(2,140) : 698,	(3,140) : 281,	(4,140) : 141,	(5,140) : 249,	(6,140) : 184,	(7,140) : 636,	(8,140) : 375,	(9,140) : 476,	(10,140) : 833,	(11,140) : 345,	(12,140) : 61,	(13,140) : 186,	(14,140) : 748,	(15,140) : 812,	(16,140) : 164,	(17,140) : 712,	(18,140) : 237,	(19,140) : 550,	(20,140) : 632,
(1,141) : 54,	(2,141) : 571,	(3,141) : 325,	(4,141) : 56,	(5,141) : 297,	(6,141) : 155,	(7,141) : 477,	(8,141) : 204,	(9,141) : 368,	(10,141) : 749,	(11,141) : 229,	(12,141) : 233,	(13,141) : 92,	(14,141) : 608,	(15,141) : 693,	(16,141) : 31,	(17,141) : 648,	(18,141) : 336,	(19,141) : 473,	(20,141) : 583,
(1,142) : 263,	(2,142) : 640,	(3,142) : 600,	(4,142) : 288,	(5,142) : 575,	(6,142) : 270,	(7,142) : 456,	(8,142) : 166,	(9,142) : 511,	(10,142) : 881,	(11,142) : 406,	(12,142) : 479,	(13,142) : 240,	(14,142) : 635,	(15,142) : 765,	(16,142) : 272,	(17,142) : 817,	(18,142) : 619,	(19,142) : 645,	(20,142) : 777,
(1,143) : 203,	(2,143) : 541,	(3,143) : 501,	(4,143) : 226,	(5,143) : 478,	(6,143) : 252,	(7,143) : 377,	(8,143) : 56,	(9,143) : 399,	(10,143) : 773,	(11,143) : 292,	(12,143) : 425,	(13,143) : 196,	(14,143) : 546,	(15,143) : 667,	(16,143) : 202,	(17,143) : 704,	(18,143) : 532,	(19,143) : 531,	(20,143) : 663,
(1,144) : 585,	(2,144) : 570,	(3,144) : 228,	(4,144) : 569,	(5,144) : 261,	(6,144) : 665,	(7,144) : 681,	(8,144) : 658,	(9,144) : 409,	(10,144) : 516,	(11,144) : 422,	(12,144) : 517,	(13,144) : 625,	(14,144) : 660,	(15,144) : 619,	(16,144) : 565,	(17,144) : 370,	(18,144) : 293,	(19,144) : 331,	(20,144) : 278,
(1,145) : 203,	(2,145) : 668,	(3,145) : 216,	(4,145) : 179,	(5,145) : 184,	(6,145) : 243,	(7,145) : 627,	(8,145) : 394,	(9,145) : 443,	(10,145) : 786,	(11,145) : 320,	(12,145) : 88,	(13,145) : 233,	(14,145) : 725,	(15,145) : 778,	(16,145) : 196,	(17,145) : 660,	(18,145) : 173,	(19,145) : 504,	(20,145) : 577,
(1,146) : 406,	(2,146) : 501,	(3,146) : 79,	(4,146) : 391,	(5,146) : 106,	(6,146) : 490,	(7,146) : 559,	(8,146) : 484,	(9,146) : 294,	(10,146) : 539,	(11,146) : 261,	(12,146) : 368,	(13,146) : 446,	(14,146) : 582,	(15,146) : 583,	(16,146) : 386,	(17,146) : 399,	(18,146) : 182,	(19,146) : 278,	(20,146) : 308,
(1,147) : 214,	(2,147) : 677,	(3,147) : 216,	(4,147) : 190,	(5,147) : 184,	(6,147) : 252,	(7,147) : 638,	(8,147) : 406,	(9,147) : 451,	(10,147) : 792,	(11,147) : 330,	(12,147) : 83,	(13,147) : 243,	(14,147) : 735,	(15,147) : 786,	(16,147) : 207,	(17,147) : 665,	(18,147) : 166,	(19,147) : 510,	(20,147) : 581,
(1,148) : 341,	(2,148) : 532,	(3,148) : 618,	(4,148) : 364,	(5,148) : 597,	(6,148) : 381,	(7,148) : 329,	(8,148) : 142,	(9,148) : 445,	(10,148) : 788,	(11,148) : 372,	(12,148) : 563,	(13,148) : 331,	(14,148) : 513,	(15,148) : 652,	(16,148) : 340,	(17,148) : 744,	(18,148) : 659,	(19,148) : 581,	(20,148) : 718,
(1,149) : 419,	(2,149) : 396,	(3,149) : 175,	(4,149) : 409,	(5,149) : 192,	(6,149) : 513,	(7,149) : 468,	(8,149) : 446,	(9,149) : 197,	(10,149) : 446,	(11,149) : 203,	(12,149) : 434,	(13,149) : 459,	(14,149) : 478,	(15,149) : 478,	(16,149) : 396,	(17,149) : 314,	(18,149) : 280,	(19,149) : 175,	(20,149) : 231,
(1,150) : 644,	(2,150) : 461,	(3,150) : 330,	(4,150) : 633,	(5,150) : 360,	(6,150) : 735,	(7,150) : 613,	(8,150) : 665,	(9,150) : 355,	(10,150) : 364,	(11,150) : 419,	(12,150) : 624,	(13,150) : 684,	(14,150) : 554,	(15,150) : 487,	(16,150) : 622,	(17,150) : 220,	(18,150) : 419,	(19,150) : 238,	(20,150) : 137,
(1,151) : 258,	(2,151) : 369,	(3,151) : 298,	(4,151) : 261,	(5,151) : 286,	(6,151) : 359,	(7,151) : 318,	(8,151) : 215,	(9,151) : 163,	(10,151) : 548,	(11,151) : 32,	(12,151) : 389,	(13,151) : 290,	(14,151) : 415,	(15,151) : 489,	(16,151) : 236,	(17,151) : 457,	(18,151) : 369,	(19,151) : 281,	(20,151) : 406,
(1,152) : 317,	(2,152) : 314,	(3,152) : 410,	(4,152) : 328,	(5,152) : 400,	(6,152) : 411,	(7,152) : 210,	(8,152) : 179,	(9,152) : 174,	(10,152) : 538,	(11,152) : 129,	(12,152) : 486,	(13,152) : 340,	(14,152) : 337,	(15,152) : 440,	(16,152) : 299,	(17,152) : 476,	(18,152) : 483,	(19,152) : 309,	(20,152) : 446,
(1,153) : 710,	(2,153) : 152,	(3,153) : 571,	(4,153) : 712,	(5,153) : 586,	(6,153) : 812,	(7,153) : 374,	(8,153) : 608,	(9,153) : 290,	(10,153) : 120,	(11,153) : 429,	(12,153) : 804,	(13,153) : 743,	(14,153) : 226,	(15,153) : 110,	(16,153) : 688,	(17,153) : 171,	(18,153) : 676,	(19,153) : 221,	(20,153) : 243,
(1,154) : 193,	(2,154) : 536,	(3,154) : 175,	(4,154) : 178,	(5,154) : 148,	(6,154) : 278,	(7,154) : 503,	(8,154) : 314,	(9,154) : 311,	(10,154) : 665,	(11,154) : 189,	(12,154) : 216,	(13,154) : 233,	(14,154) : 593,	(15,154) : 647,	(16,154) : 173,	(17,154) : 546,	(18,154) : 204,	(19,154) : 381,	(20,154) : 469,
(1,155) : 246,	(2,155) : 698,	(3,155) : 212,	(4,155) : 222,	(5,155) : 182,	(6,155) : 278,	(7,155) : 667,	(8,155) : 439,	(9,155) : 472,	(10,155) : 804,	(11,155) : 355,	(12,155) : 84,	(13,155) : 274,	(14,155) : 758,	(15,155) : 805,	(16,155) : 240,	(17,155) : 674,	(18,155) : 148,	(19,155) : 524,	(20,155) : 588,
(1,156) : 590,	(2,156) : 37,	(3,156) : 526,	(4,156) : 595,	(5,156) : 533,	(6,156) : 689,	(7,156) : 222,	(8,156) : 462,	(9,156) : 190,	(10,156) : 275,	(11,156) : 319,	(12,156) : 715,	(13,156) : 618,	(14,156) : 124,	(15,156) : 155,	(16,156) : 568,	(17,156) : 276,	(18,156) : 627,	(19,156) : 206,	(20,156) : 306,
(1,157) : 336,	(2,157) : 317,	(3,157) : 275,	(4,157) : 335,	(5,157) : 273,	(6,157) : 437,	(7,157) : 322,	(8,157) : 298,	(9,157) : 93,	(10,157) : 467,	(11,157) : 57,	(12,157) : 434,	(13,157) : 371,	(14,157) : 379,	(15,157) : 429,	(16,157) : 313,	(17,157) : 371,	(18,157) : 364,	(19,157) : 195,	(20,157) : 318,
(1,158) : 284,	(2,158) : 498,	(3,158) : 550,	(4,158) : 307,	(5,158) : 530,	(6,158) : 337,	(7,158) : 313,	(8,158) : 75,	(9,158) : 390,	(10,158) : 746,	(11,158) : 308,	(12,158) : 505,	(13,158) : 280,	(14,158) : 491,	(15,158) : 622,	(16,158) : 281,	(17,158) : 692,	(18,158) : 592,	(19,158) : 526,	(20,158) : 662,
(1,159) : 247,	(2,159) : 638,	(3,159) : 586,	(4,159) : 271,	(5,159) : 560,	(6,159) : 254,	(7,159) : 458,	(8,159) : 159,	(9,159) : 504,	(10,159) : 876,	(11,159) : 396,	(12,159) : 462,	(13,159) : 223,	(14,159) : 635,	(15,159) : 763,	(16,159) : 255,	(17,159) : 810,	(18,159) : 603,	(19,159) : 637,	(20,159) : 768,
(1,160) : 227,	(2,160) : 576,	(3,160) : 543,	(4,160) : 252,	(5,160) : 518,	(6,160) : 260,	(7,160) : 401,	(8,160) : 96,	(9,160) : 442,	(10,160) : 813,	(11,160) : 337,	(12,160) : 449,	(13,160) : 213,	(14,160) : 575,	(15,160) : 701,	(16,160) : 231,	(17,160) : 747,	(18,160) : 569,	(19,160) : 575,	(20,160) : 707,
(1,161) : 551,	(2,161) : 372,	(3,161) : 282,	(4,161) : 542,	(5,161) : 306,	(6,161) : 647,	(7,161) : 504,	(8,161) : 555,	(9,161) : 242,	(10,161) : 342,	(11,161) : 309,	(12,161) : 562,	(13,161) : 590,	(14,161) : 463,	(15,161) : 421,	(16,161) : 528,	(17,161) : 198,	(18,161) : 383,	(19,161) : 136,	(20,161) : 105,
(1,162) : 493,	(2,162) : 205,	(3,162) : 561,	(4,162) : 506,	(5,162) : 557,	(6,162) : 581,	(7,162) : 37,	(8,162) : 313,	(9,162) : 236,	(10,162) : 473,	(11,162) : 288,	(12,162) : 667,	(13,162) : 511,	(14,162) : 183,	(15,162) : 321,	(16,162) : 476,	(17,162) : 463,	(18,162) : 646,	(19,162) : 340,	(20,162) : 470,
(1,163) : 564,	(2,163) : 256,	(3,163) : 363,	(4,163) : 560,	(5,163) : 382,	(6,163) : 664,	(7,163) : 409,	(8,163) : 521,	(9,163) : 178,	(10,163) : 257,	(11,163) : 290,	(12,163) : 618,	(13,163) : 600,	(14,163) : 349,	(15,163) : 306,	(16,163) : 540,	(17,163) : 138,	(18,163) : 469,	(19,163) : 42,	(20,163) : 97,
(1,164) : 629,	(2,164) : 122,	(3,164) : 639,	(4,164) : 639,	(5,164) : 641,	(6,164) : 721,	(7,164) : 143,	(8,164) : 458,	(9,164) : 293,	(10,164) : 381,	(11,164) : 392,	(12,164) : 787,	(13,164) : 650,	(14,164) : 45,	(15,164) : 199,	(16,164) : 611,	(17,164) : 414,	(18,164) : 734,	(19,164) : 346,	(20,164) : 450,
(1,165) : 204,	(2,165) : 667,	(3,165) : 215,	(4,165) : 180,	(5,165) : 183,	(6,165) : 245,	(7,165) : 627,	(8,165) : 395,	(9,165) : 442,	(10,165) : 785,	(11,165) : 319,	(12,165) : 89,	(13,165) : 234,	(14,165) : 724,	(15,165) : 777,	(16,165) : 196,	(17,165) : 659,	(18,165) : 171,	(19,165) : 503,	(20,165) : 576,
(1,166) : 683,	(2,166) : 101,	(3,166) : 659,	(4,166) : 692,	(5,166) : 665,	(6,166) : 778,	(7,166) : 214,	(8,166) : 522,	(9,166) : 316,	(10,166) : 323,	(11,166) : 431,	(12,166) : 830,	(13,166) : 707,	(14,166) : 31,	(15,166) : 132,	(16,166) : 664,	(17,166) : 377,	(18,166) : 758,	(19,166) : 340,	(20,166) : 426,
(1,167) : 537,	(2,167) : 135,	(3,167) : 432,	(4,167) : 539,	(5,167) : 441,	(6,167) : 638,	(7,167) : 277,	(8,167) : 444,	(9,167) : 117,	(10,167) : 274,	(11,167) : 256,	(12,167) : 639,	(13,167) : 570,	(14,167) : 222,	(15,167) : 225,	(16,167) : 515,	(17,167) : 222,	(18,167) : 534,	(19,167) : 108,	(20,167) : 226,
(1,168) : 480,	(2,168) : 316,	(3,168) : 267,	(4,168) : 473,	(5,168) : 284,	(6,168) : 578,	(7,168) : 422,	(8,168) : 466,	(9,168) : 153,	(10,168) : 356,	(11,168) : 222,	(12,168) : 520,	(13,168) : 518,	(14,168) : 403,	(15,168) : 389,	(16,168) : 456,	(17,168) : 229,	(18,168) : 372,	(19,168) : 84,	(20,168) : 157,
(1,169) : 111,	(2,169) : 513,	(3,169) : 353,	(4,169) : 123,	(5,169) : 328,	(6,169) : 207,	(7,169) : 406,	(8,169) : 138,	(9,169) : 322,	(10,169) : 707,	(11,169) : 188,	(12,169) : 305,	(13,169) : 136,	(14,169) : 543,	(15,169) : 637,	(16,169) : 94,	(17,169) : 616,	(18,169) : 383,	(19,169) : 439,	(20,169) : 559,
(1,170) : 46,	(2,170) : 613,	(3,170) : 320,	(4,170) : 26,	(5,170) : 290,	(6,170) : 129,	(7,170) : 523,	(8,170) : 246,	(9,170) : 404,	(10,170) : 782,	(11,170) : 265,	(12,170) : 190,	(13,170) : 83,	(14,170) : 652,	(15,170) : 734,	(16,170) : 36,	(17,170) : 675,	(18,170) : 317,	(19,170) : 503,	(20,170) : 607,
(1,171) : 595,	(2,171) : 292,	(3,171) : 368,	(4,171) : 589,	(5,171) : 389,	(6,171) : 694,	(7,171) : 454,	(8,171) : 563,	(9,171) : 222,	(10,171) : 249,	(11,171) : 327,	(12,171) : 635,	(13,171) : 632,	(14,171) : 385,	(15,171) : 328,	(16,171) : 571,	(17,171) : 113,	(18,171) : 472,	(19,171) : 86,	(20,171) : 52,
(1,172) : 335,	(2,172) : 575,	(3,172) : 634,	(4,172) : 359,	(5,172) : 611,	(6,172) : 364,	(7,172) : 373,	(8,172) : 158,	(9,172) : 481,	(10,172) : 829,	(11,172) : 400,	(12,172) : 557,	(13,172) : 321,	(14,172) : 557,	(15,172) : 696,	(16,172) : 337,	(17,172) : 782,	(18,172) : 668,	(19,172) : 617,	(20,172) : 754,
(1,173) : 632,	(2,173) : 475,	(3,173) : 311,	(4,173) : 620,	(5,173) : 342,	(6,173) : 722,	(7,173) : 620,	(8,173) : 661,	(9,173) : 359,	(10,173) : 387,	(11,173) : 415,	(12,173) : 606,	(13,173) : 672,	(14,173) : 569,	(15,173) : 506,	(16,173) : 610,	(17,173) : 242,	(18,173) : 398,	(19,173) : 248,	(20,173) : 156,
(1,174) : 93,	(2,174) : 600,	(3,174) : 273,	(4,174) : 73,	(5,174) : 243,	(6,174) : 170,	(7,174) : 528,	(8,174) : 273,	(9,174) : 384,	(10,174) : 756,	(11,174) : 248,	(12,174) : 170,	(13,174) : 130,	(14,174) : 645,	(15,174) : 718,	(16,174) : 78,	(17,174) : 644,	(18,174) : 271,	(19,174) : 474,	(20,174) : 572,
(1,175) : 71,	(2,175) : 656,	(3,175) : 327,	(4,175) : 47,	(5,175) : 295,	(6,175) : 111,	(7,175) : 569,	(8,175) : 287,	(9,175) : 444,	(10,175) : 819,	(11,175) : 306,	(12,175) : 151,	(13,175) : 93,	(14,175) : 696,	(15,175) : 776,	(16,175) : 75,	(17,175) : 708,	(18,175) : 309,	(19,175) : 538,	(20,175) : 636,
(1,176) : 106,	(2,176) : 592,	(3,176) : 443,	(4,176) : 130,	(5,176) : 416,	(6,176) : 154,	(7,176) : 453,	(8,176) : 132,	(9,176) : 417,	(10,176) : 802,	(11,176) : 289,	(12,176) : 328,	(13,176) : 95,	(14,176) : 610,	(15,176) : 717,	(16,176) : 111,	(17,176) : 716,	(18,176) : 457,	(19,176) : 540,	(20,176) : 663,
(1,177) : 199,	(2,177) : 494,	(3,177) : 461,	(4,177) : 219,	(5,177) : 439,	(6,177) : 267,	(7,177) : 341,	(8,177) : 21,	(9,177) : 346,	(10,177) : 721,	(11,177) : 240,	(12,177) : 415,	(13,177) : 202,	(14,177) : 504,	(15,177) : 620,	(16,177) : 192,	(17,177) : 651,	(18,177) : 499,	(19,177) : 478,	(20,177) : 610,
(1,178) : 185,	(2,178) : 482,	(3,178) : 433,	(4,178) : 204,	(5,178) : 412,	(6,178) : 262,	(7,178) : 341,	(8,178) : 44,	(9,178) : 325,	(10,178) : 703,	(11,178) : 214,	(12,178) : 396,	(13,178) : 194,	(14,178) : 497,	(15,178) : 608,	(16,178) : 175,	(17,178) : 629,	(18,178) : 473,	(19,178) : 455,	(20,178) : 586,
(1,179) : 30,	(2,179) : 628,	(3,179) : 342,	(4,179) : 6,	(5,179) : 311,	(6,179) : 107,	(7,179) : 532,	(8,179) : 245,	(9,179) : 422,	(10,179) : 802,	(11,179) : 284,	(12,179) : 194,	(13,179) : 62,	(14,179) : 665,	(15,179) : 750,	(16,179) : 33,	(17,179) : 697,	(18,179) : 335,	(19,179) : 524,	(20,179) : 629,
(1,180) : 717,	(2,180) : 131,	(3,180) : 692,	(4,180) : 727,	(5,180) : 698,	(6,180) : 812,	(7,180) : 241,	(8,180) : 554,	(9,180) : 350,	(10,180) : 326,	(11,180) : 466,	(12,180) : 865,	(13,180) : 741,	(14,180) : 64,	(15,180) : 130,	(16,180) : 698,	(17,180) : 393,	(18,180) : 791,	(19,180) : 368,	(20,180) : 448,
(1,181) : 104,	(2,181) : 617,	(3,181) : 269,	(4,181) : 81,	(5,181) : 238,	(6,181) : 170,	(7,181) : 548,	(8,181) : 293,	(9,181) : 400,	(10,181) : 768,	(11,181) : 264,	(12,181) : 149,	(13,181) : 138,	(14,181) : 664,	(15,181) : 734,	(16,181) : 93,	(17,181) : 654,	(18,181) : 259,	(19,181) : 486,	(20,181) : 580,
(1,182) : 111,	(2,182) : 588,	(3,182) : 253,	(4,182) : 93,	(5,182) : 223,	(6,182) : 191,	(7,182) : 523,	(8,182) : 279,	(9,182) : 370,	(10,182) : 739,	(11,182) : 235,	(12,182) : 173,	(13,182) : 149,	(14,182) : 635,	(15,182) : 705,	(16,182) : 95,	(17,182) : 625,	(18,182) : 255,	(19,182) : 456,	(20,182) : 552,
(1,183) : 181,	(2,183) : 734,	(3,183) : 312,	(4,183) : 158,	(5,183) : 281,	(6,183) : 179,	(7,183) : 668,	(8,183) : 397,	(9,183) : 513,	(10,183) : 871,	(11,183) : 381,	(12,183) : 45,	(13,183) : 195,	(14,183) : 783,	(15,183) : 849,	(16,183) : 185,	(17,183) : 749,	(18,183) : 259,	(19,183) : 588,	(20,183) : 669,
(1,184) : 195,	(2,184) : 462,	(3,184) : 423,	(4,184) : 212,	(5,184) : 402,	(6,184) : 276,	(7,184) : 325,	(8,184) : 55,	(9,184) : 304,	(10,184) : 682,	(11,184) : 195,	(12,184) : 400,	(13,184) : 207,	(14,184) : 478,	(15,184) : 588,	(16,184) : 183,	(17,184) : 608,	(18,184) : 466,	(19,184) : 435,	(20,184) : 565,
(1,185) : 315,	(2,185) : 583,	(3,185) : 620,	(4,185) : 339,	(5,185) : 597,	(6,185) : 341,	(7,185) : 386,	(8,185) : 149,	(9,185) : 481,	(10,185) : 835,	(11,185) : 393,	(12,185) : 536,	(13,185) : 299,	(14,185) : 569,	(15,185) : 705,	(16,185) : 318,	(17,185) : 783,	(18,185) : 652,	(19,185) : 616,	(20,185) : 752,
(1,186) : 325,	(2,186) : 818,	(3,186) : 307,	(4,186) : 301,	(5,186) : 281,	(6,186) : 322,	(7,186) : 783,	(8,186) : 536,	(9,186) : 591,	(10,186) : 916,	(11,186) : 474,	(12,186) : 103,	(13,186) : 342,	(14,186) : 878,	(15,186) : 924,	(16,186) : 326,	(17,186) : 781,	(18,186) : 215,	(19,186) : 638,	(20,186) : 693,
(1,187) : 92,	(2,187) : 641,	(3,187) : 293,	(4,187) : 68,	(5,187) : 262,	(6,187) : 145,	(7,187) : 565,	(8,187) : 297,	(9,187) : 425,	(10,187) : 795,	(11,187) : 289,	(12,187) : 138,	(13,187) : 122,	(14,187) : 685,	(15,187) : 759,	(16,187) : 88,	(17,187) : 682,	(18,187) : 276,	(19,187) : 513,	(20,187) : 608,
(1,188) : 361,	(2,188) : 825,	(3,188) : 295,	(4,188) : 337,	(5,188) : 273,	(6,188) : 364,	(7,188) : 802,	(8,188) : 567,	(9,188) : 599,	(10,188) : 910,	(11,188) : 487,	(12,188) : 142,	(13,188) : 380,	(14,188) : 889,	(15,188) : 928,	(16,188) : 360,	(17,188) : 773,	(18,188) : 196,	(19,188) : 636,	(20,188) : 682,
(1,189) : 662,	(2,189) : 149,	(3,189) : 516,	(4,189) : 663,	(5,189) : 531,	(6,189) : 764,	(7,189) : 361,	(8,189) : 570,	(9,189) : 242,	(10,189) : 147,	(11,189) : 380,	(12,189) : 750,	(13,189) : 695,	(14,189) : 237,	(15,189) : 152,	(16,189) : 640,	(17,189) : 144,	(18,189) : 621,	(19,189) : 165,	(20,189) : 200,
(1,190) : 626,	(2,190) : 362,	(3,190) : 359,	(4,190) : 618,	(5,190) : 385,	(6,190) : 723,	(7,190) : 525,	(8,190) : 615,	(9,190) : 283,	(10,190) : 278,	(11,190) : 373,	(12,190) : 642,	(13,190) : 665,	(14,190) : 456,	(15,190) : 388,	(16,190) : 603,	(17,190) : 131,	(18,190) : 459,	(19,190) : 153,	(20,190) : 38,
(1,191) : 292,	(2,191) : 612,	(3,191) : 613,	(4,191) : 316,	(5,191) : 589,	(6,191) : 310,	(7,191) : 420,	(8,191) : 155,	(9,191) : 497,	(10,191) : 859,	(11,191) : 401,	(12,191) : 511,	(13,191) : 273,	(14,191) : 601,	(15,191) : 735,	(16,191) : 297,	(17,191) : 801,	(18,191) : 639,	(19,191) : 632,	(20,191) : 766,
(1,192) : 252,	(2,192) : 523,	(3,192) : 535,	(4,192) : 274,	(5,192) : 513,	(6,192) : 300,	(7,192) : 345,	(8,192) : 61,	(9,192) : 400,	(10,192) : 764,	(11,192) : 306,	(12,192) : 473,	(13,192) : 245,	(14,192) : 520,	(15,192) : 648,	(16,192) : 250,	(17,192) : 705,	(18,192) : 571,	(19,192) : 535,	(20,192) : 669,
(1,193) : 597,	(2,193) : 51,	(3,193) : 517,	(4,193) : 602,	(5,193) : 526,	(6,193) : 697,	(7,193) : 246,	(8,193) : 477,	(9,193) : 189,	(10,193) : 252,	(11,193) : 322,	(12,193) : 715,	(13,193) : 627,	(14,193) : 143,	(15,193) : 148,	(16,193) : 576,	(17,193) : 251,	(18,193) : 619,	(19,193) : 188,	(20,193) : 283,
(1,194) : 585,	(2,194) : 307,	(3,194) : 351,	(4,194) : 579,	(5,194) : 373,	(6,194) : 684,	(7,194) : 463,	(8,194) : 560,	(9,194) : 223,	(10,194) : 267,	(11,194) : 322,	(12,194) : 620,	(13,194) : 623,	(14,194) : 400,	(15,194) : 346,	(16,194) : 562,	(17,194) : 128,	(18,194) : 454,	(19,194) : 90,	(20,194) : 53,
(1,195) : 286,	(2,195) : 741,	(3,195) : 233,	(4,195) : 261,	(5,195) : 206,	(6,195) : 306,	(7,195) : 713,	(8,195) : 484,	(9,195) : 514,	(10,195) : 838,	(11,195) : 400,	(12,195) : 89,	(13,195) : 310,	(14,195) : 802,	(15,195) : 846,	(16,195) : 281,	(17,195) : 704,	(18,195) : 150,	(19,195) : 560,	(20,195) : 616,
(1,196) : 57,	(2,196) : 645,	(3,196) : 330,	(4,196) : 33,	(5,196) : 298,	(6,196) : 110,	(7,196) : 555,	(8,196) : 272,	(9,196) : 434,	(10,196) : 811,	(11,196) : 296,	(12,196) : 166,	(13,196) : 82,	(14,196) : 684,	(15,196) : 765,	(16,196) : 60,	(17,196) : 702,	(18,196) : 316,	(19,196) : 531,	(20,196) : 631,
(1,197) : 188,	(2,197) : 647,	(3,197) : 212,	(4,197) : 164,	(5,197) : 180,	(6,197) : 236,	(7,197) : 604,	(8,197) : 373,	(9,197) : 422,	(10,197) : 770,	(11,197) : 298,	(12,197) : 105,	(13,197) : 219,	(14,197) : 703,	(15,197) : 758,	(16,197) : 178,	(17,197) : 645,	(18,197) : 180,	(19,197) : 487,	(20,197) : 564,
(1,198) : 219,	(2,198) : 485,	(3,198) : 477,	(4,198) : 240,	(5,198) : 455,	(6,198) : 286,	(7,198) : 326,	(8,198) : 0,	(9,198) : 346,	(10,198) : 717,	(11,198) : 246,	(12,198) : 436,	(13,198) : 222,	(14,198) : 492,	(15,198) : 611,	(16,198) : 213,	(17,198) : 651,	(18,198) : 517,	(19,198) : 479,	(20,198) : 613,
(1,199) : 618,	(2,199) : 446,	(3,199) : 309,	(4,199) : 607,	(5,199) : 338,	(6,199) : 710,	(7,199) : 592,	(8,199) : 639,	(9,199) : 332,	(10,199) : 366,	(11,199) : 393,	(12,199) : 602,	(13,199) : 658,	(14,199) : 540,	(15,199) : 479,	(16,199) : 596,	(17,199) : 220,	(18,199) : 400,	(19,199) : 219,	(20,199) : 130,
(1,200) : 216,	(2,200) : 461,	(3,200) : 451,	(4,200) : 235,	(5,200) : 431,	(6,200) : 292,	(7,200) : 312,	(8,200) : 31,	(9,200) : 316,	(10,200) : 689,	(11,200) : 216,	(12,200) : 426,	(13,200) : 225,	(14,200) : 472,	(15,200) : 587,	(16,200) : 206,	(17,200) : 621,	(18,200) : 496,	(19,200) : 449,	(20,200) : 582,
(1,201) : 214,	(2,201) : 722,	(3,201) : 266,	(4,201) : 190,	(5,201) : 234,	(6,201) : 229,	(7,201) : 672,	(8,201) : 421,	(9,201) : 497,	(10,201) : 843,	(11,201) : 371,	(12,201) : 31,	(13,201) : 235,	(14,201) : 776,	(15,201) : 833,	(16,201) : 212,	(17,201) : 717,	(18,201) : 206,	(19,201) : 561,	(20,201) : 633,
(1,202) : 160,	(2,202) : 563,	(3,202) : 204,	(4,202) : 143,	(5,202) : 175,	(6,202) : 241,	(7,202) : 517,	(8,202) : 303,	(9,202) : 340,	(10,202) : 700,	(11,202) : 212,	(12,202) : 187,	(13,202) : 199,	(14,202) : 617,	(15,202) : 677,	(16,202) : 142,	(17,202) : 583,	(18,202) : 217,	(19,202) : 417,	(20,202) : 507,
(1,203) : 647,	(2,203) : 38,	(3,203) : 568,	(4,203) : 652,	(5,203) : 578,	(6,203) : 746,	(7,203) : 261,	(8,203) : 519,	(9,203) : 242,	(10,203) : 233,	(11,203) : 374,	(12,203) : 768,	(13,203) : 676,	(14,203) : 118,	(15,203) : 97,	(16,203) : 626,	(17,203) : 263,	(18,203) : 671,	(19,203) : 232,	(20,203) : 310,
(1,204) : 244,	(2,204) : 491,	(3,204) : 506,	(4,204) : 265,	(5,204) : 485,	(6,204) : 304,	(7,204) : 320,	(8,204) : 30,	(9,204) : 363,	(10,204) : 729,	(11,204) : 271,	(12,204) : 462,	(13,204) : 243,	(14,204) : 491,	(15,204) : 616,	(16,204) : 238,	(17,204) : 668,	(18,204) : 547,	(19,204) : 498,	(20,204) : 633,
(1,205) : 609,	(2,205) : 427,	(3,205) : 308,	(4,205) : 599,	(5,205) : 337,	(6,205) : 702,	(7,205) : 573,	(8,205) : 625,	(9,205) : 314,	(10,205) : 352,	(11,205) : 379,	(12,205) : 600,	(13,205) : 649,	(14,205) : 520,	(15,205) : 461,	(16,205) : 587,	(17,205) : 205,	(18,205) : 403,	(19,205) : 200,	(20,205) : 114,
(1,206) : 182,	(2,206) : 439,	(3,206) : 328,	(4,206) : 190,	(5,206) : 309,	(6,206) : 281,	(7,206) : 349,	(8,206) : 151,	(9,206) : 246,	(10,206) : 630,	(11,206) : 115,	(12,206) : 347,	(13,206) : 211,	(14,206) : 474,	(15,206) : 562,	(16,206) : 162,	(17,206) : 542,	(18,206) : 378,	(19,206) : 366,	(20,206) : 489,
(1,207) : 215,	(2,207) : 618,	(3,207) : 167,	(4,207) : 194,	(5,207) : 135,	(6,207) : 275,	(7,207) : 590,	(8,207) : 381,	(9,207) : 392,	(10,207) : 730,	(11,207) : 275,	(12,207) : 144,	(13,207) : 250,	(14,207) : 678,	(15,207) : 726,	(16,207) : 202,	(17,207) : 604,	(18,207) : 147,	(19,207) : 448,	(20,207) : 521,
(1,208) : 303,	(2,208) : 593,	(3,208) : 615,	(4,208) : 327,	(5,208) : 591,	(6,208) : 327,	(7,208) : 398,	(8,208) : 147,	(9,208) : 485,	(10,208) : 842,	(11,208) : 393,	(12,208) : 524,	(13,208) : 286,	(14,208) : 580,	(15,208) : 716,	(16,208) : 307,	(17,208) : 788,	(18,208) : 644,	(19,208) : 620,	(20,208) : 755,
(1,209) : 375,	(2,209) : 657,	(3,209) : 96,	(4,209) : 353,	(5,209) : 89,	(6,209) : 431,	(7,209) : 680,	(8,209) : 525,	(9,209) : 438,	(10,209) : 709,	(11,209) : 359,	(12,209) : 247,	(13,209) : 410,	(14,209) : 732,	(15,209) : 747,	(16,209) : 361,	(17,209) : 568,	(18,209) : 17,	(19,209) : 446,	(20,209) : 475,
(1,210) : 323,	(2,210) : 325,	(3,210) : 444,	(4,210) : 335,	(5,210) : 432,	(6,210) : 412,	(7,210) : 194,	(8,210) : 160,	(9,210) : 207,	(10,210) : 561,	(11,210) : 164,	(12,210) : 504,	(13,210) : 342,	(14,210) : 339,	(15,210) : 451,	(16,210) : 306,	(17,210) : 506,	(18,210) : 513,	(19,210) : 342,	(20,210) : 480,
(1,211) : 571,	(2,211) : 54,	(3,211) : 535,	(4,211) : 578,	(5,211) : 540,	(6,211) : 668,	(7,211) : 179,	(8,211) : 431,	(9,211) : 191,	(10,211) : 316,	(11,211) : 308,	(12,211) : 707,	(13,211) : 597,	(14,211) : 105,	(15,211) : 180,	(16,211) : 550,	(17,211) : 317,	(18,211) : 633,	(19,211) : 233,	(20,211) : 342,
(1,212) : 644,	(2,212) : 38,	(3,212) : 565,	(4,212) : 649,	(5,212) : 575,	(6,212) : 744,	(7,212) : 260,	(8,212) : 516,	(9,212) : 239,	(10,212) : 233,	(11,212) : 371,	(12,212) : 765,	(13,212) : 673,	(14,212) : 120,	(15,212) : 100,	(16,212) : 623,	(17,212) : 261,	(18,212) : 668,	(19,212) : 229,	(20,212) : 307,
(1,213) : 612,	(2,213) : 117,	(3,213) : 486,	(4,213) : 613,	(5,213) : 499,	(6,213) : 713,	(7,213) : 314,	(8,213) : 515,	(9,213) : 191,	(10,213) : 203,	(11,213) : 330,	(12,213) : 708,	(13,213) : 644,	(14,213) : 211,	(15,213) : 166,	(16,213) : 589,	(17,213) : 179,	(18,213) : 591,	(19,213) : 139,	(20,213) : 213,
(1,214) : 189,	(2,214) : 579,	(3,214) : 177,	(4,214) : 170,	(5,214) : 146,	(6,214) : 263,	(7,214) : 544,	(8,214) : 339,	(9,214) : 354,	(10,214) : 703,	(11,214) : 232,	(12,214) : 174,	(13,214) : 227,	(14,214) : 636,	(15,214) : 689,	(16,214) : 172,	(17,214) : 581,	(18,214) : 182,	(19,214) : 420,	(20,214) : 502,
(1,215) : 267,	(2,215) : 749,	(3,215) : 257,	(4,215) : 242,	(5,215) : 228,	(6,215) : 280,	(7,215) : 712,	(8,215) : 471,	(9,215) : 523,	(10,215) : 855,	(11,215) : 403,	(12,215) : 59,	(13,215) : 288,	(14,215) : 808,	(15,215) : 856,	(16,215) : 264,	(17,215) : 723,	(18,215) : 179,	(19,215) : 575,	(20,215) : 637,
(1,216) : 523,	(2,216) : 454,	(3,216) : 209,	(4,216) : 511,	(5,216) : 237,	(6,216) : 613,	(7,216) : 564,	(8,216) : 564,	(9,216) : 292,	(10,216) : 432,	(11,216) : 320,	(12,216) : 501,	(13,216) : 563,	(14,216) : 544,	(15,216) : 512,	(16,216) : 501,	(17,216) : 286,	(18,216) : 304,	(19,216) : 215,	(20,216) : 191,
(1,217) : 101,	(2,217) : 548,	(3,217) : 401,	(4,217) : 122,	(5,217) : 375,	(6,217) : 179,	(7,217) : 422,	(8,217) : 119,	(9,217) : 368,	(10,217) : 752,	(11,217) : 238,	(12,217) : 318,	(13,217) : 110,	(14,217) : 571,	(15,217) : 673,	(16,217) : 94,	(17,217) : 665,	(18,217) : 423,	(19,217) : 489,	(20,217) : 611,
(1,218) : 160,	(2,218) : 601,	(3,218) : 497,	(4,218) : 185,	(5,218) : 470,	(6,218) : 189,	(7,218) : 445,	(8,218) : 119,	(9,218) : 443,	(10,218) : 824,	(11,218) : 323,	(12,218) : 380,	(13,218) : 142,	(14,218) : 611,	(15,218) : 727,	(16,218) : 167,	(17,218) : 746,	(18,218) : 513,	(19,218) : 571,	(20,218) : 698,
(1,219) : 138,	(2,219) : 489,	(3,219) : 363,	(4,219) : 151,	(5,219) : 340,	(6,219) : 231,	(7,219) : 378,	(8,219) : 118,	(9,219) : 304,	(10,219) : 689,	(11,219) : 175,	(12,219) : 332,	(13,219) : 160,	(14,219) : 517,	(15,219) : 613,	(16,219) : 122,	(17,219) : 602,	(18,219) : 399,	(19,219) : 425,	(20,219) : 549,
(1,220) : 210,	(2,220) : 532,	(3,220) : 502,	(4,220) : 233,	(5,220) : 479,	(6,220) : 261,	(7,220) : 367,	(8,220) : 48,	(9,220) : 392,	(10,220) : 765,	(11,220) : 288,	(12,220) : 432,	(13,220) : 204,	(14,220) : 536,	(15,220) : 658,	(16,220) : 208,	(17,220) : 698,	(18,220) : 534,	(19,220) : 525,	(20,220) : 657,
(1,221) : 218,	(2,221) : 582,	(3,221) : 537,	(4,221) : 242,	(5,221) : 512,	(6,221) : 249,	(7,221) : 409,	(8,221) : 99,	(9,221) : 444,	(10,221) : 817,	(11,221) : 336,	(12,221) : 439,	(13,221) : 202,	(14,221) : 583,	(15,221) : 707,	(16,221) : 221,	(17,221) : 749,	(18,221) : 561,	(19,221) : 576,	(20,221) : 708,
(1,222) : 498,	(2,222) : 148,	(3,222) : 518,	(4,222) : 508,	(5,222) : 518,	(6,222) : 592,	(7,222) : 103,	(8,222) : 342,	(9,222) : 179,	(10,222) : 409,	(11,222) : 260,	(12,222) : 654,	(13,222) : 521,	(14,222) : 156,	(15,222) : 272,	(16,222) : 479,	(17,222) : 393,	(18,222) : 609,	(19,222) : 272,	(20,222) : 400,
(1,223) : 421,	(2,223) : 227,	(3,223) : 346,	(4,223) : 422,	(5,223) : 349,	(6,223) : 522,	(7,223) : 273,	(8,223) : 346,	(9,223) : 0,	(10,223) : 385,	(11,223) : 139,	(12,223) : 526,	(13,223) : 453,	(14,223) : 295,	(15,223) : 336,	(16,223) : 398,	(17,223) : 305,	(18,223) : 442,	(19,223) : 136,	(20,223) : 273,
(1,224) : 664,	(2,224) : 580,	(3,224) : 312,	(4,224) : 649,	(5,224) : 344,	(6,224) : 746,	(7,224) : 715,	(8,224) : 724,	(9,224) : 447,	(10,224) : 486,	(11,224) : 481,	(12,224) : 601,	(13,224) : 704,	(14,224) : 673,	(15,224) : 612,	(16,224) : 643,	(17,224) : 344,	(18,224) : 377,	(19,224) : 348,	(20,224) : 262,
(1,225) : 701,	(2,225) : 340,	(3,225) : 452,	(4,225) : 695,	(5,225) : 477,	(6,225) : 800,	(7,225) : 531,	(8,225) : 668,	(9,225) : 324,	(10,225) : 194,	(11,225) : 435,	(12,225) : 731,	(13,225) : 739,	(14,225) : 430,	(15,225) : 335,	(16,225) : 678,	(17,225) : 55,	(18,225) : 552,	(19,225) : 188,	(20,225) : 67,
(1,226) : 608,	(2,226) : 290,	(3,226) : 381,	(4,226) : 602,	(5,226) : 403,	(6,226) : 707,	(7,226) : 458,	(8,226) : 573,	(9,226) : 231,	(10,226) : 236,	(11,226) : 339,	(12,226) : 648,	(13,226) : 645,	(14,226) : 383,	(15,226) : 320,	(16,226) : 584,	(17,226) : 99,	(18,226) : 485,	(19,226) : 95,	(20,226) : 45,
(1,227) : 565,	(2,227) : 152,	(3,227) : 432,	(4,227) : 565,	(5,227) : 445,	(6,227) : 666,	(7,227) : 316,	(8,227) : 481,	(9,227) : 145,	(10,227) : 240,	(11,227) : 282,	(12,227) : 655,	(13,227) : 598,	(14,227) : 244,	(15,227) : 220,	(16,227) : 542,	(17,227) : 182,	(18,227) : 536,	(19,227) : 89,	(20,227) : 189,
(1,228) : 447,	(2,228) : 340,	(3,228) : 234,	(4,228) : 440,	(5,228) : 249,	(6,228) : 545,	(7,228) : 427,	(8,228) : 446,	(9,228) : 155,	(10,228) : 392,	(11,228) : 199,	(12,228) : 483,	(13,228) : 485,	(14,228) : 424,	(15,228) : 419,	(16,228) : 424,	(17,228) : 266,	(18,228) : 339,	(19,228) : 117,	(20,228) : 190,
(1,229) : 355,	(2,229) : 465,	(3,229) : 594,	(4,229) : 376,	(5,229) : 576,	(6,229) : 411,	(7,229) : 262,	(8,229) : 137,	(9,229) : 391,	(10,229) : 723,	(11,229) : 333,	(12,229) : 573,	(13,229) : 353,	(14,229) : 445,	(15,229) : 585,	(16,229) : 349,	(17,229) : 684,	(18,229) : 645,	(19,229) : 525,	(20,229) : 663,
(1,230) : 536,	(2,230) : 607,	(3,230) : 172,	(4,230) : 518,	(5,230) : 202,	(6,230) : 609,	(7,230) : 695,	(8,230) : 633,	(9,230) : 424,	(10,230) : 582,	(11,230) : 410,	(12,230) : 447,	(13,230) : 574,	(14,230) : 695,	(15,230) : 668,	(16,230) : 517,	(17,230) : 435,	(18,230) : 219,	(19,230) : 369,	(20,230) : 341,
(1,231) : 671,	(2,231) : 329,	(3,231) : 424,	(4,231) : 665,	(5,231) : 448,	(6,231) : 770,	(7,231) : 513,	(8,231) : 640,	(9,231) : 297,	(10,231) : 210,	(11,231) : 405,	(12,231) : 701,	(13,231) : 709,	(14,231) : 421,	(15,231) : 335,	(16,231) : 648,	(17,231) : 64,	(18,231) : 525,	(19,231) : 161,	(20,231) : 37,
(1,232) : 136,	(2,232) : 688,	(3,232) : 297,	(4,232) : 112,	(5,232) : 265,	(6,232) : 156,	(7,232) : 618,	(8,232) : 349,	(9,232) : 469,	(10,232) : 832,	(11,232) : 335,	(12,232) : 87,	(13,232) : 156,	(14,232) : 735,	(15,232) : 804,	(16,232) : 137,	(17,232) : 714,	(18,232) : 261,	(19,232) : 549,	(20,232) : 637,
(1,233) : 631,	(2,233) : 520,	(3,233) : 293,	(4,233) : 618,	(5,233) : 325,	(6,233) : 718,	(7,233) : 656,	(8,233) : 677,	(9,233) : 390,	(10,233) : 436,	(11,233) : 432,	(12,233) : 588,	(13,233) : 671,	(14,233) : 613,	(15,233) : 554,	(16,233) : 610,	(17,233) : 291,	(18,233) : 372,	(19,233) : 288,	(20,233) : 205,
(1,234) : 607,	(2,234) : 248,	(3,234) : 405,	(4,234) : 603,	(5,234) : 424,	(6,234) : 707,	(7,234) : 422,	(8,234) : 557,	(9,234) : 211,	(10,234) : 214,	(11,234) : 331,	(12,234) : 662,	(13,234) : 643,	(14,234) : 341,	(15,234) : 279,	(16,234) : 583,	(17,234) : 96,	(18,234) : 510,	(19,234) : 80,	(20,234) : 84,
(1,235) : 363,	(2,235) : 819,	(3,235) : 286,	(4,235) : 339,	(5,235) : 264,	(6,235) : 369,	(7,235) : 798,	(8,235) : 568,	(9,235) : 592,	(10,235) : 901,	(11,235) : 483,	(12,235) : 146,	(13,235) : 384,	(14,235) : 883,	(15,235) : 920,	(16,235) : 361,	(17,235) : 764,	(18,235) : 185,	(19,235) : 628,	(20,235) : 672,
(1,236) : 621,	(2,236) : 193,	(3,236) : 449,	(4,236) : 619,	(5,236) : 466,	(6,236) : 722,	(7,236) : 382,	(8,236) : 550,	(9,236) : 207,	(10,236) : 183,	(11,236) : 339,	(12,236) : 694,	(13,236) : 655,	(14,236) : 286,	(15,236) : 221,	(16,236) : 598,	(17,236) : 111,	(18,236) : 555,	(19,236) : 102,	(20,236) : 138,
(1,237) : 753,	(2,237) : 202,	(3,237) : 594,	(4,237) : 754,	(5,237) : 611,	(6,237) : 854,	(7,237) : 424,	(8,237) : 656,	(9,237) : 332,	(10,237) : 69,	(11,237) : 470,	(12,237) : 838,	(13,237) : 786,	(14,237) : 271,	(15,237) : 137,	(16,237) : 730,	(17,237) : 156,	(18,237) : 699,	(19,237) : 248,	(20,237) : 242,
(1,238) : 457,	(2,238) : 282,	(3,238) : 576,	(4,238) : 472,	(5,238) : 568,	(6,238) : 538,	(7,238) : 72,	(8,238) : 258,	(9,238) : 279,	(10,238) : 549,	(11,238) : 295,	(12,238) : 647,	(13,238) : 470,	(14,238) : 255,	(15,238) : 397,	(16,238) : 443,	(17,238) : 532,	(18,238) : 653,	(19,238) : 398,	(20,238) : 532,
(1,239) : 673,	(2,239) : 567,	(3,239) : 324,	(4,239) : 658,	(5,239) : 357,	(6,239) : 756,	(7,239) : 708,	(8,239) : 726,	(9,239) : 442,	(10,239) : 466,	(11,239) : 482,	(12,239) : 616,	(13,239) : 712,	(14,239) : 660,	(15,239) : 596,	(16,239) : 651,	(17,239) : 326,	(18,239) : 393,	(19,239) : 338,	(20,239) : 245,
(1,240) : 162,	(2,240) : 691,	(3,240) : 526,	(4,240) : 183,	(5,240) : 496,	(6,240) : 135,	(7,240) : 538,	(8,240) : 212,	(9,240) : 523,	(10,240) : 906,	(11,240) : 395,	(12,240) : 354,	(13,240) : 126,	(14,240) : 703,	(15,240) : 817,	(16,240) : 180,	(17,240) : 822,	(18,240) : 523,	(19,240) : 646,	(20,240) : 767,
(1,241) : 578,	(2,241) : 154,	(3,241) : 439,	(4,241) : 579,	(5,241) : 453,	(6,241) : 680,	(7,241) : 326,	(8,241) : 496,	(9,241) : 159,	(10,241) : 226,	(11,241) : 296,	(12,241) : 666,	(13,241) : 612,	(14,241) : 247,	(15,241) : 214,	(16,241) : 555,	(17,241) : 169,	(18,241) : 544,	(19,241) : 92,	(20,241) : 181,
(1,242) : 639,	(2,242) : 23,	(3,242) : 586,	(4,242) : 646,	(5,242) : 593,	(6,242) : 737,	(7,242) : 224,	(8,242) : 498,	(9,242) : 249,	(10,242) : 274,	(11,242) : 374,	(12,242) : 772,	(13,242) : 666,	(14,242) : 76,	(15,242) : 113,	(16,242) : 619,	(17,242) : 305,	(18,242) : 687,	(19,242) : 260,	(20,242) : 348,
(1,243) : 329,	(2,243) : 545,	(3,243) : 615,	(4,243) : 353,	(5,243) : 593,	(6,243) : 367,	(7,243) : 345,	(8,243) : 138,	(9,243) : 452,	(10,243) : 799,	(11,243) : 374,	(12,243) : 552,	(13,243) : 318,	(14,243) : 528,	(15,243) : 666,	(16,243) : 329,	(17,243) : 752,	(18,243) : 653,	(19,243) : 588,	(20,243) : 725,
(1,244) : 158,	(2,244) : 464,	(3,244) : 339,	(4,244) : 167,	(5,244) : 318,	(6,244) : 255,	(7,244) : 366,	(8,244) : 137,	(9,244) : 273,	(10,244) : 658,	(11,244) : 142,	(12,244) : 334,	(13,244) : 185,	(14,244) : 496,	(15,244) : 588,	(16,244) : 138,	(17,244) : 569,	(18,244) : 382,	(19,244) : 393,	(20,244) : 516,
(1,245) : 289,	(2,245) : 365,	(3,245) : 249,	(4,245) : 287,	(5,245) : 241,	(6,245) : 390,	(7,245) : 351,	(8,245) : 276,	(9,245) : 142,	(10,245) : 515,	(11,245) : 35,	(12,245) : 385,	(13,245) : 325,	(14,245) : 423,	(15,245) : 478,	(16,245) : 266,	(17,245) : 414,	(18,245) : 329,	(19,245) : 239,	(20,245) : 355,
(1,246) : 283,	(2,246) : 353,	(3,246) : 282,	(4,246) : 284,	(5,246) : 273,	(6,246) : 384,	(7,246) : 322,	(8,246) : 246,	(9,246) : 139,	(10,246) : 521,	(11,246) : 0,	(12,246) : 399,	(13,246) : 316,	(14,246) : 405,	(15,246) : 470,	(16,246) : 260,	(17,246) : 427,	(18,246) : 359,	(19,246) : 251,	(20,246) : 375,
(1,247) : 228,	(2,247) : 471,	(3,247) : 474,	(4,247) : 248,	(5,247) : 453,	(6,247) : 297,	(7,247) : 312,	(8,247) : 14,	(9,247) : 334,	(10,247) : 704,	(11,247) : 238,	(12,247) : 442,	(13,247) : 232,	(14,247) : 478,	(15,247) : 597,	(16,247) : 219,	(17,247) : 639,	(18,247) : 517,	(19,247) : 468,	(20,247) : 602,
(1,248) : 554,	(2,248) : 543,	(3,248) : 202,	(4,248) : 538,	(5,248) : 234,	(6,248) : 635,	(7,248) : 649,	(8,248) : 624,	(9,248) : 377,	(10,248) : 504,	(11,248) : 387,	(12,248) : 494,	(13,248) : 594,	(14,248) : 633,	(15,248) : 597,	(16,248) : 534,	(17,248) : 357,	(18,248) : 275,	(19,248) : 304,	(20,248) : 263,
(1,249) : 596,	(2,249) : 89,	(3,249) : 592,	(4,249) : 605,	(5,249) : 595,	(6,249) : 690,	(7,249) : 140,	(8,249) : 435,	(9,249) : 247,	(10,249) : 358,	(11,249) : 350,	(12,249) : 748,	(13,249) : 619,	(14,249) : 58,	(15,249) : 191,	(16,249) : 577,	(17,249) : 378,	(18,249) : 688,	(19,249) : 300,	(20,249) : 408,
(1,250) : 108,	(2,250) : 517,	(3,250) : 359,	(4,250) : 122,	(5,250) : 334,	(6,250) : 202,	(7,250) : 408,	(8,250) : 134,	(9,250) : 328,	(10,250) : 713,	(11,250) : 195,	(12,250) : 306,	(13,250) : 131,	(14,250) : 547,	(15,250) : 642,	(16,250) : 92,	(17,250) : 622,	(18,250) : 388,	(19,250) : 446,	(20,250) : 566,
(1,251) : 62,	(2,251) : 605,	(3,251) : 411,	(4,251) : 86,	(5,251) : 382,	(6,251) : 121,	(7,251) : 480,	(8,251) : 168,	(9,251) : 419,	(10,251) : 804,	(11,251) : 285,	(12,251) : 283,	(13,251) : 54,	(14,251) : 629,	(15,251) : 730,	(16,251) : 69,	(17,251) : 712,	(18,251) : 418,	(19,251) : 535,	(20,251) : 653,
(1,252) : 295,	(2,252) : 378,	(3,252) : 222,	(4,252) : 291,	(5,252) : 217,	(6,252) : 395,	(7,252) : 376,	(8,252) : 299,	(9,252) : 152,	(10,252) : 514,	(11,252) : 62,	(12,252) : 375,	(13,252) : 333,	(14,252) : 440,	(15,252) : 488,	(16,252) : 272,	(17,252) : 407,	(18,252) : 306,	(19,252) : 233,	(20,252) : 343,
(1,253) : 627,	(2,253) : 462,	(3,253) : 312,	(4,253) : 615,	(5,253) : 342,	(6,253) : 718,	(7,253) : 607,	(8,253) : 652,	(9,253) : 347,	(10,253) : 376,	(11,253) : 406,	(12,253) : 606,	(13,253) : 667,	(14,253) : 555,	(15,253) : 493,	(16,253) : 605,	(17,253) : 230,	(18,253) : 401,	(19,253) : 235,	(20,253) : 143,
(1,254) : 81,	(2,254) : 693,	(3,254) : 371,	(4,254) : 67,	(5,254) : 339,	(6,254) : 70,	(7,254) : 595,	(8,254) : 298,	(9,254) : 485,	(10,254) : 863,	(11,254) : 347,	(12,254) : 162,	(13,254) : 81,	(14,254) : 730,	(15,254) : 815,	(16,254) : 96,	(17,254) : 754,	(18,254) : 346,	(19,254) : 582,	(20,254) : 683,
(1,255) : 80,	(2,255) : 622,	(3,255) : 293,	(4,255) : 57,	(5,255) : 262,	(6,255) : 147,	(7,255) : 544,	(8,255) : 277,	(9,255) : 407,	(10,255) : 780,	(11,255) : 270,	(12,255) : 159,	(13,255) : 113,	(14,255) : 665,	(15,255) : 740,	(16,255) : 71,	(17,255) : 669,	(18,255) : 284,	(19,255) : 499,	(20,255) : 596,
(1,256) : 569,	(2,256) : 466,	(3,256) : 248,	(4,256) : 556,	(5,256) : 278,	(6,256) : 658,	(7,256) : 591,	(8,256) : 606,	(9,256) : 322,	(10,256) : 417,	(11,256) : 361,	(12,256) : 542,	(13,256) : 608,	(14,256) : 558,	(15,256) : 513,	(16,256) : 547,	(17,256) : 270,	(18,256) : 338,	(19,256) : 229,	(20,256) : 176,
(1,257) : 599,	(2,257) : 54,	(3,257) : 516,	(4,257) : 603,	(5,257) : 525,	(6,257) : 699,	(7,257) : 251,	(8,257) : 480,	(9,257) : 189,	(10,257) : 249,	(11,257) : 323,	(12,257) : 715,	(13,257) : 628,	(14,257) : 147,	(15,257) : 148,	(16,257) : 577,	(17,257) : 246,	(18,257) : 618,	(19,257) : 185,	(20,257) : 279,
(1,258) : 399,	(2,258) : 291,	(3,258) : 274,	(4,258) : 397,	(5,258) : 280,	(6,258) : 500,	(7,258) : 344,	(8,258) : 366,	(9,258) : 74,	(10,258) : 407,	(11,258) : 126,	(12,258) : 476,	(13,258) : 436,	(14,258) : 365,	(15,258) : 391,	(16,258) : 376,	(17,258) : 303,	(18,258) : 373,	(19,258) : 128,	(20,258) : 249,
(1,259) : 110,	(2,259) : 604,	(3,259) : 258,	(4,259) : 89,	(5,259) : 227,	(6,259) : 182,	(7,259) : 538,	(8,259) : 289,	(9,259) : 385,	(10,259) : 753,	(11,259) : 251,	(12,259) : 158,	(13,259) : 146,	(14,259) : 651,	(15,259) : 720,	(16,259) : 96,	(17,259) : 639,	(18,259) : 253,	(19,259) : 471,	(20,259) : 565,
(1,260) : 249,	(2,260) : 697,	(3,260) : 209,	(4,260) : 224,	(5,260) : 179,	(6,260) : 281,	(7,260) : 667,	(8,260) : 440,	(9,260) : 471,	(10,260) : 802,	(11,260) : 354,	(12,260) : 88,	(13,260) : 276,	(14,260) : 757,	(15,260) : 804,	(16,260) : 242,	(17,260) : 671,	(18,260) : 144,	(19,260) : 522,	(20,260) : 585,
(1,261) : 27,	(2,261) : 594,	(3,261) : 354,	(4,261) : 41,	(5,261) : 325,	(6,261) : 127,	(7,261) : 490,	(8,261) : 200,	(9,261) : 395,	(10,261) : 777,	(11,261) : 257,	(12,261) : 237,	(13,261) : 61,	(14,261) : 627,	(15,261) : 717,	(16,261) : 13,	(17,261) : 678,	(18,261) : 360,	(19,261) : 503,	(20,261) : 614,
(1,262) : 666,	(2,262) : 176,	(3,262) : 504,	(4,262) : 666,	(5,262) : 521,	(6,262) : 767,	(7,262) : 385,	(8,262) : 582,	(9,262) : 247,	(10,262) : 138,	(11,262) : 384,	(12,262) : 746,	(13,262) : 700,	(14,262) : 264,	(15,262) : 175,	(16,262) : 643,	(17,262) : 116,	(18,262) : 609,	(19,262) : 156,	(20,262) : 175,
(1,263) : 260,	(2,263) : 492,	(3,263) : 523,	(4,263) : 281,	(5,263) : 502,	(6,263) : 317,	(7,263) : 316,	(8,263) : 47,	(9,263) : 373,	(10,263) : 735,	(11,263) : 284,	(12,263) : 479,	(13,263) : 258,	(14,263) : 490,	(15,263) : 617,	(16,263) : 255,	(17,263) : 677,	(18,263) : 564,	(19,263) : 508,	(20,263) : 643,
(1,264) : 821,	(2,264) : 286,	(3,264) : 633,	(4,264) : 821,	(5,264) : 654,	(6,264) : 923,	(7,264) : 508,	(8,264) : 735,	(9,264) : 403,	(10,264) : 18,	(11,264) : 539,	(12,264) : 892,	(13,264) : 856,	(14,264) : 351,	(15,264) : 206,	(16,264) : 798,	(17,264) : 161,	(18,264) : 737,	(19,264) : 301,	(20,264) : 255,
(1,265) : 220,	(2,265) : 543,	(3,265) : 518,	(4,265) : 243,	(5,265) : 494,	(6,265) : 266,	(7,265) : 373,	(8,265) : 62,	(9,265) : 407,	(10,265) : 779,	(11,265) : 304,	(12,265) : 442,	(13,265) : 211,	(14,265) : 545,	(15,265) : 669,	(16,265) : 219,	(17,265) : 713,	(18,265) : 549,	(19,265) : 541,	(20,265) : 673,
(1,266) : 592,	(2,266) : 65,	(3,266) : 505,	(4,266) : 596,	(5,266) : 514,	(6,266) : 692,	(7,266) : 256,	(8,266) : 477,	(9,266) : 180,	(10,266) : 248,	(11,266) : 315,	(12,266) : 706,	(13,266) : 622,	(14,266) : 158,	(15,266) : 156,	(16,266) : 570,	(17,266) : 239,	(18,266) : 607,	(19,266) : 174,	(20,266) : 268,
(1,267) : 251,	(2,267) : 718,	(3,267) : 231,	(4,267) : 227,	(5,267) : 201,	(6,267) : 275,	(7,267) : 684,	(8,267) : 450,	(9,267) : 492,	(10,267) : 825,	(11,267) : 373,	(12,267) : 69,	(13,267) : 276,	(14,267) : 777,	(15,267) : 826,	(16,267) : 247,	(17,267) : 694,	(18,267) : 161,	(19,267) : 544,	(20,267) : 608,
(1,268) : 252,	(2,268) : 623,	(3,268) : 138,	(4,268) : 230,	(5,268) : 106,	(6,268) : 310,	(7,268) : 607,	(8,268) : 412,	(9,268) : 396,	(10,268) : 722,	(11,268) : 288,	(12,268) : 160,	(13,268) : 287,	(14,268) : 687,	(15,268) : 727,	(16,268) : 238,	(17,268) : 591,	(18,268) : 111,	(19,268) : 442,	(20,268) : 506,
(1,269) : 222,	(2,269) : 750,	(3,269) : 295,	(4,269) : 199,	(5,269) : 264,	(6,269) : 223,	(7,269) : 696,	(8,269) : 436,	(9,269) : 526,	(10,269) : 874,	(11,269) : 399,	(12,269) : 0,	(13,269) : 239,	(14,269) : 804,	(15,269) : 863,	(16,269) : 224,	(17,269) : 748,	(18,269) : 230,	(19,269) : 592,	(20,269) : 664,
(1,270) : 113,	(2,270) : 732,	(3,270) : 411,	(4,270) : 105,	(5,270) : 379,	(6,270) : 48,	(7,270) : 627,	(8,270) : 320,	(9,270) : 527,	(10,270) : 905,	(11,270) : 388,	(12,270) : 178,	(13,270) : 98,	(14,270) : 766,	(15,270) : 854,	(16,270) : 132,	(17,270) : 797,	(18,270) : 379,	(19,270) : 626,	(20,270) : 726,
(1,271) : 278,	(2,271) : 532,	(3,271) : 564,	(4,271) : 302,	(5,271) : 542,	(6,271) : 322,	(7,271) : 345,	(8,271) : 89,	(9,271) : 420,	(10,271) : 778,	(11,271) : 332,	(12,271) : 501,	(13,271) : 270,	(14,271) : 524,	(15,271) : 656,	(16,271) : 278,	(17,271) : 723,	(18,271) : 601,	(19,271) : 555,	(20,271) : 691,
(1,272) : 287,	(2,272) : 753,	(3,272) : 247,	(4,272) : 263,	(5,272) : 220,	(6,272) : 303,	(7,272) : 723,	(8,272) : 489,	(9,272) : 527,	(10,272) : 852,	(11,272) : 411,	(12,272) : 82,	(13,272) : 310,	(14,272) : 814,	(15,272) : 859,	(16,272) : 284,	(17,272) : 719,	(18,272) : 163,	(19,272) : 574,	(20,272) : 631,
(1,273) : 286,	(2,273) : 523,	(3,273) : 566,	(4,273) : 309,	(5,273) : 544,	(6,273) : 332,	(7,273) : 334,	(8,273) : 89,	(9,273) : 414,	(10,273) : 771,	(11,273) : 329,	(12,273) : 508,	(13,273) : 278,	(14,273) : 514,	(15,273) : 646,	(16,273) : 284,	(17,273) : 717,	(18,273) : 604,	(19,273) : 550,	(20,273) : 686,
(1,274) : 143,	(2,274) : 488,	(3,274) : 373,	(4,274) : 158,	(5,274) : 350,	(6,274) : 234,	(7,274) : 372,	(8,274) : 107,	(9,274) : 306,	(10,274) : 691,	(11,274) : 179,	(12,274) : 340,	(13,274) : 163,	(14,274) : 514,	(15,274) : 612,	(16,274) : 128,	(17,274) : 605,	(18,274) : 410,	(19,274) : 429,	(20,274) : 554,
(1,275) : 24,	(2,275) : 627,	(3,275) : 346,	(4,275) : 0,	(5,275) : 316,	(6,275) : 105,	(7,275) : 529,	(8,275) : 240,	(9,275) : 422,	(10,275) : 802,	(11,275) : 284,	(12,275) : 199,	(13,275) : 57,	(14,275) : 663,	(15,275) : 749,	(16,275) : 30,	(17,275) : 698,	(18,275) : 341,	(19,275) : 524,	(20,275) : 630,
(1,276) : 568,	(2,276) : 71,	(3,276) : 549,	(4,276) : 576,	(5,276) : 553,	(6,276) : 665,	(7,276) : 155,	(8,276) : 420,	(9,276) : 203,	(10,276) : 339,	(11,276) : 313,	(12,276) : 712,	(13,276) : 593,	(14,276) : 93,	(15,276) : 192,	(16,276) : 549,	(17,276) : 344,	(18,276) : 646,	(19,276) : 256,	(20,276) : 368,
(1,277) : 406,	(2,277) : 629,	(3,277) : 71,	(4,277) : 386,	(5,277) : 83,	(6,277) : 469,	(7,277) : 668,	(8,277) : 538,	(9,277) : 415,	(10,277) : 666,	(11,277) : 352,	(12,277) : 296,	(13,277) : 443,	(14,277) : 708,	(15,277) : 714,	(16,277) : 390,	(17,277) : 523,	(18,277) : 69,	(19,277) : 410,	(20,277) : 429,
(1,278) : 579,	(2,278) : 347,	(3,278) : 321,	(4,278) : 571,	(5,278) : 346,	(6,278) : 676,	(7,278) : 495,	(8,278) : 570,	(9,278) : 243,	(10,278) : 302,	(11,278) : 327,	(12,278) : 599,	(13,278) : 618,	(14,278) : 440,	(15,278) : 388,	(16,278) : 556,	(17,278) : 158,	(18,278) : 424,	(19,278) : 120,	(20,278) : 66,
(1,279) : 680,	(2,279) : 61,	(3,279) : 621,	(4,279) : 686,	(5,279) : 630,	(6,279) : 777,	(7,279) : 250,	(8,279) : 536,	(9,279) : 288,	(10,279) : 261,	(11,279) : 414,	(12,279) : 812,	(13,279) : 706,	(14,279) : 78,	(15,279) : 81,	(16,279) : 659,	(17,279) : 312,	(18,279) : 723,	(19,279) : 289,	(20,279) : 365,
(1,280) : 534,	(2,280) : 118,	(3,280) : 445,	(4,280) : 537,	(5,280) : 454,	(6,280) : 635,	(7,280) : 254,	(8,280) : 432,	(9,280) : 118,	(10,280) : 285,	(11,280) : 255,	(12,280) : 644,	(13,280) : 565,	(14,280) : 202,	(15,280) : 219,	(16,280) : 512,	(17,280) : 243,	(18,280) : 547,	(19,280) : 131,	(20,280) : 249,
(1,281) : 230,	(2,281) : 510,	(3,281) : 148,	(4,281) : 216,	(5,281) : 125,	(6,281) : 317,	(7,281) : 492,	(8,281) : 330,	(9,281) : 283,	(10,281) : 628,	(11,281) : 172,	(12,281) : 247,	(13,281) : 270,	(14,281) : 571,	(15,281) : 618,	(16,281) : 209,	(17,281) : 507,	(18,281) : 196,	(19,281) : 345,	(20,281) : 430,
(1,282) : 626,	(2,282) : 348,	(3,282) : 366,	(4,282) : 618,	(5,282) : 391,	(6,282) : 723,	(7,282) : 513,	(8,282) : 610,	(9,282) : 275,	(10,282) : 266,	(11,282) : 369,	(12,282) : 646,	(13,282) : 664,	(14,282) : 441,	(15,282) : 373,	(16,282) : 603,	(17,282) : 119,	(18,282) : 467,	(19,282) : 142,	(20,282) : 24,
(1,283) : 335,	(2,283) : 782,	(3,283) : 254,	(4,283) : 311,	(5,283) : 230,	(6,283) : 349,	(7,283) : 762,	(8,283) : 535,	(9,283) : 555,	(10,283) : 867,	(11,283) : 446,	(12,283) : 126,	(13,283) : 358,	(14,283) : 846,	(15,283) : 884,	(16,283) : 331,	(17,283) : 731,	(18,283) : 157,	(19,283) : 593,	(20,283) : 640,
(1,284) : 522,	(2,284) : 337,	(3,284) : 279,	(4,284) : 514,	(5,284) : 301,	(6,284) : 619,	(7,284) : 462,	(8,284) : 515,	(9,284) : 199,	(10,284) : 337,	(11,284) : 271,	(12,284) : 548,	(13,284) : 560,	(14,284) : 428,	(15,284) : 396,	(16,284) : 499,	(17,284) : 201,	(18,284) : 384,	(19,284) : 99,	(20,284) : 118,
(1,285) : 286,	(2,285) : 648,	(3,285) : 131,	(4,285) : 264,	(5,285) : 102,	(6,285) : 339,	(7,285) : 640,	(8,285) : 451,	(9,285) : 422,	(10,285) : 734,	(11,285) : 320,	(12,285) : 166,	(13,285) : 320,	(14,285) : 714,	(15,285) : 749,	(16,285) : 274,	(17,285) : 600,	(18,285) : 77,	(19,285) : 458,	(20,285) : 512,
(1,286) : 361,	(2,286) : 750,	(3,286) : 202,	(4,286) : 337,	(5,286) : 184,	(6,286) : 390,	(7,286) : 747,	(8,286) : 546,	(9,286) : 525,	(10,286) : 819,	(11,286) : 426,	(12,286) : 175,	(13,286) : 389,	(14,286) : 819,	(15,286) : 847,	(16,286) : 353,	(17,286) : 679,	(18,286) : 99,	(19,286) : 550,	(20,286) : 588,
(1,287) : 256,	(2,287) : 530,	(3,287) : 543,	(4,287) : 279,	(5,287) : 520,	(6,287) : 302,	(7,287) : 350,	(8,287) : 69,	(9,287) : 409,	(10,287) : 772,	(11,287) : 315,	(12,287) : 478,	(13,287) : 248,	(14,287) : 526,	(15,287) : 654,	(16,287) : 254,	(17,287) : 713,	(18,287) : 578,	(19,287) : 543,	(20,287) : 678,
(1,288) : 475,	(2,288) : 320,	(3,288) : 262,	(4,288) : 468,	(5,288) : 279,	(6,288) : 573,	(7,288) : 422,	(8,288) : 463,	(9,288) : 153,	(10,288) : 361,	(11,288) : 219,	(12,288) : 514,	(13,288) : 513,	(14,288) : 406,	(15,288) : 394,	(16,288) : 451,	(17,288) : 235,	(18,288) : 367,	(19,288) : 89,	(20,288) : 162,
(1,289) : 297,	(2,289) : 535,	(3,289) : 582,	(4,289) : 320,	(5,289) : 560,	(6,289) : 338,	(7,289) : 343,	(8,289) : 106,	(9,289) : 431,	(10,289) : 785,	(11,289) : 346,	(12,289) : 519,	(13,289) : 287,	(14,289) : 524,	(15,289) : 658,	(16,289) : 296,	(17,289) : 733,	(18,289) : 619,	(19,289) : 566,	(20,289) : 702,
(1,290) : 205,	(2,290) : 449,	(3,290) : 421,	(4,290) : 221,	(5,290) : 401,	(6,290) : 288,	(7,290) : 314,	(8,290) : 60,	(9,290) : 293,	(10,290) : 670,	(11,290) : 187,	(12,290) : 407,	(13,290) : 218,	(14,290) : 466,	(15,290) : 575,	(16,290) : 192,	(17,290) : 598,	(18,290) : 467,	(19,290) : 424,	(20,290) : 556,
(1,291) : 561,	(2,291) : 146,	(3,291) : 593,	(4,291) : 572,	(5,291) : 593,	(6,291) : 652,	(7,291) : 80,	(8,291) : 387,	(9,291) : 251,	(10,291) : 416,	(11,291) : 333,	(12,291) : 725,	(13,291) : 581,	(14,291) : 107,	(15,291) : 251,	(16,291) : 543,	(17,291) : 426,	(18,291) : 684,	(19,291) : 329,	(20,291) : 448,
(1,292) : 557,	(2,292) : 92,	(3,292) : 471,	(4,292) : 560,	(5,292) : 480,	(6,292) : 657,	(7,292) : 248,	(8,292) : 447,	(9,292) : 143,	(10,292) : 272,	(11,292) : 279,	(12,292) : 670,	(13,292) : 587,	(14,292) : 179,	(15,292) : 193,	(16,292) : 535,	(17,292) : 244,	(18,292) : 573,	(19,292) : 151,	(20,292) : 260,
(1,293) : 513,	(2,293) : 128,	(3,293) : 436,	(4,293) : 516,	(5,293) : 443,	(6,293) : 613,	(7,293) : 239,	(8,293) : 409,	(9,293) : 101,	(10,293) : 309,	(11,293) : 235,	(12,293) : 627,	(13,293) : 543,	(14,293) : 205,	(15,293) : 236,	(16,293) : 491,	(17,293) : 264,	(18,293) : 536,	(19,293) : 139,	(20,293) : 264,
(1,294) : 533,	(2,294) : 535,	(3,294) : 183,	(4,294) : 518,	(5,294) : 215,	(6,294) : 615,	(7,294) : 635,	(8,294) : 604,	(9,294) : 363,	(10,294) : 506,	(11,294) : 369,	(12,294) : 476,	(13,294) : 573,	(14,294) : 624,	(15,294) : 593,	(16,294) : 513,	(17,294) : 359,	(18,294) : 259,	(19,294) : 296,	(20,294) : 265,
(1,295) : 234,	(2,295) : 496,	(3,295) : 500,	(4,295) : 255,	(5,295) : 478,	(6,295) : 294,	(7,295) : 328,	(8,295) : 23,	(9,295) : 364,	(10,295) : 732,	(11,295) : 268,	(12,295) : 452,	(13,295) : 233,	(14,295) : 498,	(15,295) : 621,	(16,295) : 229,	(17,295) : 669,	(18,295) : 539,	(19,295) : 498,	(20,295) : 632,
(1,296) : 278,	(2,296) : 824,	(3,296) : 353,	(4,296) : 257,	(5,296) : 324,	(6,296) : 255,	(7,296) : 767,	(8,296) : 497,	(9,296) : 600,	(10,296) : 944,	(11,296) : 472,	(12,296) : 74,	(13,296) : 288,	(14,296) : 877,	(15,296) : 936,	(16,296) : 284,	(17,296) : 816,	(18,296) : 274,	(19,296) : 663,	(20,296) : 730,
(1,297) : 728,	(2,297) : 114,	(3,297) : 632,	(4,297) : 733,	(5,297) : 644,	(6,297) : 827,	(7,297) : 326,	(8,297) : 599,	(9,297) : 318,	(10,297) : 189,	(11,297) : 453,	(12,297) : 844,	(13,297) : 757,	(14,297) : 154,	(15,297) : 20,	(16,297) : 706,	(17,297) : 264,	(18,297) : 736,	(19,297) : 285,	(20,297) : 332,
(1,298) : 571,	(2,298) : 220,	(3,298) : 392,	(4,298) : 569,	(5,298) : 409,	(6,298) : 672,	(7,298) : 381,	(8,298) : 514,	(9,298) : 168,	(10,298) : 238,	(11,298) : 292,	(12,298) : 637,	(13,298) : 607,	(14,298) : 313,	(15,298) : 270,	(16,298) : 548,	(17,298) : 137,	(18,298) : 497,	(19,298) : 46,	(20,298) : 122,
(1,299) : 329,	(2,299) : 508,	(3,299) : 596,	(4,299) : 352,	(5,299) : 576,	(6,299) : 376,	(7,299) : 309,	(8,299) : 123,	(9,299) : 419,	(10,299) : 763,	(11,299) : 347,	(12,299) : 551,	(13,299) : 323,	(14,299) : 492,	(15,299) : 630,	(16,299) : 327,	(17,299) : 718,	(18,299) : 640,	(19,299) : 555,	(20,299) : 692,
(1,300) : 324,	(2,300) : 601,	(3,300) : 636,	(4,300) : 349,	(5,300) : 612,	(6,300) : 346,	(7,300) : 402,	(8,300) : 166,	(9,300) : 499,	(10,300) : 853,	(11,300) : 411,	(12,300) : 545,	(13,300) : 307,	(14,300) : 585,	(15,300) : 722,	(16,300) : 328,	(17,300) : 802,	(18,300) : 666,	(19,300) : 635,	(20,300) : 771,
(1,301) : 29,	(2,301) : 630,	(3,301) : 344,	(4,301) : 5,	(5,301) : 313,	(6,301) : 105,	(7,301) : 533,	(8,301) : 245,	(9,301) : 424,	(10,301) : 803,	(11,301) : 285,	(12,301) : 194,	(13,301) : 60,	(14,301) : 666,	(15,301) : 751,	(16,301) : 34,	(17,301) : 698,	(18,301) : 337,	(19,301) : 525,	(20,301) : 630,
(1,302) : 566,	(2,302) : 70,	(3,302) : 545,	(4,302) : 574,	(5,302) : 549,	(6,302) : 663,	(7,302) : 156,	(8,302) : 419,	(9,302) : 200,	(10,302) : 338,	(11,302) : 310,	(12,302) : 709,	(13,302) : 591,	(14,302) : 96,	(15,302) : 193,	(16,302) : 546,	(17,302) : 341,	(18,302) : 642,	(19,302) : 252,	(20,302) : 365,
(1,303) : 309,	(2,303) : 599,	(3,303) : 622,	(4,303) : 333,	(5,303) : 599,	(6,303) : 331,	(7,303) : 404,	(8,303) : 155,	(9,303) : 492,	(10,303) : 849,	(11,303) : 401,	(12,303) : 530,	(13,303) : 291,	(14,303) : 586,	(15,303) : 722,	(16,303) : 313,	(17,303) : 796,	(18,303) : 651,	(19,303) : 628,	(20,303) : 763,
(1,304) : 606,	(2,304) : 530,	(3,304) : 261,	(4,304) : 591,	(5,304) : 294,	(6,304) : 690,	(7,304) : 656,	(8,304) : 660,	(9,304) : 387,	(10,304) : 462,	(11,304) : 418,	(12,304) : 555,	(13,304) : 645,	(14,304) : 622,	(15,304) : 572,	(16,304) : 584,	(17,304) : 316,	(18,304) : 338,	(19,304) : 294,	(20,304) : 226,
(1,305) : 144,	(2,305) : 478,	(3,305) : 339,	(4,305) : 153,	(5,305) : 317,	(6,305) : 242,	(7,305) : 378,	(8,305) : 139,	(9,305) : 286,	(10,305) : 671,	(11,305) : 153,	(12,305) : 323,	(13,305) : 171,	(14,305) : 510,	(15,305) : 601,	(16,305) : 125,	(17,305) : 581,	(18,305) : 378,	(19,305) : 404,	(20,305) : 526,
(1,306) : 526,	(2,306) : 153,	(3,306) : 414,	(4,306) : 528,	(5,306) : 424,	(6,306) : 628,	(7,306) : 287,	(8,306) : 440,	(9,306) : 106,	(10,306) : 281,	(11,306) : 244,	(12,306) : 624,	(13,306) : 559,	(14,306) : 240,	(15,306) : 242,	(16,306) : 504,	(17,306) : 219,	(18,306) : 516,	(19,306) : 93,	(20,306) : 215,
(1,307) : 326,	(2,307) : 298,	(3,307) : 392,	(4,307) : 334,	(5,307) : 384,	(6,307) : 422,	(7,307) : 216,	(8,307) : 203,	(9,307) : 147,	(10,307) : 515,	(11,307) : 111,	(12,307) : 484,	(13,307) : 351,	(14,307) : 329,	(15,307) : 424,	(16,307) : 306,	(17,307) : 450,	(18,307) : 469,	(19,307) : 282,	(20,307) : 419,
(1,308) : 340,	(2,308) : 706,	(3,308) : 685,	(4,308) : 364,	(5,308) : 659,	(6,308) : 329,	(7,308) : 510,	(8,308) : 247,	(9,308) : 592,	(10,308) : 955,	(11,308) : 490,	(12,308) : 547,	(13,308) : 312,	(14,308) : 693,	(15,308) : 829,	(16,308) : 351,	(17,308) : 897,	(18,308) : 700,	(19,308) : 726,	(20,308) : 859,
(1,309) : 720,	(2,309) : 300,	(3,309) : 493,	(4,309) : 715,	(5,309) : 516,	(6,309) : 820,	(7,309) : 504,	(8,309) : 668,	(9,309) : 322,	(10,309) : 137,	(11,309) : 445,	(12,309) : 765,	(13,309) : 756,	(14,309) : 388,	(15,309) : 282,	(16,309) : 696,	(17,309) : 18,	(18,309) : 596,	(19,309) : 194,	(20,309) : 108,
(1,310) : 576,	(2,310) : 425,	(3,310) : 275,	(4,310) : 565,	(5,310) : 304,	(6,310) : 668,	(7,310) : 559,	(8,310) : 597,	(9,310) : 294,	(10,310) : 372,	(11,310) : 350,	(12,310) : 566,	(13,310) : 615,	(14,310) : 517,	(15,310) : 468,	(16,310) : 553,	(17,310) : 225,	(18,310) : 371,	(19,310) : 190,	(20,310) : 130,
(1,311) : 266,	(2,311) : 612,	(3,311) : 118,	(4,311) : 245,	(5,311) : 86,	(6,311) : 328,	(7,311) : 603,	(8,311) : 419,	(9,311) : 386,	(10,311) : 705,	(11,311) : 282,	(12,311) : 180,	(13,311) : 302,	(14,311) : 678,	(15,311) : 714,	(16,311) : 252,	(17,311) : 574,	(18,311) : 99,	(19,311) : 427,	(20,311) : 487,
(1,312) : 333,	(2,312) : 658,	(3,312) : 665,	(4,312) : 358,	(5,312) : 640,	(6,312) : 338,	(7,312) : 460,	(8,312) : 209,	(9,312) : 550,	(10,312) : 909,	(11,312) : 455,	(12,312) : 549,	(13,312) : 310,	(14,312) : 643,	(15,312) : 780,	(16,312) : 341,	(17,312) : 854,	(18,312) : 687,	(19,312) : 686,	(20,312) : 820,
(1,313) : 645,	(2,313) : 46,	(3,313) : 605,	(4,313) : 653,	(5,313) : 612,	(6,313) : 742,	(7,313) : 210,	(8,313) : 497,	(9,313) : 265,	(10,313) : 294,	(11,313) : 385,	(12,313) : 784,	(13,313) : 671,	(14,313) : 50,	(15,313) : 122,	(16,313) : 625,	(17,313) : 331,	(18,313) : 705,	(19,313) : 285,	(20,313) : 375,
(1,314) : 529,	(2,314) : 196,	(3,314) : 382,	(4,314) : 528,	(5,314) : 395,	(6,314) : 630,	(7,314) : 334,	(8,314) : 462,	(9,314) : 117,	(10,314) : 274,	(11,314) : 247,	(12,314) : 609,	(13,314) : 563,	(14,314) : 285,	(15,314) : 270,	(16,314) : 506,	(17,314) : 189,	(18,314) : 486,	(19,314) : 44,	(20,314) : 171,
(1,315) : 514,	(2,315) : 108,	(3,315) : 469,	(4,315) : 520,	(5,315) : 474,	(6,315) : 613,	(7,315) : 194,	(8,315) : 391,	(9,315) : 124,	(10,315) : 331,	(11,315) : 245,	(12,315) : 643,	(13,315) : 543,	(14,315) : 170,	(15,315) : 230,	(16,315) : 493,	(17,315) : 302,	(18,315) : 567,	(19,315) : 184,	(20,315) : 308,
(1,316) : 86,	(2,316) : 683,	(3,316) : 447,	(4,316) : 101,	(5,316) : 417,	(6,316) : 52,	(7,316) : 556,	(8,316) : 236,	(9,316) : 494,	(10,316) : 879,	(11,316) : 358,	(12,316) : 263,	(13,316) : 46,	(14,316) : 708,	(15,316) : 808,	(16,316) : 109,	(17,316) : 783,	(18,316) : 437,	(19,316) : 607,	(20,316) : 720,
(1,317) : 581,	(2,317) : 155,	(3,317) : 440,	(4,317) : 581,	(5,317) : 454,	(6,317) : 682,	(7,317) : 328,	(8,317) : 498,	(9,317) : 161,	(10,317) : 224,	(11,317) : 298,	(12,317) : 668,	(13,317) : 614,	(14,317) : 248,	(15,317) : 213,	(16,317) : 558,	(17,317) : 166,	(18,317) : 545,	(19,317) : 92,	(20,317) : 180,
(1,318) : 655,	(2,318) : 39,	(3,318) : 580,	(4,318) : 660,	(5,318) : 590,	(6,318) : 754,	(7,318) : 259,	(8,318) : 523,	(9,318) : 252,	(10,318) : 237,	(11,318) : 383,	(12,318) : 778,	(13,318) : 683,	(14,318) : 109,	(15,318) : 90,	(16,318) : 634,	(17,318) : 272,	(18,318) : 683,	(19,318) : 245,	(20,318) : 321,
(1,319) : 237,	(2,319) : 669,	(3,319) : 587,	(4,319) : 261,	(5,319) : 560,	(6,319) : 230,	(7,319) : 493,	(8,319) : 186,	(9,319) : 528,	(10,319) : 903,	(11,319) : 413,	(12,319) : 445,	(13,319) : 208,	(14,319) : 669,	(15,319) : 795,	(16,319) : 249,	(17,319) : 833,	(18,319) : 598,	(19,319) : 658,	(20,319) : 787,
(1,320) : 367,	(2,320) : 323,	(3,320) : 247,	(4,320) : 363,	(5,320) : 251,	(6,320) : 467,	(7,320) : 358,	(8,320) : 347,	(9,320) : 99,	(10,320) : 442,	(11,320) : 102,	(12,320) : 441,	(13,320) : 404,	(14,320) : 393,	(15,320) : 425,	(16,320) : 344,	(17,320) : 335,	(18,320) : 344,	(19,320) : 161,	(20,320) : 275,
(1,321) : 86,	(2,321) : 612,	(3,321) : 437,	(4,321) : 110,	(5,321) : 408,	(6,321) : 127,	(7,321) : 479,	(8,321) : 159,	(9,321) : 432,	(10,321) : 817,	(11,321) : 301,	(12,321) : 305,	(13,321) : 69,	(14,321) : 633,	(15,321) : 738,	(16,321) : 95,	(17,321) : 728,	(18,321) : 444,	(19,321) : 552,	(20,321) : 672,
(1,322) : 236,	(2,322) : 493,	(3,322) : 499,	(4,322) : 257,	(5,322) : 478,	(6,322) : 296,	(7,322) : 325,	(8,322) : 23,	(9,322) : 362,	(10,322) : 730,	(11,322) : 267,	(12,322) : 454,	(13,322) : 235,	(14,322) : 495,	(15,322) : 618,	(16,322) : 230,	(17,322) : 667,	(18,322) : 540,	(19,322) : 496,	(20,322) : 630,
(1,323) : 677,	(2,323) : 99,	(3,323) : 655,	(4,323) : 686,	(5,323) : 660,	(6,323) : 772,	(7,323) : 208,	(8,323) : 516,	(9,323) : 312,	(10,323) : 326,	(11,323) : 426,	(12,323) : 825,	(13,323) : 701,	(14,323) : 25,	(15,323) : 136,	(16,323) : 658,	(17,323) : 377,	(18,323) : 754,	(19,323) : 338,	(20,323) : 425,
(1,324) : 247,	(2,324) : 542,	(3,324) : 542,	(4,324) : 270,	(5,324) : 519,	(6,324) : 290,	(7,324) : 363,	(8,324) : 74,	(9,324) : 417,	(10,324) : 783,	(11,324) : 320,	(12,324) : 469,	(13,324) : 238,	(14,324) : 539,	(15,324) : 667,	(16,324) : 247,	(17,324) : 722,	(18,324) : 575,	(19,324) : 551,	(20,324) : 685,
(1,325) : 536,	(2,325) : 146,	(3,325) : 566,	(4,325) : 547,	(5,325) : 566,	(6,325) : 628,	(7,325) : 78,	(8,325) : 367,	(9,325) : 226,	(10,325) : 416,	(11,325) : 306,	(12,325) : 698,	(13,325) : 557,	(14,325) : 125,	(15,325) : 260,	(16,325) : 518,	(17,325) : 416,	(18,325) : 657,	(19,325) : 310,	(20,325) : 432,
(1,326) : 791,	(2,326) : 516,	(3,326) : 477,	(4,326) : 781,	(5,326) : 508,	(6,326) : 884,	(7,326) : 701,	(8,326) : 798,	(9,326) : 469,	(10,326) : 336,	(11,326) : 554,	(12,326) : 772,	(13,326) : 831,	(14,326) : 607,	(15,326) : 506,	(16,326) : 769,	(17,326) : 228,	(18,326) : 561,	(19,326) : 336,	(20,326) : 200,
(1,327) : 725,	(2,327) : 121,	(3,327) : 617,	(4,327) : 729,	(5,327) : 631,	(6,327) : 825,	(7,327) : 339,	(8,327) : 603,	(9,327) : 311,	(10,327) : 167,	(11,327) : 448,	(12,327) : 836,	(13,327) : 755,	(14,327) : 173,	(15,327) : 41,	(16,327) : 704,	(17,327) : 240,	(18,327) : 722,	(19,327) : 268,	(20,327) : 309,
(1,328) : 321,	(2,328) : 753,	(3,328) : 226,	(4,328) : 296,	(5,328) : 202,	(6,328) : 342,	(7,328) : 735,	(8,328) : 515,	(9,328) : 527,	(10,328) : 838,	(11,328) : 418,	(12,328) : 124,	(13,328) : 346,	(14,328) : 818,	(15,328) : 855,	(16,328) : 315,	(17,328) : 702,	(18,328) : 132,	(19,328) : 564,	(20,328) : 612,
(1,329) : 238,	(2,329) : 589,	(3,329) : 132,	(4,329) : 218,	(5,329) : 100,	(6,329) : 306,	(7,329) : 572,	(8,329) : 385,	(9,329) : 362,	(10,329) : 694,	(11,329) : 253,	(12,329) : 182,	(13,329) : 275,	(14,329) : 652,	(15,329) : 694,	(16,329) : 222,	(17,329) : 566,	(18,329) : 133,	(19,329) : 412,	(20,329) : 482,
(1,330) : 198,	(2,330) : 768,	(3,330) : 346,	(4,330) : 177,	(5,330) : 314,	(6,330) : 177,	(7,330) : 697,	(8,330) : 417,	(9,330) : 548,	(10,330) : 907,	(11,330) : 415,	(12,330) : 59,	(13,330) : 206,	(14,330) : 816,	(15,330) : 884,	(16,330) : 206,	(17,330) : 785,	(18,330) : 287,	(19,330) : 624,	(20,330) : 705,
(1,331) : 194,	(2,331) : 801,	(3,331) : 419,	(4,331) : 180,	(5,331) : 387,	(6,331) : 132,	(7,331) : 709,	(8,331) : 407,	(9,331) : 588,	(10,331) : 958,	(11,331) : 451,	(12,331) : 139,	(13,331) : 185,	(14,331) : 841,	(15,331) : 921,	(16,331) : 209,	(17,331) : 842,	(18,331) : 366,	(19,331) : 676,	(20,331) : 765,
(1,332) : 344,	(2,332) : 625,	(3,332) : 71,	(4,332) : 323,	(5,332) : 54,	(6,332) : 405,	(7,332) : 642,	(8,332) : 487,	(9,332) : 403,	(10,332) : 687,	(11,332) : 321,	(12,332) : 237,	(13,332) : 380,	(14,332) : 698,	(15,332) : 718,	(16,332) : 329,	(17,332) : 549,	(18,332) : 39,	(19,332) : 419,	(20,332) : 458,
(1,333) : 281,	(2,333) : 488,	(3,333) : 114,	(4,333) : 267,	(5,333) : 101,	(6,333) : 369,	(7,333) : 493,	(8,333) : 365,	(9,333) : 262,	(10,333) : 588,	(11,333) : 173,	(12,333) : 286,	(13,333) : 321,	(14,333) : 555,	(15,333) : 590,	(16,333) : 260,	(17,333) : 463,	(18,333) : 188,	(19,333) : 307,	(20,333) : 383,
(1,334) : 223,	(2,334) : 727,	(3,334) : 265,	(4,334) : 199,	(5,334) : 234,	(6,334) : 237,	(7,334) : 680,	(8,334) : 430,	(9,334) : 503,	(10,334) : 846,	(11,334) : 378,	(12,334) : 30,	(13,334) : 244,	(14,334) : 783,	(15,334) : 838,	(16,334) : 221,	(17,334) : 719,	(18,334) : 201,	(19,334) : 564,	(20,334) : 635,
(1,335) : 211,	(2,335) : 661,	(3,335) : 204,	(4,335) : 187,	(5,335) : 172,	(6,335) : 255,	(7,335) : 624,	(8,335) : 397,	(9,335) : 435,	(10,335) : 776,	(11,335) : 314,	(12,335) : 99,	(13,335) : 241,	(14,335) : 719,	(15,335) : 770,	(16,335) : 202,	(17,335) : 649,	(18,335) : 161,	(19,335) : 494,	(20,335) : 566,
(1,336) : 75,	(2,336) : 601,	(3,336) : 421,	(4,336) : 99,	(5,336) : 393,	(6,336) : 130,	(7,336) : 473,	(8,336) : 157,	(9,336) : 419,	(10,336) : 804,	(11,336) : 286,	(12,336) : 297,	(13,336) : 65,	(14,336) : 624,	(15,336) : 727,	(16,336) : 82,	(17,336) : 714,	(18,336) : 430,	(19,336) : 537,	(20,336) : 656,
(1,337) : 665,	(2,337) : 114,	(3,337) : 658,	(4,337) : 675,	(5,337) : 662,	(6,337) : 758,	(7,337) : 184,	(8,337) : 498,	(9,337) : 313,	(10,337) : 354,	(11,337) : 420,	(12,337) : 818,	(13,337) : 687,	(14,337) : 21,	(15,337) : 165,	(16,337) : 646,	(17,337) : 400,	(18,337) : 755,	(19,337) : 350,	(20,337) : 444,
(1,338) : 76,	(2,338) : 685,	(3,338) : 362,	(4,338) : 60,	(5,338) : 330,	(6,338) : 78,	(7,338) : 590,	(8,338) : 295,	(9,338) : 476,	(10,338) : 853,	(11,338) : 338,	(12,338) : 159,	(13,338) : 81,	(14,338) : 723,	(15,338) : 806,	(16,338) : 90,	(17,338) : 744,	(18,338) : 338,	(19,338) : 573,	(20,338) : 673,
(1,339) : 78,	(2,339) : 547,	(3,339) : 360,	(4,339) : 93,	(5,339) : 334,	(6,339) : 173,	(7,339) : 437,	(8,339) : 153,	(9,339) : 354,	(10,339) : 739,	(11,339) : 219,	(12,339) : 283,	(13,339) : 102,	(14,339) : 577,	(15,339) : 671,	(16,339) : 63,	(17,339) : 646,	(18,339) : 381,	(19,339) : 470,	(20,339) : 587,
(1,340) : 232,	(2,340) : 482,	(3,340) : 488,	(4,340) : 253,	(5,340) : 467,	(6,340) : 297,	(7,340) : 318,	(8,340) : 13,	(9,340) : 349,	(10,340) : 718,	(11,340) : 253,	(12,340) : 449,	(13,340) : 234,	(14,340) : 486,	(15,340) : 608,	(16,340) : 226,	(17,340) : 654,	(18,340) : 530,	(19,340) : 483,	(20,340) : 617,
(1,341) : 801,	(2,341) : 180,	(3,341) : 719,	(4,341) : 807,	(5,341) : 731,	(6,341) : 899,	(7,341) : 361,	(8,341) : 658,	(9,341) : 400,	(10,341) : 231,	(11,341) : 532,	(12,341) : 926,	(13,341) : 828,	(14,341) : 176,	(15,341) : 67,	(16,341) : 780,	(17,341) : 336,	(18,341) : 823,	(19,341) : 372,	(20,341) : 412,
(1,342) : 62,	(2,342) : 560,	(3,342) : 351,	(4,342) : 75,	(5,342) : 324,	(6,342) : 160,	(7,342) : 455,	(8,342) : 171,	(9,342) : 364,	(10,342) : 748,	(11,342) : 227,	(12,342) : 265,	(13,342) : 90,	(14,342) : 592,	(15,342) : 683,	(16,342) : 45,	(17,342) : 651,	(18,342) : 368,	(19,342) : 476,	(20,342) : 591,
(1,343) : 535,	(2,343) : 152,	(3,343) : 570,	(4,343) : 546,	(5,343) : 569,	(6,343) : 627,	(7,343) : 71,	(8,343) : 364,	(9,343) : 230,	(10,343) : 422,	(11,343) : 308,	(12,343) : 699,	(13,343) : 556,	(14,343) : 128,	(15,343) : 265,	(16,343) : 517,	(17,343) : 423,	(18,343) : 660,	(19,343) : 316,	(20,343) : 439,
(1,344) : 169,	(2,344) : 649,	(3,344) : 230,	(4,344) : 145,	(5,344) : 198,	(6,344) : 216,	(7,344) : 599,	(8,344) : 359,	(9,344) : 425,	(10,344) : 778,	(11,344) : 298,	(12,344) : 102,	(13,344) : 200,	(14,344) : 703,	(15,344) : 761,	(16,344) : 161,	(17,344) : 656,	(18,344) : 200,	(19,344) : 495,	(20,344) : 576,
(1,345) : 564,	(2,345) : 108,	(3,345) : 462,	(4,345) : 566,	(5,345) : 472,	(6,345) : 664,	(7,345) : 272,	(8,345) : 463,	(9,345) : 145,	(10,345) : 255,	(11,345) : 283,	(12,345) : 669,	(13,345) : 595,	(14,345) : 198,	(15,345) : 195,	(16,345) : 541,	(17,345) : 220,	(18,345) : 564,	(19,345) : 131,	(20,345) : 236,
(1,346) : 198,	(2,346) : 777,	(3,346) : 361,	(4,346) : 178,	(5,346) : 329,	(6,346) : 169,	(7,346) : 702,	(8,346) : 417,	(9,346) : 558,	(10,346) : 919,	(11,346) : 424,	(12,346) : 73,	(13,346) : 203,	(14,346) : 823,	(15,346) : 893,	(16,346) : 207,	(17,346) : 798,	(18,346) : 302,	(19,346) : 636,	(20,346) : 718,
(1,347) : 74,	(2,347) : 630,	(3,347) : 433,	(4,347) : 97,	(5,347) : 404,	(6,347) : 104,	(7,347) : 500,	(8,347) : 181,	(9,347) : 446,	(10,347) : 831,	(11,347) : 313,	(12,347) : 288,	(13,347) : 49,	(14,347) : 653,	(15,347) : 755,	(16,347) : 88,	(17,347) : 740,	(18,347) : 436,	(19,347) : 563,	(20,347) : 681,
(1,348) : 240,	(2,348) : 695,	(3,348) : 214,	(4,348) : 216,	(5,348) : 184,	(6,348) : 272,	(7,348) : 662,	(8,348) : 433,	(9,348) : 469,	(10,348) : 803,	(11,348) : 351,	(12,348) : 81,	(13,348) : 267,	(14,348) : 755,	(15,348) : 803,	(16,348) : 233,	(17,348) : 674,	(18,348) : 153,	(19,348) : 523,	(20,348) : 588,
(1,349) : 744,	(2,349) : 262,	(3,349) : 545,	(4,349) : 742,	(5,349) : 566,	(6,349) : 845,	(7,349) : 477,	(8,349) : 673,	(9,349) : 332,	(10,349) : 73,	(11,349) : 464,	(12,349) : 806,	(13,349) : 780,	(14,349) : 343,	(15,349) : 222,	(16,349) : 721,	(17,349) : 75,	(18,349) : 649,	(19,349) : 219,	(20,349) : 169,
(1,350) : 97,	(2,350) : 554,	(3,350) : 403,	(4,350) : 118,	(5,350) : 377,	(6,350) : 173,	(7,350) : 428,	(8,350) : 122,	(9,350) : 374,	(10,350) : 759,	(11,350) : 244,	(12,350) : 316,	(13,350) : 104,	(14,350) : 578,	(15,350) : 680,	(16,350) : 92,	(17,350) : 671,	(18,350) : 424,	(19,350) : 495,	(20,350) : 617,
(1,351) : 484,	(2,351) : 445,	(3,351) : 178,	(4,351) : 472,	(5,351) : 205,	(6,351) : 574,	(7,351) : 541,	(8,351) : 527,	(9,351) : 268,	(10,351) : 447,	(11,351) : 285,	(12,351) : 467,	(13,351) : 524,	(14,351) : 533,	(15,351) : 512,	(16,351) : 462,	(17,351) : 304,	(18,351) : 278,	(19,351) : 209,	(20,351) : 211,
(1,352) : 142,	(2,352) : 608,	(3,352) : 484,	(4,352) : 166,	(5,352) : 456,	(6,352) : 170,	(7,352) : 457,	(8,352) : 131,	(9,352) : 444,	(10,352) : 826,	(11,352) : 320,	(12,352) : 361,	(13,352) : 123,	(14,352) : 620,	(15,352) : 734,	(16,352) : 150,	(17,352) : 746,	(18,352) : 497,	(19,352) : 570,	(20,352) : 695,
(1,353) : 283,	(2,353) : 450,	(3,353) : 516,	(4,353) : 303,	(5,353) : 498,	(6,353) : 349,	(7,353) : 273,	(8,353) : 64,	(9,353) : 339,	(10,353) : 695,	(11,353) : 263,	(12,353) : 497,	(13,353) : 286,	(14,353) : 446,	(15,353) : 574,	(16,353) : 275,	(17,353) : 641,	(18,353) : 565,	(19,353) : 475,	(20,353) : 611,
(1,354) : 498,	(2,354) : 355,	(3,354) : 250,	(4,354) : 490,	(5,354) : 271,	(6,354) : 595,	(7,354) : 467,	(8,354) : 501,	(9,354) : 198,	(10,354) : 366,	(11,354) : 255,	(12,354) : 520,	(13,354) : 537,	(14,354) : 444,	(15,354) : 420,	(16,354) : 475,	(17,354) : 230,	(18,354) : 355,	(19,354) : 117,	(20,354) : 145,
(1,355) : 615,	(2,355) : 554,	(3,355) : 265,	(4,355) : 600,	(5,355) : 297,	(6,355) : 698,	(7,355) : 679,	(8,355) : 676,	(9,355) : 409,	(10,355) : 483,	(11,355) : 435,	(12,355) : 556,	(13,355) : 655,	(14,355) : 646,	(15,355) : 596,	(16,355) : 594,	(17,355) : 338,	(18,355) : 335,	(19,355) : 318,	(20,355) : 249,
(1,356) : 74,	(2,356) : 694,	(3,356) : 392,	(4,356) : 68,	(5,356) : 360,	(6,356) : 46,	(7,356) : 588,	(8,356) : 284,	(9,356) : 490,	(10,356) : 870,	(11,356) : 352,	(12,356) : 187,	(13,356) : 63,	(14,356) : 728,	(15,356) : 816,	(16,356) : 94,	(17,356) : 764,	(18,356) : 370,	(19,356) : 591,	(20,356) : 695,
(1,357) : 545,	(2,357) : 504,	(3,357) : 206,	(4,357) : 530,	(5,357) : 237,	(6,357) : 630,	(7,357) : 614,	(8,357) : 601,	(9,357) : 342,	(10,357) : 469,	(11,357) : 361,	(12,357) : 501,	(13,357) : 584,	(14,357) : 594,	(15,357) : 559,	(16,357) : 523,	(17,357) : 322,	(18,357) : 290,	(19,357) : 266,	(20,357) : 227,
(1,358) : 580,	(2,358) : 111,	(3,358) : 468,	(4,358) : 582,	(5,358) : 479,	(6,358) : 681,	(7,358) : 288,	(8,358) : 481,	(9,358) : 160,	(10,358) : 236,	(11,358) : 299,	(12,358) : 681,	(13,358) : 612,	(14,358) : 203,	(15,358) : 185,	(16,358) : 557,	(17,358) : 204,	(18,358) : 571,	(19,358) : 130,	(20,358) : 225,
(1,359) : 597,	(2,359) : 137,	(3,359) : 463,	(4,359) : 598,	(5,359) : 477,	(6,359) : 698,	(7,359) : 323,	(8,359) : 508,	(9,359) : 176,	(10,359) : 211,	(11,359) : 314,	(12,359) : 689,	(13,359) : 630,	(14,359) : 231,	(15,359) : 190,	(16,359) : 574,	(17,359) : 169,	(18,359) : 568,	(19,359) : 115,	(20,359) : 194,
(1,360) : 635,	(2,360) : 404,	(3,360) : 346,	(4,360) : 625,	(5,360) : 374,	(6,360) : 729,	(7,360) : 562,	(8,360) : 637,	(9,360) : 313,	(10,360) : 311,	(11,360) : 392,	(12,360) : 635,	(13,360) : 674,	(14,360) : 497,	(15,360) : 429,	(16,360) : 612,	(17,360) : 165,	(18,360) : 443,	(19,360) : 188,	(20,360) : 78,
(1,361) : 122,	(2,361) : 519,	(3,361) : 392,	(4,361) : 140,	(5,361) : 368,	(6,361) : 206,	(7,361) : 396,	(8,361) : 104,	(9,361) : 340,	(10,361) : 724,	(11,361) : 213,	(12,361) : 332,	(13,361) : 136,	(14,361) : 543,	(15,361) : 644,	(16,361) : 111,	(17,361) : 639,	(18,361) : 421,	(19,361) : 463,	(20,361) : 587,
(1,362) : 203,	(2,362) : 728,	(3,362) : 284,	(4,362) : 179,	(5,362) : 252,	(6,362) : 211,	(7,362) : 673,	(8,362) : 414,	(9,362) : 505,	(10,362) : 855,	(11,362) : 376,	(12,362) : 23,	(13,362) : 221,	(14,362) : 781,	(15,362) : 841,	(16,362) : 203,	(17,362) : 730,	(18,362) : 226,	(19,362) : 572,	(20,362) : 648,
(1,363) : 69,	(2,363) : 688,	(3,363) : 386,	(4,363) : 61,	(5,363) : 354,	(6,363) : 52,	(7,363) : 584,	(8,363) : 281,	(9,363) : 484,	(10,363) : 863,	(11,363) : 345,	(12,363) : 185,	(13,363) : 60,	(14,363) : 722,	(15,363) : 810,	(16,363) : 88,	(17,363) : 757,	(18,363) : 365,	(19,363) : 585,	(20,363) : 688,
(1,364) : 22,	(2,364) : 613,	(3,364) : 342,	(4,364) : 15,	(5,364) : 313,	(6,364) : 116,	(7,364) : 515,	(8,364) : 228,	(9,364) : 408,	(10,364) : 789,	(11,364) : 270,	(12,364) : 209,	(13,364) : 61,	(14,364) : 648,	(15,364) : 735,	(16,364) : 15,	(17,364) : 686,	(18,364) : 341,	(19,364) : 512,	(20,364) : 619,
(1,365) : 182,	(2,365) : 626,	(3,365) : 526,	(4,365) : 206,	(5,365) : 499,	(6,365) : 195,	(7,365) : 463,	(8,365) : 141,	(9,365) : 473,	(10,365) : 852,	(11,365) : 354,	(12,365) : 398,	(13,365) : 159,	(14,365) : 632,	(15,365) : 752,	(16,365) : 191,	(17,365) : 776,	(18,365) : 539,	(19,365) : 601,	(20,365) : 729,
(1,366) : 233,	(2,366) : 402,	(3,366) : 271,	(4,366) : 234,	(5,366) : 256,	(6,366) : 334,	(7,366) : 353,	(8,366) : 223,	(9,366) : 189,	(10,366) : 570,	(11,366) : 50,	(12,366) : 354,	(13,366) : 268,	(14,366) : 450,	(15,366) : 520,	(16,366) : 210,	(17,366) : 473,	(18,366) : 336,	(19,366) : 297,	(20,366) : 416,
(1,367) : 230,	(2,367) : 715,	(3,367) : 244,	(4,367) : 205,	(5,367) : 213,	(6,367) : 251,	(7,367) : 673,	(8,367) : 432,	(9,367) : 489,	(10,367) : 829,	(11,367) : 367,	(12,367) : 51,	(13,367) : 254,	(14,367) : 772,	(15,367) : 824,	(16,367) : 226,	(17,367) : 701,	(18,367) : 181,	(19,367) : 548,	(20,367) : 616,
(1,368) : 396,	(2,368) : 577,	(3,368) : 33,	(4,368) : 378,	(5,368) : 62,	(6,368) : 468,	(7,368) : 621,	(8,368) : 509,	(9,368) : 365,	(10,368) : 616,	(11,368) : 309,	(12,368) : 316,	(13,368) : 434,	(14,368) : 656,	(15,368) : 661,	(16,368) : 378,	(17,368) : 475,	(18,368) : 107,	(19,368) : 358,	(20,368) : 382,
(1,369) : 540,	(2,369) : 120,	(3,369) : 552,	(4,369) : 550,	(5,369) : 553,	(6,369) : 634,	(7,369) : 106,	(8,369) : 380,	(9,369) : 208,	(10,369) : 389,	(11,369) : 299,	(12,369) : 695,	(13,369) : 563,	(14,369) : 114,	(15,369) : 238,	(16,369) : 521,	(17,369) : 388,	(18,369) : 645,	(19,369) : 285,	(20,369) : 405,
(1,370) : 246,	(2,370) : 574,	(3,370) : 120,	(4,370) : 227,	(5,370) : 89,	(6,370) : 318,	(7,370) : 561,	(8,370) : 383,	(9,370) : 347,	(10,370) : 676,	(11,370) : 241,	(12,370) : 200,	(13,370) : 284,	(14,370) : 638,	(15,370) : 678,	(16,370) : 229,	(17,370) : 548,	(18,370) : 134,	(19,370) : 395,	(20,370) : 465,
(1,371) : 219,	(2,371) : 627,	(3,371) : 170,	(4,371) : 197,	(5,371) : 137,	(6,371) : 276,	(7,371) : 599,	(8,371) : 388,	(9,371) : 401,	(10,371) : 738,	(11,371) : 284,	(12,371) : 137,	(13,371) : 253,	(14,371) : 687,	(15,371) : 734,	(16,371) : 207,	(17,371) : 610,	(18,371) : 144,	(19,371) : 456,	(20,371) : 527,
(1,372) : 8,	(2,372) : 619,	(3,372) : 357,	(4,372) : 18,	(5,372) : 327,	(6,372) : 105,	(7,372) : 516,	(8,372) : 223,	(9,372) : 417,	(10,372) : 799,	(11,372) : 279,	(12,372) : 217,	(13,372) : 47,	(14,372) : 653,	(15,372) : 742,	(16,372) : 19,	(17,372) : 697,	(18,372) : 355,	(19,372) : 523,	(20,372) : 631,
(1,373) : 618,	(2,373) : 81,	(3,373) : 513,	(4,373) : 621,	(5,373) : 525,	(6,373) : 719,	(7,373) : 287,	(8,373) : 509,	(9,373) : 201,	(10,373) : 214,	(11,373) : 339,	(12,373) : 725,	(13,373) : 649,	(14,373) : 174,	(15,373) : 140,	(16,373) : 596,	(17,373) : 212,	(18,373) : 617,	(19,373) : 171,	(20,373) : 250,
(1,374) : 134,	(2,374) : 543,	(3,374) : 237,	(4,374) : 121,	(5,374) : 210,	(6,374) : 225,	(7,374) : 484,	(8,374) : 262,	(9,374) : 325,	(10,374) : 694,	(11,374) : 190,	(12,374) : 213,	(13,374) : 173,	(14,374) : 592,	(15,374) : 660,	(16,374) : 112,	(17,374) : 583,	(18,374) : 258,	(19,374) : 413,	(20,374) : 512,
(1,375) : 531,	(2,375) : 504,	(3,375) : 192,	(4,375) : 517,	(5,375) : 224,	(6,375) : 616,	(7,375) : 608,	(8,375) : 590,	(9,375) : 336,	(10,375) : 476,	(11,375) : 351,	(12,375) : 487,	(13,375) : 571,	(14,375) : 593,	(15,375) : 561,	(16,375) : 510,	(17,375) : 329,	(18,375) : 278,	(19,375) : 265,	(20,375) : 234,
(1,376) : 375,	(2,376) : 789,	(3,376) : 734,	(4,376) : 397,	(5,376) : 706,	(6,376) : 341,	(7,376) : 594,	(8,376) : 322,	(9,376) : 668,	(10,376) : 1035,	(11,376) : 560,	(12,376) : 565,	(13,376) : 342,	(14,376) : 777,	(15,376) : 913,	(16,376) : 390,	(17,376) : 973,	(18,376) : 738,	(19,376) : 801,	(20,376) : 932,
(1,377) : 346,	(2,377) : 639,	(3,377) : 84,	(4,377) : 325,	(5,377) : 68,	(6,377) : 404,	(7,377) : 654,	(8,377) : 494,	(9,377) : 417,	(10,377) : 701,	(11,377) : 333,	(12,377) : 229,	(13,377) : 382,	(14,377) : 711,	(15,377) : 732,	(16,377) : 332,	(17,377) : 562,	(18,377) : 26,	(19,377) : 433,	(20,377) : 471,
(1,378) : 290,	(2,378) : 706,	(3,378) : 189,	(4,378) : 266,	(5,378) : 162,	(6,378) : 325,	(7,378) : 689,	(8,378) : 476,	(9,378) : 480,	(10,378) : 796,	(11,378) : 371,	(12,378) : 122,	(13,378) : 319,	(14,378) : 770,	(15,378) : 809,	(16,378) : 282,	(17,378) : 662,	(18,378) : 108,	(19,378) : 520,	(20,378) : 573,
(1,379) : 671,	(2,379) : 136,	(3,379) : 533,	(4,379) : 672,	(5,379) : 548,	(6,379) : 772,	(7,379) : 353,	(8,379) : 572,	(9,379) : 250,	(10,379) : 147,	(11,379) : 389,	(12,379) : 764,	(13,379) : 703,	(14,379) : 221,	(15,379) : 132,	(16,379) : 648,	(17,379) : 161,	(18,379) : 638,	(19,379) : 182,	(20,379) : 220,
(1,380) : 315,	(2,380) : 787,	(3,380) : 273,	(4,380) : 290,	(5,380) : 248,	(6,380) : 322,	(7,380) : 758,	(8,380) : 520,	(9,380) : 561,	(10,380) : 882,	(11,380) : 446,	(12,380) : 98,	(13,380) : 335,	(14,380) : 849,	(15,380) : 892,	(16,380) : 313,	(17,380) : 748,	(18,380) : 182,	(19,380) : 605,	(20,380) : 659,
(1,381) : 301,	(2,381) : 457,	(3,381) : 128,	(4,381) : 289,	(5,381) : 122,	(6,381) : 392,	(7,381) : 471,	(8,381) : 365,	(9,381) : 232,	(10,381) : 556,	(11,381) : 154,	(12,381) : 318,	(13,381) : 340,	(14,381) : 526,	(15,381) : 558,	(16,381) : 278,	(17,381) : 431,	(18,381) : 213,	(19,381) : 274,	(20,381) : 353,
(1,382) : 612,	(2,382) : 30,	(3,382) : 572,	(4,382) : 620,	(5,382) : 579,	(6,382) : 709,	(7,382) : 195,	(8,382) : 468,	(9,382) : 231,	(10,382) : 300,	(11,382) : 351,	(12,382) : 749,	(13,382) : 638,	(14,382) : 71,	(15,382) : 144,	(16,382) : 592,	(17,382) : 320,	(18,382) : 672,	(19,382) : 258,	(20,382) : 356,
(1,383) : 644,	(2,383) : 572,	(3,383) : 292,	(4,383) : 629,	(5,383) : 324,	(6,383) : 726,	(7,383) : 703,	(8,383) : 705,	(9,383) : 434,	(10,383) : 488,	(11,383) : 464,	(12,383) : 582,	(13,383) : 684,	(14,383) : 665,	(15,383) : 609,	(16,383) : 623,	(17,383) : 345,	(18,383) : 358,	(19,383) : 339,	(20,383) : 259,
(1,384) : 426,	(2,384) : 257,	(3,384) : 307,	(4,384) : 424,	(5,384) : 314,	(6,384) : 527,	(7,384) : 323,	(8,384) : 376,	(9,384) : 51,	(10,384) : 378,	(11,384) : 146,	(12,384) : 510,	(13,384) : 461,	(14,384) : 333,	(15,384) : 356,	(16,384) : 403,	(17,384) : 281,	(18,384) : 407,	(19,384) : 105,	(20,384) : 236,
(1,385) : 205,	(2,385) : 699,	(3,385) : 246,	(4,385) : 181,	(5,385) : 215,	(6,385) : 231,	(7,385) : 652,	(8,385) : 407,	(9,385) : 474,	(10,385) : 820,	(11,385) : 349,	(12,385) : 55,	(13,385) : 230,	(14,385) : 754,	(15,385) : 810,	(16,385) : 201,	(17,385) : 693,	(18,385) : 193,	(19,385) : 537,	(20,385) : 610,
(1,386) : 341,	(2,386) : 395,	(3,386) : 536,	(4,386) : 360,	(5,386) : 521,	(6,386) : 413,	(7,386) : 207,	(8,386) : 128,	(9,386) : 314,	(10,386) : 649,	(11,386) : 265,	(12,386) : 549,	(13,386) : 348,	(14,386) : 385,	(15,386) : 518,	(16,386) : 331,	(17,386) : 607,	(18,386) : 596,	(19,386) : 448,	(20,386) : 586,
(1,387) : 595,	(2,387) : 74,	(3,387) : 582,	(4,387) : 604,	(5,387) : 586,	(6,387) : 690,	(7,387) : 151,	(8,387) : 439,	(9,387) : 237,	(10,387) : 345,	(11,387) : 344,	(12,387) : 743,	(13,387) : 619,	(14,387) : 61,	(15,387) : 182,	(16,387) : 575,	(17,387) : 362,	(18,387) : 679,	(19,387) : 286,	(20,387) : 393,
(1,388) : 274,	(2,388) : 646,	(3,388) : 139,	(4,388) : 251,	(5,388) : 109,	(6,388) : 326,	(7,388) : 635,	(8,388) : 440,	(9,388) : 420,	(10,388) : 738,	(11,388) : 315,	(12,388) : 156,	(13,388) : 307,	(14,388) : 712,	(15,388) : 749,	(16,388) : 262,	(17,388) : 605,	(18,388) : 90,	(19,388) : 460,	(20,388) : 517,
(1,389) : 157,	(2,389) : 468,	(3,389) : 360,	(4,389) : 169,	(5,389) : 338,	(6,389) : 252,	(7,389) : 360,	(8,389) : 117,	(9,389) : 284,	(10,389) : 669,	(11,389) : 157,	(12,389) : 345,	(13,389) : 180,	(14,389) : 497,	(15,389) : 593,	(16,389) : 140,	(17,389) : 583,	(18,389) : 401,	(19,389) : 407,	(20,389) : 532,
(1,390) : 697,	(2,390) : 290,	(3,390) : 473,	(4,390) : 693,	(5,390) : 496,	(6,390) : 797,	(7,390) : 489,	(8,390) : 647,	(9,390) : 301,	(10,390) : 151,	(11,390) : 422,	(12,390) : 743,	(13,390) : 734,	(14,390) : 379,	(15,390) : 281,	(16,390) : 674,	(17,390) : 5,	(18,390) : 576,	(19,390) : 171,	(20,390) : 91,
(1,391) : 170,	(2,391) : 668,	(3,391) : 245,	(4,391) : 146,	(5,391) : 213,	(6,391) : 208,	(7,391) : 615,	(8,391) : 368,	(9,391) : 444,	(10,391) : 798,	(11,391) : 316,	(12,391) : 83,	(13,391) : 198,	(14,391) : 721,	(15,391) : 781,	(16,391) : 164,	(17,391) : 675,	(18,391) : 208,	(19,391) : 515,	(20,391) : 595,
(1,392) : 170,	(2,392) : 490,	(3,392) : 232,	(4,392) : 163,	(5,392) : 209,	(6,392) : 268,	(7,392) : 438,	(8,392) : 247,	(9,392) : 271,	(10,392) : 644,	(11,392) : 137,	(12,392) : 264,	(13,392) : 209,	(14,392) : 540,	(15,392) : 606,	(16,392) : 147,	(17,392) : 535,	(18,392) : 273,	(19,392) : 363,	(20,392) : 468,
(1,393) : 501,	(2,393) : 469,	(3,393) : 179,	(4,393) : 488,	(5,393) : 208,	(6,393) : 589,	(7,393) : 568,	(8,393) : 552,	(9,393) : 296,	(10,393) : 459,	(11,393) : 311,	(12,393) : 472,	(13,393) : 541,	(14,393) : 558,	(15,393) : 532,	(16,393) : 480,	(17,393) : 314,	(18,393) : 274,	(19,393) : 232,	(20,393) : 220,
(1,394) : 557,	(2,394) : 459,	(3,394) : 239,	(4,394) : 544,	(5,394) : 269,	(6,394) : 646,	(7,394) : 580,	(8,394) : 594,	(9,394) : 311,	(10,394) : 417,	(11,394) : 349,	(12,394) : 533,	(13,394) : 597,	(14,394) : 550,	(15,394) : 508,	(16,394) : 535,	(17,394) : 270,	(18,394) : 331,	(19,394) : 221,	(20,394) : 175,
(1,395) : 230,	(2,395) : 685,	(3,395) : 210,	(4,395) : 206,	(5,395) : 179,	(6,395) : 266,	(7,395) : 650,	(8,395) : 421,	(9,395) : 459,	(10,395) : 795,	(11,395) : 339,	(12,395) : 85,	(13,395) : 258,	(14,395) : 744,	(15,395) : 793,	(16,395) : 223,	(17,395) : 666,	(18,395) : 155,	(19,395) : 514,	(20,395) : 581,
(1,396) : 327,	(2,396) : 524,	(3,396) : 602,	(4,396) : 350,	(5,396) : 581,	(6,396) : 370,	(7,396) : 324,	(8,396) : 126,	(9,396) : 432,	(10,396) : 778,	(11,396) : 357,	(12,396) : 549,	(13,396) : 319,	(14,396) : 507,	(15,396) : 645,	(16,396) : 325,	(17,396) : 732,	(18,396) : 643,	(19,396) : 568,	(20,396) : 705,
(1,397) : 392,	(2,397) : 352,	(3,397) : 558,	(4,397) : 410,	(5,397) : 546,	(6,397) : 467,	(7,397) : 153,	(8,397) : 183,	(9,397) : 302,	(10,397) : 613,	(11,397) : 279,	(12,397) : 594,	(13,397) : 401,	(14,397) : 334,	(15,397) : 472,	(16,397) : 380,	(17,397) : 582,	(18,397) : 625,	(19,397) : 432,	(20,397) : 569,
(1,398) : 74,	(2,398) : 593,	(3,398) : 290,	(4,398) : 57,	(5,398) : 260,	(6,398) : 160,	(7,398) : 514,	(8,398) : 252,	(9,398) : 380,	(10,398) : 756,	(11,398) : 243,	(12,398) : 187,	(13,398) : 113,	(14,398) : 635,	(15,398) : 712,	(16,398) : 57,	(17,398) : 647,	(18,398) : 292,	(19,398) : 475,	(20,398) : 577,
(1,399) : 82,	(2,399) : 662,	(3,399) : 447,	(4,399) : 102,	(5,399) : 417,	(6,399) : 77,	(7,399) : 532,	(8,399) : 212,	(9,399) : 477,	(10,399) : 862,	(11,399) : 342,	(12,399) : 278,	(13,399) : 45,	(14,399) : 685,	(15,399) : 788,	(16,399) : 102,	(17,399) : 768,	(18,399) : 442,	(19,399) : 592,	(20,399) : 708,
(1,400) : 151,	(2,400) : 503,	(3,400) : 412,	(4,400) : 170,	(5,400) : 389,	(6,400) : 230,	(7,400) : 371,	(8,400) : 74,	(9,400) : 334,	(10,400) : 716,	(11,400) : 213,	(12,400) : 362,	(13,400) : 161,	(14,400) : 522,	(15,400) : 629,	(16,400) : 141,	(17,400) : 636,	(18,400) : 446,	(19,400) : 461,	(20,400) : 588,
(1,401) : 265,	(2,401) : 793,	(3,401) : 318,	(4,401) : 242,	(5,401) : 289,	(6,401) : 256,	(7,401) : 742,	(8,401) : 481,	(9,401) : 568,	(10,401) : 910,	(11,401) : 443,	(12,401) : 46,	(13,401) : 279,	(14,401) : 848,	(15,401) : 904,	(16,401) : 268,	(17,401) : 781,	(18,401) : 240,	(19,401) : 629,	(20,401) : 695,
(1,402) : 251,	(2,402) : 471,	(3,402) : 155,	(4,402) : 240,	(5,402) : 139,	(6,402) : 344,	(7,402) : 461,	(8,402) : 324,	(9,402) : 245,	(10,402) : 590,	(11,402) : 139,	(12,402) : 287,	(13,402) : 291,	(14,402) : 534,	(15,402) : 578,	(16,402) : 229,	(17,402) : 471,	(18,402) : 221,	(19,402) : 307,	(20,402) : 396,
(1,403) : 290,	(2,403) : 724,	(3,403) : 210,	(4,403) : 266,	(5,403) : 183,	(6,403) : 317,	(7,403) : 702,	(8,403) : 482,	(9,403) : 498,	(10,403) : 817,	(11,403) : 386,	(12,403) : 107,	(13,403) : 316,	(14,403) : 787,	(15,403) : 828,	(16,403) : 283,	(17,403) : 683,	(18,403) : 127,	(19,403) : 540,	(20,403) : 594,
(1,404) : 538,	(2,404) : 284,	(3,404) : 326,	(4,404) : 533,	(5,404) : 345,	(6,404) : 637,	(7,404) : 421,	(8,404) : 508,	(9,404) : 172,	(10,404) : 293,	(11,404) : 271,	(12,404) : 584,	(13,404) : 575,	(14,404) : 375,	(15,404) : 341,	(16,404) : 514,	(17,404) : 167,	(18,404) : 432,	(19,404) : 48,	(20,404) : 105,
(1,405) : 340,	(2,405) : 771,	(3,405) : 238,	(4,405) : 316,	(5,405) : 215,	(6,405) : 359,	(7,405) : 756,	(8,405) : 536,	(9,405) : 545,	(10,405) : 853,	(11,405) : 438,	(12,405) : 138,	(13,405) : 365,	(14,405) : 837,	(15,405) : 872,	(16,405) : 335,	(17,405) : 715,	(18,405) : 139,	(19,405) : 579,	(20,405) : 625,
(1,406) : 636,	(2,406) : 88,	(3,406) : 525,	(4,406) : 639,	(5,406) : 537,	(6,406) : 737,	(7,406) : 301,	(8,406) : 527,	(9,406) : 219,	(10,406) : 197,	(11,406) : 356,	(12,406) : 741,	(13,406) : 667,	(14,406) : 178,	(15,406) : 127,	(16,406) : 614,	(17,406) : 203,	(18,406) : 629,	(19,406) : 180,	(20,406) : 248,
(1,407) : 543,	(2,407) : 274,	(3,407) : 337,	(4,407) : 539,	(5,407) : 356,	(6,407) : 643,	(7,407) : 415,	(8,407) : 509,	(9,407) : 171,	(10,407) : 283,	(11,407) : 274,	(12,407) : 592,	(13,407) : 581,	(14,407) : 366,	(15,407) : 330,	(16,407) : 520,	(17,407) : 159,	(18,407) : 442,	(19,407) : 41,	(20,407) : 103,
(1,408) : 188,	(2,408) : 460,	(3,408) : 409,	(4,408) : 204,	(5,408) : 389,	(6,408) : 272,	(7,408) : 330,	(8,408) : 68,	(9,408) : 297,	(10,408) : 677,	(11,408) : 184,	(12,408) : 390,	(13,408) : 202,	(14,408) : 479,	(15,408) : 586,	(16,408) : 175,	(17,408) : 601,	(18,408) : 453,	(19,408) : 426,	(20,408) : 556,
(1,409) : 540,	(2,409) : 208,	(3,409) : 379,	(4,409) : 538,	(5,409) : 393,	(6,409) : 641,	(7,409) : 353,	(8,409) : 479,	(9,409) : 133,	(10,409) : 265,	(11,409) : 260,	(12,409) : 613,	(13,409) : 575,	(14,409) : 300,	(15,409) : 275,	(16,409) : 517,	(17,409) : 172,	(18,409) : 484,	(19,409) : 31,	(20,409) : 151,
(1,410) : 589,	(2,410) : 53,	(3,410) : 512,	(4,410) : 593,	(5,410) : 520,	(6,410) : 689,	(7,410) : 240,	(8,410) : 469,	(9,410) : 181,	(10,410) : 260,	(11,410) : 314,	(12,410) : 708,	(13,410) : 618,	(14,410) : 144,	(15,410) : 156,	(16,410) : 567,	(17,410) : 255,	(18,410) : 613,	(19,410) : 186,	(20,410) : 284,
(1,411) : 182,	(2,411) : 460,	(3,411) : 253,	(4,411) : 179,	(5,411) : 233,	(6,411) : 282,	(7,411) : 404,	(8,411) : 224,	(9,411) : 246,	(10,411) : 623,	(11,411) : 108,	(12,411) : 297,	(13,411) : 219,	(14,411) : 508,	(15,411) : 578,	(16,411) : 158,	(17,411) : 520,	(18,411) : 303,	(19,411) : 346,	(20,411) : 457,
(1,412) : 275,	(2,412) : 506,	(3,412) : 546,	(4,412) : 298,	(5,412) : 525,	(6,412) : 327,	(7,412) : 322,	(8,412) : 70,	(9,412) : 393,	(10,412) : 752,	(11,412) : 308,	(12,412) : 496,	(13,412) : 270,	(14,412) : 500,	(15,412) : 630,	(16,412) : 272,	(17,412) : 697,	(18,412) : 587,	(19,412) : 529,	(20,412) : 665,
(1,413) : 207,	(2,413) : 742,	(3,413) : 298,	(4,413) : 183,	(5,413) : 267,	(6,413) : 208,	(7,413) : 684,	(8,413) : 421,	(9,413) : 519,	(10,413) : 870,	(11,413) : 390,	(12,413) : 16,	(13,413) : 223,	(14,413) : 794,	(15,413) : 855,	(16,413) : 209,	(17,413) : 746,	(18,413) : 238,	(19,413) : 588,	(20,413) : 663,
(1,414) : 110,	(2,414) : 513,	(3,414) : 313,	(4,414) : 114,	(5,414) : 288,	(6,414) : 211,	(7,414) : 423,	(8,414) : 176,	(9,414) : 311,	(10,414) : 694,	(11,414) : 173,	(12,414) : 277,	(13,414) : 143,	(14,414) : 550,	(15,414) : 635,	(16,414) : 88,	(17,414) : 596,	(18,414) : 342,	(19,414) : 421,	(20,414) : 536,
(1,415) : 683,	(2,415) : 511,	(3,415) : 354,	(4,415) : 670,	(5,415) : 385,	(6,415) : 772,	(7,415) : 666,	(8,415) : 714,	(9,415) : 409,	(10,415) : 396,	(11,415) : 468,	(12,415) : 649,	(13,415) : 723,	(14,415) : 605,	(15,415) : 532,	(16,415) : 661,	(17,415) : 257,	(18,415) : 435,	(19,415) : 292,	(20,415) : 183,
(1,416) : 247,	(2,416) : 624,	(3,416) : 581,	(4,416) : 272,	(5,416) : 555,	(6,416) : 260,	(7,416) : 443,	(8,416) : 146,	(9,416) : 492,	(10,416) : 863,	(11,416) : 385,	(12,416) : 465,	(13,416) : 226,	(14,416) : 620,	(15,416) : 749,	(16,416) : 255,	(17,416) : 797,	(18,416) : 600,	(19,416) : 625,	(20,416) : 756,
(1,417) : 471,	(2,417) : 287,	(3,417) : 286,	(4,417) : 467,	(5,417) : 301,	(6,417) : 571,	(7,417) : 389,	(8,417) : 445,	(9,417) : 122,	(10,417) : 348,	(11,417) : 204,	(12,417) : 526,	(13,417) : 509,	(14,417) : 373,	(15,417) : 366,	(16,417) : 448,	(17,417) : 231,	(18,417) : 391,	(19,417) : 66,	(20,417) : 170,
(1,418) : 339,	(2,418) : 748,	(3,418) : 210,	(4,418) : 315,	(5,418) : 188,	(6,418) : 366,	(7,418) : 738,	(8,418) : 528,	(9,418) : 522,	(10,418) : 825,	(11,418) : 419,	(12,418) : 150,	(13,418) : 366,	(14,418) : 815,	(15,418) : 847,	(16,418) : 332,	(17,418) : 687,	(18,418) : 111,	(19,418) : 553,	(20,418) : 596,
(1,419) : 85,	(2,419) : 673,	(3,419) : 332,	(4,419) : 63,	(5,419) : 300,	(6,419) : 108,	(7,419) : 587,	(8,419) : 303,	(9,419) : 461,	(10,419) : 834,	(11,419) : 323,	(12,419) : 139,	(13,419) : 101,	(14,419) : 714,	(15,419) : 793,	(16,419) : 91,	(17,419) : 722,	(18,419) : 308,	(19,419) : 552,	(20,419) : 649,
(1,420) : 307,	(2,420) : 509,	(3,420) : 77,	(4,420) : 292,	(5,420) : 67,	(6,420) : 391,	(7,420) : 525,	(8,420) : 401,	(9,420) : 285,	(10,420) : 593,	(11,420) : 207,	(12,420) : 286,	(13,420) : 347,	(14,420) : 580,	(15,420) : 607,	(16,420) : 287,	(17,420) : 463,	(18,420) : 158,	(19,420) : 315,	(20,420) : 378,
(1,421) : 245,	(2,421) : 419,	(3,421) : 220,	(4,421) : 240,	(5,421) : 207,	(6,421) : 345,	(7,421) : 394,	(8,421) : 270,	(9,421) : 196,	(10,421) : 565,	(11,421) : 73,	(12,421) : 331,	(13,421) : 283,	(14,421) : 476,	(15,421) : 532,	(16,421) : 222,	(17,421) : 458,	(18,421) : 290,	(19,421) : 285,	(20,421) : 393,
(1,422) : 84,	(2,422) : 613,	(3,422) : 435,	(4,422) : 108,	(5,422) : 407,	(6,422) : 125,	(7,422) : 480,	(8,422) : 161,	(9,422) : 433,	(10,422) : 817,	(11,422) : 301,	(12,422) : 303,	(13,422) : 67,	(14,422) : 634,	(15,422) : 738,	(16,422) : 93,	(17,422) : 728,	(18,422) : 443,	(19,422) : 552,	(20,422) : 671,
(1,423) : 269,	(2,423) : 498,	(3,423) : 535,	(4,423) : 291,	(5,423) : 514,	(6,423) : 323,	(7,423) : 317,	(8,423) : 59,	(9,423) : 382,	(10,423) : 742,	(11,423) : 296,	(12,423) : 489,	(13,423) : 265,	(14,423) : 493,	(15,423) : 622,	(16,423) : 265,	(17,423) : 686,	(18,423) : 576,	(19,423) : 518,	(20,423) : 653,
(1,424) : 373,	(2,424) : 878,	(3,424) : 361,	(4,424) : 350,	(5,424) : 337,	(6,424) : 357,	(7,424) : 842,	(8,424) : 588,	(9,424) : 652,	(10,424) : 973,	(11,424) : 534,	(12,424) : 152,	(13,424) : 386,	(14,424) : 938,	(15,424) : 984,	(16,424) : 376,	(17,424) : 837,	(18,424) : 263,	(19,424) : 697,	(20,424) : 747,
(1,425) : 177,	(2,425) : 570,	(3,425) : 495,	(4,425) : 200,	(5,425) : 469,	(6,425) : 218,	(7,425) : 411,	(8,425) : 86,	(9,425) : 418,	(10,425) : 797,	(11,425) : 304,	(12,425) : 399,	(13,425) : 165,	(14,425) : 578,	(15,425) : 696,	(16,425) : 179,	(17,425) : 723,	(18,425) : 518,	(19,425) : 548,	(20,425) : 678,
(1,426) : 202,	(2,426) : 562,	(3,426) : 513,	(4,426) : 226,	(5,426) : 488,	(6,426) : 243,	(7,426) : 395,	(8,426) : 77,	(9,426) : 419,	(10,426) : 794,	(11,426) : 310,	(12,426) : 424,	(13,426) : 191,	(14,426) : 566,	(15,426) : 688,	(16,426) : 203,	(17,426) : 725,	(18,426) : 539,	(19,426) : 551,	(20,426) : 682,
(1,427) : 164,	(2,427) : 749,	(3,427) : 354,	(4,427) : 144,	(5,427) : 322,	(6,427) : 140,	(7,427) : 669,	(8,427) : 383,	(9,427) : 532,	(10,427) : 897,	(11,427) : 396,	(12,427) : 86,	(13,427) : 169,	(14,427) : 793,	(15,427) : 867,	(16,427) : 173,	(17,427) : 779,	(18,427) : 305,	(19,427) : 614,	(20,427) : 701,
(1,428) : 85,	(2,428) : 674,	(3,428) : 333,	(4,428) : 63,	(5,428) : 301,	(6,428) : 107,	(7,428) : 588,	(8,428) : 303,	(9,428) : 462,	(10,428) : 835,	(11,428) : 324,	(12,428) : 138,	(13,428) : 101,	(14,428) : 715,	(15,428) : 794,	(16,428) : 92,	(17,428) : 723,	(18,428) : 309,	(19,428) : 554,	(20,428) : 650,
(1,429) : 608,	(2,429) : 59,	(3,429) : 519,	(4,429) : 612,	(5,429) : 529,	(6,429) : 709,	(7,429) : 263,	(8,429) : 492,	(9,429) : 196,	(10,429) : 236,	(11,429) : 332,	(12,429) : 722,	(13,429) : 638,	(14,429) : 153,	(15,429) : 140,	(16,429) : 587,	(17,429) : 236,	(18,429) : 621,	(19,429) : 184,	(20,429) : 271,
(1,430) : 247,	(2,430) : 518,	(3,430) : 527,	(4,430) : 269,	(5,430) : 505,	(6,430) : 298,	(7,430) : 342,	(8,430) : 53,	(9,430) : 393,	(10,430) : 758,	(11,430) : 298,	(12,430) : 468,	(13,430) : 241,	(14,430) : 516,	(15,430) : 643,	(16,430) : 244,	(17,430) : 698,	(18,430) : 564,	(19,430) : 527,	(20,430) : 662,
(1,431) : 276,	(2,431) : 622,	(3,431) : 117,	(4,431) : 255,	(5,431) : 86,	(6,431) : 335,	(7,431) : 615,	(8,431) : 432,	(9,431) : 396,	(10,431) : 712,	(11,431) : 294,	(12,431) : 179,	(13,431) : 311,	(14,431) : 689,	(15,431) : 724,	(16,431) : 262,	(17,431) : 579,	(18,431) : 88,	(19,431) : 434,	(20,431) : 492,
(1,432) : 650,	(2,432) : 434,	(3,432) : 349,	(4,432) : 640,	(5,432) : 378,	(6,432) : 744,	(7,432) : 593,	(8,432) : 661,	(9,432) : 342,	(10,432) : 332,	(11,432) : 415,	(12,432) : 641,	(13,432) : 690,	(14,432) : 528,	(15,432) : 457,	(16,432) : 628,	(17,432) : 188,	(18,432) : 441,	(19,432) : 219,	(20,432) : 107,
(1,433) : 247,	(2,433) : 696,	(3,433) : 210,	(4,433) : 222,	(5,433) : 179,	(6,433) : 279,	(7,433) : 665,	(8,433) : 439,	(9,433) : 470,	(10,433) : 802,	(11,433) : 353,	(12,433) : 87,	(13,433) : 274,	(14,433) : 756,	(15,433) : 803,	(16,433) : 240,	(17,433) : 671,	(18,433) : 146,	(19,433) : 521,	(20,433) : 585,
(1,434) : 735,	(2,434) : 493,	(3,434) : 419,	(4,434) : 724,	(5,434) : 450,	(6,434) : 827,	(7,434) : 667,	(8,434) : 748,	(9,434) : 424,	(10,434) : 343,	(11,434) : 502,	(12,434) : 714,	(13,434) : 774,	(14,434) : 586,	(15,434) : 497,	(16,434) : 712,	(17,434) : 216,	(18,434) : 505,	(19,434) : 296,	(20,434) : 165,
(1,435) : 592,	(2,435) : 209,	(3,435) : 416,	(4,435) : 590,	(5,435) : 433,	(6,435) : 693,	(7,435) : 382,	(8,435) : 530,	(9,435) : 184,	(10,435) : 214,	(11,435) : 312,	(12,435) : 661,	(13,435) : 628,	(14,435) : 302,	(15,435) : 250,	(16,435) : 569,	(17,435) : 123,	(18,435) : 521,	(19,435) : 69,	(20,435) : 124,
(1,436) : 354,	(2,436) : 837,	(3,436) : 315,	(4,436) : 330,	(5,436) : 292,	(6,436) : 350,	(7,436) : 807,	(8,436) : 564,	(9,436) : 610,	(10,436) : 928,	(11,436) : 496,	(12,436) : 132,	(13,436) : 371,	(14,436) : 899,	(15,436) : 941,	(16,436) : 354,	(17,436) : 792,	(18,436) : 218,	(19,436) : 653,	(20,436) : 702,
(1,437) : 559,	(2,437) : 360,	(3,437) : 295,	(4,437) : 551,	(5,437) : 320,	(6,437) : 655,	(7,437) : 498,	(8,437) : 558,	(9,437) : 239,	(10,437) : 327,	(11,437) : 313,	(12,437) : 574,	(13,437) : 598,	(14,437) : 452,	(15,437) : 407,	(16,437) : 536,	(17,437) : 184,	(18,437) : 398,	(19,437) : 126,	(20,437) : 91,
(1,438) : 146,	(2,438) : 633,	(3,438) : 239,	(4,438) : 123,	(5,438) : 207,	(6,438) : 201,	(7,438) : 578,	(8,438) : 335,	(9,438) : 411,	(10,438) : 770,	(11,438) : 281,	(12,438) : 119,	(13,438) : 179,	(14,438) : 685,	(15,438) : 747,	(16,438) : 137,	(17,438) : 650,	(18,438) : 219,	(19,438) : 486,	(20,438) : 572,
(1,439) : 479,	(2,439) : 229,	(3,439) : 337,	(4,439) : 478,	(5,439) : 348,	(6,439) : 580,	(7,439) : 333,	(8,439) : 424,	(9,439) : 80,	(10,439) : 325,	(11,439) : 199,	(12,439) : 558,	(13,439) : 514,	(14,439) : 313,	(15,439) : 316,	(16,439) : 456,	(17,439) : 230,	(18,439) : 441,	(19,439) : 56,	(20,439) : 193,
(1,440) : 349,	(2,440) : 487,	(3,440) : 602,	(4,440) : 372,	(5,440) : 583,	(6,440) : 401,	(7,440) : 283,	(8,440) : 136,	(9,440) : 408,	(10,440) : 744,	(11,440) : 345,	(12,440) : 569,	(13,440) : 345,	(14,440) : 467,	(15,440) : 607,	(16,440) : 345,	(17,440) : 703,	(18,440) : 650,	(19,440) : 544,	(20,440) : 681,
(1,441) : 297,	(2,441) : 518,	(3,441) : 573,	(4,441) : 320,	(5,441) : 552,	(6,441) : 343,	(7,441) : 327,	(8,441) : 97,	(9,441) : 415,	(10,441) : 768,	(11,441) : 333,	(12,441) : 518,	(13,441) : 290,	(14,441) : 508,	(15,441) : 641,	(16,441) : 295,	(17,441) : 717,	(18,441) : 613,	(19,441) : 551,	(20,441) : 687,
(1,442) : 647,	(2,442) : 485,	(3,442) : 324,	(4,442) : 635,	(5,442) : 354,	(6,442) : 737,	(7,442) : 633,	(8,442) : 676,	(9,442) : 373,	(10,442) : 388,	(11,442) : 430,	(12,442) : 619,	(13,442) : 687,	(14,442) : 578,	(15,442) : 512,	(16,442) : 625,	(17,442) : 245,	(18,442) : 409,	(19,442) : 260,	(20,442) : 162,
(1,443) : 260,	(2,443) : 535,	(3,443) : 550,	(4,443) : 284,	(5,443) : 528,	(6,443) : 304,	(7,443) : 354,	(8,443) : 77,	(9,443) : 416,	(10,443) : 779,	(11,443) : 323,	(12,443) : 482,	(13,443) : 252,	(14,443) : 531,	(15,443) : 660,	(16,443) : 260,	(17,443) : 720,	(18,443) : 585,	(19,443) : 551,	(20,443) : 686,
(1,444) : 149,	(2,444) : 672,	(3,444) : 511,	(4,444) : 171,	(5,444) : 482,	(6,444) : 134,	(7,444) : 521,	(8,444) : 195,	(9,444) : 504,	(10,444) : 887,	(11,444) : 376,	(12,444) : 349,	(13,444) : 116,	(14,444) : 686,	(15,444) : 798,	(16,444) : 166,	(17,444) : 803,	(18,444) : 512,	(19,444) : 627,	(20,444) : 749,
(1,445) : 79,	(2,445) : 685,	(3,445) : 358,	(4,445) : 62,	(5,445) : 326,	(6,445) : 82,	(7,445) : 591,	(8,445) : 298,	(9,445) : 476,	(10,445) : 852,	(11,445) : 338,	(12,445) : 154,	(13,445) : 86,	(14,445) : 724,	(15,445) : 806,	(16,445) : 92,	(17,445) : 742,	(18,445) : 333,	(19,445) : 572,	(20,445) : 671,
(1,446) : 171,	(2,446) : 596,	(3,446) : 199,	(4,446) : 150,	(5,446) : 167,	(6,446) : 240,	(7,446) : 553,	(8,446) : 334,	(9,446) : 372,	(10,446) : 725,	(11,446) : 246,	(12,446) : 154,	(13,446) : 207,	(14,446) : 651,	(15,446) : 708,	(16,446) : 156,	(17,446) : 604,	(18,446) : 194,	(19,446) : 442,	(20,446) : 526,
(1,447) : 266,	(2,447) : 564,	(3,447) : 98,	(4,447) : 248,	(5,447) : 68,	(6,447) : 340,	(7,447) : 560,	(8,447) : 395,	(9,447) : 338,	(10,447) : 659,	(11,447) : 239,	(12,447) : 219,	(13,447) : 305,	(14,447) : 631,	(15,447) : 666,	(16,447) : 249,	(17,447) : 530,	(18,447) : 126,	(19,447) : 380,	(20,447) : 445,
(1,448) : 562,	(2,448) : 86,	(3,448) : 478,	(4,448) : 565,	(5,448) : 487,	(6,448) : 662,	(7,448) : 246,	(8,448) : 450,	(9,448) : 149,	(10,448) : 270,	(11,448) : 284,	(12,448) : 676,	(13,448) : 592,	(14,448) : 173,	(15,448) : 187,	(16,448) : 540,	(17,448) : 245,	(18,448) : 580,	(19,448) : 156,	(20,448) : 264,
(1,449) : 623,	(2,449) : 412,	(3,449) : 330,	(4,449) : 613,	(5,449) : 358,	(6,449) : 717,	(7,449) : 565,	(8,449) : 631,	(9,449) : 311,	(10,449) : 328,	(11,449) : 385,	(12,449) : 620,	(13,449) : 662,	(14,449) : 505,	(15,449) : 441,	(16,449) : 600,	(17,449) : 181,	(18,449) : 426,	(19,449) : 190,	(20,449) : 91,
(1,450) : 96,	(2,450) : 703,	(3,450) : 364,	(4,450) : 80,	(5,450) : 332,	(6,450) : 83,	(7,450) : 609,	(8,450) : 315,	(9,450) : 492,	(10,450) : 867,	(11,450) : 354,	(12,450) : 145,	(13,450) : 99,	(14,450) : 741,	(15,450) : 823,	(16,450) : 110,	(17,450) : 756,	(18,450) : 335,	(19,450) : 586,	(20,450) : 684,
(1,451) : 80,	(2,451) : 544,	(3,451) : 316,	(4,451) : 83,	(5,451) : 290,	(6,451) : 182,	(7,451) : 452,	(8,451) : 190,	(9,451) : 340,	(10,451) : 723,	(11,451) : 202,	(12,451) : 252,	(13,451) : 116,	(14,451) : 581,	(15,451) : 666,	(16,451) : 58,	(17,451) : 623,	(18,451) : 336,	(19,451) : 448,	(20,451) : 560,
(1,452) : 309,	(2,452) : 700,	(3,452) : 170,	(4,452) : 285,	(5,452) : 145,	(6,452) : 347,	(7,452) : 690,	(8,452) : 488,	(9,452) : 474,	(10,452) : 782,	(11,452) : 370,	(12,452) : 147,	(13,452) : 339,	(14,452) : 766,	(15,452) : 800,	(16,452) : 300,	(17,452) : 646,	(18,452) : 84,	(19,452) : 508,	(20,452) : 556,
(1,453) : 66,	(2,453) : 558,	(3,453) : 359,	(4,453) : 81,	(5,453) : 332,	(6,453) : 161,	(7,453) : 449,	(8,453) : 163,	(9,453) : 364,	(10,453) : 748,	(11,453) : 228,	(12,453) : 273,	(13,453) : 91,	(14,453) : 588,	(15,453) : 682,	(16,453) : 52,	(17,453) : 654,	(18,453) : 377,	(19,453) : 478,	(20,453) : 594,
(1,454) : 655,	(2,454) : 572,	(3,454) : 303,	(4,454) : 639,	(5,454) : 336,	(6,454) : 737,	(7,454) : 707,	(8,454) : 714,	(9,454) : 438,	(10,454) : 482,	(11,454) : 472,	(12,454) : 593,	(13,454) : 694,	(14,454) : 665,	(15,454) : 606,	(16,454) : 634,	(17,454) : 340,	(18,454) : 370,	(19,454) : 340,	(20,454) : 256,
(1,455) : 306,	(2,455) : 461,	(3,455) : 546,	(4,455) : 327,	(5,455) : 528,	(6,455) : 366,	(7,455) : 273,	(8,455) : 87,	(9,455) : 362,	(10,455) : 711,	(11,455) : 292,	(12,455) : 522,	(13,455) : 306,	(14,455) : 451,	(15,455) : 584,	(16,455) : 299,	(17,455) : 662,	(18,455) : 595,	(19,455) : 498,	(20,455) : 635,
(1,456) : 619,	(2,456) : 437,	(3,456) : 313,	(4,456) : 608,	(5,456) : 342,	(6,456) : 711,	(7,456) : 585,	(8,456) : 636,	(9,456) : 326,	(10,456) : 356,	(11,456) : 390,	(12,456) : 606,	(13,456) : 658,	(14,456) : 531,	(15,456) : 469,	(16,456) : 596,	(17,456) : 210,	(18,456) : 406,	(19,456) : 211,	(20,456) : 121,
(1,457) : 358,	(2,457) : 747,	(3,457) : 200,	(4,457) : 334,	(5,457) : 181,	(6,457) : 388,	(7,457) : 744,	(8,457) : 543,	(9,457) : 522,	(10,457) : 816,	(11,457) : 424,	(12,457) : 173,	(13,457) : 386,	(14,457) : 816,	(15,457) : 844,	(16,457) : 351,	(17,457) : 677,	(18,457) : 97,	(19,457) : 547,	(20,457) : 585,
(1,458) : 570,	(2,458) : 373,	(3,458) : 298,	(4,458) : 562,	(5,458) : 324,	(6,458) : 666,	(7,458) : 513,	(8,458) : 572,	(9,458) : 255,	(10,458) : 329,	(11,458) : 327,	(12,458) : 581,	(13,458) : 609,	(14,458) : 465,	(15,458) : 417,	(16,458) : 547,	(17,458) : 184,	(18,458) : 399,	(19,458) : 141,	(20,458) : 90,
(1,459) : 60,	(2,459) : 640,	(3,459) : 323,	(4,459) : 36,	(5,459) : 291,	(6,459) : 117,	(7,459) : 553,	(8,459) : 272,	(9,459) : 429,	(10,459) : 805,	(11,459) : 291,	(12,459) : 164,	(13,459) : 88,	(14,459) : 680,	(15,459) : 760,	(16,459) : 60,	(17,459) : 695,	(18,459) : 310,	(19,459) : 524,	(20,459) : 624,
(1,460) : 74,	(2,460) : 554,	(3,460) : 370,	(4,460) : 91,	(5,460) : 343,	(6,460) : 165,	(7,460) : 442,	(8,460) : 151,	(9,460) : 364,	(10,460) : 749,	(11,460) : 229,	(12,460) : 285,	(13,460) : 94,	(14,460) : 583,	(15,460) : 679,	(16,460) : 62,	(17,460) : 656,	(18,460) : 388,	(19,460) : 480,	(20,460) : 598,
(1,461) : 279,	(2,461) : 423,	(3,461) : 490,	(4,461) : 298,	(5,461) : 472,	(6,461) : 353,	(7,461) : 256,	(8,461) : 70,	(9,461) : 306,	(10,461) : 664,	(11,461) : 231,	(12,461) : 487,	(13,461) : 287,	(14,461) : 424,	(15,461) : 548,	(16,461) : 269,	(17,461) : 608,	(18,461) : 543,	(19,461) : 442,	(20,461) : 578,
(1,462) : 184,	(2,462) : 534,	(3,462) : 478,	(4,462) : 207,	(5,462) : 454,	(6,462) : 240,	(7,462) : 377,	(8,462) : 52,	(9,462) : 384,	(10,462) : 761,	(11,462) : 273,	(12,462) : 405,	(13,462) : 180,	(14,462) : 543,	(15,462) : 660,	(16,462) : 182,	(17,462) : 689,	(18,462) : 508,	(19,462) : 515,	(20,462) : 646,
(1,463) : 325,	(2,463) : 429,	(3,463) : 544,	(4,463) : 345,	(5,463) : 527,	(6,463) : 391,	(7,463) : 240,	(8,463) : 106,	(9,463) : 340,	(10,463) : 681,	(11,463) : 280,	(12,463) : 538,	(13,463) : 329,	(14,463) : 418,	(15,463) : 551,	(16,463) : 317,	(17,463) : 636,	(18,463) : 598,	(19,463) : 475,	(20,463) : 613,
(1,464) : 171,	(2,464) : 698,	(3,464) : 275,	(4,464) : 147,	(5,464) : 243,	(6,464) : 192,	(7,464) : 639,	(8,464) : 380,	(9,464) : 476,	(10,464) : 831,	(11,464) : 345,	(12,464) : 58,	(13,464) : 193,	(14,464) : 749,	(15,464) : 812,	(16,464) : 170,	(17,464) : 709,	(18,464) : 230,	(19,464) : 548,	(20,464) : 628,
(1,465) : 179,	(2,465) : 732,	(3,465) : 312,	(4,465) : 156,	(5,465) : 280,	(6,465) : 177,	(7,465) : 666,	(8,465) : 395,	(9,465) : 511,	(10,465) : 869,	(11,465) : 379,	(12,465) : 47,	(13,465) : 193,	(14,465) : 781,	(15,465) : 847,	(16,465) : 182,	(17,465) : 747,	(18,465) : 259,	(19,465) : 586,	(20,465) : 667,
(1,466) : 632,	(2,466) : 182,	(3,466) : 466,	(4,466) : 631,	(5,466) : 483,	(6,466) : 733,	(7,466) : 377,	(8,466) : 556,	(9,466) : 216,	(10,466) : 171,	(11,466) : 350,	(12,466) : 709,	(13,466) : 666,	(14,466) : 274,	(15,466) : 204,	(16,466) : 609,	(17,466) : 115,	(18,466) : 572,	(19,466) : 118,	(20,466) : 152,
(1,467) : 470,	(2,467) : 166,	(3,467) : 404,	(4,467) : 473,	(5,467) : 409,	(6,467) : 570,	(7,467) : 236,	(8,467) : 371,	(9,467) : 61,	(10,467) : 347,	(11,467) : 192,	(12,467) : 585,	(13,467) : 501,	(14,467) : 235,	(15,467) : 279,	(16,467) : 448,	(17,467) : 289,	(18,467) : 502,	(19,467) : 141,	(20,467) : 276,
(1,468) : 440,	(2,468) : 181,	(3,468) : 429,	(4,468) : 447,	(5,468) : 429,	(6,468) : 539,	(7,468) : 181,	(8,468) : 318,	(9,468) : 92,	(10,468) : 400,	(11,468) : 180,	(12,468) : 578,	(13,468) : 468,	(14,468) : 226,	(15,468) : 304,	(16,468) : 420,	(17,468) : 352,	(18,468) : 521,	(19,468) : 205,	(20,468) : 341,
(1,469) : 63,	(2,469) : 630,	(3,469) : 312,	(4,469) : 39,	(5,469) : 281,	(6,469) : 128,	(7,469) : 545,	(8,469) : 270,	(9,469) : 418,	(10,469) : 793,	(11,469) : 280,	(12,469) : 166,	(13,469) : 95,	(14,469) : 671,	(15,469) : 750,	(16,469) : 59,	(17,469) : 684,	(18,469) : 302,	(19,469) : 513,	(20,469) : 613,
(1,470) : 75,	(2,470) : 556,	(3,470) : 375,	(4,470) : 93,	(5,470) : 348,	(6,470) : 164,	(7,470) : 442,	(8,470) : 148,	(9,470) : 367,	(10,470) : 752,	(11,470) : 233,	(12,470) : 287,	(13,470) : 93,	(14,470) : 584,	(15,470) : 681,	(16,470) : 64,	(17,470) : 660,	(18,470) : 393,	(19,470) : 483,	(20,470) : 602,
(1,471) : 257,	(2,471) : 779,	(3,471) : 304,	(4,471) : 234,	(5,471) : 274,	(6,471) : 253,	(7,471) : 729,	(8,471) : 471,	(9,471) : 554,	(10,471) : 895,	(11,471) : 429,	(12,471) : 35,	(13,471) : 273,	(14,471) : 834,	(15,471) : 889,	(16,471) : 259,	(17,471) : 766,	(18,471) : 228,	(19,471) : 613,	(20,471) : 680,
(1,472) : 342,	(2,472) : 786,	(3,472) : 255,	(4,472) : 318,	(5,472) : 232,	(6,472) : 356,	(7,472) : 767,	(8,472) : 542,	(9,472) : 560,	(10,472) : 869,	(11,472) : 451,	(12,472) : 133,	(13,472) : 365,	(14,472) : 851,	(15,472) : 888,	(16,472) : 339,	(17,472) : 732,	(18,472) : 157,	(19,472) : 596,	(20,472) : 642,
(1,473) : 621,	(2,473) : 0,	(3,473) : 563,	(4,473) : 627,	(5,473) : 571,	(6,473) : 719,	(7,473) : 223,	(8,473) : 485,	(9,473) : 227,	(10,473) : 271,	(11,473) : 353,	(12,473) : 750,	(13,473) : 648,	(14,473) : 93,	(15,473) : 126,	(16,473) : 600,	(17,473) : 292,	(18,473) : 664,	(19,473) : 239,	(20,473) : 330,
(1,474) : 233,	(2,474) : 482,	(3,474) : 164,	(4,474) : 222,	(5,474) : 145,	(6,474) : 325,	(7,474) : 464,	(8,474) : 313,	(9,474) : 256,	(10,474) : 607,	(11,474) : 143,	(12,474) : 272,	(13,474) : 273,	(14,474) : 543,	(15,474) : 591,	(16,474) : 211,	(17,474) : 489,	(18,474) : 221,	(19,474) : 324,	(20,474) : 414,
(1,475) : 335,	(2,475) : 415,	(3,475) : 544,	(4,475) : 355,	(5,475) : 528,	(6,475) : 404,	(7,475) : 224,	(8,475) : 118,	(9,475) : 331,	(10,475) : 668,	(11,475) : 276,	(12,475) : 546,	(13,475) : 340,	(14,475) : 403,	(15,475) : 537,	(16,475) : 326,	(17,475) : 625,	(18,475) : 600,	(19,475) : 466,	(20,475) : 603,
(1,476) : 117,	(2,476) : 574,	(3,476) : 248,	(4,476) : 100,	(5,476) : 218,	(6,476) : 201,	(7,476) : 510,	(8,476) : 273,	(9,476) : 355,	(10,476) : 725,	(11,476) : 221,	(12,476) : 186,	(13,476) : 156,	(14,476) : 622,	(15,476) : 690,	(16,476) : 98,	(17,476) : 612,	(18,476) : 255,	(19,476) : 443,	(20,476) : 540,
(1,477) : 577,	(2,477) : 44,	(3,477) : 528,	(4,477) : 583,	(5,477) : 534,	(6,477) : 675,	(7,477) : 198,	(8,477) : 443,	(9,477) : 187,	(10,477) : 298,	(11,477) : 310,	(12,477) : 708,	(13,477) : 604,	(14,477) : 114,	(15,477) : 169,	(16,477) : 556,	(17,477) : 298,	(18,477) : 627,	(19,477) : 218,	(20,477) : 324,
(1,478) : 79,	(2,478) : 592,	(3,478) : 285,	(4,478) : 62,	(5,478) : 255,	(6,478) : 164,	(7,478) : 515,	(8,478) : 256,	(9,478) : 379,	(10,478) : 753,	(11,478) : 241,	(12,478) : 185,	(13,478) : 118,	(14,478) : 635,	(15,478) : 711,	(16,478) : 62,	(17,478) : 644,	(18,478) : 287,	(19,478) : 473,	(20,478) : 574,
(1,479) : 327,	(2,479) : 741,	(3,479) : 208,	(4,479) : 303,	(5,479) : 185,	(6,479) : 354,	(7,479) : 729,	(8,479) : 517,	(9,479) : 515,	(10,479) : 822,	(11,479) : 410,	(12,479) : 139,	(13,479) : 354,	(14,479) : 807,	(15,479) : 842,	(16,479) : 320,	(17,479) : 686,	(18,479) : 113,	(19,479) : 549,	(20,479) : 595,
(1,480) : 279,	(2,480) : 365,	(3,480) : 264,	(4,480) : 279,	(5,480) : 255,	(6,480) : 381,	(7,480) : 339,	(8,480) : 257,	(9,480) : 145,	(10,480) : 524,	(11,480) : 18,	(12,480) : 386,	(13,480) : 314,	(14,480) : 419,	(15,480) : 480,	(16,480) : 256,	(17,480) : 426,	(18,480) : 341,	(19,480) : 250,	(20,480) : 370,
(1,481) : 155,	(2,481) : 746,	(3,481) : 360,	(4,481) : 136,	(5,481) : 328,	(6,481) : 128,	(7,481) : 663,	(8,481) : 374,	(9,481) : 530,	(10,481) : 898,	(11,481) : 394,	(12,481) : 98,	(13,481) : 158,	(14,481) : 789,	(15,481) : 864,	(16,481) : 165,	(17,481) : 781,	(18,481) : 314,	(19,481) : 615,	(20,481) : 703,
(1,482) : 329,	(2,482) : 744,	(3,482) : 210,	(4,482) : 304,	(5,482) : 187,	(6,482) : 355,	(7,482) : 731,	(8,482) : 519,	(9,482) : 518,	(10,482) : 824,	(11,482) : 412,	(12,482) : 139,	(13,482) : 355,	(14,482) : 810,	(15,482) : 844,	(16,482) : 322,	(17,482) : 688,	(18,482) : 114,	(19,482) : 551,	(20,482) : 597,
(1,483) : 175,	(2,483) : 537,	(3,483) : 471,	(4,483) : 197,	(5,483) : 447,	(6,483) : 231,	(7,483) : 383,	(8,483) : 58,	(9,483) : 384,	(10,483) : 762,	(11,483) : 270,	(12,483) : 396,	(13,483) : 171,	(14,483) : 547,	(15,483) : 663,	(16,483) : 172,	(17,483) : 688,	(18,483) : 500,	(19,483) : 514,	(20,483) : 643,
(1,484) : 287,	(2,484) : 496,	(3,484) : 551,	(4,484) : 309,	(5,484) : 531,	(6,484) : 340,	(7,484) : 310,	(8,484) : 77,	(9,484) : 389,	(10,484) : 744,	(11,484) : 308,	(12,484) : 507,	(13,484) : 283,	(14,484) : 488,	(15,484) : 620,	(16,484) : 283,	(17,484) : 691,	(18,484) : 594,	(19,484) : 525,	(20,484) : 661,
(1,485) : 335,	(2,485) : 748,	(3,485) : 212,	(4,485) : 310,	(5,485) : 190,	(6,485) : 361,	(7,485) : 736,	(8,485) : 525,	(9,485) : 522,	(10,485) : 827,	(11,485) : 417,	(12,485) : 144,	(13,485) : 361,	(14,485) : 814,	(15,485) : 848,	(16,485) : 328,	(17,485) : 690,	(18,485) : 115,	(19,485) : 554,	(20,485) : 599,
(1,486) : 363,	(2,486) : 664,	(3,486) : 105,	(4,486) : 341,	(5,486) : 93,	(6,486) : 416,	(7,486) : 681,	(8,486) : 517,	(9,486) : 442,	(10,486) : 721,	(11,486) : 359,	(12,486) : 230,	(13,486) : 397,	(14,486) : 737,	(15,486) : 756,	(16,486) : 349,	(17,486) : 581,	(18,486) : 0,	(19,486) : 455,	(20,486) : 489,
(1,487) : 373,	(2,487) : 560,	(3,487) : 10,	(4,487) : 355,	(5,487) : 42,	(6,487) : 448,	(7,487) : 597,	(8,487) : 483,	(9,487) : 344,	(10,487) : 610,	(11,487) : 284,	(12,487) : 305,	(13,487) : 412,	(14,487) : 637,	(15,487) : 647,	(16,487) : 355,	(17,487) : 470,	(18,487) : 112,	(19,487) : 345,	(20,487) : 379,
(1,488) : 531,	(2,488) : 118,	(3,488) : 538,	(4,488) : 540,	(5,488) : 539,	(6,488) : 626,	(7,488) : 114,	(8,488) : 375,	(9,488) : 194,	(10,488) : 383,	(11,488) : 287,	(12,488) : 684,	(13,488) : 555,	(14,488) : 123,	(15,488) : 239,	(16,488) : 512,	(17,488) : 378,	(18,488) : 631,	(19,488) : 271,	(20,488) : 393,
(1,489) : 226,	(2,489) : 618,	(3,489) : 157,	(4,489) : 205,	(5,489) : 125,	(6,489) : 286,	(7,489) : 593,	(8,489) : 390,	(9,489) : 391,	(10,489) : 726,	(11,489) : 277,	(12,489) : 150,	(13,489) : 262,	(14,489) : 679,	(15,489) : 724,	(16,489) : 213,	(17,489) : 598,	(18,489) : 136,	(19,489) : 444,	(20,489) : 514,
(1,490) : 50,	(2,490) : 665,	(3,490) : 365,	(4,490) : 37,	(5,490) : 333,	(6,490) : 75,	(7,490) : 565,	(8,490) : 268,	(9,490) : 459,	(10,490) : 838,	(11,490) : 320,	(12,490) : 182,	(13,490) : 57,	(14,490) : 700,	(15,490) : 787,	(16,490) : 66,	(17,490) : 732,	(18,490) : 349,	(19,490) : 559,	(20,490) : 663,
(1,491) : 240,	(2,491) : 586,	(3,491) : 558,	(4,491) : 264,	(5,491) : 533,	(6,491) : 267,	(7,491) : 407,	(8,491) : 110,	(9,491) : 455,	(10,491) : 825,	(11,491) : 352,	(12,491) : 461,	(13,491) : 223,	(14,491) : 583,	(15,491) : 711,	(16,491) : 244,	(17,491) : 761,	(18,491) : 583,	(19,491) : 589,	(20,491) : 721,
(1,492) : 670,	(2,492) : 94,	(3,492) : 648,	(4,492) : 679,	(5,492) : 654,	(6,492) : 765,	(7,492) : 203,	(8,492) : 510,	(9,492) : 305,	(10,492) : 326,	(11,492) : 419,	(12,492) : 818,	(13,492) : 694,	(14,492) : 19,	(15,492) : 138,	(16,492) : 651,	(17,492) : 375,	(18,492) : 747,	(19,492) : 332,	(20,492) : 422,
(1,493) : 649,	(2,493) : 194,	(3,493) : 475,	(4,493) : 648,	(5,493) : 493,	(6,493) : 750,	(7,493) : 394,	(8,493) : 574,	(9,493) : 233,	(10,493) : 155,	(11,493) : 367,	(12,493) : 722,	(13,493) : 683,	(14,493) : 285,	(15,493) : 205,	(16,493) : 626,	(17,493) : 99,	(18,493) : 581,	(19,493) : 131,	(20,493) : 146,
(1,494) : 192,	(2,494) : 729,	(3,494) : 557,	(4,494) : 211,	(5,494) : 527,	(6,494) : 145,	(7,494) : 572,	(8,494) : 248,	(9,494) : 562,	(10,494) : 945,	(11,494) : 434,	(12,494) : 369,	(13,494) : 154,	(14,494) : 740,	(15,494) : 855,	(16,494) : 212,	(17,494) : 861,	(18,494) : 549,	(19,494) : 685,	(20,494) : 806,
(1,495) : 547,	(2,495) : 498,	(3,495) : 211,	(4,495) : 533,	(5,495) : 243,	(6,495) : 633,	(7,495) : 610,	(8,495) : 601,	(9,495) : 338,	(10,495) : 461,	(11,495) : 360,	(12,495) : 506,	(13,495) : 587,	(14,495) : 589,	(15,495) : 551,	(16,495) : 526,	(17,495) : 314,	(18,495) : 297,	(19,495) : 259,	(20,495) : 220,
(1,496) : 110,	(2,496) : 511,	(3,496) : 326,	(4,496) : 117,	(5,496) : 301,	(6,496) : 210,	(7,496) : 416,	(8,496) : 163,	(9,496) : 313,	(10,496) : 697,	(11,496) : 176,	(12,496) : 287,	(13,496) : 141,	(14,496) : 546,	(15,496) : 634,	(16,496) : 89,	(17,496) : 601,	(18,496) : 356,	(19,496) : 426,	(20,496) : 542,
(1,497) : 84,	(2,497) : 632,	(3,497) : 295,	(4,497) : 60,	(5,497) : 264,	(6,497) : 143,	(7,497) : 555,	(8,497) : 287,	(9,497) : 417,	(10,497) : 789,	(11,497) : 281,	(12,497) : 149,	(13,497) : 115,	(14,497) : 676,	(15,497) : 751,	(16,497) : 78,	(17,497) : 677,	(18,497) : 282,	(19,497) : 508,	(20,497) : 604,
(1,498) : 571,	(2,498) : 279,	(3,498) : 355,	(4,498) : 566,	(5,498) : 375,	(6,498) : 670,	(7,498) : 433,	(8,498) : 536,	(9,498) : 196,	(10,498) : 262,	(11,498) : 302,	(12,498) : 616,	(13,498) : 608,	(14,498) : 372,	(15,498) : 324,	(16,498) : 547,	(17,498) : 133,	(18,498) : 460,	(19,498) : 61,	(20,498) : 77,
(1,499) : 225,	(2,499) : 713,	(3,499) : 245,	(4,499) : 201,	(5,499) : 215,	(6,499) : 248,	(7,499) : 670,	(8,499) : 428,	(9,499) : 488,	(10,499) : 829,	(11,499) : 365,	(12,499) : 50,	(13,499) : 249,	(14,499) : 770,	(15,499) : 823,	(16,499) : 222,	(17,499) : 701,	(18,499) : 184,	(19,499) : 547,	(20,499) : 616,
(1,500) : 137,	(2,500) : 506,	(3,500) : 397,	(4,500) : 155,	(5,500) : 373,	(6,500) : 221,	(7,500) : 381,	(8,500) : 92,	(9,500) : 331,	(10,500) : 715,	(11,500) : 206,	(12,500) : 346,	(13,500) : 151,	(14,500) : 529,	(15,500) : 632,	(16,500) : 126,	(17,500) : 632,	(18,500) : 429,	(19,500) : 456,	(20,500) : 581,
(1,501) : 348,	(2,501) : 318,	(3,501) : 264,	(4,501) : 346,	(5,501) : 264,	(6,501) : 449,	(7,501) : 335,	(8,501) : 317,	(9,501) : 91,	(10,501) : 456,	(11,501) : 74,	(12,501) : 437,	(13,501) : 384,	(14,501) : 383,	(15,501) : 426,	(16,501) : 325,	(17,501) : 356,	(18,501) : 356,	(19,501) : 180,	(20,501) : 301,
(1,502) : 89,	(2,502) : 611,	(3,502) : 280,	(4,502) : 67,	(5,502) : 249,	(6,502) : 161,	(7,502) : 537,	(8,502) : 278,	(9,502) : 395,	(10,502) : 767,	(11,502) : 259,	(12,502) : 162,	(13,502) : 125,	(14,502) : 656,	(15,502) : 729,	(16,502) : 77,	(17,502) : 655,	(18,502) : 274,	(19,502) : 485,	(20,502) : 582,
(1,503) : 134,	(2,503) : 628,	(3,503) : 247,	(4,503) : 111,	(5,503) : 215,	(6,503) : 192,	(7,503) : 569,	(8,503) : 322,	(9,503) : 407,	(10,503) : 768,	(11,503) : 275,	(12,503) : 127,	(13,503) : 167,	(14,503) : 678,	(15,503) : 743,	(16,503) : 124,	(17,503) : 650,	(18,503) : 230,	(19,503) : 485,	(20,503) : 573,
(1,504) : 601,	(2,504) : 81,	(3,504) : 592,	(4,504) : 610,	(5,504) : 596,	(6,504) : 696,	(7,504) : 149,	(8,504) : 443,	(9,504) : 247,	(10,504) : 349,	(11,504) : 353,	(12,504) : 751,	(13,504) : 625,	(14,504) : 53,	(15,504) : 182,	(16,504) : 582,	(17,504) : 370,	(18,504) : 689,	(19,504) : 296,	(20,504) : 402,
(1,505) : 724,	(2,505) : 110,	(3,505) : 630,	(4,505) : 729,	(5,505) : 642,	(6,505) : 824,	(7,505) : 322,	(8,505) : 595,	(9,505) : 315,	(10,505) : 192,	(11,505) : 450,	(12,505) : 841,	(13,505) : 753,	(14,505) : 150,	(15,505) : 22,	(16,505) : 703,	(17,505) : 265,	(18,505) : 734,	(19,505) : 283,	(20,505) : 332,
(1,506) : 97,	(2,506) : 564,	(3,506) : 413,	(4,506) : 119,	(5,506) : 386,	(6,506) : 167,	(7,506) : 434,	(8,506) : 124,	(9,506) : 385,	(10,506) : 769,	(11,506) : 255,	(12,506) : 317,	(13,506) : 99,	(14,506) : 586,	(15,506) : 689,	(16,506) : 94,	(17,506) : 682,	(18,506) : 432,	(19,506) : 506,	(20,506) : 628,
(1,507) : 131,	(2,507) : 604,	(3,507) : 239,	(4,507) : 110,	(5,507) : 208,	(6,507) : 200,	(7,507) : 546,	(8,507) : 306,	(9,507) : 383,	(10,507) : 747,	(11,507) : 251,	(12,507) : 151,	(13,507) : 167,	(14,507) : 654,	(15,507) : 719,	(16,507) : 117,	(17,507) : 630,	(18,507) : 232,	(19,507) : 464,	(20,507) : 554,
(1,508) : 117,	(2,508) : 511,	(3,508) : 365,	(4,508) : 131,	(5,508) : 341,	(6,508) : 210,	(7,508) : 399,	(8,508) : 125,	(9,508) : 324,	(10,508) : 709,	(11,508) : 193,	(12,508) : 316,	(13,508) : 138,	(14,508) : 539,	(15,508) : 635,	(16,508) : 101,	(17,508) : 620,	(18,508) : 396,	(19,508) : 444,	(20,508) : 565,
(1,509) : 303,	(2,509) : 742,	(3,509) : 223,	(4,509) : 279,	(5,509) : 197,	(6,509) : 326,	(7,509) : 720,	(8,509) : 498,	(9,509) : 515,	(10,509) : 832,	(11,509) : 404,	(12,509) : 110,	(13,509) : 328,	(14,509) : 805,	(15,509) : 845,	(16,509) : 297,	(17,509) : 697,	(18,509) : 134,	(19,509) : 556,	(20,509) : 608,
(1,510) : 346,	(2,510) : 833,	(3,510) : 315,	(4,510) : 322,	(5,510) : 291,	(6,510) : 341,	(7,510) : 801,	(8,510) : 556,	(9,510) : 607,	(10,510) : 927,	(11,510) : 491,	(12,510) : 124,	(13,510) : 362,	(14,510) : 895,	(15,510) : 938,	(16,510) : 346,	(17,510) : 792,	(18,510) : 219,	(19,510) : 651,	(20,510) : 702,
(1,511) : 297,	(2,511) : 433,	(3,511) : 155,	(4,511) : 287,	(5,511) : 149,	(6,511) : 391,	(7,511) : 443,	(8,511) : 345,	(9,511) : 207,	(10,511) : 542,	(11,511) : 127,	(12,511) : 334,	(13,511) : 336,	(14,511) : 501,	(15,511) : 536,	(16,511) : 274,	(17,511) : 421,	(18,511) : 240,	(19,511) : 259,	(20,511) : 346,
(1,512) : 27,	(2,512) : 628,	(3,512) : 344,	(4,512) : 3,	(5,512) : 313,	(6,512) : 106,	(7,512) : 531,	(8,512) : 243,	(9,512) : 422,	(10,512) : 802,	(11,512) : 284,	(12,512) : 196,	(13,512) : 60,	(14,512) : 664,	(15,512) : 750,	(16,512) : 32,	(17,512) : 697,	(18,512) : 338,	(19,512) : 524,	(20,512) : 629,
(1,513) : 120,	(2,513) : 566,	(3,513) : 439,	(4,513) : 143,	(5,513) : 413,	(6,513) : 179,	(7,513) : 427,	(8,513) : 107,	(9,513) : 395,	(10,513) : 778,	(11,513) : 270,	(12,513) : 342,	(13,513) : 116,	(14,513) : 584,	(15,513) : 692,	(16,513) : 120,	(17,513) : 696,	(18,513) : 459,	(19,513) : 520,	(20,513) : 644,
(1,514) : 747,	(2,514) : 557,	(3,514) : 411,	(4,514) : 734,	(5,514) : 443,	(6,514) : 835,	(7,514) : 722,	(8,514) : 779,	(9,514) : 470,	(10,514) : 413,	(11,514) : 533,	(12,514) : 705,	(13,514) : 787,	(14,514) : 651,	(15,514) : 566,	(16,514) : 725,	(17,514) : 286,	(18,514) : 486,	(19,514) : 348,	(20,514) : 227,
(1,515) : 610,	(2,515) : 188,	(3,515) : 443,	(4,515) : 609,	(5,515) : 460,	(6,515) : 711,	(7,515) : 372,	(8,515) : 538,	(9,515) : 196,	(10,515) : 193,	(11,515) : 329,	(12,515) : 685,	(13,515) : 645,	(14,515) : 281,	(15,515) : 222,	(16,515) : 587,	(17,515) : 121,	(18,515) : 548,	(19,515) : 94,	(20,515) : 143,
(1,516) : 65,	(2,516) : 563,	(3,516) : 369,	(4,516) : 83,	(5,516) : 342,	(6,516) : 157,	(7,516) : 451,	(8,516) : 159,	(9,516) : 371,	(10,516) : 756,	(11,516) : 236,	(12,516) : 277,	(13,516) : 85,	(14,516) : 592,	(15,516) : 687,	(16,516) : 54,	(17,516) : 662,	(18,516) : 386,	(19,516) : 486,	(20,516) : 603,
(1,517) : 367,	(2,517) : 781,	(3,517) : 237,	(4,517) : 342,	(5,517) : 218,	(6,517) : 387,	(7,517) : 773,	(8,517) : 560,	(9,517) : 556,	(10,517) : 854,	(11,517) : 453,	(12,517) : 165,	(13,517) : 392,	(14,517) : 849,	(15,517) : 880,	(16,517) : 361,	(17,517) : 715,	(18,517) : 134,	(19,517) : 584,	(20,517) : 623,
(1,518) : 312,	(2,518) : 491,	(3,518) : 89,	(4,518) : 298,	(5,518) : 83,	(6,518) : 398,	(7,518) : 510,	(8,518) : 396,	(9,518) : 268,	(10,518) : 576,	(11,518) : 193,	(12,518) : 302,	(13,518) : 352,	(14,518) : 562,	(15,518) : 588,	(16,518) : 291,	(17,518) : 446,	(18,518) : 175,	(19,518) : 297,	(20,518) : 363,
(1,519) : 619,	(2,519) : 370,	(3,519) : 348,	(4,519) : 611,	(5,519) : 374,	(6,519) : 715,	(7,519) : 529,	(8,519) : 612,	(9,519) : 283,	(10,519) : 290,	(11,519) : 369,	(12,519) : 632,	(13,519) : 658,	(14,519) : 463,	(15,519) : 398,	(16,519) : 596,	(17,519) : 143,	(18,519) : 447,	(19,519) : 155,	(20,519) : 50,
(1,520) : 551,	(2,520) : 185,	(3,520) : 402,	(4,520) : 550,	(5,520) : 415,	(6,520) : 652,	(7,520) : 338,	(8,520) : 481,	(9,520) : 137,	(10,520) : 252,	(11,520) : 269,	(12,520) : 632,	(13,520) : 585,	(14,520) : 277,	(15,520) : 251,	(16,520) : 528,	(17,520) : 172,	(18,520) : 506,	(19,520) : 55,	(20,520) : 165,
(1,521) : 276,	(2,521) : 561,	(3,521) : 88,	(4,521) : 258,	(5,521) : 58,	(6,521) : 351,	(7,521) : 561,	(8,521) : 402,	(9,521) : 336,	(10,521) : 653,	(11,521) : 240,	(12,521) : 228,	(13,521) : 315,	(14,521) : 629,	(15,521) : 662,	(16,521) : 258,	(17,521) : 522,	(18,521) : 122,	(19,521) : 374,	(20,521) : 437,
(1,522) : 127,	(2,522) : 506,	(3,522) : 280,	(4,522) : 126,	(5,522) : 255,	(6,522) : 228,	(7,522) : 431,	(8,522) : 205,	(9,522) : 297,	(10,522) : 677,	(11,522) : 158,	(12,522) : 265,	(13,522) : 164,	(14,522) : 549,	(15,522) : 627,	(16,522) : 104,	(17,522) : 575,	(18,522) : 312,	(19,522) : 400,	(20,522) : 511,
(1,523) : 76,	(2,523) : 545,	(3,523) : 337,	(4,523) : 85,	(5,523) : 310,	(6,523) : 176,	(7,523) : 445,	(8,523) : 173,	(9,523) : 347,	(10,523) : 730,	(11,523) : 209,	(12,523) : 266,	(13,523) : 107,	(14,523) : 579,	(15,523) : 668,	(16,523) : 56,	(17,523) : 633,	(18,523) : 357,	(19,523) : 458,	(20,523) : 572,
(1,524) : 187,	(2,524) : 457,	(3,524) : 403,	(4,524) : 203,	(5,524) : 383,	(6,524) : 273,	(7,524) : 329,	(8,524) : 74,	(9,524) : 291,	(10,524) : 672,	(11,524) : 178,	(12,524) : 387,	(13,524) : 203,	(14,524) : 477,	(15,524) : 583,	(16,524) : 173,	(17,524) : 595,	(18,524) : 448,	(19,524) : 421,	(20,524) : 550,
(1,525) : 376,	(2,525) : 247,	(3,525) : 366,	(4,525) : 381,	(5,525) : 364,	(6,525) : 476,	(7,525) : 232,	(8,525) : 280,	(9,525) : 69,	(10,525) : 442,	(11,525) : 108,	(12,525) : 507,	(13,525) : 406,	(14,525) : 297,	(15,525) : 368,	(16,525) : 355,	(17,525) : 372,	(18,525) : 455,	(19,525) : 205,	(20,525) : 342,
(1,526) : 109,	(2,526) : 695,	(3,526) : 334,	(4,526) : 88,	(5,526) : 302,	(6,526) : 115,	(7,526) : 611,	(8,526) : 328,	(9,526) : 480,	(10,526) : 851,	(11,526) : 343,	(12,526) : 117,	(13,526) : 122,	(14,526) : 737,	(15,526) : 813,	(16,526) : 116,	(17,526) : 737,	(18,526) : 302,	(19,526) : 568,	(20,526) : 662,
(1,527) : 144,	(2,527) : 619,	(3,527) : 490,	(4,527) : 168,	(5,527) : 463,	(6,527) : 165,	(7,527) : 467,	(8,527) : 141,	(9,527) : 455,	(10,527) : 837,	(11,527) : 331,	(12,527) : 361,	(13,527) : 122,	(14,527) : 631,	(15,527) : 745,	(16,527) : 153,	(17,527) : 756,	(18,527) : 502,	(19,527) : 581,	(20,527) : 705,
(1,528) : 308,	(2,528) : 558,	(3,528) : 57,	(4,528) : 289,	(5,528) : 28,	(6,528) : 382,	(7,528) : 571,	(8,528) : 427,	(9,528) : 335,	(10,528) : 637,	(11,528) : 250,	(12,528) : 252,	(13,528) : 346,	(14,528) : 629,	(15,528) : 655,	(16,528) : 289,	(17,528) : 503,	(18,528) : 110,	(19,528) : 361,	(20,528) : 416,
(1,529) : 502,	(2,529) : 201,	(3,529) : 367,	(4,529) : 502,	(5,529) : 378,	(6,529) : 604,	(7,529) : 321,	(8,529) : 436,	(9,529) : 90,	(10,529) : 301,	(11,529) : 221,	(12,529) : 586,	(13,529) : 537,	(14,529) : 288,	(15,529) : 286,	(16,529) : 479,	(17,529) : 215,	(18,529) : 470,	(19,529) : 54,	(20,529) : 190,
(1,530) : 534,	(2,530) : 133,	(3,530) : 554,	(4,530) : 544,	(5,530) : 555,	(6,530) : 627,	(7,530) : 93,	(8,530) : 371,	(9,530) : 212,	(10,530) : 401,	(11,530) : 298,	(12,530) : 692,	(13,530) : 556,	(14,530) : 122,	(15,530) : 250,	(16,530) : 515,	(17,530) : 400,	(18,530) : 646,	(19,530) : 293,	(20,530) : 415,
(1,531) : 236,	(2,531) : 424,	(3,531) : 225,	(4,531) : 231,	(5,531) : 211,	(6,531) : 336,	(7,531) : 394,	(8,531) : 262,	(9,531) : 202,	(10,531) : 573,	(11,531) : 75,	(12,531) : 326,	(13,531) : 273,	(14,531) : 479,	(15,531) : 538,	(16,531) : 212,	(17,531) : 466,	(18,531) : 291,	(19,531) : 293,	(20,531) : 402,
(1,532) : 197,	(2,532) : 460,	(3,532) : 423,	(4,532) : 214,	(5,532) : 402,	(6,532) : 278,	(7,532) : 323,	(8,532) : 55,	(9,532) : 302,	(10,532) : 680,	(11,532) : 194,	(12,532) : 402,	(13,532) : 209,	(14,532) : 476,	(15,532) : 586,	(16,532) : 185,	(17,532) : 607,	(18,532) : 467,	(19,532) : 433,	(20,532) : 564,
(1,533) : 249,	(2,533) : 408,	(3,533) : 435,	(4,533) : 265,	(5,533) : 418,	(6,533) : 334,	(7,533) : 268,	(8,533) : 81,	(9,533) : 266,	(10,533) : 636,	(11,533) : 178,	(12,533) : 446,	(13,533) : 264,	(14,533) : 421,	(15,533) : 534,	(16,533) : 235,	(17,533) : 571,	(18,533) : 490,	(19,533) : 400,	(20,533) : 535,
(1,534) : 198,	(2,534) : 643,	(3,534) : 546,	(4,534) : 222,	(5,534) : 519,	(6,534) : 202,	(7,534) : 475,	(8,534) : 157,	(9,534) : 493,	(10,534) : 871,	(11,534) : 375,	(12,534) : 411,	(13,534) : 172,	(14,534) : 647,	(15,534) : 768,	(16,534) : 209,	(17,534) : 797,	(18,534) : 558,	(19,534) : 622,	(20,534) : 750,
(1,535) : 191,	(2,535) : 803,	(3,535) : 429,	(4,535) : 178,	(5,535) : 397,	(6,535) : 123,	(7,535) : 706,	(8,535) : 402,	(9,535) : 591,	(10,535) : 963,	(11,535) : 453,	(12,535) : 152,	(13,535) : 179,	(14,535) : 841,	(15,535) : 923,	(16,535) : 207,	(17,535) : 848,	(18,535) : 378,	(19,535) : 681,	(20,535) : 772,
(1,536) : 440,	(2,536) : 185,	(3,536) : 449,	(4,536) : 447,	(5,536) : 448,	(6,536) : 536,	(7,536) : 155,	(8,536) : 304,	(9,536) : 118,	(10,536) : 418,	(11,536) : 190,	(12,536) : 587,	(13,536) : 465,	(14,536) : 218,	(15,536) : 310,	(16,536) : 420,	(17,536) : 377,	(18,536) : 539,	(19,536) : 232,	(20,536) : 368,
(1,537) : 268,	(2,537) : 526,	(3,537) : 551,	(4,537) : 291,	(5,537) : 529,	(6,537) : 314,	(7,537) : 342,	(8,537) : 76,	(9,537) : 410,	(10,537) : 770,	(11,537) : 320,	(12,537) : 490,	(13,537) : 260,	(14,537) : 520,	(15,537) : 650,	(16,537) : 266,	(17,537) : 714,	(18,537) : 588,	(19,537) : 545,	(20,537) : 680,
(1,538) : 274,	(2,538) : 674,	(3,538) : 165,	(4,538) : 251,	(5,538) : 136,	(6,538) : 318,	(7,538) : 657,	(8,538) : 452,	(9,538) : 447,	(10,538) : 767,	(11,538) : 339,	(12,538) : 133,	(13,538) : 305,	(14,538) : 738,	(15,538) : 777,	(16,538) : 264,	(17,538) : 634,	(18,538) : 101,	(19,538) : 489,	(20,538) : 546,
(1,539) : 132,	(2,539) : 532,	(3,539) : 422,	(4,539) : 153,	(5,539) : 397,	(6,539) : 205,	(7,539) : 397,	(8,539) : 87,	(9,539) : 361,	(10,539) : 744,	(11,539) : 238,	(12,539) : 350,	(13,539) : 137,	(14,539) : 551,	(15,539) : 658,	(16,539) : 126,	(17,539) : 663,	(18,539) : 449,	(19,539) : 487,	(20,539) : 613,
(1,540) : 161,	(2,540) : 670,	(3,540) : 255,	(4,540) : 136,	(5,540) : 223,	(6,540) : 196,	(7,540) : 613,	(8,540) : 362,	(9,540) : 447,	(10,540) : 803,	(11,540) : 317,	(12,540) : 83,	(13,540) : 187,	(14,540) : 721,	(15,540) : 783,	(16,540) : 156,	(17,540) : 682,	(18,540) : 220,	(19,540) : 520,	(20,540) : 602,
(1,541) : 202,	(2,541) : 748,	(3,541) : 312,	(4,541) : 179,	(5,541) : 280,	(6,541) : 197,	(7,541) : 686,	(8,541) : 418,	(9,541) : 526,	(10,541) : 880,	(11,541) : 396,	(12,541) : 27,	(13,541) : 216,	(14,541) : 799,	(15,541) : 862,	(16,541) : 205,	(17,541) : 756,	(18,541) : 252,	(19,541) : 597,	(20,541) : 674,
(1,542) : 213,	(2,542) : 788,	(3,542) : 362,	(4,542) : 193,	(5,542) : 330,	(6,542) : 183,	(7,542) : 715,	(8,542) : 432,	(9,542) : 568,	(10,542) : 926,	(11,542) : 435,	(12,542) : 69,	(13,542) : 218,	(14,542) : 835,	(15,542) : 904,	(16,542) : 222,	(17,542) : 804,	(18,542) : 299,	(19,542) : 643,	(20,542) : 723,
(1,543) : 709,	(2,543) : 264,	(3,543) : 500,	(4,543) : 706,	(5,543) : 522,	(6,543) : 809,	(7,543) : 471,	(8,543) : 647,	(9,543) : 303,	(10,543) : 118,	(11,543) : 430,	(12,543) : 764,	(13,543) : 745,	(14,543) : 352,	(15,543) : 246,	(16,543) : 685,	(17,543) : 35,	(18,543) : 605,	(19,543) : 181,	(20,543) : 125,
(1,544) : 53,	(2,544) : 604,	(3,544) : 402,	(4,544) : 77,	(5,544) : 373,	(6,544) : 119,	(7,544) : 483,	(8,544) : 173,	(9,544) : 416,	(10,544) : 800,	(11,544) : 281,	(12,544) : 275,	(13,544) : 50,	(14,544) : 630,	(15,544) : 729,	(16,544) : 60,	(17,544) : 707,	(18,544) : 409,	(19,544) : 531,	(20,544) : 648,
(1,545) : 63,	(2,545) : 572,	(3,545) : 382,	(4,545) : 83,	(5,545) : 354,	(6,545) : 148,	(7,545) : 456,	(8,545) : 157,	(9,545) : 383,	(10,545) : 768,	(11,545) : 248,	(12,545) : 281,	(13,545) : 77,	(14,545) : 600,	(15,545) : 697,	(16,545) : 57,	(17,545) : 675,	(18,545) : 396,	(19,545) : 499,	(20,545) : 616,
(1,546) : 314,	(2,546) : 625,	(3,546) : 637,	(4,546) : 339,	(5,546) : 613,	(6,546) : 328,	(7,546) : 429,	(8,546) : 176,	(9,546) : 516,	(10,546) : 875,	(11,546) : 423,	(12,546) : 533,	(13,546) : 294,	(14,546) : 612,	(15,546) : 748,	(16,546) : 320,	(17,546) : 820,	(18,546) : 663,	(19,546) : 652,	(20,546) : 787,
(1,547) : 145,	(2,547) : 580,	(3,547) : 472,	(4,547) : 169,	(5,547) : 446,	(6,547) : 188,	(7,547) : 430,	(8,547) : 105,	(9,547) : 418,	(10,547) : 800,	(11,547) : 297,	(12,547) : 367,	(13,547) : 133,	(14,547) : 593,	(15,547) : 706,	(16,547) : 148,	(17,547) : 721,	(18,547) : 491,	(19,547) : 545,	(20,547) : 671,
(1,548) : 272,	(2,548) : 680,	(3,548) : 173,	(4,548) : 248,	(5,548) : 144,	(6,548) : 313,	(7,548) : 661,	(8,548) : 452,	(9,548) : 453,	(10,548) : 774,	(11,548) : 344,	(12,548) : 126,	(13,548) : 302,	(14,548) : 743,	(15,548) : 783,	(16,548) : 262,	(17,548) : 641,	(18,548) : 107,	(19,548) : 496,	(20,548) : 554,
(1,549) : 285,	(2,549) : 481,	(3,549) : 117,	(4,549) : 272,	(5,549) : 105,	(6,549) : 374,	(7,549) : 488,	(8,549) : 364,	(9,549) : 255,	(10,549) : 581,	(11,549) : 168,	(12,549) : 293,	(13,549) : 325,	(14,549) : 549,	(15,549) : 583,	(16,549) : 263,	(17,549) : 456,	(18,549) : 193,	(19,549) : 300,	(20,549) : 376,
(1,550) : 355,	(2,550) : 463,	(3,550) : 593,	(4,550) : 376,	(5,550) : 576,	(6,550) : 411,	(7,550) : 260,	(8,550) : 137,	(9,550) : 389,	(10,550) : 722,	(11,550) : 332,	(12,550) : 573,	(13,550) : 353,	(14,550) : 444,	(15,550) : 583,	(16,550) : 349,	(17,550) : 683,	(18,550) : 644,	(19,550) : 524,	(20,550) : 662,
(1,551) : 140,	(2,551) : 679,	(3,551) : 283,	(4,551) : 116,	(5,551) : 251,	(6,551) : 168,	(7,551) : 613,	(8,551) : 349,	(9,551) : 459,	(10,551) : 821,	(11,551) : 326,	(12,551) : 87,	(13,551) : 163,	(14,551) : 727,	(15,551) : 795,	(16,551) : 139,	(17,551) : 701,	(18,551) : 248,	(19,551) : 537,	(20,551) : 623,
(1,552) : 484,	(2,552) : 147,	(3,552) : 424,	(4,552) : 488,	(5,552) : 429,	(6,552) : 584,	(7,552) : 223,	(8,552) : 378,	(9,552) : 80,	(10,552) : 340,	(11,552) : 208,	(12,552) : 603,	(13,552) : 514,	(14,552) : 215,	(15,552) : 262,	(16,552) : 462,	(17,552) : 290,	(18,552) : 522,	(19,552) : 151,	(20,552) : 283,
(1,553) : 217,	(2,553) : 680,	(3,553) : 217,	(4,553) : 193,	(5,553) : 185,	(6,553) : 253,	(7,553) : 642,	(8,553) : 409,	(9,553) : 454,	(10,553) : 795,	(11,553) : 333,	(12,553) : 81,	(13,553) : 245,	(14,553) : 738,	(15,553) : 789,	(16,553) : 210,	(17,553) : 667,	(18,553) : 166,	(19,553) : 513,	(20,553) : 583,
(1,554) : 662,	(2,554) : 377,	(3,554) : 389,	(4,554) : 654,	(5,554) : 416,	(6,554) : 759,	(7,554) : 550,	(8,554) : 650,	(9,554) : 315,	(10,554) : 264,	(11,554) : 409,	(12,554) : 675,	(13,554) : 701,	(14,554) : 470,	(15,554) : 391,	(16,554) : 639,	(17,554) : 120,	(18,554) : 488,	(19,554) : 182,	(20,554) : 48,
(1,555) : 104,	(2,555) : 593,	(3,555) : 442,	(4,555) : 128,	(5,555) : 414,	(6,555) : 152,	(7,555) : 455,	(8,555) : 134,	(9,555) : 418,	(10,555) : 802,	(11,555) : 290,	(12,555) : 325,	(13,555) : 92,	(14,555) : 611,	(15,555) : 719,	(16,555) : 109,	(17,555) : 717,	(18,555) : 455,	(19,555) : 540,	(20,555) : 663,
(1,556) : 345,	(2,556) : 359,	(3,556) : 213,	(4,556) : 340,	(5,556) : 215,	(6,556) : 444,	(7,556) : 388,	(8,556) : 347,	(9,556) : 135,	(10,556) : 472,	(11,556) : 102,	(12,556) : 407,	(13,556) : 383,	(14,556) : 430,	(15,556) : 461,	(16,556) : 322,	(17,556) : 359,	(18,556) : 308,	(19,556) : 189,	(20,556) : 292,
(1,557) : 404,	(2,557) : 277,	(3,557) : 513,	(4,557) : 418,	(5,557) : 504,	(6,557) : 491,	(7,557) : 111,	(8,557) : 221,	(9,557) : 226,	(10,557) : 533,	(11,557) : 231,	(12,557) : 588,	(13,557) : 421,	(14,557) : 272,	(15,557) : 400,	(16,557) : 389,	(17,557) : 500,	(18,557) : 589,	(19,557) : 353,	(20,557) : 490,
(1,558) : 616,	(2,558) : 391,	(3,558) : 334,	(4,558) : 607,	(5,558) : 361,	(6,558) : 711,	(7,558) : 546,	(8,558) : 618,	(9,558) : 294,	(10,558) : 312,	(11,558) : 373,	(12,558) : 621,	(13,558) : 655,	(14,558) : 485,	(15,558) : 421,	(16,558) : 593,	(17,558) : 166,	(18,558) : 432,	(19,558) : 171,	(20,558) : 73,
(1,559) : 318,	(2,559) : 838,	(3,559) : 341,	(4,559) : 295,	(5,559) : 314,	(6,559) : 302,	(7,559) : 793,	(8,559) : 534,	(9,559) : 612,	(10,559) : 945,	(11,559) : 490,	(12,559) : 99,	(13,559) : 330,	(14,559) : 895,	(15,559) : 947,	(16,559) : 321,	(17,559) : 813,	(18,559) : 252,	(19,559) : 665,	(20,559) : 725,
(1,560) : 281,	(2,560) : 344,	(3,560) : 379,	(4,560) : 291,	(5,560) : 367,	(6,560) : 377,	(7,560) : 250,	(8,560) : 167,	(9,560) : 179,	(10,560) : 556,	(11,560) : 102,	(12,560) : 447,	(13,560) : 306,	(14,560) : 373,	(15,560) : 469,	(16,560) : 262,	(17,560) : 485,	(18,560) : 448,	(19,560) : 313,	(20,560) : 447,
(1,561) : 285,	(2,561) : 340,	(3,561) : 314,	(4,561) : 289,	(5,561) : 305,	(6,561) : 386,	(7,561) : 293,	(8,561) : 223,	(9,561) : 138,	(10,561) : 523,	(11,561) : 33,	(12,561) : 418,	(13,561) : 317,	(14,561) : 386,	(15,561) : 460,	(16,561) : 263,	(17,561) : 438,	(18,561) : 390,	(19,561) : 262,	(20,561) : 391,
(1,562) : 264,	(2,562) : 500,	(3,562) : 532,	(4,562) : 287,	(5,562) : 511,	(6,562) : 319,	(7,562) : 320,	(8,562) : 56,	(9,562) : 382,	(10,562) : 743,	(11,562) : 295,	(12,562) : 484,	(13,562) : 261,	(14,562) : 496,	(15,562) : 624,	(16,562) : 261,	(17,562) : 686,	(18,562) : 573,	(19,562) : 518,	(20,562) : 653,
(1,563) : 372,	(2,563) : 516,	(3,563) : 49,	(4,563) : 356,	(5,563) : 70,	(6,563) : 454,	(7,563) : 558,	(8,563) : 462,	(9,563) : 301,	(10,563) : 569,	(11,563) : 250,	(12,563) : 330,	(13,563) : 412,	(14,563) : 593,	(15,563) : 603,	(16,563) : 352,	(17,563) : 431,	(18,563) : 154,	(19,563) : 301,	(20,563) : 341,
(1,564) : 209,	(2,564) : 543,	(3,564) : 157,	(4,564) : 193,	(5,564) : 129,	(6,564) : 291,	(7,564) : 517,	(8,564) : 333,	(9,564) : 317,	(10,564) : 664,	(11,564) : 200,	(12,564) : 212,	(13,564) : 248,	(14,564) : 603,	(15,564) : 652,	(16,564) : 190,	(17,564) : 542,	(18,564) : 184,	(19,564) : 381,	(20,564) : 464,
(1,565) : 224,	(2,565) : 769,	(3,565) : 321,	(4,565) : 201,	(5,565) : 290,	(6,565) : 213,	(7,565) : 709,	(8,565) : 441,	(9,565) : 546,	(10,565) : 897,	(11,565) : 417,	(12,565) : 26,	(13,565) : 236,	(14,565) : 821,	(15,565) : 883,	(16,565) : 228,	(17,565) : 772,	(18,565) : 255,	(19,565) : 615,	(20,565) : 689,
(1,566) : 404,	(2,566) : 224,	(3,566) : 442,	(4,566) : 413,	(5,566) : 438,	(6,566) : 500,	(7,566) : 155,	(8,566) : 264,	(9,566) : 132,	(10,566) : 458,	(11,566) : 170,	(12,566) : 560,	(13,566) : 429,	(14,566) : 250,	(15,566) : 350,	(16,566) : 385,	(17,566) : 411,	(18,566) : 527,	(19,566) : 259,	(20,566) : 396,
(1,567) : 320,	(2,567) : 689,	(3,567) : 152,	(4,567) : 297,	(5,567) : 129,	(6,567) : 363,	(7,567) : 686,	(8,567) : 493,	(9,567) : 464,	(10,567) : 766,	(11,567) : 365,	(12,567) : 167,	(13,567) : 351,	(14,567) : 758,	(15,567) : 788,	(16,567) : 310,	(17,567) : 629,	(18,567) : 64,	(19,567) : 493,	(20,567) : 539,
(1,568) : 263,	(2,568) : 458,	(3,568) : 501,	(4,568) : 283,	(5,568) : 482,	(6,568) : 330,	(7,568) : 287,	(8,568) : 44,	(9,568) : 337,	(10,568) : 698,	(11,568) : 253,	(12,568) : 477,	(13,568) : 266,	(14,568) : 458,	(15,568) : 583,	(16,568) : 255,	(17,568) : 641,	(18,568) : 548,	(19,568) : 472,	(20,568) : 608,
(1,569) : 415,	(2,569) : 385,	(3,569) : 184,	(4,569) : 406,	(5,569) : 199,	(6,569) : 510,	(7,569) : 454,	(8,569) : 436,	(9,569) : 184,	(10,569) : 441,	(11,569) : 192,	(12,569) : 436,	(13,569) : 454,	(14,569) : 466,	(15,569) : 469,	(16,569) : 392,	(17,569) : 311,	(18,569) : 288,	(19,569) : 167,	(20,569) : 230,
(1,570) : 222,	(2,570) : 462,	(3,570) : 459,	(4,570) : 241,	(5,570) : 439,	(6,570) : 295,	(7,570) : 310,	(8,570) : 26,	(9,570) : 320,	(10,570) : 692,	(11,570) : 222,	(12,570) : 433,	(13,570) : 229,	(14,570) : 472,	(15,570) : 588,	(16,570) : 212,	(17,570) : 626,	(18,570) : 503,	(19,570) : 454,	(20,570) : 587,
(1,571) : 616,	(2,571) : 527,	(3,571) : 273,	(4,571) : 601,	(5,571) : 305,	(6,571) : 701,	(7,571) : 657,	(8,571) : 667,	(9,571) : 389,	(10,571) : 453,	(11,571) : 424,	(12,571) : 568,	(13,571) : 655,	(14,571) : 620,	(15,571) : 566,	(16,571) : 594,	(17,571) : 308,	(18,571) : 350,	(19,571) : 293,	(20,571) : 219,
(1,572) : 208,	(2,572) : 742,	(3,572) : 297,	(4,572) : 185,	(5,572) : 266,	(6,572) : 210,	(7,572) : 685,	(8,572) : 422,	(9,572) : 519,	(10,572) : 870,	(11,572) : 390,	(12,572) : 14,	(13,572) : 225,	(14,572) : 795,	(15,572) : 855,	(16,572) : 210,	(17,572) : 745,	(18,572) : 236,	(19,572) : 587,	(20,572) : 662,
(1,573) : 278,	(2,573) : 691,	(3,573) : 181,	(4,573) : 254,	(5,573) : 153,	(6,573) : 316,	(7,573) : 672,	(8,573) : 461,	(9,573) : 464,	(10,573) : 784,	(11,573) : 355,	(12,573) : 122,	(13,573) : 307,	(14,573) : 755,	(15,573) : 794,	(16,573) : 269,	(17,573) : 651,	(18,573) : 109,	(19,573) : 507,	(20,573) : 563,
(1,574) : 168,	(2,574) : 549,	(3,574) : 198,	(4,574) : 153,	(5,574) : 169,	(6,574) : 253,	(7,574) : 506,	(8,574) : 300,	(9,574) : 326,	(10,574) : 685,	(11,574) : 198,	(12,574) : 201,	(13,574) : 208,	(14,574) : 603,	(15,574) : 662,	(16,574) : 149,	(17,574) : 568,	(18,574) : 217,	(19,574) : 402,	(20,574) : 493,
(1,575) : 13,	(2,575) : 631,	(3,575) : 377,	(4,575) : 33,	(5,575) : 347,	(6,575) : 90,	(7,575) : 522,	(8,575) : 221,	(9,575) : 432,	(10,575) : 815,	(11,575) : 294,	(12,575) : 226,	(13,575) : 26,	(14,575) : 662,	(15,575) : 754,	(16,575) : 36,	(17,575) : 715,	(18,575) : 373,	(19,575) : 540,	(20,575) : 650,
(1,576) : 634,	(2,576) : 524,	(3,576) : 294,	(4,576) : 620,	(5,576) : 326,	(6,576) : 720,	(7,576) : 661,	(8,576) : 680,	(9,576) : 394,	(10,576) : 440,	(11,576) : 436,	(12,576) : 589,	(13,576) : 673,	(14,576) : 617,	(15,576) : 559,	(16,576) : 612,	(17,576) : 295,	(18,576) : 372,	(19,576) : 292,	(20,576) : 209,
(1,577) : 233,	(2,577) : 693,	(3,577) : 590,	(4,577) : 256,	(5,577) : 562,	(6,577) : 215,	(7,577) : 519,	(8,577) : 208,	(9,577) : 546,	(10,577) : 923,	(11,577) : 428,	(12,577) : 434,	(13,577) : 201,	(14,577) : 694,	(15,577) : 818,	(16,577) : 247,	(17,577) : 850,	(18,577) : 596,	(19,577) : 675,	(20,577) : 802,
(1,578) : 290,	(2,578) : 368,	(3,578) : 447,	(4,578) : 305,	(5,578) : 433,	(6,578) : 376,	(7,578) : 225,	(8,578) : 118,	(9,578) : 241,	(10,578) : 602,	(11,578) : 176,	(12,578) : 482,	(13,578) : 307,	(14,578) : 379,	(15,578) : 494,	(16,578) : 276,	(17,578) : 544,	(18,578) : 510,	(19,578) : 377,	(20,578) : 513,
(1,579) : 521,	(2,579) : 372,	(3,579) : 256,	(4,579) : 512,	(5,579) : 280,	(6,579) : 617,	(7,579) : 492,	(8,579) : 529,	(9,579) : 225,	(10,579) : 363,	(11,579) : 283,	(12,579) : 534,	(13,579) : 561,	(14,579) : 462,	(15,579) : 430,	(16,579) : 498,	(17,579) : 222,	(18,579) : 359,	(19,579) : 133,	(20,579) : 132,
(1,580) : 929,	(2,580) : 639,	(3,580) : 601,	(4,580) : 917,	(5,580) : 633,	(6,580) : 1020,	(7,580) : 835,	(8,580) : 941,	(9,580) : 610,	(10,580) : 421,	(11,580) : 697,	(12,580) : 895,	(13,580) : 969,	(14,580) : 726,	(15,580) : 609,	(16,580) : 907,	(17,580) : 347,	(18,580) : 676,	(19,580) : 476,	(20,580) : 339,
(1,581) : 286,	(2,581) : 494,	(3,581) : 549,	(4,581) : 308,	(5,581) : 529,	(6,581) : 340,	(7,581) : 309,	(8,581) : 75,	(9,581) : 387,	(10,581) : 742,	(11,581) : 306,	(12,581) : 506,	(13,581) : 283,	(14,581) : 487,	(15,581) : 618,	(16,581) : 282,	(17,581) : 689,	(18,581) : 592,	(19,581) : 523,	(20,581) : 659,
(1,582) : 207,	(2,582) : 583,	(3,582) : 529,	(4,582) : 231,	(5,582) : 503,	(6,582) : 238,	(7,582) : 414,	(8,582) : 99,	(9,582) : 442,	(10,582) : 816,	(11,582) : 332,	(12,582) : 428,	(13,582) : 192,	(14,582) : 586,	(15,582) : 709,	(16,582) : 211,	(17,582) : 747,	(18,582) : 552,	(19,582) : 573,	(20,582) : 704,
(1,583) : 91,	(2,583) : 536,	(3,583) : 305,	(4,583) : 92,	(5,583) : 279,	(6,583) : 192,	(7,583) : 449,	(8,583) : 195,	(9,583) : 331,	(10,583) : 712,	(11,583) : 192,	(12,583) : 252,	(13,583) : 128,	(14,583) : 575,	(15,583) : 657,	(16,583) : 68,	(17,583) : 611,	(18,583) : 328,	(19,583) : 436,	(20,583) : 548,
(1,584) : 119,	(2,584) : 637,	(3,584) : 475,	(4,584) : 142,	(5,584) : 447,	(6,584) : 131,	(7,584) : 493,	(8,584) : 168,	(9,584) : 465,	(10,584) : 849,	(11,584) : 336,	(12,584) : 330,	(13,584) : 91,	(14,584) : 654,	(15,584) : 763,	(16,584) : 132,	(17,584) : 763,	(18,584) : 481,	(19,584) : 587,	(20,584) : 708,
(1,585) : 285,	(2,585) : 836,	(3,585) : 366,	(4,585) : 264,	(5,585) : 337,	(6,585) : 257,	(7,585) : 777,	(8,585) : 504,	(9,585) : 612,	(10,585) : 958,	(11,585) : 484,	(12,585) : 86,	(13,585) : 292,	(14,585) : 888,	(15,585) : 948,	(16,585) : 292,	(17,585) : 829,	(18,585) : 286,	(19,585) : 676,	(20,585) : 744,
(1,586) : 93,	(2,586) : 576,	(3,586) : 420,	(4,586) : 116,	(5,586) : 393,	(6,586) : 157,	(7,586) : 445,	(8,586) : 131,	(9,586) : 397,	(10,586) : 782,	(11,586) : 268,	(12,586) : 315,	(13,586) : 91,	(14,586) : 598,	(15,586) : 702,	(16,586) : 93,	(17,586) : 695,	(18,586) : 436,	(19,586) : 519,	(20,586) : 640,
(1,587) : 88,	(2,587) : 707,	(3,587) : 395,	(4,587) : 81,	(5,587) : 363,	(6,587) : 47,	(7,587) : 603,	(8,587) : 299,	(9,587) : 502,	(10,587) : 881,	(11,587) : 364,	(12,587) : 178,	(13,587) : 77,	(14,587) : 742,	(15,587) : 830,	(16,587) : 107,	(17,587) : 774,	(18,587) : 369,	(19,587) : 602,	(20,587) : 704,
(1,588) : 705,	(2,588) : 354,	(3,588) : 448,	(4,588) : 698,	(5,588) : 474,	(6,588) : 803,	(7,588) : 544,	(8,588) : 676,	(9,588) : 333,	(10,588) : 206,	(11,588) : 441,	(12,588) : 730,	(13,588) : 743,	(14,588) : 445,	(15,588) : 350,	(16,588) : 682,	(17,588) : 69,	(18,588) : 548,	(19,588) : 197,	(20,588) : 68,
(1,589) : 542,	(2,589) : 447,	(3,589) : 230,	(4,589) : 530,	(5,589) : 259,	(6,589) : 632,	(7,589) : 565,	(8,589) : 577,	(9,589) : 295,	(10,589) : 415,	(11,589) : 332,	(12,589) : 522,	(13,589) : 582,	(14,589) : 538,	(15,589) : 500,	(16,589) : 520,	(17,589) : 268,	(18,589) : 325,	(19,589) : 209,	(20,589) : 173,
(1,590) : 656,	(2,590) : 495,	(3,590) : 329,	(4,590) : 643,	(5,590) : 360,	(6,590) : 745,	(7,590) : 644,	(8,590) : 687,	(9,590) : 384,	(10,590) : 394,	(11,590) : 441,	(12,590) : 624,	(13,590) : 695,	(14,590) : 588,	(15,590) : 521,	(16,590) : 634,	(17,590) : 251,	(18,590) : 413,	(19,590) : 270,	(20,590) : 170,
(1,591) : 764,	(2,591) : 277,	(3,591) : 561,	(4,591) : 762,	(5,591) : 582,	(6,591) : 865,	(7,591) : 494,	(8,591) : 693,	(9,591) : 352,	(10,591) : 61,	(11,591) : 484,	(12,591) : 824,	(13,591) : 799,	(14,591) : 356,	(15,591) : 229,	(16,591) : 741,	(17,591) : 86,	(18,591) : 665,	(19,591) : 238,	(20,591) : 181,
(1,592) : 661,	(2,592) : 467,	(3,592) : 346,	(4,592) : 649,	(5,592) : 376,	(6,592) : 752,	(7,592) : 623,	(8,592) : 681,	(9,592) : 368,	(10,592) : 361,	(11,592) : 435,	(12,592) : 640,	(13,592) : 700,	(14,592) : 561,	(15,592) : 490,	(16,592) : 638,	(17,592) : 219,	(18,592) : 434,	(19,592) : 249,	(20,592) : 140,
(1,593) : 636,	(2,593) : 470,	(3,593) : 317,	(4,593) : 624,	(5,593) : 347,	(6,593) : 726,	(7,593) : 618,	(8,593) : 662,	(9,593) : 357,	(10,593) : 380,	(11,593) : 416,	(12,593) : 612,	(13,593) : 675,	(14,593) : 564,	(15,593) : 500,	(16,593) : 613,	(17,593) : 235,	(18,593) : 405,	(19,593) : 244,	(20,593) : 150,
(1,594) : 615,	(2,594) : 92,	(3,594) : 504,	(4,594) : 617,	(5,594) : 516,	(6,594) : 716,	(7,594) : 294,	(8,594) : 509,	(9,594) : 196,	(10,594) : 211,	(11,594) : 335,	(12,594) : 719,	(13,594) : 646,	(14,594) : 185,	(15,594) : 148,	(16,594) : 593,	(17,594) : 203,	(18,594) : 608,	(19,594) : 161,	(20,594) : 239,
(1,595) : 143,	(2,595) : 478,	(3,595) : 332,	(4,595) : 151,	(5,595) : 310,	(6,595) : 241,	(7,595) : 382,	(8,595) : 146,	(9,595) : 284,	(10,595) : 669,	(11,595) : 151,	(12,595) : 318,	(13,595) : 171,	(14,595) : 512,	(15,595) : 602,	(16,595) : 123,	(17,595) : 578,	(18,595) : 372,	(19,595) : 402,	(20,595) : 522,
(1,596) : 89,	(2,596) : 644,	(3,596) : 451,	(4,596) : 111,	(5,596) : 422,	(6,596) : 101,	(7,596) : 510,	(8,596) : 188,	(9,596) : 463,	(10,596) : 848,	(11,596) : 330,	(12,596) : 297,	(13,596) : 58,	(14,596) : 665,	(15,596) : 770,	(16,596) : 105,	(17,596) : 757,	(18,596) : 452,	(19,596) : 581,	(20,596) : 699,
(1,597) : 397,	(2,597) : 346,	(3,597) : 559,	(4,597) : 415,	(5,597) : 547,	(6,597) : 473,	(7,597) : 146,	(8,597) : 189,	(9,597) : 299,	(10,597) : 608,	(11,597) : 279,	(12,597) : 598,	(13,597) : 407,	(14,597) : 327,	(15,597) : 466,	(16,597) : 385,	(17,597) : 577,	(18,597) : 627,	(19,597) : 429,	(20,597) : 566,
(1,598) : 54,	(2,598) : 580,	(3,598) : 317,	(4,598) : 49,	(5,598) : 288,	(6,598) : 152,	(7,598) : 489,	(8,598) : 218,	(9,598) : 373,	(10,598) : 753,	(11,598) : 235,	(12,598) : 219,	(13,598) : 93,	(14,598) : 618,	(15,598) : 701,	(16,598) : 31,	(17,598) : 650,	(18,598) : 325,	(19,598) : 476,	(20,598) : 584,
(1,599) : 654,	(2,599) : 93,	(3,599) : 639,	(4,599) : 663,	(5,599) : 644,	(6,599) : 748,	(7,599) : 185,	(8,599) : 492,	(9,599) : 295,	(10,599) : 339,	(11,599) : 405,	(12,599) : 804,	(13,599) : 677,	(14,599) : 0,	(15,599) : 154,	(16,599) : 635,	(17,599) : 381,	(18,599) : 737,	(19,599) : 329,	(20,599) : 424,
(1,600) : 617,	(2,600) : 44,	(3,600) : 534,	(4,600) : 622,	(5,600) : 544,	(6,600) : 717,	(7,600) : 256,	(8,600) : 496,	(9,600) : 209,	(10,600) : 239,	(11,600) : 342,	(12,600) : 735,	(13,600) : 647,	(14,600) : 137,	(15,600) : 128,	(16,600) : 596,	(17,600) : 248,	(18,600) : 637,	(19,600) : 200,	(20,600) : 286,
(1,601) : 661,	(2,601) : 478,	(3,601) : 342,	(4,601) : 649,	(5,601) : 372,	(6,601) : 752,	(7,601) : 632,	(8,601) : 685,	(9,601) : 375,	(10,601) : 372,	(11,601) : 439,	(12,601) : 637,	(13,601) : 701,	(14,601) : 571,	(15,601) : 501,	(16,601) : 639,	(17,601) : 230,	(18,601) : 428,	(19,601) : 258,	(20,601) : 151,
(1,602) : 360,	(2,602) : 429,	(3,602) : 134,	(4,602) : 349,	(5,602) : 143,	(6,602) : 452,	(7,602) : 471,	(8,602) : 406,	(9,602) : 212,	(10,602) : 504,	(11,602) : 174,	(12,602) : 372,	(13,602) : 400,	(14,602) : 505,	(15,602) : 521,	(16,602) : 337,	(17,602) : 375,	(18,602) : 235,	(19,602) : 226,	(20,602) : 294,
(1,603) : 561,	(2,603) : 170,	(3,603) : 418,	(4,603) : 561,	(5,603) : 431,	(6,603) : 662,	(7,603) : 330,	(8,603) : 485,	(9,603) : 143,	(10,603) : 242,	(11,603) : 278,	(12,603) : 645,	(13,603) : 595,	(14,603) : 262,	(15,603) : 235,	(16,603) : 538,	(17,603) : 172,	(18,603) : 522,	(19,603) : 72,	(20,603) : 173,
(1,604) : 161,	(2,604) : 578,	(3,604) : 204,	(4,604) : 142,	(5,604) : 173,	(6,604) : 237,	(7,604) : 533,	(8,604) : 315,	(9,604) : 355,	(10,604) : 713,	(11,604) : 227,	(12,604) : 172,	(13,604) : 199,	(14,604) : 632,	(15,604) : 691,	(16,604) : 144,	(17,604) : 594,	(18,604) : 209,	(19,604) : 429,	(20,604) : 517,
(1,605) : 737,	(2,605) : 525,	(3,605) : 411,	(4,605) : 725,	(5,605) : 442,	(6,605) : 827,	(7,605) : 693,	(8,605) : 761,	(9,605) : 444,	(10,605) : 379,	(11,605) : 514,	(12,605) : 706,	(13,605) : 777,	(14,605) : 618,	(15,605) : 531,	(16,605) : 715,	(17,605) : 251,	(18,605) : 491,	(19,605) : 320,	(20,605) : 195,
(1,606) : 612,	(2,606) : 542,	(3,606) : 265,	(4,606) : 597,	(5,606) : 297,	(6,606) : 696,	(7,606) : 668,	(8,606) : 670,	(9,606) : 398,	(10,606) : 472,	(11,606) : 428,	(12,606) : 558,	(13,606) : 652,	(14,606) : 634,	(15,606) : 583,	(16,606) : 591,	(17,606) : 326,	(18,606) : 338,	(19,606) : 306,	(20,606) : 237,
(1,607) : 186,	(2,607) : 692,	(3,607) : 548,	(4,607) : 208,	(5,607) : 519,	(6,607) : 161,	(7,607) : 532,	(8,607) : 210,	(9,607) : 531,	(10,607) : 913,	(11,607) : 406,	(12,607) : 381,	(13,607) : 152,	(14,607) : 701,	(15,607) : 819,	(16,607) : 202,	(17,607) : 832,	(18,607) : 548,	(19,607) : 656,	(20,607) : 780,
(1,608) : 629,	(2,608) : 353,	(3,608) : 367,	(4,608) : 621,	(5,608) : 392,	(6,608) : 726,	(7,608) : 518,	(8,608) : 614,	(9,608) : 279,	(10,608) : 267,	(11,608) : 373,	(12,608) : 648,	(13,608) : 667,	(14,608) : 446,	(15,608) : 377,	(16,608) : 606,	(17,608) : 120,	(18,608) : 468,	(19,608) : 147,	(20,608) : 27,
(1,609) : 351,	(2,609) : 332,	(3,609) : 243,	(4,609) : 347,	(5,609) : 245,	(6,609) : 451,	(7,609) : 357,	(8,609) : 333,	(9,609) : 106,	(10,609) : 458,	(11,609) : 87,	(12,609) : 427,	(13,609) : 387,	(14,609) : 400,	(15,609) : 437,	(16,609) : 327,	(17,609) : 351,	(18,609) : 337,	(19,609) : 177,	(20,609) : 291,
(1,610) : 31,	(2,610) : 640,	(3,610) : 351,	(4,610) : 13,	(5,610) : 320,	(6,610) : 95,	(7,610) : 542,	(8,610) : 250,	(9,610) : 434,	(10,610) : 814,	(11,610) : 296,	(12,610) : 191,	(13,610) : 56,	(14,610) : 676,	(15,610) : 762,	(16,610) : 42,	(17,610) : 709,	(18,610) : 341,	(19,610) : 536,	(20,610) : 641,
(1,611) : 108,	(2,611) : 611,	(3,611) : 263,	(4,611) : 86,	(5,611) : 232,	(6,611) : 177,	(7,611) : 544,	(8,611) : 292,	(9,611) : 393,	(10,611) : 761,	(11,611) : 258,	(12,611) : 152,	(13,611) : 143,	(14,611) : 658,	(15,611) : 728,	(16,611) : 95,	(17,611) : 647,	(18,611) : 255,	(19,611) : 479,	(20,611) : 573,
(1,612) : 325,	(2,612) : 871,	(3,612) : 388,	(4,612) : 303,	(5,612) : 360,	(6,612) : 293,	(7,612) : 816,	(8,612) : 543,	(9,612) : 647,	(10,612) : 987,	(11,612) : 520,	(12,612) : 122,	(13,612) : 331,	(14,612) : 925,	(15,612) : 982,	(16,612) : 331,	(17,612) : 856,	(18,612) : 301,	(19,612) : 706,	(20,612) : 769,
(1,613) : 218,	(2,613) : 731,	(3,613) : 273,	(4,613) : 194,	(5,613) : 242,	(6,613) : 229,	(7,613) : 681,	(8,613) : 427,	(9,613) : 507,	(10,613) : 852,	(11,613) : 381,	(12,613) : 22,	(13,613) : 238,	(14,613) : 786,	(15,613) : 843,	(16,613) : 218,	(17,613) : 726,	(18,613) : 211,	(19,613) : 570,	(20,613) : 642,
(1,614) : 529,	(2,614) : 158,	(3,614) : 411,	(4,614) : 530,	(5,614) : 421,	(6,614) : 631,	(7,614) : 295,	(8,614) : 446,	(9,614) : 109,	(10,614) : 276,	(11,614) : 247,	(12,614) : 624,	(13,614) : 563,	(14,614) : 246,	(15,614) : 243,	(16,614) : 507,	(17,614) : 212,	(18,614) : 514,	(19,614) : 85,	(20,614) : 207,
(1,615) : 431,	(2,615) : 227,	(3,615) : 499,	(4,615) : 443,	(5,615) : 494,	(6,615) : 522,	(7,615) : 99,	(8,615) : 263,	(9,615) : 189,	(10,615) : 481,	(11,615) : 223,	(12,615) : 601,	(13,615) : 451,	(14,615) : 229,	(15,615) : 351,	(16,615) : 413,	(17,615) : 450,	(18,615) : 582,	(19,615) : 309,	(20,615) : 444,
(1,616) : 275,	(2,616) : 506,	(3,616) : 107,	(4,616) : 260,	(5,616) : 89,	(6,616) : 360,	(7,616) : 508,	(8,616) : 370,	(9,616) : 280,	(10,616) : 605,	(11,616) : 187,	(12,616) : 270,	(13,616) : 315,	(14,616) : 573,	(15,616) : 608,	(16,616) : 254,	(17,616) : 479,	(18,616) : 173,	(19,616) : 324,	(20,616) : 397,
(1,617) : 254,	(2,617) : 801,	(3,617) : 339,	(4,617) : 232,	(5,617) : 309,	(6,617) : 236,	(7,617) : 742,	(8,617) : 472,	(9,617) : 577,	(10,617) : 925,	(11,617) : 449,	(12,617) : 51,	(13,617) : 265,	(14,617) : 853,	(15,617) : 913,	(16,617) : 260,	(17,617) : 797,	(18,617) : 265,	(19,617) : 642,	(20,617) : 713,
(1,618) : 173,	(2,618) : 694,	(3,618) : 270,	(4,618) : 148,	(5,618) : 238,	(6,618) : 196,	(7,618) : 637,	(8,618) : 380,	(9,618) : 472,	(10,618) : 827,	(11,618) : 342,	(12,618) : 59,	(13,618) : 196,	(14,618) : 746,	(15,618) : 808,	(16,618) : 171,	(17,618) : 704,	(18,618) : 225,	(19,618) : 543,	(20,618) : 623,
(1,619) : 165,	(2,619) : 697,	(3,619) : 279,	(4,619) : 141,	(5,619) : 247,	(6,619) : 185,	(7,619) : 635,	(8,619) : 375,	(9,619) : 475,	(10,619) : 832,	(11,619) : 344,	(12,619) : 62,	(13,619) : 187,	(14,619) : 747,	(15,619) : 811,	(16,619) : 164,	(17,619) : 710,	(18,619) : 236,	(19,619) : 548,	(20,619) : 630,
(1,620) : 335,	(2,620) : 461,	(3,620) : 574,	(4,620) : 356,	(5,620) : 556,	(6,620) : 393,	(7,620) : 264,	(8,620) : 117,	(9,620) : 377,	(10,620) : 716,	(11,620) : 315,	(12,620) : 552,	(13,620) : 334,	(14,620) : 445,	(15,620) : 582,	(16,620) : 329,	(17,620) : 673,	(18,620) : 624,	(19,620) : 513,	(20,620) : 650,
(1,621) : 521,	(2,621) : 205,	(3,621) : 371,	(4,621) : 520,	(5,621) : 384,	(6,621) : 622,	(7,621) : 338,	(8,621) : 458,	(9,621) : 113,	(10,621) : 283,	(11,621) : 240,	(12,621) : 599,	(13,621) : 556,	(14,621) : 294,	(15,621) : 281,	(16,621) : 498,	(17,621) : 193,	(18,621) : 476,	(19,621) : 37,	(20,621) : 169,
(1,622) : 701,	(2,622) : 505,	(3,622) : 376,	(4,622) : 688,	(5,622) : 407,	(6,622) : 790,	(7,622) : 667,	(8,622) : 726,	(9,622) : 413,	(10,622) : 378,	(11,622) : 479,	(12,622) : 671,	(13,622) : 740,	(14,622) : 599,	(15,622) : 520,	(16,622) : 678,	(17,622) : 243,	(18,622) : 458,	(19,622) : 292,	(20,622) : 175,
(1,623) : 623,	(2,623) : 82,	(3,623) : 612,	(4,623) : 633,	(5,623) : 616,	(6,623) : 718,	(7,623) : 163,	(8,623) : 463,	(9,623) : 266,	(10,623) : 343,	(11,623) : 375,	(12,623) : 773,	(13,623) : 647,	(14,623) : 31,	(15,623) : 168,	(16,623) : 604,	(17,623) : 373,	(18,623) : 709,	(19,623) : 309,	(20,623) : 410,
(1,624) : 97,	(2,624) : 552,	(3,624) : 277,	(4,624) : 88,	(5,624) : 249,	(6,624) : 193,	(7,624) : 477,	(8,624) : 232,	(9,624) : 340,	(10,624) : 716,	(11,624) : 202,	(12,624) : 220,	(13,624) : 137,	(14,624) : 595,	(15,624) : 671,	(16,624) : 75,	(17,624) : 610,	(18,624) : 293,	(19,624) : 437,	(20,624) : 542,
(1,625) : 239,	(2,625) : 781,	(3,625) : 324,	(4,625) : 216,	(5,625) : 293,	(6,625) : 227,	(7,625) : 723,	(8,625) : 456,	(9,625) : 558,	(10,625) : 906,	(11,625) : 429,	(12,625) : 32,	(13,625) : 251,	(14,625) : 833,	(15,625) : 894,	(16,625) : 243,	(17,625) : 779,	(18,625) : 253,	(19,625) : 623,	(20,625) : 695,
(1,626) : 243,	(2,626) : 451,	(3,626) : 183,	(4,626) : 235,	(5,626) : 168,	(6,626) : 339,	(7,626) : 433,	(8,626) : 298,	(9,626) : 225,	(10,626) : 581,	(11,626) : 112,	(12,626) : 302,	(13,626) : 282,	(14,626) : 511,	(15,626) : 561,	(16,626) : 220,	(17,626) : 467,	(18,626) : 250,	(19,626) : 299,	(20,626) : 396,
(1,627) : 313,	(2,627) : 835,	(3,627) : 341,	(4,627) : 290,	(5,627) : 314,	(6,627) : 297,	(7,627) : 789,	(8,627) : 529,	(9,627) : 609,	(10,627) : 943,	(11,627) : 487,	(12,627) : 94,	(13,627) : 325,	(14,627) : 892,	(15,627) : 944,	(16,627) : 316,	(17,627) : 812,	(18,627) : 253,	(19,627) : 664,	(20,627) : 724,
(1,628) : 614,	(2,628) : 464,	(3,628) : 296,	(4,628) : 602,	(5,628) : 326,	(6,628) : 704,	(7,628) : 605,	(8,628) : 643,	(9,628) : 342,	(10,628) : 387,	(11,628) : 396,	(12,628) : 591,	(13,628) : 654,	(14,628) : 557,	(15,628) : 499,	(16,628) : 592,	(17,628) : 241,	(18,628) : 385,	(19,628) : 234,	(20,628) : 151,
(1,629) : 594,	(2,629) : 85,	(3,629) : 493,	(4,629) : 597,	(5,629) : 504,	(6,629) : 695,	(7,629) : 275,	(8,629) : 486,	(9,629) : 177,	(10,629) : 234,	(11,629) : 315,	(12,629) : 702,	(13,629) : 625,	(14,629) : 178,	(15,629) : 162,	(16,629) : 572,	(17,629) : 218,	(18,629) : 596,	(19,629) : 156,	(20,629) : 247,
(1,630) : 771,	(2,630) : 161,	(3,630) : 721,	(4,630) : 779,	(5,630) : 730,	(6,630) : 867,	(7,630) : 305,	(8,630) : 614,	(9,630) : 387,	(10,630) : 290,	(11,630) : 511,	(12,630) : 909,	(13,630) : 795,	(14,630) : 123,	(15,630) : 98,	(16,630) : 751,	(17,630) : 378,	(18,630) : 823,	(19,630) : 384,	(20,630) : 445,
(1,631) : 167,	(2,631) : 648,	(3,631) : 521,	(4,631) : 191,	(5,631) : 493,	(6,631) : 169,	(7,631) : 489,	(8,631) : 165,	(9,631) : 488,	(10,631) : 869,	(11,631) : 365,	(12,631) : 377,	(13,631) : 140,	(14,631) : 657,	(15,631) : 774,	(16,631) : 180,	(17,631) : 790,	(18,631) : 529,	(19,631) : 614,	(20,631) : 739,
(1,632) : 681,	(2,632) : 497,	(3,632) : 357,	(4,632) : 669,	(5,632) : 388,	(6,632) : 771,	(7,632) : 654,	(8,632) : 708,	(9,632) : 399,	(10,632) : 381,	(11,632) : 462,	(12,632) : 652,	(13,632) : 721,	(14,632) : 591,	(15,632) : 517,	(16,632) : 659,	(17,632) : 242,	(18,632) : 441,	(19,632) : 280,	(20,632) : 168,
(1,633) : 59,	(2,633) : 651,	(3,633) : 424,	(4,633) : 79,	(5,633) : 394,	(6,633) : 74,	(7,633) : 528,	(8,633) : 212,	(9,633) : 461,	(10,633) : 846,	(11,633) : 326,	(12,633) : 260,	(13,633) : 23,	(14,633) : 677,	(15,633) : 776,	(16,633) : 80,	(17,633) : 750,	(18,633) : 419,	(19,633) : 575,	(20,633) : 688,
(1,634) : 136,	(2,634) : 540,	(3,634) : 236,	(4,634) : 124,	(5,634) : 208,	(6,634) : 228,	(7,634) : 482,	(8,634) : 261,	(9,634) : 321,	(10,634) : 691,	(11,634) : 187,	(12,634) : 216,	(13,634) : 176,	(14,634) : 589,	(15,634) : 656,	(16,634) : 115,	(17,634) : 580,	(18,634) : 257,	(19,634) : 409,	(20,634) : 509,
(1,635) : 331,	(2,635) : 643,	(3,635) : 658,	(4,635) : 356,	(5,635) : 633,	(6,635) : 340,	(7,635) : 444,	(8,635) : 198,	(9,635) : 537,	(10,635) : 894,	(11,635) : 444,	(12,635) : 548,	(13,635) : 309,	(14,635) : 627,	(15,635) : 765,	(16,635) : 338,	(17,635) : 841,	(18,635) : 682,	(19,635) : 673,	(20,635) : 808,
(1,636) : 92,	(2,636) : 536,	(3,636) : 304,	(4,636) : 92,	(5,636) : 278,	(6,636) : 193,	(7,636) : 449,	(8,636) : 196,	(9,636) : 330,	(10,636) : 711,	(11,636) : 191,	(12,636) : 251,	(13,636) : 128,	(14,636) : 574,	(15,636) : 657,	(16,636) : 69,	(17,636) : 610,	(18,636) : 327,	(19,636) : 436,	(20,636) : 547,
(1,637) : 94,	(2,637) : 659,	(3,637) : 307,	(4,637) : 70,	(5,637) : 275,	(6,637) : 132,	(7,637) : 580,	(8,637) : 306,	(9,637) : 444,	(10,637) : 814,	(11,637) : 307,	(12,637) : 130,	(13,637) : 118,	(14,637) : 703,	(15,637) : 778,	(16,637) : 94,	(17,637) : 701,	(18,637) : 285,	(19,637) : 532,	(20,637) : 627,
(1,638) : 369,	(2,638) : 422,	(3,638) : 581,	(4,638) : 389,	(5,638) : 566,	(6,638) : 433,	(7,638) : 218,	(8,638) : 150,	(9,638) : 357,	(10,638) : 681,	(11,638) : 312,	(12,638) : 582,	(13,638) : 372,	(14,638) : 401,	(15,638) : 541,	(16,638) : 361,	(17,638) : 646,	(18,638) : 639,	(19,638) : 490,	(20,638) : 628,
(1,639) : 286,	(2,639) : 630,	(3,639) : 616,	(4,639) : 310,	(5,639) : 591,	(6,639) : 297,	(7,639) : 440,	(8,639) : 166,	(9,639) : 511,	(10,639) : 875,	(11,639) : 411,	(12,639) : 504,	(13,639) : 264,	(14,639) : 621,	(15,639) : 754,	(16,639) : 293,	(17,639) : 816,	(18,639) : 638,	(19,639) : 645,	(20,639) : 778,
(1,640) : 324,	(2,640) : 781,	(3,640) : 259,	(4,640) : 300,	(5,640) : 234,	(6,640) : 336,	(7,640) : 756,	(8,640) : 526,	(9,640) : 554,	(10,640) : 870,	(11,640) : 442,	(12,640) : 113,	(13,640) : 346,	(14,640) : 844,	(15,640) : 884,	(16,640) : 321,	(17,640) : 735,	(18,640) : 165,	(19,640) : 595,	(20,640) : 645,
(1,641) : 191,	(2,641) : 726,	(3,641) : 291,	(4,641) : 168,	(5,641) : 259,	(6,641) : 199,	(7,641) : 667,	(8,641) : 404,	(9,641) : 503,	(10,641) : 857,	(11,641) : 374,	(12,641) : 31,	(13,641) : 209,	(14,641) : 777,	(15,641) : 840,	(16,641) : 193,	(17,641) : 733,	(18,641) : 236,	(19,641) : 574,	(20,641) : 651,
(1,642) : 231,	(2,642) : 518,	(3,642) : 513,	(4,642) : 254,	(5,642) : 490,	(6,642) : 284,	(7,642) : 347,	(8,642) : 42,	(9,642) : 387,	(10,642) : 755,	(11,642) : 288,	(12,642) : 452,	(13,642) : 227,	(14,642) : 519,	(15,642) : 643,	(16,642) : 228,	(17,642) : 692,	(18,642) : 548,	(19,642) : 521,	(20,642) : 654,
(1,643) : 416,	(2,643) : 633,	(3,643) : 78,	(4,643) : 396,	(5,643) : 92,	(6,643) : 479,	(7,643) : 674,	(8,643) : 547,	(9,643) : 420,	(10,643) : 665,	(11,643) : 360,	(12,643) : 305,	(13,643) : 453,	(14,643) : 712,	(15,643) : 716,	(16,643) : 400,	(17,643) : 522,	(18,643) : 76,	(19,643) : 411,	(20,643) : 428,
(1,644) : 655,	(2,644) : 343,	(3,644) : 400,	(4,644) : 649,	(5,644) : 425,	(6,644) : 753,	(7,644) : 520,	(8,644) : 632,	(9,644) : 293,	(10,644) : 237,	(11,644) : 394,	(12,644) : 679,	(13,644) : 694,	(14,644) : 436,	(15,644) : 357,	(16,644) : 632,	(17,644) : 90,	(18,644) : 500,	(19,644) : 157,	(20,644) : 20,
(1,645) : 582,	(2,645) : 421,	(3,645) : 283,	(4,645) : 571,	(5,645) : 311,	(6,645) : 674,	(7,645) : 558,	(8,645) : 600,	(9,645) : 295,	(10,645) : 365,	(11,645) : 354,	(12,645) : 573,	(13,645) : 621,	(14,645) : 514,	(15,645) : 463,	(16,645) : 559,	(17,645) : 218,	(18,645) : 379,	(19,645) : 188,	(20,645) : 123,
(1,646) : 424,	(2,646) : 329,	(3,646) : 575,	(4,646) : 441,	(5,646) : 564,	(6,646) : 501,	(7,646) : 121,	(8,646) : 217,	(9,646) : 300,	(10,646) : 593,	(11,646) : 294,	(12,646) : 623,	(13,646) : 434,	(14,646) : 304,	(15,646) : 446,	(16,646) : 412,	(17,646) : 569,	(18,646) : 645,	(19,646) : 426,	(20,646) : 563,
(1,647) : 221,	(2,647) : 602,	(3,647) : 154,	(4,647) : 200,	(5,647) : 121,	(6,647) : 286,	(7,647) : 577,	(8,647) : 377,	(9,647) : 375,	(10,647) : 712,	(11,647) : 261,	(12,647) : 162,	(13,647) : 257,	(14,647) : 663,	(15,647) : 709,	(16,647) : 206,	(17,647) : 586,	(18,647) : 144,	(19,647) : 431,	(20,647) : 503,
(1,648) : 661,	(2,648) : 194,	(3,648) : 488,	(4,648) : 660,	(5,648) : 506,	(6,648) : 762,	(7,648) : 398,	(8,648) : 584,	(9,648) : 245,	(10,648) : 142,	(11,648) : 379,	(12,648) : 735,	(13,648) : 695,	(14,648) : 284,	(15,648) : 198,	(16,648) : 638,	(17,648) : 97,	(18,648) : 593,	(19,648) : 143,	(20,648) : 153,
(1,649) : 178,	(2,649) : 698,	(3,649) : 270,	(4,649) : 154,	(5,649) : 238,	(6,649) : 200,	(7,649) : 642,	(8,649) : 386,	(9,649) : 476,	(10,649) : 829,	(11,649) : 346,	(12,649) : 54,	(13,649) : 201,	(14,649) : 750,	(15,649) : 812,	(16,649) : 176,	(17,649) : 706,	(18,649) : 223,	(19,649) : 546,	(20,649) : 625,
(1,650) : 179,	(2,650) : 774,	(3,650) : 381,	(4,650) : 161,	(5,650) : 348,	(6,650) : 138,	(7,650) : 689,	(8,650) : 397,	(9,650) : 558,	(10,650) : 925,	(11,650) : 422,	(12,650) : 103,	(13,650) : 178,	(14,650) : 817,	(15,650) : 892,	(16,650) : 191,	(17,650) : 807,	(18,650) : 328,	(19,650) : 642,	(20,650) : 729,
(1,651) : 416,	(2,651) : 301,	(3,651) : 546,	(4,651) : 432,	(5,651) : 537,	(6,651) : 498,	(7,651) : 108,	(8,651) : 219,	(9,651) : 265,	(10,651) : 563,	(11,651) : 265,	(12,651) : 608,	(13,651) : 430,	(14,651) : 284,	(15,651) : 421,	(16,651) : 402,	(17,651) : 536,	(18,651) : 620,	(19,651) : 391,	(20,651) : 528,
(1,652) : 334,	(2,652) : 792,	(3,652) : 697,	(4,652) : 355,	(5,652) : 668,	(6,652) : 291,	(7,652) : 606,	(8,652) : 313,	(9,652) : 656,	(10,652) : 1030,	(11,652) : 541,	(12,652) : 515,	(13,652) : 299,	(14,652) : 786,	(15,652) : 917,	(16,652) : 351,	(17,652) : 961,	(18,652) : 695,	(19,652) : 786,	(20,652) : 915,
(1,653) : 187,	(2,653) : 633,	(3,653) : 533,	(4,653) : 211,	(5,653) : 506,	(6,653) : 196,	(7,653) : 469,	(8,653) : 149,	(9,653) : 481,	(10,653) : 860,	(11,653) : 363,	(12,653) : 401,	(13,653) : 162,	(14,653) : 639,	(15,653) : 759,	(16,653) : 197,	(17,653) : 785,	(18,653) : 545,	(19,653) : 610,	(20,653) : 737,
(1,654) : 197,	(2,654) : 640,	(3,654) : 544,	(4,654) : 221,	(5,654) : 517,	(6,654) : 202,	(7,654) : 473,	(8,654) : 155,	(9,654) : 490,	(10,654) : 868,	(11,654) : 372,	(12,654) : 410,	(13,654) : 171,	(14,654) : 645,	(15,654) : 766,	(16,654) : 207,	(17,654) : 794,	(18,654) : 556,	(19,654) : 619,	(20,654) : 747,
(1,655) : 725,	(2,655) : 130,	(3,655) : 609,	(4,655) : 729,	(5,655) : 623,	(6,655) : 826,	(7,655) : 350,	(8,655) : 607,	(9,655) : 309,	(10,655) : 151,	(11,655) : 446,	(12,655) : 832,	(13,655) : 756,	(14,655) : 187,	(15,655) : 57,	(16,655) : 703,	(17,655) : 223,	(18,655) : 714,	(19,655) : 259,	(20,655) : 295,
(1,656) : 637,	(2,656) : 330,	(3,656) : 387,	(4,656) : 630,	(5,656) : 411,	(6,656) : 735,	(7,656) : 503,	(8,656) : 613,	(9,656) : 273,	(10,656) : 242,	(11,656) : 375,	(12,656) : 664,	(13,656) : 675,	(14,656) : 424,	(15,656) : 351,	(16,656) : 614,	(17,656) : 95,	(18,656) : 489,	(19,656) : 138,	(20,656) : 0,
(1,657) : 272,	(2,657) : 396,	(3,657) : 454,	(4,657) : 288,	(5,657) : 438,	(6,657) : 354,	(7,657) : 247,	(8,657) : 89,	(9,657) : 267,	(10,657) : 630,	(11,657) : 191,	(12,657) : 470,	(13,657) : 285,	(14,657) : 405,	(15,657) : 522,	(16,657) : 259,	(17,657) : 570,	(18,657) : 511,	(19,657) : 402,	(20,657) : 538,
(1,658) : 337,	(2,658) : 731,	(3,658) : 191,	(4,658) : 313,	(5,658) : 170,	(6,658) : 369,	(7,658) : 725,	(8,658) : 522,	(9,658) : 506,	(10,658) : 806,	(11,658) : 404,	(12,658) : 158,	(13,658) : 366,	(14,658) : 799,	(15,658) : 830,	(16,658) : 329,	(17,658) : 668,	(18,658) : 92,	(19,658) : 535,	(20,658) : 578,
(1,659) : 344,	(2,659) : 505,	(3,659) : 607,	(4,659) : 367,	(5,659) : 588,	(6,659) : 391,	(7,659) : 303,	(8,659) : 136,	(9,659) : 423,	(10,659) : 762,	(11,659) : 355,	(12,659) : 565,	(13,659) : 338,	(14,659) : 486,	(15,659) : 626,	(16,659) : 341,	(17,659) : 720,	(18,659) : 652,	(19,659) : 559,	(20,659) : 696,
(1,660) : 632,	(2,660) : 341,	(3,660) : 376,	(4,660) : 625,	(5,660) : 400,	(6,660) : 729,	(7,660) : 510,	(8,660) : 612,	(9,660) : 275,	(10,660) : 255,	(11,660) : 373,	(12,660) : 655,	(13,660) : 670,	(14,660) : 435,	(15,660) : 364,	(16,660) : 608,	(17,660) : 109,	(18,660) : 477,	(19,660) : 141,	(20,660) : 14,
(1,661) : 360,	(2,661) : 537,	(3,661) : 26,	(4,661) : 343,	(5,661) : 45,	(6,661) : 438,	(7,661) : 573,	(8,661) : 461,	(9,661) : 320,	(10,661) : 594,	(11,661) : 260,	(12,661) : 307,	(13,661) : 399,	(14,661) : 613,	(15,661) : 627,	(16,661) : 341,	(17,661) : 457,	(18,661) : 130,	(19,661) : 326,	(20,661) : 367,
(1,662) : 327,	(2,662) : 500,	(3,662) : 72,	(4,662) : 312,	(5,662) : 71,	(6,662) : 412,	(7,662) : 525,	(8,662) : 414,	(9,662) : 279,	(10,662) : 576,	(11,662) : 210,	(12,662) : 306,	(13,662) : 367,	(14,662) : 573,	(15,662) : 595,	(16,662) : 307,	(17,662) : 444,	(18,662) : 164,	(19,662) : 300,	(20,662) : 359,
(1,663) : 333,	(2,663) : 341,	(3,663) : 243,	(4,663) : 330,	(5,663) : 241,	(6,663) : 434,	(7,663) : 355,	(8,663) : 317,	(9,663) : 114,	(10,663) : 474,	(11,663) : 71,	(12,663) : 415,	(13,663) : 370,	(14,663) : 407,	(15,663) : 448,	(16,663) : 310,	(17,663) : 369,	(18,663) : 333,	(19,663) : 194,	(20,663) : 309,
(1,664) : 218,	(2,664) : 611,	(3,664) : 161,	(4,664) : 197,	(5,664) : 128,	(6,664) : 280,	(7,664) : 585,	(8,664) : 380,	(9,664) : 385,	(10,664) : 723,	(11,664) : 269,	(12,664) : 152,	(13,664) : 254,	(14,664) : 672,	(15,664) : 719,	(16,664) : 204,	(17,664) : 596,	(18,664) : 145,	(19,664) : 441,	(20,664) : 513,
(1,665) : 244,	(2,665) : 649,	(3,665) : 165,	(4,665) : 222,	(5,665) : 134,	(6,665) : 294,	(7,665) : 626,	(8,665) : 418,	(9,665) : 423,	(10,665) : 751,	(11,665) : 310,	(12,665) : 131,	(13,665) : 277,	(14,665) : 711,	(15,665) : 755,	(16,665) : 233,	(17,665) : 621,	(18,665) : 121,	(19,665) : 471,	(20,665) : 536,
(1,666) : 40,	(2,666) : 648,	(3,666) : 403,	(4,666) : 57,	(5,666) : 373,	(6,666) : 71,	(7,666) : 532,	(8,666) : 222,	(9,666) : 453,	(10,666) : 837,	(11,666) : 316,	(12,666) : 239,	(13,666) : 0,	(14,666) : 677,	(15,666) : 772,	(16,666) : 62,	(17,666) : 739,	(18,666) : 397,	(19,666) : 564,	(20,666) : 675,
(1,667) : 318,	(2,667) : 742,	(3,667) : 214,	(4,667) : 293,	(5,667) : 190,	(6,667) : 343,	(7,667) : 726,	(8,667) : 510,	(9,667) : 515,	(10,667) : 826,	(11,667) : 408,	(12,667) : 127,	(13,667) : 344,	(14,667) : 807,	(15,667) : 843,	(16,667) : 311,	(17,667) : 690,	(18,667) : 121,	(19,667) : 552,	(20,667) : 600,
(1,668) : 553,	(2,668) : 273,	(3,668) : 344,	(4,668) : 548,	(5,668) : 363,	(6,668) : 652,	(7,668) : 419,	(8,668) : 518,	(9,668) : 178,	(10,668) : 275,	(11,668) : 283,	(12,668) : 601,	(13,668) : 590,	(14,668) : 365,	(15,668) : 326,	(16,668) : 529,	(17,668) : 150,	(18,668) : 449,	(19,668) : 45,	(20,668) : 96,
(1,669) : 301,	(2,669) : 555,	(3,669) : 595,	(4,669) : 324,	(5,669) : 573,	(6,669) : 336,	(7,669) : 361,	(8,669) : 121,	(9,669) : 450,	(10,669) : 805,	(11,669) : 364,	(12,669) : 523,	(13,669) : 289,	(14,669) : 543,	(15,669) : 678,	(16,669) : 302,	(17,669) : 753,	(18,669) : 630,	(19,669) : 586,	(20,669) : 722,
(1,670) : 715,	(2,670) : 347,	(3,670) : 463,	(4,670) : 709,	(5,670) : 488,	(6,670) : 814,	(7,670) : 542,	(8,670) : 681,	(9,670) : 337,	(10,670) : 190,	(11,670) : 448,	(12,670) : 743,	(13,670) : 753,	(14,670) : 438,	(15,670) : 338,	(16,670) : 692,	(17,670) : 58,	(18,670) : 563,	(19,670) : 202,	(20,670) : 80,
(1,671) : 177,	(2,671) : 480,	(3,671) : 420,	(4,671) : 196,	(5,671) : 398,	(6,671) : 257,	(7,671) : 344,	(8,671) : 57,	(9,671) : 317,	(10,671) : 697,	(11,671) : 203,	(12,671) : 386,	(13,671) : 189,	(14,671) : 497,	(15,671) : 606,	(16,671) : 166,	(17,671) : 621,	(18,671) : 460,	(19,671) : 447,	(20,671) : 576,
(1,672) : 697,	(2,672) : 348,	(3,672) : 443,	(4,672) : 691,	(5,672) : 468,	(6,672) : 796,	(7,672) : 537,	(8,672) : 668,	(9,672) : 325,	(10,672) : 206,	(11,672) : 433,	(12,672) : 724,	(13,672) : 735,	(14,672) : 439,	(15,672) : 346,	(16,672) : 674,	(17,672) : 66,	(18,672) : 543,	(19,672) : 189,	(20,672) : 61,
(1,673) : 741,	(2,673) : 165,	(3,673) : 604,	(4,673) : 743,	(5,673) : 620,	(6,673) : 842,	(7,673) : 388,	(8,673) : 633,	(9,673) : 321,	(10,673) : 110,	(11,673) : 460,	(12,673) : 837,	(13,673) : 773,	(14,673) : 229,	(15,673) : 93,	(16,673) : 719,	(17,673) : 191,	(18,673) : 709,	(19,673) : 254,	(20,673) : 270,
(1,674) : 598,	(2,674) : 382,	(3,674) : 320,	(4,674) : 589,	(5,674) : 346,	(6,674) : 693,	(7,674) : 531,	(8,674) : 599,	(9,674) : 278,	(10,674) : 317,	(11,674) : 354,	(12,674) : 605,	(13,674) : 637,	(14,674) : 475,	(15,674) : 417,	(16,674) : 575,	(17,674) : 170,	(18,674) : 419,	(19,674) : 157,	(20,674) : 76,
(1,675) : 472,	(2,675) : 387,	(3,675) : 209,	(4,675) : 462,	(5,675) : 231,	(6,675) : 566,	(7,675) : 484,	(8,675) : 491,	(9,675) : 212,	(10,675) : 408,	(11,675) : 245,	(12,675) : 482,	(13,675) : 511,	(14,675) : 474,	(15,675) : 458,	(16,675) : 449,	(17,675) : 270,	(18,675) : 314,	(19,675) : 153,	(20,675) : 183,
(1,676) : 375,	(2,676) : 672,	(3,676) : 112,	(4,676) : 353,	(5,676) : 102,	(6,676) : 427,	(7,676) : 692,	(8,676) : 530,	(9,676) : 452,	(10,676) : 725,	(11,676) : 371,	(12,676) : 238,	(13,676) : 409,	(14,676) : 746,	(15,676) : 763,	(16,676) : 362,	(17,676) : 584,	(18,676) : 13,	(19,676) : 462,	(20,676) : 492,
(1,677) : 153,	(2,677) : 774,	(3,677) : 466,	(4,677) : 152,	(5,677) : 433,	(6,677) : 55,	(7,677) : 656,	(8,677) : 339,	(9,677) : 573,	(10,677) : 954,	(11,677) : 435,	(12,677) : 219,	(13,677) : 126,	(14,677) : 803,	(15,677) : 897,	(16,677) : 175,	(17,677) : 849,	(18,677) : 431,	(19,677) : 676,	(20,677) : 779,
(1,678) : 348,	(2,678) : 371,	(3,678) : 198,	(4,678) : 341,	(5,678) : 201,	(6,678) : 446,	(7,678) : 403,	(8,678) : 359,	(9,678) : 148,	(10,678) : 476,	(11,678) : 116,	(12,678) : 401,	(13,678) : 386,	(14,678) : 443,	(15,678) : 471,	(16,678) : 325,	(17,678) : 359,	(18,678) : 294,	(19,678) : 192,	(20,678) : 289,
(1,679) : 335,	(2,679) : 489,	(3,679) : 79,	(4,679) : 321,	(5,679) : 82,	(6,679) : 421,	(7,679) : 518,	(8,679) : 416,	(9,679) : 269,	(10,679) : 563,	(11,679) : 206,	(12,679) : 318,	(13,679) : 375,	(14,679) : 563,	(15,679) : 583,	(16,679) : 314,	(17,679) : 431,	(18,679) : 175,	(19,679) : 287,	(20,679) : 346,
(1,680) : 251,	(2,680) : 510,	(3,680) : 526,	(4,680) : 273,	(5,680) : 504,	(6,680) : 304,	(7,680) : 333,	(8,680) : 50,	(9,680) : 386,	(10,680) : 751,	(11,680) : 294,	(12,680) : 471,	(13,680) : 246,	(14,680) : 508,	(15,680) : 634,	(16,680) : 247,	(17,680) : 691,	(18,680) : 564,	(19,680) : 521,	(20,680) : 656,
(1,681) : 309,	(2,681) : 530,	(3,681) : 63,	(4,681) : 292,	(5,681) : 47,	(6,681) : 389,	(7,681) : 546,	(8,681) : 414,	(9,681) : 307,	(10,681) : 610,	(11,681) : 226,	(12,681) : 273,	(13,681) : 348,	(14,681) : 601,	(15,681) : 627,	(16,681) : 290,	(17,681) : 478,	(18,681) : 137,	(19,681) : 334,	(20,681) : 392,
(1,682) : 381,	(2,682) : 702,	(3,682) : 142,	(4,682) : 358,	(5,682) : 131,	(6,682) : 426,	(7,682) : 717,	(8,682) : 545,	(9,682) : 480,	(10,682) : 756,	(11,682) : 396,	(12,682) : 226,	(13,682) : 413,	(14,682) : 775,	(15,682) : 793,	(16,682) : 369,	(17,682) : 614,	(18,682) : 38,	(19,682) : 492,	(20,682) : 522,
(1,683) : 337,	(2,683) : 300,	(3,683) : 431,	(4,683) : 348,	(5,683) : 421,	(6,683) : 429,	(7,683) : 187,	(8,683) : 187,	(9,683) : 178,	(10,683) : 532,	(11,683) : 149,	(12,683) : 509,	(13,683) : 358,	(14,683) : 319,	(15,683) : 426,	(16,683) : 319,	(17,683) : 476,	(18,683) : 505,	(19,683) : 313,	(20,683) : 451,
(1,684) : 521,	(2,684) : 502,	(3,684) : 182,	(4,684) : 506,	(5,684) : 213,	(6,684) : 605,	(7,684) : 603,	(8,684) : 581,	(9,684) : 331,	(10,684) : 480,	(11,684) : 342,	(12,684) : 477,	(13,684) : 560,	(14,684) : 591,	(15,684) : 561,	(16,684) : 500,	(17,684) : 334,	(18,684) : 269,	(19,684) : 264,	(20,684) : 239,
(1,685) : 25,	(2,685) : 646,	(3,685) : 376,	(4,685) : 30,	(5,685) : 345,	(6,685) : 77,	(7,685) : 540,	(8,685) : 239,	(9,685) : 445,	(10,685) : 827,	(11,685) : 307,	(12,685) : 211,	(13,685) : 30,	(14,685) : 679,	(15,685) : 769,	(16,685) : 47,	(17,685) : 725,	(18,685) : 367,	(19,685) : 551,	(20,685) : 659,
(1,686) : 705,	(2,686) : 384,	(3,686) : 434,	(4,686) : 698,	(5,686) : 461,	(6,686) : 802,	(7,686) : 569,	(8,686) : 686,	(9,686) : 347,	(10,686) : 238,	(11,686) : 448,	(12,686) : 720,	(13,686) : 744,	(14,686) : 476,	(15,686) : 383,	(16,686) : 682,	(17,686) : 103,	(18,686) : 531,	(19,686) : 211,	(20,686) : 74,
(1,687) : 687,	(2,687) : 93,	(3,687) : 654,	(4,687) : 695,	(5,687) : 661,	(6,687) : 782,	(7,687) : 225,	(8,687) : 530,	(9,687) : 313,	(10,687) : 308,	(11,687) : 431,	(12,687) : 830,	(13,687) : 711,	(14,687) : 41,	(15,687) : 117,	(16,687) : 667,	(17,687) : 363,	(18,687) : 754,	(19,687) : 331,	(20,687) : 414,
(1,688) : 660,	(2,688) : 515,	(3,688) : 327,	(4,688) : 647,	(5,688) : 358,	(6,688) : 748,	(7,688) : 662,	(8,688) : 698,	(9,688) : 400,	(10,688) : 414,	(11,688) : 452,	(12,688) : 622,	(13,688) : 700,	(14,688) : 609,	(15,688) : 542,	(16,688) : 638,	(17,688) : 273,	(18,688) : 407,	(19,688) : 290,	(20,688) : 192,
(1,689) : 546,	(2,689) : 113,	(3,689) : 452,	(4,689) : 549,	(5,689) : 461,	(6,689) : 647,	(7,689) : 261,	(8,689) : 444,	(9,689) : 129,	(10,689) : 273,	(11,689) : 266,	(12,689) : 654,	(13,689) : 577,	(14,689) : 200,	(15,689) : 209,	(16,689) : 524,	(17,689) : 234,	(18,689) : 554,	(19,689) : 131,	(20,689) : 244,
(1,690) : 67,	(2,690) : 566,	(3,690) : 378,	(4,690) : 87,	(5,690) : 351,	(6,690) : 155,	(7,690) : 450,	(8,690) : 153,	(9,690) : 376,	(10,690) : 761,	(11,690) : 242,	(12,690) : 283,	(13,690) : 84,	(14,690) : 593,	(15,690) : 690,	(16,690) : 59,	(17,690) : 668,	(18,690) : 394,	(19,690) : 492,	(20,690) : 610,
(1,691) : 364,	(2,691) : 563,	(3,691) : 0,	(4,691) : 346,	(5,691) : 32,	(6,691) : 438,	(7,691) : 597,	(8,691) : 477,	(9,691) : 346,	(10,691) : 617,	(11,691) : 282,	(12,691) : 295,	(13,691) : 403,	(14,691) : 639,	(15,691) : 652,	(16,691) : 346,	(17,691) : 478,	(18,691) : 105,	(19,691) : 350,	(20,691) : 387,
(1,692) : 427,	(2,692) : 197,	(3,692) : 442,	(4,692) : 435,	(5,692) : 440,	(6,692) : 524,	(7,692) : 159,	(8,692) : 293,	(9,692) : 116,	(10,692) : 428,	(11,692) : 180,	(12,692) : 575,	(13,692) : 453,	(14,692) : 230,	(15,692) : 322,	(16,692) : 407,	(17,692) : 383,	(18,692) : 531,	(19,692) : 235,	(20,692) : 372,
(1,693) : 579,	(2,693) : 114,	(3,693) : 466,	(4,693) : 581,	(5,693) : 477,	(6,693) : 680,	(7,693) : 291,	(8,693) : 482,	(9,693) : 159,	(10,693) : 236,	(11,693) : 298,	(12,693) : 680,	(13,693) : 611,	(14,693) : 207,	(15,693) : 187,	(16,693) : 557,	(17,693) : 201,	(18,693) : 569,	(19,693) : 127,	(20,693) : 222,
(1,694) : 215,	(2,694) : 453,	(3,694) : 440,	(4,694) : 233,	(5,694) : 421,	(6,694) : 294,	(7,694) : 308,	(8,694) : 43,	(9,694) : 304,	(10,694) : 679,	(11,694) : 203,	(12,694) : 422,	(13,694) : 226,	(14,694) : 466,	(15,694) : 579,	(16,694) : 204,	(17,694) : 610,	(18,694) : 486,	(19,694) : 437,	(20,694) : 570,
(1,695) : 544,	(2,695) : 277,	(3,695) : 336,	(4,695) : 540,	(5,695) : 355,	(6,695) : 644,	(7,695) : 418,	(8,695) : 511,	(9,695) : 173,	(10,695) : 283,	(11,695) : 275,	(12,695) : 592,	(13,695) : 582,	(14,695) : 368,	(15,695) : 332,	(16,695) : 521,	(17,695) : 159,	(18,695) : 441,	(19,695) : 44,	(20,695) : 101,
(1,696) : 303,	(2,696) : 481,	(3,696) : 556,	(4,696) : 325,	(5,696) : 537,	(6,696) : 359,	(7,696) : 291,	(8,696) : 87,	(9,696) : 381,	(10,696) : 731,	(11,696) : 306,	(12,696) : 522,	(13,696) : 301,	(14,696) : 470,	(15,696) : 604,	(16,696) : 298,	(17,696) : 681,	(18,696) : 602,	(19,696) : 517,	(20,696) : 653,
(1,697) : 625,	(2,697) : 252,	(3,697) : 419,	(4,697) : 621,	(5,697) : 439,	(6,697) : 725,	(7,697) : 433,	(8,697) : 574,	(9,697) : 229,	(10,697) : 198,	(11,697) : 349,	(12,697) : 679,	(13,697) : 661,	(14,697) : 345,	(15,697) : 275,	(16,697) : 602,	(17,697) : 78,	(18,697) : 524,	(19,697) : 98,	(20,697) : 79,
(1,698) : 12,	(2,698) : 625,	(3,698) : 377,	(4,698) : 35,	(5,698) : 347,	(6,698) : 95,	(7,698) : 515,	(8,698) : 215,	(9,698) : 427,	(10,698) : 810,	(11,698) : 289,	(12,698) : 231,	(13,698) : 29,	(14,698) : 656,	(15,698) : 749,	(16,698) : 33,	(17,698) : 710,	(18,698) : 375,	(19,698) : 536,	(20,698) : 646,
(1,699) : 92,	(2,699) : 686,	(3,699) : 341,	(4,699) : 72,	(5,699) : 309,	(6,699) : 102,	(7,699) : 598,	(8,699) : 311,	(9,699) : 474,	(10,699) : 847,	(11,699) : 336,	(12,699) : 135,	(13,699) : 104,	(14,699) : 727,	(15,699) : 806,	(16,699) : 101,	(17,699) : 735,	(18,699) : 314,	(19,699) : 565,	(20,699) : 661,
(1,700) : 595,	(2,700) : 458,	(3,700) : 279,	(4,700) : 583,	(5,700) : 309,	(6,700) : 686,	(7,700) : 594,	(8,700) : 625,	(9,700) : 329,	(10,700) : 392,	(11,700) : 379,	(12,700) : 573,	(13,700) : 635,	(14,700) : 551,	(15,700) : 498,	(16,700) : 573,	(17,700) : 246,	(18,700) : 370,	(19,700) : 225,	(20,700) : 153,
(1,701) : 137,	(2,701) : 491,	(3,701) : 288,	(4,701) : 138,	(5,701) : 265,	(6,701) : 238,	(7,701) : 414,	(8,701) : 192,	(9,701) : 284,	(10,701) : 666,	(11,701) : 146,	(12,701) : 282,	(13,701) : 172,	(14,701) : 533,	(15,701) : 612,	(16,701) : 114,	(17,701) : 566,	(18,701) : 325,	(19,701) : 391,	(20,701) : 505,
(1,702) : 506,	(2,702) : 610,	(3,702) : 141,	(4,702) : 487,	(5,702) : 171,	(6,702) : 576,	(7,702) : 687,	(8,702) : 610,	(9,702) : 419,	(10,702) : 600,	(11,702) : 393,	(12,702) : 412,	(13,702) : 544,	(14,702) : 696,	(15,702) : 678,	(16,702) : 487,	(17,702) : 454,	(18,702) : 184,	(19,702) : 375,	(20,702) : 359,
(1,703) : 327,	(2,703) : 595,	(3,703) : 54,	(4,703) : 307,	(5,703) : 27,	(6,703) : 394,	(7,703) : 610,	(8,703) : 460,	(9,703) : 373,	(10,703) : 664,	(11,703) : 290,	(12,703) : 241,	(13,703) : 364,	(14,703) : 667,	(15,703) : 690,	(16,703) : 310,	(17,703) : 528,	(18,703) : 71,	(19,703) : 393,	(20,703) : 438,
(1,704) : 570,	(2,704) : 192,	(3,704) : 409,	(4,704) : 569,	(5,704) : 424,	(6,704) : 671,	(7,704) : 356,	(8,704) : 502,	(9,704) : 158,	(10,704) : 234,	(11,704) : 289,	(12,704) : 646,	(13,704) : 605,	(14,704) : 285,	(15,704) : 248,	(16,704) : 547,	(17,704) : 150,	(18,704) : 514,	(19,704) : 58,	(20,704) : 147,
(1,705) : 237,	(2,705) : 776,	(3,705) : 318,	(4,705) : 214,	(5,705) : 288,	(6,705) : 227,	(7,705) : 719,	(8,705) : 453,	(9,705) : 552,	(10,705) : 900,	(11,705) : 424,	(12,705) : 26,	(13,705) : 250,	(14,705) : 829,	(15,705) : 889,	(16,705) : 241,	(17,705) : 774,	(18,705) : 249,	(19,705) : 618,	(20,705) : 690,
(1,706) : 250,	(2,706) : 518,	(3,706) : 530,	(4,706) : 272,	(5,706) : 508,	(6,706) : 300,	(7,706) : 341,	(8,706) : 55,	(9,706) : 394,	(10,706) : 759,	(11,706) : 300,	(12,706) : 471,	(13,706) : 244,	(14,706) : 515,	(15,706) : 642,	(16,706) : 247,	(17,706) : 699,	(18,706) : 567,	(19,706) : 529,	(20,706) : 663,
(1,707) : 244,	(2,707) : 412,	(3,707) : 432,	(4,707) : 260,	(5,707) : 415,	(6,707) : 329,	(7,707) : 273,	(8,707) : 79,	(9,707) : 268,	(10,707) : 639,	(11,707) : 178,	(12,707) : 442,	(13,707) : 259,	(14,707) : 426,	(15,707) : 538,	(16,707) : 230,	(17,707) : 573,	(18,707) : 487,	(19,707) : 402,	(20,707) : 536,
(1,708) : 76,	(2,708) : 695,	(3,708) : 390,	(4,708) : 69,	(5,708) : 359,	(6,708) : 48,	(7,708) : 591,	(8,708) : 287,	(9,708) : 491,	(10,708) : 871,	(11,708) : 352,	(12,708) : 184,	(13,708) : 66,	(14,708) : 730,	(15,708) : 818,	(16,708) : 95,	(17,708) : 764,	(18,708) : 368,	(19,708) : 592,	(20,708) : 695,
(1,709) : 29,	(2,709) : 647,	(3,709) : 367,	(4,709) : 24,	(5,709) : 337,	(6,709) : 81,	(7,709) : 544,	(8,709) : 246,	(9,709) : 445,	(10,709) : 825,	(11,709) : 306,	(12,709) : 201,	(13,709) : 40,	(14,709) : 682,	(15,709) : 770,	(16,709) : 47,	(17,709) : 722,	(18,709) : 358,	(19,709) : 548,	(20,709) : 655,
(1,710) : 412,	(2,710) : 477,	(3,710) : 103,	(4,710) : 398,	(5,710) : 127,	(6,710) : 499,	(7,710) : 539,	(8,710) : 478,	(9,710) : 273,	(10,710) : 514,	(11,710) : 249,	(12,710) : 386,	(13,710) : 452,	(14,710) : 558,	(15,710) : 557,	(16,710) : 391,	(17,710) : 375,	(18,710) : 207,	(19,710) : 253,	(20,710) : 285,
(1,711) : 592,	(2,711) : 33,	(3,711) : 549,	(4,711) : 599,	(5,711) : 555,	(6,711) : 690,	(7,711) : 193,	(8,711) : 453,	(9,711) : 207,	(10,711) : 301,	(11,711) : 328,	(12,711) : 727,	(13,711) : 619,	(14,711) : 93,	(15,711) : 158,	(16,711) : 572,	(17,711) : 310,	(18,711) : 648,	(19,711) : 238,	(20,711) : 341,
(1,712) : 80,	(2,712) : 573,	(3,712) : 287,	(4,712) : 69,	(5,712) : 258,	(6,712) : 173,	(7,712) : 493,	(8,712) : 237,	(9,712) : 361,	(10,712) : 737,	(11,712) : 223,	(12,712) : 206,	(13,712) : 120,	(14,712) : 615,	(15,712) : 692,	(16,712) : 59,	(17,712) : 630,	(18,712) : 296,	(19,712) : 458,	(20,712) : 562,
(1,713) : 685,	(2,713) : 393,	(3,713) : 407,	(4,713) : 677,	(5,713) : 434,	(6,713) : 782,	(7,713) : 571,	(8,713) : 674,	(9,713) : 339,	(10,713) : 263,	(11,713) : 433,	(12,713) : 694,	(13,713) : 724,	(14,713) : 486,	(15,713) : 401,	(16,713) : 662,	(17,713) : 123,	(18,713) : 503,	(19,713) : 205,	(20,713) : 68,
(1,714) : 624,	(2,714) : 185,	(3,714) : 457,	(4,714) : 623,	(5,714) : 473,	(6,714) : 725,	(7,714) : 377,	(8,714) : 550,	(9,714) : 208,	(10,714) : 179,	(11,714) : 342,	(12,714) : 699,	(13,714) : 658,	(14,714) : 278,	(15,714) : 212,	(16,714) : 601,	(17,714) : 115,	(18,714) : 562,	(19,714) : 109,	(20,714) : 147,
(1,715) : 589,	(2,715) : 148,	(3,715) : 450,	(4,715) : 589,	(5,715) : 464,	(6,715) : 690,	(7,715) : 328,	(8,715) : 504,	(9,715) : 169,	(10,715) : 216,	(11,715) : 306,	(12,715) : 677,	(13,715) : 622,	(14,715) : 242,	(15,715) : 203,	(16,715) : 566,	(17,715) : 165,	(18,715) : 555,	(19,715) : 102,	(20,715) : 184,
(1,716) : 257,	(2,716) : 505,	(3,716) : 529,	(4,716) : 279,	(5,716) : 508,	(6,716) : 311,	(7,716) : 328,	(8,716) : 52,	(9,716) : 385,	(10,716) : 748,	(11,716) : 294,	(12,716) : 477,	(13,716) : 253,	(14,716) : 502,	(15,716) : 630,	(16,716) : 254,	(17,716) : 689,	(18,716) : 568,	(19,716) : 520,	(20,716) : 655,
(1,717) : 421,	(2,717) : 284,	(3,717) : 538,	(4,717) : 436,	(5,717) : 530,	(6,717) : 505,	(7,717) : 97,	(8,717) : 229,	(9,717) : 250,	(10,717) : 545,	(11,717) : 257,	(12,717) : 609,	(13,717) : 436,	(14,717) : 270,	(15,717) : 404,	(16,717) : 406,	(17,717) : 518,	(18,717) : 614,	(19,717) : 375,	(20,717) : 511,
(1,718) : 702,	(2,718) : 292,	(3,718) : 478,	(4,718) : 698,	(5,718) : 501,	(6,718) : 802,	(7,718) : 492,	(8,718) : 651,	(9,718) : 305,	(10,718) : 147,	(11,718) : 427,	(12,718) : 748,	(13,718) : 739,	(14,718) : 381,	(15,718) : 280,	(16,718) : 679,	(17,718) : 0,	(18,718) : 581,	(19,718) : 176,	(20,718) : 95,
(1,719) : 331,	(2,719) : 897,	(3,719) : 424,	(4,719) : 312,	(5,719) : 396,	(6,719) : 288,	(7,719) : 833,	(8,719) : 550,	(9,719) : 673,	(10,719) : 1019,	(11,719) : 544,	(12,719) : 147,	(13,719) : 333,	(14,719) : 948,	(15,719) : 1009,	(16,719) : 341,	(17,719) : 890,	(18,719) : 340,	(19,719) : 737,	(20,719) : 804,
(1,720) : 289,	(2,720) : 629,	(3,720) : 618,	(4,720) : 313,	(5,720) : 592,	(6,720) : 300,	(7,720) : 439,	(8,720) : 166,	(9,720) : 511,	(10,720) : 874,	(11,720) : 411,	(12,720) : 506,	(13,720) : 267,	(14,720) : 619,	(15,720) : 753,	(16,720) : 296,	(17,720) : 815,	(18,720) : 640,	(19,720) : 645,	(20,720) : 779,
(1,721) : 667,	(2,721) : 153,	(3,721) : 518,	(4,721) : 668,	(5,721) : 534,	(6,721) : 769,	(7,721) : 366,	(8,721) : 575,	(9,721) : 247,	(10,721) : 142,	(11,721) : 385,	(12,721) : 755,	(13,721) : 700,	(14,721) : 240,	(15,721) : 152,	(16,721) : 645,	(17,721) : 141,	(18,721) : 624,	(19,721) : 169,	(20,721) : 199,
(1,722) : 486,	(2,722) : 160,	(3,722) : 405,	(4,722) : 489,	(5,722) : 411,	(6,722) : 587,	(7,722) : 252,	(8,722) : 392,	(9,722) : 70,	(10,722) : 327,	(11,722) : 206,	(12,722) : 596,	(13,722) : 518,	(14,722) : 235,	(15,722) : 267,	(16,722) : 464,	(17,722) : 267,	(18,722) : 505,	(19,722) : 123,	(20,722) : 256,
(1,723) : 130,	(2,723) : 540,	(3,723) : 427,	(4,723) : 152,	(5,723) : 402,	(6,723) : 199,	(7,723) : 404,	(8,723) : 90,	(9,723) : 370,	(10,723) : 753,	(11,723) : 246,	(12,723) : 349,	(13,723) : 133,	(14,723) : 559,	(15,723) : 666,	(16,723) : 125,	(17,723) : 672,	(18,723) : 453,	(19,723) : 496,	(20,723) : 621,
(1,724) : 259,	(2,724) : 686,	(3,724) : 189,	(4,724) : 235,	(5,724) : 159,	(6,724) : 297,	(7,724) : 661,	(8,724) : 444,	(9,724) : 459,	(10,724) : 785,	(11,724) : 346,	(12,724) : 108,	(13,724) : 288,	(14,724) : 748,	(15,724) : 791,	(16,724) : 250,	(17,724) : 654,	(18,724) : 125,	(19,724) : 506,	(20,724) : 567,
(1,725) : 357,	(2,725) : 818,	(3,725) : 288,	(4,725) : 333,	(5,725) : 265,	(6,725) : 362,	(7,725) : 795,	(8,725) : 562,	(9,725) : 591,	(10,725) : 903,	(11,725) : 481,	(12,725) : 139,	(13,725) : 377,	(14,725) : 882,	(15,725) : 920,	(16,725) : 355,	(17,725) : 765,	(18,725) : 189,	(19,725) : 629,	(20,725) : 675,
(1,726) : 179,	(2,726) : 492,	(3,726) : 436,	(4,726) : 199,	(5,726) : 414,	(6,726) : 254,	(7,726) : 349,	(8,726) : 44,	(9,726) : 334,	(10,726) : 713,	(11,726) : 222,	(12,726) : 392,	(13,726) : 186,	(14,726) : 506,	(15,726) : 618,	(16,726) : 170,	(17,726) : 638,	(18,726) : 474,	(19,726) : 464,	(20,726) : 594,
(1,727) : 240,	(2,727) : 541,	(3,727) : 126,	(4,727) : 223,	(5,727) : 99,	(6,727) : 320,	(7,727) : 528,	(8,727) : 359,	(9,727) : 314,	(10,727) : 650,	(11,727) : 207,	(12,727) : 224,	(13,727) : 279,	(14,727) : 604,	(15,727) : 647,	(16,727) : 220,	(17,727) : 525,	(18,727) : 162,	(19,727) : 368,	(20,727) : 444,
(1,728) : 434,	(2,728) : 595,	(3,728) : 72,	(4,728) : 415,	(5,728) : 100,	(6,728) : 505,	(7,728) : 649,	(8,728) : 547,	(9,728) : 388,	(10,728) : 617,	(11,728) : 342,	(12,728) : 344,	(13,728) : 472,	(14,728) : 676,	(15,728) : 673,	(16,728) : 416,	(17,728) : 473,	(18,728) : 122,	(19,728) : 368,	(20,728) : 379,
(1,729) : 99,	(2,729) : 673,	(3,729) : 318,	(4,729) : 75,	(5,729) : 286,	(6,729) : 124,	(7,729) : 593,	(8,729) : 315,	(9,729) : 458,	(10,729) : 829,	(11,729) : 322,	(12,729) : 124,	(13,729) : 118,	(14,729) : 717,	(15,729) : 792,	(16,729) : 102,	(17,729) : 715,	(18,729) : 292,	(19,729) : 547,	(20,729) : 641,
(1,730) : 336,	(2,730) : 345,	(3,730) : 487,	(4,730) : 351,	(5,730) : 474,	(6,730) : 419,	(7,730) : 183,	(8,730) : 146,	(9,730) : 250,	(10,730) : 591,	(11,730) : 209,	(12,730) : 529,	(13,730) : 350,	(14,730) : 346,	(15,730) : 470,	(16,730) : 321,	(17,730) : 544,	(18,730) : 553,	(19,730) : 384,	(20,730) : 522,
(1,731) : 115,	(2,731) : 508,	(3,731) : 349,	(4,731) : 127,	(5,731) : 324,	(6,731) : 211,	(7,731) : 403,	(8,731) : 139,	(9,731) : 316,	(10,731) : 701,	(11,731) : 183,	(12,731) : 306,	(13,731) : 141,	(14,731) : 539,	(15,731) : 632,	(16,731) : 97,	(17,731) : 610,	(18,731) : 380,	(19,731) : 434,	(20,731) : 554,
(1,732) : 214,	(2,732) : 788,	(3,732) : 360,	(4,732) : 194,	(5,732) : 329,	(6,732) : 185,	(7,732) : 716,	(8,732) : 434,	(9,732) : 568,	(10,732) : 926,	(11,732) : 435,	(12,732) : 68,	(13,732) : 220,	(14,732) : 835,	(15,732) : 904,	(16,732) : 223,	(17,732) : 804,	(18,732) : 298,	(19,732) : 643,	(20,732) : 722,
(1,733) : 151,	(2,733) : 511,	(3,733) : 237,	(4,733) : 143,	(5,733) : 212,	(6,733) : 248,	(7,733) : 454,	(8,733) : 247,	(9,733) : 293,	(10,733) : 665,	(11,733) : 158,	(12,733) : 245,	(13,733) : 190,	(14,733) : 560,	(15,733) : 628,	(16,733) : 128,	(17,733) : 556,	(18,733) : 270,	(19,733) : 384,	(20,733) : 488,
(1,734) : 357,	(2,734) : 353,	(3,734) : 520,	(4,734) : 374,	(5,734) : 507,	(6,734) : 436,	(7,734) : 172,	(8,734) : 155,	(9,734) : 277,	(10,734) : 606,	(11,734) : 243,	(12,734) : 556,	(13,734) : 368,	(14,734) : 345,	(15,734) : 476,	(16,734) : 344,	(17,734) : 566,	(18,734) : 586,	(19,734) : 410,	(20,734) : 548,
(1,735) : 158,	(2,735) : 506,	(3,735) : 232,	(4,735) : 150,	(5,735) : 208,	(6,735) : 255,	(7,735) : 452,	(8,735) : 250,	(9,735) : 287,	(10,735) : 658,	(11,735) : 152,	(12,735) : 249,	(13,735) : 198,	(14,735) : 555,	(15,735) : 622,	(16,735) : 135,	(17,735) : 549,	(18,735) : 268,	(19,735) : 378,	(20,735) : 480,
(1,736) : 211,	(2,736) : 523,	(3,736) : 161,	(4,736) : 197,	(5,736) : 136,	(6,736) : 298,	(7,736) : 497,	(8,736) : 321,	(9,736) : 297,	(10,736) : 647,	(11,736) : 179,	(12,736) : 231,	(13,736) : 251,	(14,736) : 582,	(15,736) : 632,	(16,736) : 191,	(17,736) : 527,	(18,736) : 199,	(19,736) : 363,	(20,736) : 450,
(1,737) : 184,	(2,737) : 603,	(3,737) : 189,	(4,737) : 163,	(5,737) : 157,	(6,737) : 250,	(7,737) : 564,	(8,737) : 348,	(9,737) : 378,	(10,737) : 727,	(11,737) : 254,	(12,737) : 150,	(13,737) : 220,	(14,737) : 659,	(15,737) : 713,	(16,737) : 170,	(17,737) : 604,	(18,737) : 180,	(19,737) : 444,	(20,737) : 524,
(1,738) : 563,	(2,738) : 308,	(3,738) : 330,	(4,738) : 556,	(5,738) : 352,	(6,738) : 661,	(7,738) : 454,	(8,738) : 540,	(9,738) : 207,	(10,738) : 286,	(11,738) : 301,	(12,738) : 598,	(13,738) : 601,	(14,738) : 401,	(15,738) : 355,	(16,738) : 539,	(17,738) : 150,	(18,738) : 435,	(19,738) : 80,	(20,738) : 75,
(1,739) : 213,	(2,739) : 779,	(3,739) : 346,	(4,739) : 192,	(5,739) : 315,	(6,739) : 191,	(7,739) : 710,	(8,739) : 432,	(9,739) : 558,	(10,739) : 914,	(11,739) : 426,	(12,739) : 53,	(13,739) : 221,	(14,739) : 827,	(15,739) : 894,	(16,739) : 220,	(17,739) : 791,	(18,739) : 283,	(19,739) : 631,	(20,739) : 709,
(1,740) : 81,	(2,740) : 577,	(3,740) : 285,	(4,740) : 68,	(5,740) : 256,	(6,740) : 172,	(7,740) : 499,	(8,740) : 243,	(9,740) : 365,	(10,740) : 741,	(11,740) : 227,	(12,740) : 200,	(13,740) : 120,	(14,740) : 620,	(15,740) : 697,	(16,740) : 61,	(17,740) : 633,	(18,740) : 292,	(19,740) : 461,	(20,740) : 564,
(1,741) : 612,	(2,741) : 479,	(3,741) : 288,	(4,741) : 600,	(5,741) : 319,	(6,741) : 702,	(7,741) : 617,	(8,741) : 647,	(9,741) : 352,	(10,741) : 404,	(11,741) : 401,	(12,741) : 583,	(13,741) : 652,	(14,741) : 572,	(15,741) : 516,	(16,741) : 590,	(17,741) : 258,	(18,741) : 375,	(19,741) : 248,	(20,741) : 168,
(1,742) : 519,	(2,742) : 381,	(3,742) : 249,	(4,742) : 510,	(5,742) : 273,	(6,742) : 614,	(7,742) : 499,	(8,742) : 531,	(9,742) : 230,	(10,742) : 371,	(11,742) : 284,	(12,742) : 528,	(13,742) : 558,	(14,742) : 471,	(15,742) : 439,	(16,742) : 496,	(17,742) : 230,	(18,742) : 351,	(19,742) : 142,	(20,742) : 139,
(1,743) : 664,	(2,743) : 489,	(3,743) : 341,	(4,743) : 652,	(5,743) : 371,	(6,743) : 754,	(7,743) : 642,	(8,743) : 692,	(9,743) : 385,	(10,743) : 383,	(11,743) : 445,	(12,743) : 636,	(13,743) : 704,	(14,743) : 583,	(15,743) : 513,	(16,743) : 642,	(17,743) : 242,	(18,743) : 425,	(19,743) : 268,	(20,743) : 163,
(1,744) : 359,	(2,744) : 845,	(3,744) : 323,	(4,744) : 335,	(5,744) : 299,	(6,744) : 353,	(7,744) : 814,	(8,744) : 570,	(9,744) : 618,	(10,744) : 936,	(11,744) : 503,	(12,744) : 137,	(13,744) : 375,	(14,744) : 907,	(15,744) : 949,	(16,744) : 360,	(17,744) : 800,	(18,744) : 225,	(19,744) : 661,	(20,744) : 710,
(1,745) : 242,	(2,745) : 438,	(3,745) : 199,	(4,745) : 235,	(5,745) : 185,	(6,745) : 340,	(7,745) : 417,	(8,745) : 285,	(9,745) : 213,	(10,745) : 575,	(11,745) : 95,	(12,745) : 313,	(13,745) : 281,	(14,745) : 496,	(15,745) : 549,	(16,745) : 219,	(17,745) : 464,	(18,745) : 267,	(19,745) : 293,	(20,745) : 395,
(1,746) : 614,	(2,746) : 292,	(3,746) : 386,	(4,746) : 609,	(5,746) : 408,	(6,746) : 714,	(7,746) : 462,	(8,746) : 580,	(9,746) : 237,	(10,746) : 232,	(11,746) : 346,	(12,746) : 654,	(13,746) : 652,	(14,746) : 386,	(15,746) : 320,	(16,746) : 591,	(17,746) : 94,	(18,746) : 490,	(19,746) : 101,	(20,746) : 41,
(1,747) : 291,	(2,747) : 649,	(3,747) : 627,	(4,747) : 315,	(5,747) : 601,	(6,747) : 295,	(7,747) : 459,	(8,747) : 183,	(9,747) : 528,	(10,747) : 894,	(11,747) : 427,	(12,747) : 506,	(13,747) : 267,	(14,747) : 640,	(15,747) : 773,	(16,747) : 299,	(17,747) : 834,	(18,747) : 646,	(19,747) : 663,	(20,747) : 796,
(1,748) : 212,	(2,748) : 524,	(3,748) : 160,	(4,748) : 197,	(5,748) : 135,	(6,748) : 298,	(7,748) : 499,	(8,748) : 323,	(9,748) : 298,	(10,748) : 648,	(11,748) : 181,	(12,748) : 230,	(13,748) : 251,	(14,748) : 584,	(15,748) : 634,	(16,748) : 191,	(17,748) : 527,	(18,748) : 197,	(19,748) : 364,	(20,748) : 450,
(1,749) : 73,	(2,749) : 638,	(3,749) : 435,	(4,749) : 96,	(5,749) : 405,	(6,749) : 96,	(7,749) : 508,	(8,749) : 190,	(9,749) : 453,	(10,749) : 838,	(11,749) : 319,	(12,749) : 283,	(13,749) : 44,	(14,749) : 661,	(15,749) : 763,	(16,749) : 89,	(17,749) : 746,	(18,749) : 436,	(19,749) : 570,	(20,749) : 686,
(1,750) : 736,	(2,750) : 435,	(3,750) : 445,	(4,750) : 727,	(5,750) : 474,	(6,750) : 831,	(7,750) : 620,	(8,750) : 728,	(9,750) : 392,	(10,750) : 273,	(11,750) : 486,	(12,750) : 737,	(13,750) : 775,	(14,750) : 526,	(15,750) : 430,	(16,750) : 713,	(17,750) : 150,	(18,750) : 538,	(19,750) : 258,	(20,750) : 120,
(1,751) : 285,	(2,751) : 336,	(3,751) : 362,	(4,751) : 293,	(5,751) : 351,	(6,751) : 383,	(7,751) : 257,	(8,751) : 185,	(9,751) : 162,	(10,751) : 542,	(11,751) : 83,	(12,751) : 442,	(13,751) : 312,	(14,751) : 371,	(15,751) : 461,	(16,751) : 265,	(17,751) : 467,	(18,751) : 434,	(19,751) : 295,	(20,751) : 428,
(1,752) : 748,	(2,752) : 435,	(3,752) : 459,	(4,752) : 739,	(5,752) : 488,	(6,752) : 843,	(7,752) : 623,	(8,752) : 737,	(9,752) : 400,	(10,752) : 264,	(11,752) : 496,	(12,752) : 750,	(13,752) : 787,	(14,752) : 526,	(15,752) : 426,	(16,752) : 725,	(17,752) : 147,	(18,752) : 552,	(19,752) : 265,	(20,752) : 127,
(1,753) : 204,	(2,753) : 606,	(3,753) : 537,	(4,753) : 228,	(5,753) : 511,	(6,753) : 226,	(7,753) : 436,	(8,753) : 121,	(9,753) : 461,	(10,753) : 837,	(11,753) : 349,	(12,753) : 423,	(13,753) : 185,	(14,753) : 609,	(15,753) : 731,	(16,753) : 210,	(17,753) : 766,	(18,753) : 556,	(19,753) : 592,	(20,753) : 722,
(1,754) : 681,	(2,754) : 260,	(3,754) : 472,	(4,754) : 678,	(5,754) : 493,	(6,754) : 781,	(7,754) : 459,	(8,754) : 623,	(9,754) : 278,	(10,754) : 145,	(11,754) : 404,	(12,754) : 735,	(13,754) : 717,	(14,754) : 350,	(15,754) : 256,	(16,754) : 658,	(17,754) : 33,	(18,754) : 576,	(19,754) : 153,	(20,754) : 102,
(1,755) : 521,	(2,755) : 171,	(3,755) : 398,	(4,755) : 521,	(5,755) : 408,	(6,755) : 622,	(7,755) : 302,	(8,755) : 442,	(9,755) : 101,	(10,755) : 284,	(11,755) : 238,	(12,755) : 613,	(13,755) : 554,	(14,755) : 258,	(15,755) : 256,	(16,755) : 498,	(17,755) : 213,	(18,755) : 501,	(19,755) : 75,	(20,755) : 202,
(1,756) : 321,	(2,756) : 826,	(3,756) : 321,	(4,756) : 297,	(5,756) : 295,	(6,756) : 312,	(7,756) : 786,	(8,756) : 534,	(9,756) : 599,	(10,756) : 928,	(11,756) : 480,	(12,756) : 99,	(13,756) : 336,	(14,756) : 884,	(15,756) : 933,	(16,756) : 322,	(17,756) : 794,	(18,756) : 230,	(19,756) : 649,	(20,756) : 706,
(1,757) : 147,	(2,757) : 690,	(3,757) : 289,	(4,757) : 123,	(5,757) : 257,	(6,757) : 168,	(7,757) : 624,	(8,757) : 359,	(9,757) : 470,	(10,757) : 831,	(11,757) : 337,	(12,757) : 77,	(13,757) : 168,	(14,757) : 739,	(15,757) : 806,	(16,757) : 147,	(17,757) : 712,	(18,757) : 250,	(19,757) : 548,	(20,757) : 633,
(1,758) : 333,	(2,758) : 703,	(3,758) : 679,	(4,758) : 357,	(5,758) : 652,	(6,758) : 323,	(7,758) : 508,	(8,758) : 242,	(9,758) : 587,	(10,758) : 951,	(11,758) : 485,	(12,758) : 541,	(13,758) : 305,	(14,758) : 690,	(15,758) : 826,	(16,758) : 344,	(17,758) : 892,	(18,758) : 693,	(19,758) : 721,	(20,758) : 854,
(1,759) : 346,	(2,759) : 861,	(3,759) : 354,	(4,759) : 323,	(5,759) : 328,	(6,759) : 329,	(7,759) : 819,	(8,759) : 562,	(9,759) : 635,	(10,759) : 962,	(11,759) : 514,	(12,759) : 127,	(13,759) : 358,	(14,759) : 919,	(15,759) : 968,	(16,759) : 350,	(17,759) : 829,	(18,759) : 260,	(19,759) : 684,	(20,759) : 740,
(1,760) : 724,	(2,760) : 125,	(3,760) : 688,	(4,760) : 733,	(5,760) : 695,	(6,760) : 820,	(7,760) : 256,	(8,760) : 566,	(9,760) : 349,	(10,760) : 306,	(11,760) : 468,	(12,760) : 867,	(13,760) : 748,	(14,760) : 74,	(15,760) : 109,	(16,760) : 705,	(17,760) : 376,	(18,760) : 788,	(19,760) : 359,	(20,760) : 433,
(1,761) : 440,	(2,761) : 484,	(3,761) : 116,	(4,761) : 426,	(5,761) : 144,	(6,761) : 526,	(7,761) : 557,	(8,761) : 506,	(9,761) : 288,	(10,761) : 506,	(11,761) : 274,	(12,761) : 407,	(13,761) : 480,	(14,761) : 568,	(15,761) : 560,	(16,761) : 419,	(17,761) : 365,	(18,761) : 217,	(19,761) : 255,	(20,761) : 272,
(1,762) : 588,	(2,762) : 223,	(3,762) : 403,	(4,762) : 586,	(5,762) : 421,	(6,762) : 689,	(7,762) : 393,	(8,762) : 531,	(9,762) : 186,	(10,762) : 222,	(11,762) : 310,	(12,762) : 652,	(13,762) : 624,	(14,762) : 317,	(15,762) : 265,	(16,762) : 565,	(17,762) : 120,	(18,762) : 509,	(19,762) : 62,	(20,762) : 111,
(1,763) : 155,	(2,763) : 738,	(3,763) : 347,	(4,763) : 135,	(5,763) : 315,	(6,763) : 137,	(7,763) : 659,	(8,763) : 374,	(9,763) : 521,	(10,763) : 888,	(11,763) : 386,	(12,763) : 87,	(13,763) : 162,	(14,763) : 783,	(15,763) : 856,	(16,763) : 164,	(17,763) : 770,	(18,763) : 301,	(19,763) : 604,	(20,763) : 692,
(1,764) : 675,	(2,764) : 96,	(3,764) : 651,	(4,764) : 684,	(5,764) : 657,	(6,764) : 770,	(7,764) : 208,	(8,764) : 515,	(9,764) : 308,	(10,764) : 324,	(11,764) : 423,	(12,764) : 822,	(13,764) : 698,	(14,764) : 23,	(15,764) : 135,	(16,764) : 656,	(17,764) : 375,	(18,764) : 750,	(19,764) : 334,	(20,764) : 422,
(1,765) : 236,	(2,765) : 420,	(3,765) : 230,	(4,765) : 232,	(5,765) : 216,	(6,765) : 336,	(7,765) : 388,	(8,765) : 257,	(9,765) : 199,	(10,765) : 571,	(11,765) : 70,	(12,765) : 330,	(13,765) : 273,	(14,765) : 475,	(15,765) : 535,	(16,765) : 213,	(17,765) : 466,	(18,765) : 297,	(19,765) : 292,	(20,765) : 402,
(1,766) : 147,	(2,766) : 636,	(3,766) : 500,	(4,766) : 171,	(5,766) : 472,	(6,766) : 157,	(7,766) : 483,	(8,766) : 158,	(9,766) : 472,	(10,766) : 854,	(11,766) : 347,	(12,766) : 360,	(13,766) : 121,	(14,766) : 648,	(15,766) : 762,	(16,766) : 159,	(17,766) : 773,	(18,766) : 508,	(19,766) : 597,	(20,766) : 721,
(1,767) : 209,	(2,767) : 779,	(3,767) : 568,	(4,767) : 223,	(5,767) : 537,	(6,767) : 134,	(7,767) : 628,	(8,767) : 303,	(9,767) : 604,	(10,767) : 989,	(11,767) : 472,	(12,767) : 352,	(13,767) : 169,	(14,767) : 793,	(15,767) : 905,	(16,767) : 231,	(17,767) : 899,	(18,767) : 549,	(19,767) : 723,	(20,767) : 840,
(1,768) : 663,	(2,768) : 69,	(3,768) : 567,	(4,768) : 667,	(5,768) : 578,	(6,768) : 763,	(7,768) : 292,	(8,768) : 542,	(9,768) : 251,	(10,768) : 202,	(11,768) : 387,	(12,768) : 777,	(13,768) : 693,	(14,768) : 147,	(15,768) : 86,	(16,768) : 642,	(17,768) : 236,	(18,768) : 671,	(19,768) : 223,	(20,768) : 289,
(1,769) : 318,	(2,769) : 626,	(3,769) : 640,	(4,769) : 342,	(5,769) : 616,	(6,769) : 332,	(7,769) : 429,	(8,769) : 179,	(9,769) : 518,	(10,769) : 876,	(11,769) : 425,	(12,769) : 536,	(13,769) : 297,	(14,769) : 612,	(15,769) : 748,	(16,769) : 324,	(17,769) : 822,	(18,769) : 666,	(19,769) : 653,	(20,769) : 788,
(1,770) : 353,	(2,770) : 786,	(3,770) : 250,	(4,770) : 329,	(5,770) : 229,	(6,770) : 369,	(7,770) : 771,	(8,770) : 551,	(9,770) : 560,	(10,770) : 866,	(11,770) : 454,	(12,770) : 146,	(13,770) : 377,	(14,770) : 852,	(15,770) : 887,	(16,770) : 349,	(17,770) : 728,	(18,770) : 150,	(19,770) : 593,	(20,770) : 637,
(1,771) : 250,	(2,771) : 740,	(3,771) : 259,	(4,771) : 226,	(5,771) : 229,	(6,771) : 264,	(7,771) : 700,	(8,771) : 456,	(9,771) : 515,	(10,771) : 851,	(11,771) : 393,	(12,771) : 46,	(13,771) : 272,	(14,771) : 798,	(15,771) : 849,	(16,771) : 248,	(17,771) : 721,	(18,771) : 186,	(19,771) : 570,	(20,771) : 635,
(1,772) : 246,	(2,772) : 455,	(3,772) : 175,	(4,772) : 237,	(5,772) : 160,	(6,772) : 342,	(7,772) : 440,	(8,772) : 306,	(9,772) : 229,	(10,772) : 583,	(11,772) : 119,	(12,772) : 299,	(13,772) : 286,	(14,772) : 516,	(15,772) : 564,	(16,772) : 223,	(17,772) : 467,	(18,772) : 242,	(19,772) : 300,	(20,772) : 395,
(1,773) : 164,	(2,773) : 720,	(3,773) : 528,	(4,773) : 182,	(5,773) : 498,	(6,773) : 115,	(7,773) : 571,	(8,773) : 245,	(9,773) : 546,	(10,773) : 931,	(11,773) : 415,	(12,773) : 338,	(13,773) : 125,	(14,773) : 735,	(15,773) : 846,	(16,773) : 184,	(17,773) : 843,	(18,773) : 519,	(19,773) : 666,	(20,773) : 785,
(1,774) : 265,	(2,774) : 616,	(3,774) : 593,	(4,774) : 290,	(5,774) : 568,	(6,774) : 281,	(7,774) : 431,	(8,774) : 146,	(9,774) : 491,	(10,774) : 858,	(11,774) : 389,	(12,774) : 484,	(13,774) : 245,	(14,774) : 610,	(15,774) : 740,	(16,774) : 272,	(17,774) : 797,	(18,774) : 615,	(19,774) : 625,	(20,774) : 758,
(1,775) : 557,	(2,775) : 211,	(3,775) : 388,	(4,775) : 555,	(5,775) : 404,	(6,775) : 658,	(7,775) : 366,	(8,775) : 497,	(9,775) : 151,	(10,775) : 249,	(11,775) : 277,	(12,775) : 628,	(13,775) : 593,	(14,775) : 303,	(15,775) : 269,	(16,775) : 534,	(17,775) : 154,	(18,775) : 493,	(19,775) : 38,	(20,775) : 137,
(1,776) : 488,	(2,776) : 156,	(3,776) : 511,	(4,776) : 497,	(5,776) : 510,	(6,776) : 582,	(7,776) : 104,	(8,776) : 332,	(9,776) : 173,	(10,776) : 414,	(11,776) : 250,	(12,776) : 644,	(13,776) : 511,	(14,776) : 166,	(15,776) : 280,	(16,776) : 469,	(17,776) : 395,	(18,776) : 601,	(19,776) : 270,	(20,776) : 400,
(1,777) : 755,	(2,777) : 371,	(3,777) : 498,	(4,777) : 749,	(5,777) : 524,	(6,777) : 854,	(7,777) : 572,	(8,777) : 720,	(9,777) : 375,	(10,777) : 182,	(11,777) : 489,	(12,777) : 781,	(13,777) : 793,	(14,777) : 459,	(15,777) : 349,	(16,777) : 732,	(17,777) : 81,	(18,777) : 597,	(19,777) : 241,	(20,777) : 119,
(1,778) : 137,	(2,778) : 683,	(3,778) : 291,	(4,778) : 113,	(5,778) : 258,	(6,778) : 161,	(7,778) : 615,	(8,778) : 349,	(9,778) : 464,	(10,778) : 827,	(11,778) : 330,	(12,778) : 87,	(13,778) : 159,	(14,778) : 731,	(15,778) : 800,	(16,778) : 137,	(17,778) : 708,	(18,778) : 255,	(19,778) : 544,	(20,778) : 630,
(1,779) : 205,	(2,779) : 460,	(3,779) : 435,	(4,779) : 223,	(5,779) : 414,	(6,779) : 284,	(7,779) : 318,	(8,779) : 44,	(9,779) : 307,	(10,779) : 684,	(11,779) : 203,	(12,779) : 412,	(13,779) : 216,	(14,779) : 474,	(15,779) : 586,	(16,779) : 194,	(17,779) : 613,	(18,779) : 479,	(19,779) : 440,	(20,779) : 571,
(1,780) : 205,	(2,780) : 674,	(3,780) : 561,	(4,780) : 228,	(5,780) : 533,	(6,780) : 193,	(7,780) : 506,	(8,780) : 189,	(9,780) : 521,	(10,780) : 901,	(11,780) : 401,	(12,780) : 409,	(13,780) : 175,	(14,780) : 679,	(15,780) : 800,	(16,780) : 219,	(17,780) : 825,	(18,780) : 567,	(19,780) : 650,	(20,780) : 776,
(1,781) : 722,	(2,781) : 576,	(3,781) : 377,	(4,781) : 708,	(5,781) : 410,	(6,781) : 807,	(7,781) : 730,	(8,781) : 767,	(9,781) : 470,	(10,781) : 449,	(11,781) : 521,	(12,781) : 669,	(13,781) : 762,	(14,781) : 669,	(15,781) : 593,	(16,781) : 701,	(17,781) : 316,	(18,781) : 447,	(19,781) : 356,	(20,781) : 247,
(1,782) : 601,	(2,782) : 342,	(3,782) : 345,	(4,782) : 593,	(5,782) : 369,	(6,782) : 698,	(7,782) : 498,	(8,782) : 586,	(9,782) : 254,	(10,782) : 281,	(11,782) : 345,	(12,782) : 623,	(13,782) : 639,	(14,782) : 435,	(15,782) : 375,	(16,782) : 577,	(17,782) : 135,	(18,782) : 447,	(19,782) : 125,	(20,782) : 42,
(1,783) : 214,	(2,783) : 573,	(3,783) : 151,	(4,783) : 195,	(5,783) : 121,	(6,783) : 288,	(7,783) : 548,	(8,783) : 356,	(9,783) : 347,	(10,783) : 688,	(11,783) : 231,	(12,783) : 186,	(13,783) : 252,	(14,783) : 633,	(15,783) : 681,	(16,783) : 197,	(17,783) : 563,	(18,783) : 162,	(19,783) : 405,	(20,783) : 483,
(1,784) : 666,	(2,784) : 436,	(3,784) : 366,	(4,784) : 656,	(5,784) : 395,	(6,784) : 760,	(7,784) : 599,	(8,784) : 674,	(9,784) : 351,	(10,784) : 322,	(11,784) : 429,	(12,784) : 658,	(13,784) : 706,	(14,784) : 529,	(15,784) : 453,	(16,784) : 643,	(17,784) : 181,	(18,784) : 458,	(19,784) : 225,	(20,784) : 106,
(1,785) : 269,	(2,785) : 596,	(3,785) : 107,	(4,785) : 248,	(5,785) : 74,	(6,785) : 335,	(7,785) : 590,	(8,785) : 413,	(9,785) : 370,	(10,785) : 689,	(11,785) : 269,	(12,785) : 195,	(13,785) : 305,	(14,785) : 662,	(15,785) : 698,	(16,785) : 253,	(17,785) : 557,	(18,785) : 104,	(19,785) : 410,	(20,785) : 471,
(1,786) : 389,	(2,786) : 365,	(3,786) : 563,	(4,786) : 407,	(5,786) : 550,	(6,786) : 462,	(7,786) : 164,	(8,786) : 177,	(9,786) : 312,	(10,786) : 625,	(11,786) : 285,	(12,786) : 593,	(13,786) : 397,	(14,786) : 345,	(15,786) : 484,	(16,786) : 377,	(17,786) : 594,	(18,786) : 628,	(19,786) : 443,	(20,786) : 581,
(1,787) : 739,	(2,787) : 118,	(3,787) : 667,	(4,787) : 745,	(5,787) : 678,	(6,787) : 837,	(7,787) : 303,	(8,787) : 596,	(9,787) : 341,	(10,787) : 238,	(11,787) : 471,	(12,787) : 867,	(13,787) : 766,	(14,787) : 121,	(15,787) : 41,	(16,787) : 718,	(17,787) : 317,	(18,787) : 770,	(19,787) : 326,	(20,787) : 383,
(1,788) : 705,	(2,788) : 338,	(3,788) : 457,	(4,788) : 699,	(5,788) : 482,	(6,788) : 804,	(7,788) : 531,	(8,788) : 670,	(9,788) : 326,	(10,788) : 189,	(11,788) : 438,	(12,788) : 736,	(13,788) : 743,	(14,788) : 429,	(15,788) : 332,	(16,788) : 682,	(17,788) : 51,	(18,788) : 557,	(19,788) : 191,	(20,788) : 71,
(1,789) : 206,	(2,789) : 818,	(3,789) : 440,	(4,789) : 194,	(5,789) : 408,	(6,789) : 135,	(7,789) : 722,	(8,789) : 416,	(9,789) : 606,	(10,789) : 978,	(11,789) : 469,	(12,789) : 158,	(13,789) : 194,	(14,789) : 857,	(15,789) : 939,	(16,789) : 223,	(17,789) : 862,	(18,789) : 386,	(19,789) : 695,	(20,789) : 786,
(1,790) : 325,	(2,790) : 508,	(3,790) : 592,	(4,790) : 347,	(5,790) : 572,	(6,790) : 372,	(7,790) : 310,	(8,790) : 118,	(9,790) : 417,	(10,790) : 762,	(11,790) : 344,	(12,790) : 546,	(13,790) : 318,	(14,790) : 493,	(15,790) : 630,	(16,790) : 322,	(17,790) : 716,	(18,790) : 635,	(19,790) : 553,	(20,790) : 690,
(1,791) : 744,	(2,791) : 126,	(3,791) : 652,	(4,791) : 749,	(5,791) : 664,	(6,791) : 843,	(7,791) : 331,	(8,791) : 611,	(9,791) : 336,	(10,791) : 197,	(11,791) : 470,	(12,791) : 863,	(13,791) : 772,	(14,791) : 154,	(15,791) : 0,	(16,791) : 723,	(17,791) : 280,	(18,791) : 756,	(19,791) : 305,	(20,791) : 351,
(1,792) : 239,	(2,792) : 764,	(3,792) : 299,	(4,792) : 216,	(5,792) : 268,	(6,792) : 238,	(7,792) : 712,	(8,792) : 452,	(9,792) : 539,	(10,792) : 884,	(11,792) : 413,	(12,792) : 17,	(13,792) : 255,	(14,792) : 818,	(15,792) : 875,	(16,792) : 241,	(17,792) : 756,	(18,792) : 228,	(19,792) : 602,	(20,792) : 672,
(1,793) : 150,	(2,793) : 679,	(3,793) : 275,	(4,793) : 126,	(5,793) : 243,	(6,793) : 179,	(7,793) : 617,	(8,793) : 357,	(9,793) : 458,	(10,793) : 818,	(11,793) : 326,	(12,793) : 80,	(13,793) : 174,	(14,793) : 729,	(15,793) : 794,	(16,793) : 148,	(17,793) : 698,	(18,793) : 238,	(19,793) : 535,	(20,793) : 619,
(1,794) : 673,	(2,794) : 205,	(3,794) : 494,	(4,794) : 672,	(5,794) : 513,	(6,794) : 774,	(7,794) : 411,	(8,794) : 598,	(9,794) : 258,	(10,794) : 131,	(11,794) : 391,	(12,794) : 744,	(13,794) : 708,	(14,794) : 294,	(15,794) : 201,	(16,794) : 650,	(17,794) : 87,	(18,794) : 599,	(19,794) : 153,	(20,794) : 150,
(1,795) : 709,	(2,795) : 91,	(3,795) : 649,	(4,795) : 716,	(5,795) : 658,	(6,795) : 807,	(7,795) : 271,	(8,795) : 563,	(9,795) : 317,	(10,795) : 257,	(11,795) : 444,	(12,795) : 841,	(13,795) : 735,	(14,795) : 90,	(15,795) : 64,	(16,795) : 689,	(17,795) : 322,	(18,795) : 751,	(19,795) : 313,	(20,795) : 381,
(1,796) : 606,	(2,796) : 78,	(3,796) : 506,	(4,796) : 609,	(5,796) : 517,	(6,796) : 707,	(7,796) : 277,	(8,796) : 496,	(9,796) : 190,	(10,796) : 226,	(11,796) : 327,	(12,796) : 715,	(13,796) : 637,	(14,796) : 172,	(15,796) : 149,	(16,796) : 584,	(17,796) : 219,	(18,796) : 609,	(19,796) : 167,	(20,796) : 252,
(1,797) : 512,	(2,797) : 369,	(3,797) : 250,	(4,797) : 503,	(5,797) : 273,	(6,797) : 607,	(7,797) : 485,	(8,797) : 519,	(9,797) : 217,	(10,797) : 367,	(11,797) : 273,	(12,797) : 526,	(13,797) : 551,	(14,797) : 459,	(15,797) : 430,	(16,797) : 489,	(17,797) : 227,	(18,797) : 354,	(19,797) : 130,	(20,797) : 139,
(1,798) : 711,	(2,798) : 101,	(3,798) : 663,	(4,798) : 718,	(5,798) : 672,	(6,798) : 807,	(7,798) : 258,	(8,798) : 558,	(9,798) : 327,	(10,798) : 282,	(11,798) : 450,	(12,798) : 849,	(13,798) : 736,	(14,798) : 73,	(15,798) : 87,	(16,798) : 691,	(17,798) : 348,	(18,798) : 765,	(19,798) : 332,	(20,798) : 405,
(1,799) : 109,	(2,799) : 539,	(3,799) : 400,	(4,799) : 129,	(5,799) : 374,	(6,799) : 189,	(7,799) : 413,	(8,799) : 112,	(9,799) : 359,	(10,799) : 744,	(11,799) : 231,	(12,799) : 324,	(13,799) : 119,	(14,799) : 562,	(15,799) : 664,	(16,799) : 101,	(17,799) : 658,	(18,799) : 424,	(19,799) : 482,	(20,799) : 604,
(1,800) : 285,	(2,800) : 787,	(3,800) : 293,	(4,800) : 261,	(5,800) : 265,	(6,800) : 284,	(7,800) : 746,	(8,800) : 496,	(9,800) : 561,	(10,800) : 894,	(11,800) : 440,	(12,800) : 63,	(13,800) : 302,	(14,800) : 845,	(15,800) : 895,	(16,800) : 285,	(17,800) : 762,	(18,800) : 210,	(19,800) : 614,	(20,800) : 675,
(1,801) : 336,	(2,801) : 333,	(3,801) : 475,	(4,801) : 351,	(5,801) : 463,	(6,801) : 422,	(7,801) : 180,	(8,801) : 155,	(9,801) : 233,	(10,801) : 577,	(11,801) : 196,	(12,801) : 525,	(13,801) : 352,	(14,801) : 337,	(15,801) : 458,	(16,801) : 321,	(17,801) : 528,	(18,801) : 543,	(19,801) : 368,	(20,801) : 506,
(1,802) : 588,	(2,802) : 34,	(3,802) : 539,	(4,802) : 594,	(5,802) : 545,	(6,802) : 686,	(7,802) : 200,	(8,802) : 452,	(9,802) : 198,	(10,802) : 294,	(11,802) : 322,	(12,802) : 720,	(13,802) : 615,	(14,802) : 104,	(15,802) : 159,	(16,802) : 567,	(17,802) : 300,	(18,802) : 638,	(19,802) : 226,	(20,802) : 329,
(1,803) : 451,	(2,803) : 327,	(3,803) : 246,	(4,803) : 444,	(5,803) : 261,	(6,803) : 549,	(7,803) : 417,	(8,803) : 443,	(9,803) : 145,	(10,803) : 383,	(11,803) : 198,	(12,803) : 492,	(13,803) : 489,	(14,803) : 411,	(15,803) : 407,	(16,803) : 428,	(17,803) : 258,	(18,803) : 351,	(19,803) : 105,	(20,803) : 186,
(1,804) : 69,	(2,804) : 681,	(3,804) : 424,	(4,804) : 79,	(5,804) : 393,	(6,804) : 40,	(7,804) : 561,	(8,804) : 247,	(9,804) : 486,	(10,804) : 870,	(11,804) : 349,	(12,804) : 236,	(13,804) : 33,	(14,804) : 709,	(15,804) : 805,	(16,804) : 92,	(17,804) : 770,	(18,804) : 411,	(19,804) : 595,	(20,804) : 705,
(1,805) : 540,	(2,805) : 143,	(3,805) : 426,	(4,805) : 542,	(5,805) : 437,	(6,805) : 642,	(7,805) : 289,	(8,805) : 452,	(9,805) : 120,	(10,805) : 268,	(11,805) : 258,	(12,805) : 638,	(13,805) : 573,	(14,805) : 231,	(15,805) : 228,	(16,805) : 518,	(17,805) : 212,	(18,805) : 529,	(19,805) : 98,	(20,805) : 214,
(1,806) : 717,	(2,806) : 545,	(3,806) : 381,	(4,806) : 704,	(5,806) : 412,	(6,806) : 804,	(7,806) : 704,	(8,806) : 752,	(9,806) : 447,	(10,806) : 416,	(11,806) : 506,	(12,806) : 675,	(13,806) : 757,	(14,806) : 639,	(15,806) : 560,	(16,806) : 695,	(17,806) : 282,	(18,806) : 456,	(19,806) : 329,	(20,806) : 215,
(1,807) : 295,	(2,807) : 618,	(3,807) : 98,	(4,807) : 274,	(5,807) : 68,	(6,807) : 356,	(7,807) : 619,	(8,807) : 445,	(9,807) : 393,	(10,807) : 700,	(11,807) : 297,	(12,807) : 197,	(13,807) : 331,	(14,807) : 687,	(15,807) : 717,	(16,807) : 280,	(17,807) : 566,	(18,807) : 72,	(19,807) : 425,	(20,807) : 478,
(1,808) : 415,	(2,808) : 693,	(3,808) : 131,	(4,808) : 393,	(5,808) : 130,	(6,808) : 465,	(7,808) : 722,	(8,808) : 569,	(9,808) : 476,	(10,808) : 732,	(11,808) : 403,	(12,808) : 270,	(13,808) : 449,	(14,808) : 770,	(15,808) : 780,	(16,808) : 402,	(17,808) : 589,	(18,808) : 53,	(19,808) : 476,	(20,808) : 495,
(1,809) : 293,	(2,809) : 709,	(3,809) : 190,	(4,809) : 269,	(5,809) : 164,	(6,809) : 326,	(7,809) : 692,	(8,809) : 479,	(9,809) : 482,	(10,809) : 798,	(11,809) : 374,	(12,809) : 123,	(13,809) : 321,	(14,809) : 773,	(15,809) : 811,	(16,809) : 285,	(17,809) : 664,	(18,809) : 109,	(19,809) : 522,	(20,809) : 575,
(1,810) : 177,	(2,810) : 744,	(3,810) : 331,	(4,810) : 155,	(5,810) : 299,	(6,810) : 165,	(7,810) : 673,	(8,810) : 395,	(9,810) : 524,	(10,810) : 885,	(11,810) : 391,	(12,810) : 59,	(13,810) : 188,	(14,810) : 791,	(15,810) : 860,	(16,810) : 183,	(17,810) : 764,	(18,810) : 278,	(19,810) : 602,	(20,810) : 685,
(1,811) : 163,	(2,811) : 635,	(3,811) : 513,	(4,811) : 187,	(5,811) : 485,	(6,811) : 172,	(7,811) : 477,	(8,811) : 153,	(9,811) : 475,	(10,811) : 856,	(11,811) : 353,	(12,811) : 376,	(13,811) : 137,	(14,811) : 644,	(15,811) : 761,	(16,811) : 174,	(17,811) : 777,	(18,811) : 523,	(19,811) : 602,	(20,811) : 727,
(1,812) : 241,	(2,812) : 417,	(3,812) : 435,	(4,812) : 257,	(5,812) : 417,	(6,812) : 325,	(7,812) : 277,	(8,812) : 73,	(9,812) : 273,	(10,812) : 645,	(11,812) : 182,	(12,812) : 440,	(13,812) : 255,	(14,812) : 431,	(15,812) : 544,	(16,812) : 228,	(17,812) : 579,	(18,812) : 488,	(19,812) : 408,	(20,812) : 542,
(1,813) : 602,	(2,813) : 116,	(3,813) : 480,	(4,813) : 604,	(5,813) : 492,	(6,813) : 703,	(7,813) : 307,	(8,813) : 505,	(9,813) : 181,	(10,813) : 213,	(11,813) : 320,	(12,813) : 700,	(13,813) : 634,	(14,813) : 209,	(15,813) : 172,	(16,813) : 579,	(17,813) : 186,	(18,813) : 584,	(19,813) : 135,	(20,813) : 215,
(1,814) : 347,	(2,814) : 495,	(3,814) : 69,	(4,814) : 332,	(5,814) : 77,	(6,814) : 432,	(7,814) : 529,	(8,814) : 429,	(9,814) : 276,	(10,814) : 563,	(11,814) : 218,	(12,814) : 323,	(13,814) : 386,	(14,814) : 570,	(15,814) : 587,	(16,814) : 326,	(17,814) : 429,	(18,814) : 169,	(19,814) : 289,	(20,814) : 342,
(1,815) : 593,	(2,815) : 65,	(3,815) : 574,	(4,815) : 601,	(5,815) : 579,	(6,815) : 688,	(7,815) : 159,	(8,815) : 440,	(9,815) : 229,	(10,815) : 336,	(11,815) : 339,	(12,815) : 738,	(13,815) : 617,	(14,815) : 66,	(15,815) : 177,	(16,815) : 573,	(17,815) : 352,	(18,815) : 671,	(19,815) : 275,	(20,815) : 382,
(1,816) : 33,	(2,816) : 588,	(3,816) : 351,	(4,816) : 45,	(5,816) : 322,	(6,816) : 133,	(7,816) : 485,	(8,816) : 197,	(9,816) : 389,	(10,816) : 771,	(11,816) : 251,	(12,816) : 239,	(13,816) : 66,	(14,816) : 621,	(15,816) : 711,	(16,816) : 16,	(17,816) : 672,	(18,816) : 359,	(19,816) : 497,	(20,816) : 609,
(1,817) : 549,	(2,817) : 242,	(3,817) : 362,	(4,817) : 546,	(5,817) : 379,	(6,817) : 650,	(7,817) : 390,	(8,817) : 502,	(9,817) : 158,	(10,817) : 265,	(11,817) : 274,	(12,817) : 609,	(13,817) : 586,	(14,817) : 334,	(15,817) : 299,	(16,817) : 526,	(17,817) : 154,	(18,817) : 467,	(19,817) : 23,	(20,817) : 118,
(1,818) : 202,	(2,818) : 638,	(3,818) : 548,	(4,818) : 226,	(5,818) : 521,	(6,818) : 209,	(7,818) : 470,	(8,818) : 153,	(9,818) : 490,	(10,818) : 868,	(11,818) : 374,	(12,818) : 416,	(13,818) : 177,	(14,818) : 642,	(15,818) : 764,	(16,818) : 212,	(17,818) : 794,	(18,818) : 561,	(19,818) : 620,	(20,818) : 748,
(1,819) : 368,	(2,819) : 455,	(3,819) : 108,	(4,819) : 355,	(5,819) : 121,	(6,819) : 458,	(7,819) : 500,	(8,819) : 428,	(9,819) : 240,	(10,819) : 519,	(11,819) : 200,	(12,819) : 363,	(13,819) : 408,	(14,819) : 532,	(15,819) : 544,	(16,819) : 346,	(17,819) : 386,	(18,819) : 212,	(19,819) : 245,	(20,819) : 301,
(1,820) : 214,	(2,820) : 693,	(3,820) : 232,	(4,820) : 190,	(5,820) : 201,	(6,820) : 244,	(7,820) : 651,	(8,820) : 412,	(9,820) : 467,	(10,820) : 810,	(11,820) : 344,	(12,820) : 66,	(13,820) : 241,	(14,820) : 750,	(15,820) : 803,	(16,820) : 209,	(17,820) : 682,	(18,820) : 178,	(19,820) : 528,	(20,820) : 599,
(1,821) : 351,	(2,821) : 338,	(3,821) : 500,	(4,821) : 366,	(5,821) : 487,	(6,821) : 433,	(7,821) : 169,	(8,821) : 158,	(9,821) : 254,	(10,821) : 588,	(11,821) : 221,	(12,821) : 544,	(13,821) : 364,	(14,821) : 335,	(15,821) : 462,	(16,821) : 337,	(17,821) : 545,	(18,821) : 567,	(19,821) : 387,	(20,821) : 525,
(1,822) : 684,	(2,822) : 304,	(3,822) : 452,	(4,822) : 680,	(5,822) : 475,	(6,822) : 784,	(7,822) : 497,	(8,822) : 642,	(9,822) : 296,	(10,822) : 175,	(11,822) : 413,	(12,822) : 724,	(13,822) : 722,	(14,822) : 395,	(15,822) : 302,	(16,822) : 661,	(17,822) : 28,	(18,822) : 554,	(19,822) : 163,	(20,822) : 67,
(1,823) : 752,	(2,823) : 137,	(3,823) : 694,	(4,823) : 760,	(5,823) : 703,	(6,823) : 849,	(7,823) : 299,	(8,823) : 602,	(9,823) : 363,	(10,823) : 267,	(11,823) : 489,	(12,823) : 887,	(13,823) : 778,	(14,823) : 115,	(15,823) : 72,	(16,823) : 732,	(17,823) : 350,	(18,823) : 796,	(19,823) : 355,	(20,823) : 416,
(1,824) : 277,	(2,824) : 502,	(3,824) : 546,	(4,824) : 299,	(5,824) : 525,	(6,824) : 329,	(7,824) : 319,	(8,824) : 70,	(9,824) : 391,	(10,824) : 749,	(11,824) : 306,	(12,824) : 498,	(13,824) : 273,	(14,824) : 496,	(15,824) : 626,	(16,824) : 274,	(17,824) : 694,	(18,824) : 587,	(19,824) : 526,	(20,824) : 662,
(1,825) : 225,	(2,825) : 557,	(3,825) : 530,	(4,825) : 249,	(5,825) : 507,	(6,825) : 265,	(7,825) : 384,	(8,825) : 77,	(9,825) : 423,	(10,825) : 793,	(11,825) : 319,	(12,825) : 447,	(13,825) : 214,	(14,825) : 557,	(15,825) : 682,	(16,825) : 226,	(17,825) : 728,	(18,825) : 560,	(19,825) : 556,	(20,825) : 688,
(1,826) : 720,	(2,826) : 396,	(3,826) : 445,	(4,826) : 713,	(5,826) : 473,	(6,826) : 817,	(7,826) : 584,	(8,826) : 703,	(9,826) : 363,	(10,826) : 239,	(11,826) : 464,	(12,826) : 733,	(13,826) : 759,	(14,826) : 488,	(15,826) : 391,	(16,826) : 697,	(17,826) : 111,	(18,826) : 542,	(19,826) : 227,	(20,826) : 90,
(1,827) : 601,	(2,827) : 240,	(3,827) : 404,	(4,827) : 598,	(5,827) : 423,	(6,827) : 702,	(7,827) : 412,	(8,827) : 549,	(9,827) : 203,	(10,827) : 216,	(11,827) : 324,	(12,827) : 659,	(13,827) : 637,	(14,827) : 333,	(15,827) : 274,	(16,827) : 578,	(17,827) : 103,	(18,827) : 509,	(19,827) : 74,	(20,827) : 92,
(1,828) : 327,	(2,828) : 377,	(3,828) : 201,	(4,828) : 321,	(5,828) : 200,	(6,828) : 426,	(7,828) : 397,	(8,828) : 338,	(9,828) : 152,	(10,828) : 492,	(11,828) : 98,	(12,828) : 386,	(13,828) : 365,	(14,828) : 446,	(15,828) : 480,	(16,828) : 304,	(17,828) : 378,	(18,828) : 292,	(19,828) : 209,	(20,828) : 310,
(1,829) : 819,	(2,829) : 583,	(3,829) : 487,	(4,829) : 807,	(5,829) : 519,	(6,829) : 908,	(7,829) : 763,	(8,829) : 842,	(9,829) : 521,	(10,829) : 404,	(11,829) : 596,	(12,829) : 781,	(13,829) : 859,	(14,829) : 674,	(15,829) : 575,	(16,829) : 797,	(17,829) : 296,	(18,829) : 562,	(19,829) : 393,	(20,829) : 260,
(1,830) : 354,	(2,830) : 346,	(3,830) : 511,	(4,830) : 370,	(5,830) : 498,	(6,830) : 434,	(7,830) : 170,	(8,830) : 156,	(9,830) : 266,	(10,830) : 598,	(11,830) : 232,	(12,830) : 550,	(13,830) : 366,	(14,830) : 340,	(15,830) : 469,	(16,830) : 341,	(17,830) : 556,	(18,830) : 577,	(19,830) : 399,	(20,830) : 537,
(1,831) : 289,	(2,831) : 839,	(3,831) : 368,	(4,831) : 268,	(5,831) : 339,	(6,831) : 261,	(7,831) : 781,	(8,831) : 508,	(9,831) : 615,	(10,831) : 960,	(11,831) : 488,	(12,831) : 89,	(13,831) : 297,	(14,831) : 892,	(15,831) : 951,	(16,831) : 296,	(17,831) : 831,	(18,831) : 287,	(19,831) : 678,	(20,831) : 746,
(1,832) : 212,	(2,832) : 611,	(3,832) : 166,	(4,832) : 191,	(5,832) : 134,	(6,832) : 274,	(7,832) : 583,	(8,832) : 375,	(9,832) : 385,	(10,832) : 725,	(11,832) : 268,	(12,832) : 149,	(13,832) : 248,	(14,832) : 671,	(15,832) : 720,	(16,832) : 198,	(17,832) : 599,	(18,832) : 151,	(19,832) : 443,	(20,832) : 516,
(1,833) : 81,	(2,833) : 699,	(3,833) : 385,	(4,833) : 71,	(5,833) : 353,	(6,833) : 55,	(7,833) : 596,	(8,833) : 294,	(9,833) : 493,	(10,833) : 871,	(11,833) : 354,	(12,833) : 175,	(13,833) : 74,	(14,833) : 734,	(15,833) : 821,	(16,833) : 99,	(17,833) : 764,	(18,833) : 361,	(19,833) : 592,	(20,833) : 694,
(1,834) : 607,	(2,834) : 72,	(3,834) : 510,	(4,834) : 610,	(5,834) : 520,	(6,834) : 707,	(7,834) : 273,	(8,834) : 495,	(9,834) : 192,	(10,834) : 229,	(11,834) : 328,	(12,834) : 717,	(13,834) : 637,	(14,834) : 166,	(15,834) : 146,	(16,834) : 585,	(17,834) : 224,	(18,834) : 613,	(19,834) : 172,	(20,834) : 258,
(1,835) : 164,	(2,835) : 777,	(3,835) : 415,	(4,835) : 152,	(5,835) : 383,	(6,835) : 101,	(7,835) : 680,	(8,835) : 376,	(9,835) : 566,	(10,835) : 940,	(11,835) : 428,	(12,835) : 149,	(13,835) : 154,	(14,835) : 815,	(15,835) : 898,	(16,835) : 181,	(17,835) : 827,	(18,835) : 370,	(19,835) : 658,	(20,835) : 752,
(1,836) : 318,	(2,836) : 913,	(3,836) : 475,	(4,836) : 302,	(5,836) : 445,	(6,836) : 254,	(7,836) : 831,	(8,836) : 533,	(9,836) : 694,	(10,836) : 1052,	(11,836) : 560,	(12,836) : 182,	(13,836) : 311,	(14,836) : 958,	(15,836) : 1030,	(16,836) : 332,	(17,836) : 929,	(18,836) : 398,	(19,836) : 769,	(20,836) : 846,
(1,837) : 206,	(2,837) : 783,	(3,837) : 362,	(4,837) : 186,	(5,837) : 331,	(6,837) : 176,	(7,837) : 709,	(8,837) : 425,	(9,837) : 564,	(10,837) : 924,	(11,837) : 430,	(12,837) : 72,	(13,837) : 211,	(14,837) : 830,	(15,837) : 900,	(16,837) : 215,	(17,837) : 802,	(18,837) : 302,	(19,837) : 641,	(20,837) : 722,
(1,838) : 694,	(2,838) : 523,	(3,838) : 362,	(4,838) : 681,	(5,838) : 393,	(6,838) : 782,	(7,838) : 679,	(8,838) : 726,	(9,838) : 421,	(10,838) : 403,	(11,838) : 480,	(12,838) : 657,	(13,838) : 733,	(14,838) : 616,	(15,838) : 542,	(16,838) : 672,	(17,838) : 266,	(18,838) : 441,	(19,838) : 304,	(20,838) : 194,
(1,839) : 645,	(2,839) : 173,	(3,839) : 484,	(4,839) : 645,	(5,839) : 500,	(6,839) : 746,	(7,839) : 375,	(8,839) : 564,	(9,839) : 227,	(10,839) : 158,	(11,839) : 363,	(12,839) : 725,	(13,839) : 679,	(14,839) : 264,	(15,839) : 188,	(16,839) : 622,	(17,839) : 119,	(18,839) : 589,	(19,839) : 135,	(20,839) : 166,
(1,840) : 744,	(2,840) : 138,	(3,840) : 700,	(4,840) : 752,	(5,840) : 708,	(6,840) : 840,	(7,840) : 278,	(8,840) : 587,	(9,840) : 364,	(10,840) : 295,	(11,840) : 486,	(12,840) : 885,	(13,840) : 769,	(14,840) : 96,	(15,840) : 98,	(16,840) : 725,	(17,840) : 373,	(18,840) : 802,	(19,840) : 367,	(20,840) : 435,
(1,841) : 289,	(2,841) : 736,	(3,841) : 648,	(4,841) : 311,	(5,841) : 619,	(6,841) : 259,	(7,841) : 553,	(8,841) : 255,	(9,841) : 598,	(10,841) : 972,	(11,841) : 483,	(12,841) : 482,	(13,841) : 255,	(14,841) : 732,	(15,841) : 861,	(16,841) : 304,	(17,841) : 903,	(18,841) : 651,	(19,841) : 729,	(20,841) : 857,
(1,842) : 647,	(2,842) : 374,	(3,842) : 374,	(4,842) : 638,	(5,842) : 401,	(6,842) : 743,	(7,842) : 542,	(8,842) : 636,	(9,842) : 303,	(10,842) : 273,	(11,842) : 394,	(12,842) : 659,	(13,842) : 685,	(14,842) : 467,	(15,842) : 393,	(16,842) : 623,	(17,842) : 127,	(18,842) : 473,	(19,842) : 171,	(20,842) : 44,
(1,843) : 268,	(2,843) : 726,	(3,843) : 627,	(4,843) : 290,	(5,843) : 599,	(6,843) : 239,	(7,843) : 547,	(8,843) : 243,	(9,843) : 583,	(10,843) : 960,	(11,843) : 466,	(12,843) : 461,	(13,843) : 234,	(14,843) : 724,	(15,843) : 851,	(16,843) : 283,	(17,843) : 888,	(18,843) : 630,	(19,843) : 713,	(20,843) : 841,
(1,844) : 256,	(2,844) : 819,	(3,844) : 366,	(4,844) : 236,	(5,844) : 336,	(6,844) : 227,	(7,844) : 754,	(8,844) : 476,	(9,844) : 596,	(10,844) : 948,	(11,844) : 466,	(12,844) : 74,	(13,844) : 263,	(14,844) : 869,	(15,844) : 933,	(16,844) : 264,	(17,844) : 822,	(18,844) : 293,	(19,844) : 665,	(20,844) : 738,
(1,845) : 635,	(2,845) : 352,	(3,845) : 373,	(4,845) : 627,	(5,845) : 398,	(6,845) : 732,	(7,845) : 520,	(8,845) : 618,	(9,845) : 283,	(10,845) : 262,	(11,845) : 378,	(12,845) : 654,	(13,845) : 673,	(14,845) : 445,	(15,845) : 374,	(16,845) : 611,	(17,845) : 115,	(18,845) : 474,	(19,845) : 150,	(20,845) : 23,
(1,846) : 666,	(2,846) : 395,	(3,846) : 384,	(4,846) : 657,	(5,846) : 412,	(6,846) : 762,	(7,846) : 566,	(8,846) : 659,	(9,846) : 327,	(10,846) : 280,	(11,846) : 416,	(12,846) : 672,	(13,846) : 705,	(14,846) : 488,	(15,846) : 409,	(16,846) : 643,	(17,846) : 136,	(18,846) : 481,	(19,846) : 196,	(20,846) : 65,
(1,847) : 300,	(2,847) : 595,	(3,847) : 613,	(4,847) : 324,	(5,847) : 589,	(6,847) : 323,	(7,847) : 401,	(8,847) : 147,	(9,847) : 485,	(10,847) : 844,	(11,847) : 393,	(12,847) : 521,	(13,847) : 283,	(14,847) : 583,	(15,847) : 718,	(16,847) : 304,	(17,847) : 789,	(18,847) : 642,	(19,847) : 621,	(20,847) : 756,
(1,848) : 329,	(2,848) : 907,	(3,848) : 447,	(4,848) : 311,	(5,848) : 418,	(6,848) : 277,	(7,848) : 836,	(8,848) : 547,	(9,848) : 685,	(10,848) : 1036,	(11,848) : 554,	(12,848) : 162,	(13,848) : 327,	(14,848) : 956,	(15,848) : 1022,	(16,848) : 340,	(17,848) : 909,	(18,848) : 365,	(19,848) : 754,	(20,848) : 824,
(1,849) : 73,	(2,849) : 574,	(3,849) : 295,	(4,849) : 63,	(5,849) : 266,	(6,849) : 168,	(7,849) : 492,	(8,849) : 232,	(9,849) : 363,	(10,849) : 741,	(11,849) : 225,	(12,849) : 210,	(13,849) : 113,	(14,849) : 615,	(15,849) : 694,	(16,849) : 52,	(17,849) : 635,	(18,849) : 304,	(19,849) : 462,	(20,849) : 567,
(1,850) : 591,	(2,850) : 95,	(3,850) : 485,	(4,850) : 593,	(5,850) : 496,	(6,850) : 692,	(7,850) : 281,	(8,850) : 487,	(9,850) : 173,	(10,850) : 232,	(11,850) : 311,	(12,850) : 696,	(13,850) : 622,	(14,850) : 188,	(15,850) : 169,	(16,850) : 569,	(17,850) : 211,	(18,850) : 588,	(19,850) : 147,	(20,850) : 238,
(1,851) : 551,	(2,851) : 340,	(3,851) : 301,	(4,851) : 543,	(5,851) : 324,	(6,851) : 648,	(7,851) : 476,	(8,851) : 542,	(9,851) : 219,	(10,851) : 317,	(11,851) : 299,	(12,851) : 575,	(13,851) : 590,	(14,851) : 432,	(15,851) : 389,	(16,851) : 528,	(17,851) : 177,	(18,851) : 405,	(19,851) : 105,	(20,851) : 90,
(1,852) : 341,	(2,852) : 853,	(3,852) : 346,	(4,852) : 318,	(5,852) : 320,	(6,852) : 327,	(7,852) : 812,	(8,852) : 556,	(9,852) : 627,	(10,852) : 954,	(11,852) : 507,	(12,852) : 121,	(13,852) : 354,	(14,852) : 912,	(15,852) : 960,	(16,852) : 344,	(17,852) : 820,	(18,852) : 252,	(19,852) : 676,	(20,852) : 731,
(1,853) : 372,	(2,853) : 268,	(3,853) : 450,	(4,853) : 383,	(5,853) : 442,	(6,853) : 464,	(7,853) : 157,	(8,853) : 217,	(9,853) : 169,	(10,853) : 506,	(11,853) : 169,	(12,853) : 541,	(13,853) : 393,	(14,853) : 284,	(15,853) : 394,	(16,853) : 354,	(17,853) : 458,	(18,853) : 528,	(19,853) : 301,	(20,853) : 439,
(1,854) : 42,	(2,854) : 595,	(3,854) : 323,	(4,854) : 34,	(5,854) : 294,	(6,854) : 138,	(7,854) : 503,	(8,854) : 226,	(9,854) : 389,	(10,854) : 768,	(11,854) : 250,	(12,854) : 209,	(13,854) : 82,	(14,854) : 633,	(15,854) : 716,	(16,854) : 23,	(17,854) : 664,	(18,854) : 326,	(19,854) : 491,	(20,854) : 597,
(1,855) : 281,	(2,855) : 532,	(3,855) : 88,	(4,855) : 264,	(5,855) : 65,	(6,855) : 361,	(7,855) : 536,	(8,855) : 390,	(9,855) : 306,	(10,855) : 625,	(11,855) : 214,	(12,855) : 253,	(13,855) : 320,	(14,855) : 600,	(15,855) : 633,	(16,855) : 261,	(17,855) : 495,	(18,855) : 145,	(19,855) : 345,	(20,855) : 412,
(1,856) : 341,	(2,856) : 581,	(3,856) : 33,	(4,856) : 321,	(5,856) : 11,	(6,856) : 411,	(7,856) : 604,	(8,856) : 465,	(9,856) : 360,	(10,856) : 645,	(11,856) : 284,	(12,856) : 262,	(13,856) : 378,	(14,856) : 655,	(15,856) : 674,	(16,856) : 323,	(17,856) : 507,	(18,856) : 83,	(19,856) : 375,	(20,856) : 417,
(1,857) : 178,	(2,857) : 646,	(3,857) : 530,	(4,857) : 202,	(5,857) : 502,	(6,857) : 180,	(7,857) : 485,	(8,857) : 162,	(9,857) : 489,	(10,857) : 870,	(11,857) : 368,	(12,857) : 389,	(13,857) : 151,	(14,857) : 654,	(15,857) : 772,	(16,857) : 189,	(17,857) : 792,	(18,857) : 539,	(19,857) : 617,	(20,857) : 743,
(1,858) : 466,	(2,858) : 618,	(3,858) : 105,	(4,858) : 446,	(5,858) : 132,	(6,858) : 533,	(7,858) : 680,	(8,858) : 581,	(9,858) : 416,	(10,858) : 627,	(11,858) : 375,	(12,858) : 365,	(13,858) : 503,	(14,858) : 701,	(15,858) : 693,	(16,858) : 448,	(17,858) : 482,	(18,858) : 137,	(19,858) : 388,	(20,858) : 387,
(1,859) : 304,	(2,859) : 508,	(3,859) : 80,	(4,859) : 289,	(5,859) : 69,	(6,859) : 388,	(7,859) : 523,	(8,859) : 398,	(9,859) : 284,	(10,859) : 594,	(11,859) : 204,	(12,859) : 285,	(13,859) : 344,	(14,859) : 579,	(15,859) : 606,	(16,859) : 284,	(17,859) : 464,	(18,859) : 159,	(19,859) : 315,	(20,859) : 380,
(1,860) : 209,	(2,860) : 534,	(3,860) : 502,	(4,860) : 232,	(5,860) : 478,	(6,860) : 259,	(7,860) : 369,	(8,860) : 50,	(9,860) : 394,	(10,860) : 767,	(11,860) : 289,	(12,860) : 430,	(13,860) : 203,	(14,860) : 538,	(15,860) : 660,	(16,860) : 207,	(17,860) : 699,	(18,860) : 534,	(19,860) : 527,	(20,860) : 659,
(1,861) : 220,	(2,861) : 657,	(3,861) : 569,	(4,861) : 244,	(5,861) : 542,	(6,861) : 218,	(7,861) : 485,	(8,861) : 173,	(9,861) : 512,	(10,861) : 889,	(11,861) : 396,	(12,861) : 430,	(13,861) : 193,	(14,861) : 659,	(15,861) : 783,	(16,861) : 231,	(17,861) : 817,	(18,861) : 581,	(19,861) : 642,	(20,861) : 771,
(1,862) : 190,	(2,862) : 610,	(3,862) : 526,	(4,862) : 214,	(5,862) : 500,	(6,862) : 211,	(7,862) : 444,	(8,862) : 125,	(9,862) : 461,	(10,862) : 838,	(11,862) : 345,	(12,862) : 409,	(13,862) : 170,	(14,862) : 615,	(15,862) : 736,	(16,862) : 197,	(17,862) : 765,	(18,862) : 543,	(19,862) : 590,	(20,862) : 719,
(1,863) : 138,	(2,863) : 573,	(3,863) : 226,	(4,863) : 121,	(5,863) : 196,	(6,863) : 219,	(7,863) : 519,	(8,863) : 291,	(9,863) : 352,	(10,863) : 716,	(11,863) : 220,	(12,863) : 180,	(13,863) : 177,	(14,863) : 624,	(15,863) : 688,	(16,863) : 120,	(17,863) : 601,	(18,863) : 234,	(19,863) : 434,	(20,863) : 527,
(1,864) : 217,	(2,864) : 413,	(3,864) : 281,	(4,864) : 219,	(5,864) : 265,	(6,864) : 318,	(7,864) : 355,	(8,864) : 207,	(9,864) : 204,	(10,864) : 586,	(11,864) : 66,	(12,864) : 347,	(13,864) : 251,	(14,864) : 458,	(15,864) : 532,	(16,864) : 194,	(17,864) : 491,	(18,864) : 342,	(19,864) : 315,	(20,864) : 434,
(1,865) : 335,	(2,865) : 347,	(3,865) : 488,	(4,865) : 351,	(5,865) : 476,	(6,865) : 418,	(7,865) : 184,	(8,865) : 144,	(9,865) : 252,	(10,865) : 594,	(11,865) : 211,	(12,865) : 529,	(13,865) : 349,	(14,865) : 348,	(15,865) : 472,	(16,865) : 321,	(17,865) : 547,	(18,865) : 554,	(19,865) : 387,	(20,865) : 525,
(1,866) : 655,	(2,866) : 104,	(3,866) : 646,	(4,866) : 665,	(5,866) : 651,	(6,866) : 749,	(7,866) : 180,	(8,866) : 490,	(9,866) : 301,	(10,866) : 349,	(11,866) : 409,	(12,866) : 807,	(13,866) : 678,	(14,866) : 11,	(15,866) : 162,	(16,866) : 637,	(17,866) : 391,	(18,866) : 744,	(19,866) : 339,	(20,866) : 434,
(1,867) : 780,	(2,867) : 165,	(3,867) : 721,	(4,867) : 788,	(5,867) : 731,	(6,867) : 877,	(7,867) : 320,	(8,867) : 627,	(9,867) : 391,	(10,867) : 275,	(11,867) : 518,	(12,867) : 915,	(13,867) : 805,	(14,867) : 137,	(15,867) : 87,	(16,867) : 760,	(17,867) : 368,	(18,867) : 824,	(19,867) : 381,	(20,867) : 437,
(1,868) : 298,	(2,868) : 726,	(3,868) : 207,	(4,868) : 274,	(5,868) : 181,	(6,868) : 326,	(7,868) : 707,	(8,868) : 489,	(9,868) : 500,	(10,868) : 816,	(11,868) : 390,	(12,868) : 116,	(13,868) : 325,	(14,868) : 790,	(15,868) : 829,	(16,868) : 292,	(17,868) : 681,	(18,868) : 121,	(19,868) : 540,	(20,868) : 592,
(1,869) : 102,	(2,869) : 542,	(3,869) : 395,	(4,869) : 122,	(5,869) : 370,	(6,869) : 183,	(7,869) : 419,	(8,869) : 119,	(9,869) : 361,	(10,869) : 746,	(11,869) : 231,	(12,869) : 317,	(13,869) : 113,	(14,869) : 566,	(15,869) : 668,	(16,869) : 94,	(17,869) : 658,	(18,869) : 419,	(19,869) : 482,	(20,869) : 604,
(1,870) : 246,	(2,870) : 696,	(3,870) : 210,	(4,870) : 222,	(5,870) : 179,	(6,870) : 279,	(7,870) : 665,	(8,870) : 438,	(9,870) : 469,	(10,870) : 801,	(11,870) : 352,	(12,870) : 87,	(13,870) : 274,	(14,870) : 756,	(15,870) : 803,	(16,870) : 239,	(17,870) : 671,	(18,870) : 146,	(19,870) : 521,	(20,870) : 585,
(1,871) : 102,	(2,871) : 549,	(3,871) : 404,	(4,871) : 123,	(5,871) : 378,	(6,871) : 179,	(7,871) : 422,	(8,871) : 117,	(9,871) : 370,	(10,871) : 754,	(11,871) : 241,	(12,871) : 320,	(13,871) : 110,	(14,871) : 572,	(15,871) : 675,	(16,871) : 96,	(17,871) : 668,	(18,871) : 426,	(19,871) : 491,	(20,871) : 614,
(1,872) : 168,	(2,872) : 710,	(3,872) : 293,	(4,872) : 144,	(5,872) : 261,	(6,872) : 179,	(7,872) : 646,	(8,872) : 381,	(9,872) : 489,	(10,872) : 847,	(11,872) : 357,	(12,872) : 55,	(13,872) : 187,	(14,872) : 760,	(15,872) : 825,	(16,872) : 169,	(17,872) : 725,	(18,872) : 246,	(19,872) : 564,	(20,872) : 645,
(1,873) : 666,	(2,873) : 58,	(3,873) : 580,	(4,873) : 671,	(5,873) : 591,	(6,873) : 765,	(7,873) : 279,	(8,873) : 539,	(9,873) : 258,	(10,873) : 216,	(11,873) : 392,	(12,873) : 785,	(13,873) : 695,	(14,873) : 128,	(15,873) : 79,	(16,873) : 645,	(17,873) : 255,	(18,873) : 683,	(19,873) : 239,	(20,873) : 308,
(1,874) : 398,	(2,874) : 956,	(3,874) : 465,	(4,874) : 378,	(5,874) : 439,	(6,874) : 352,	(7,874) : 898,	(8,874) : 617,	(9,874) : 732,	(10,874) : 1070,	(11,874) : 605,	(12,874) : 206,	(13,874) : 399,	(14,874) : 1010,	(15,874) : 1067,	(16,874) : 407,	(17,874) : 938,	(18,874) : 373,	(19,874) : 790,	(20,874) : 850,
(1,875) : 188,	(2,875) : 606,	(3,875) : 523,	(4,875) : 212,	(5,875) : 497,	(6,875) : 211,	(7,875) : 441,	(8,875) : 121,	(9,875) : 457,	(10,875) : 834,	(11,875) : 341,	(12,875) : 407,	(13,875) : 169,	(14,875) : 612,	(15,875) : 732,	(16,875) : 195,	(17,875) : 761,	(18,875) : 541,	(19,875) : 586,	(20,875) : 715,
(1,876) : 371,	(2,876) : 877,	(3,876) : 361,	(4,876) : 348,	(5,876) : 337,	(6,876) : 354,	(7,876) : 840,	(8,876) : 586,	(9,876) : 651,	(10,876) : 973,	(11,876) : 533,	(12,876) : 150,	(13,876) : 383,	(14,876) : 937,	(15,876) : 983,	(16,876) : 374,	(17,876) : 838,	(18,876) : 264,	(19,876) : 697,	(20,876) : 748,
(1,877) : 421,	(2,877) : 200,	(3,877) : 417,	(4,877) : 427,	(5,877) : 416,	(6,877) : 519,	(7,877) : 186,	(8,877) : 301,	(9,877) : 88,	(10,877) : 417,	(11,877) : 162,	(12,877) : 560,	(13,877) : 449,	(14,877) : 243,	(15,877) : 324,	(16,877) : 400,	(17,877) : 364,	(18,877) : 507,	(19,877) : 210,	(20,877) : 348,
(1,878) : 179,	(2,878) : 688,	(3,878) : 541,	(4,878) : 201,	(5,878) : 512,	(6,878) : 157,	(7,878) : 529,	(8,878) : 205,	(9,878) : 525,	(10,878) : 908,	(11,878) : 400,	(12,878) : 375,	(13,878) : 145,	(14,878) : 697,	(15,878) : 814,	(16,878) : 195,	(17,878) : 826,	(18,878) : 542,	(19,878) : 650,	(20,878) : 774,
(1,879) : 328,	(2,879) : 577,	(3,879) : 628,	(4,879) : 352,	(5,879) : 606,	(6,879) : 357,	(7,879) : 377,	(8,879) : 154,	(9,879) : 480,	(10,879) : 830,	(11,879) : 397,	(12,879) : 550,	(13,879) : 313,	(14,879) : 560,	(15,879) : 698,	(16,879) : 330,	(17,879) : 781,	(18,879) : 662,	(19,879) : 616,	(20,879) : 752,
(1,880) : 744,	(2,880) : 193,	(3,880) : 588,	(4,880) : 745,	(5,880) : 605,	(6,880) : 845,	(7,880) : 415,	(8,880) : 647,	(9,880) : 323,	(10,880) : 78,	(11,880) : 462,	(12,880) : 830,	(13,880) : 777,	(14,880) : 263,	(15,880) : 132,	(16,880) : 721,	(17,880) : 156,	(18,880) : 693,	(19,880) : 241,	(20,880) : 239,
(1,881) : 437,	(2,881) : 198,	(3,881) : 380,	(4,881) : 440,	(5,881) : 383,	(6,881) : 537,	(7,881) : 241,	(8,881) : 344,	(9,881) : 34,	(10,881) : 377,	(11,881) : 158,	(12,881) : 553,	(13,881) : 468,	(14,881) : 262,	(15,881) : 312,	(16,881) : 415,	(17,881) : 310,	(18,881) : 475,	(19,881) : 150,	(20,881) : 288,
(1,882) : 595,	(2,882) : 301,	(3,882) : 363,	(4,882) : 590,	(5,882) : 385,	(6,882) : 694,	(7,882) : 462,	(8,882) : 567,	(9,882) : 227,	(10,882) : 254,	(11,882) : 330,	(12,882) : 632,	(13,882) : 633,	(14,882) : 394,	(15,882) : 336,	(16,882) : 572,	(17,882) : 116,	(18,882) : 467,	(19,882) : 92,	(20,882) : 46,
(1,883) : 60,	(2,883) : 595,	(3,883) : 304,	(4,883) : 45,	(5,883) : 274,	(6,883) : 149,	(7,883) : 511,	(8,883) : 243,	(9,883) : 385,	(10,883) : 762,	(11,883) : 247,	(12,883) : 194,	(13,883) : 99,	(14,883) : 636,	(15,883) : 716,	(16,883) : 44,	(17,883) : 655,	(18,883) : 306,	(19,883) : 483,	(20,883) : 586,
(1,884) : 767,	(2,884) : 148,	(3,884) : 700,	(4,884) : 774,	(5,884) : 710,	(6,884) : 864,	(7,884) : 318,	(8,884) : 619,	(9,884) : 373,	(10,884) : 254,	(11,884) : 501,	(12,884) : 898,	(13,884) : 793,	(14,884) : 134,	(15,884) : 63,	(16,884) : 746,	(17,884) : 343,	(18,884) : 803,	(19,884) : 358,	(20,884) : 412,
(1,885) : 709,	(2,885) : 88,	(3,885) : 634,	(4,885) : 715,	(5,885) : 644,	(6,885) : 808,	(7,885) : 289,	(8,885) : 571,	(9,885) : 308,	(10,885) : 229,	(11,885) : 439,	(12,885) : 834,	(13,885) : 737,	(14,885) : 113,	(15,885) : 42,	(16,885) : 688,	(17,885) : 294,	(18,885) : 737,	(19,885) : 293,	(20,885) : 355,
(1,886) : 167,	(2,886) : 454,	(3,886) : 321,	(4,886) : 174,	(5,886) : 300,	(6,886) : 267,	(7,886) : 366,	(8,886) : 156,	(9,886) : 258,	(10,886) : 642,	(11,886) : 124,	(12,886) : 331,	(13,886) : 197,	(14,886) : 490,	(15,886) : 577,	(16,886) : 146,	(17,886) : 551,	(18,886) : 367,	(19,886) : 375,	(20,886) : 496,
(1,887) : 605,	(2,887) : 340,	(3,887) : 350,	(4,887) : 598,	(5,887) : 374,	(6,887) : 703,	(7,887) : 499,	(8,887) : 590,	(9,887) : 256,	(10,887) : 276,	(11,887) : 348,	(12,887) : 628,	(13,887) : 644,	(14,887) : 434,	(15,887) : 372,	(16,887) : 582,	(17,887) : 130,	(18,887) : 452,	(19,887) : 126,	(20,887) : 37,
(1,888) : 149,	(2,888) : 723,	(3,888) : 511,	(4,888) : 165,	(5,888) : 480,	(6,888) : 90,	(7,888) : 581,	(8,888) : 255,	(9,888) : 544,	(10,888) : 929,	(11,888) : 411,	(12,888) : 314,	(13,888) : 109,	(14,888) : 741,	(15,888) : 849,	(16,888) : 171,	(17,888) : 837,	(18,888) : 498,	(19,888) : 661,	(20,888) : 778,
(1,889) : 231,	(2,889) : 553,	(3,889) : 134,	(4,889) : 213,	(5,889) : 105,	(6,889) : 309,	(7,889) : 535,	(8,889) : 358,	(9,889) : 326,	(10,889) : 663,	(11,889) : 216,	(12,889) : 211,	(13,889) : 269,	(14,889) : 615,	(15,889) : 659,	(16,889) : 212,	(17,889) : 538,	(18,889) : 160,	(19,889) : 381,	(20,889) : 457,
(1,890) : 335,	(2,890) : 753,	(3,890) : 218,	(4,890) : 311,	(5,890) : 195,	(6,890) : 359,	(7,890) : 740,	(8,890) : 526,	(9,890) : 527,	(10,890) : 832,	(11,890) : 421,	(12,890) : 142,	(13,890) : 361,	(14,890) : 819,	(15,890) : 853,	(16,890) : 329,	(17,890) : 695,	(18,890) : 120,	(19,890) : 559,	(20,890) : 604,
(1,891) : 42,	(2,891) : 654,	(3,891) : 403,	(4,891) : 57,	(5,891) : 373,	(6,891) : 65,	(7,891) : 538,	(8,891) : 229,	(9,891) : 459,	(10,891) : 842,	(11,891) : 321,	(12,891) : 234,	(13,891) : 7,	(14,891) : 683,	(15,891) : 778,	(16,891) : 65,	(17,891) : 743,	(18,891) : 395,	(19,891) : 568,	(20,891) : 679,
(1,892) : 536,	(2,892) : 600,	(3,892) : 172,	(4,892) : 518,	(5,892) : 203,	(6,892) : 610,	(7,892) : 689,	(8,892) : 630,	(9,892) : 418,	(10,892) : 575,	(11,892) : 406,	(12,892) : 450,	(13,892) : 575,	(14,892) : 688,	(15,892) : 661,	(16,892) : 517,	(17,892) : 428,	(18,892) : 223,	(19,892) : 362,	(20,892) : 333,
(1,893) : 805,	(2,893) : 185,	(3,893) : 729,	(4,893) : 812,	(5,893) : 741,	(6,893) : 903,	(7,893) : 357,	(8,893) : 659,	(9,893) : 407,	(10,893) : 246,	(11,893) : 538,	(12,893) : 933,	(13,893) : 832,	(14,893) : 173,	(15,893) : 79,	(16,893) : 785,	(17,893) : 351,	(18,893) : 833,	(19,893) : 384,	(20,893) : 426,
(1,894) : 357,	(2,894) : 540,	(3,894) : 24,	(4,894) : 340,	(5,894) : 41,	(6,894) : 435,	(7,894) : 574,	(8,894) : 460,	(9,894) : 322,	(10,894) : 598,	(11,894) : 260,	(12,894) : 304,	(13,894) : 397,	(14,894) : 616,	(15,894) : 629,	(16,894) : 338,	(17,894) : 460,	(18,894) : 127,	(19,894) : 329,	(20,894) : 370,
(1,895) : 375,	(2,895) : 921,	(3,895) : 425,	(4,895) : 354,	(5,895) : 399,	(6,895) : 338,	(7,895) : 867,	(8,895) : 594,	(9,895) : 696,	(10,895) : 1031,	(11,895) : 571,	(12,895) : 173,	(13,895) : 380,	(14,895) : 976,	(15,895) : 1031,	(16,895) : 382,	(17,895) : 898,	(18,895) : 333,	(19,895) : 751,	(20,895) : 810,
(1,896) : 634,	(2,896) : 63,	(3,896) : 608,	(4,896) : 642,	(5,896) : 614,	(6,896) : 729,	(7,896) : 185,	(8,896) : 479,	(9,896) : 265,	(10,896) : 320,	(11,896) : 379,	(12,896) : 778,	(13,896) : 658,	(14,896) : 32,	(15,896) : 146,	(16,896) : 614,	(17,896) : 353,	(18,896) : 707,	(19,896) : 297,	(20,896) : 393,
(1,897) : 500,	(2,897) : 168,	(3,897) : 395,	(4,897) : 501,	(5,897) : 404,	(6,897) : 601,	(7,897) : 280,	(8,897) : 416,	(9,897) : 79,	(10,897) : 307,	(11,897) : 218,	(12,897) : 599,	(13,897) : 533,	(14,897) : 250,	(15,897) : 265,	(16,897) : 477,	(17,897) : 239,	(18,897) : 497,	(19,897) : 95,	(20,897) : 227,
(1,898) : 718,	(2,898) : 421,	(3,898) : 430,	(4,898) : 709,	(5,898) : 459,	(6,898) : 813,	(7,898) : 603,	(8,898) : 710,	(9,898) : 374,	(10,898) : 270,	(11,898) : 468,	(12,898) : 721,	(13,898) : 757,	(14,898) : 513,	(15,898) : 421,	(16,898) : 695,	(17,898) : 140,	(18,898) : 524,	(19,898) : 240,	(20,898) : 103,
(1,899) : 155,	(2,899) : 566,	(3,899) : 472,	(4,899) : 179,	(5,899) : 447,	(6,899) : 203,	(7,899) : 415,	(8,899) : 89,	(9,899) : 407,	(10,899) : 788,	(11,899) : 289,	(12,899) : 377,	(13,899) : 146,	(14,899) : 578,	(15,899) : 692,	(16,899) : 156,	(17,899) : 711,	(18,899) : 495,	(19,899) : 536,	(20,899) : 663,
(1,900) : 301,	(2,900) : 840,	(3,900) : 359,	(4,900) : 279,	(5,900) : 330,	(6,900) : 277,	(7,900) : 787,	(8,900) : 518,	(9,900) : 615,	(10,900) : 956,	(11,900) : 490,	(12,900) : 91,	(13,900) : 310,	(14,900) : 894,	(15,900) : 951,	(16,900) : 306,	(17,900) : 826,	(18,900) : 275,	(19,900) : 675,	(20,900) : 739,
(1,901) : 619,	(2,901) : 56,	(3,901) : 529,	(4,901) : 623,	(5,901) : 539,	(6,901) : 719,	(7,901) : 266,	(8,901) : 501,	(9,901) : 207,	(10,901) : 230,	(11,901) : 342,	(12,901) : 733,	(13,901) : 649,	(14,901) : 149,	(15,901) : 129,	(16,901) : 597,	(17,901) : 236,	(18,901) : 631,	(19,901) : 192,	(20,901) : 275,
(1,902) : 148,	(2,902) : 675,	(3,902) : 510,	(4,902) : 170,	(5,902) : 481,	(6,902) : 131,	(7,902) : 524,	(8,902) : 199,	(9,902) : 506,	(10,902) : 889,	(11,902) : 377,	(12,902) : 346,	(13,902) : 114,	(14,902) : 688,	(15,902) : 801,	(16,902) : 165,	(17,902) : 804,	(18,902) : 510,	(19,902) : 628,	(20,902) : 750,
(1,903) : 115,	(2,903) : 633,	(3,903) : 470,	(4,903) : 138,	(5,903) : 442,	(6,903) : 131,	(7,903) : 490,	(8,903) : 165,	(9,903) : 460,	(10,903) : 844,	(11,903) : 331,	(12,903) : 328,	(13,903) : 89,	(14,903) : 649,	(15,903) : 759,	(16,903) : 127,	(17,903) : 758,	(18,903) : 476,	(19,903) : 582,	(20,903) : 703,
(1,904) : 363,	(2,904) : 845,	(3,904) : 321,	(4,904) : 340,	(5,904) : 298,	(6,904) : 359,	(7,904) : 816,	(8,904) : 574,	(9,904) : 619,	(10,904) : 934,	(11,904) : 504,	(12,904) : 141,	(13,904) : 380,	(14,904) : 907,	(15,904) : 949,	(16,904) : 364,	(17,904) : 798,	(18,904) : 222,	(19,904) : 660,	(20,904) : 708,
(1,905) : 256,	(2,905) : 618,	(3,905) : 586,	(4,905) : 281,	(5,905) : 561,	(6,905) : 271,	(7,905) : 435,	(8,905) : 144,	(9,905) : 490,	(10,905) : 859,	(11,905) : 386,	(12,905) : 475,	(13,905) : 236,	(14,905) : 613,	(15,905) : 743,	(16,905) : 263,	(17,905) : 796,	(18,905) : 608,	(19,905) : 624,	(20,905) : 756,
(1,906) : 321,	(2,906) : 611,	(3,906) : 71,	(4,906) : 301,	(5,906) : 45,	(6,906) : 385,	(7,906) : 622,	(8,906) : 463,	(9,906) : 387,	(10,906) : 682,	(11,906) : 300,	(12,906) : 225,	(13,906) : 358,	(14,906) : 682,	(15,906) : 706,	(16,906) : 306,	(17,906) : 546,	(18,906) : 59,	(19,906) : 410,	(20,906) : 456,
(1,907) : 84,	(2,907) : 700,	(3,907) : 381,	(4,907) : 72,	(5,907) : 349,	(6,907) : 60,	(7,907) : 599,	(8,907) : 298,	(9,907) : 493,	(10,907) : 871,	(11,907) : 354,	(12,907) : 169,	(13,907) : 79,	(14,907) : 735,	(15,907) : 821,	(16,907) : 101,	(17,907) : 763,	(18,907) : 356,	(19,907) : 591,	(20,907) : 692,
(1,908) : 89,	(2,908) : 683,	(3,908) : 451,	(4,908) : 105,	(5,908) : 421,	(6,908) : 56,	(7,908) : 554,	(8,908) : 234,	(9,908) : 495,	(10,908) : 880,	(11,908) : 360,	(12,908) : 268,	(13,908) : 49,	(14,908) : 707,	(15,908) : 808,	(16,908) : 111,	(17,908) : 784,	(18,908) : 441,	(19,908) : 609,	(20,908) : 722,
(1,909) : 327,	(2,909) : 729,	(3,909) : 680,	(4,909) : 350,	(5,909) : 653,	(6,909) : 307,	(7,909) : 537,	(8,909) : 259,	(9,909) : 605,	(10,909) : 973,	(11,909) : 498,	(12,909) : 528,	(13,909) : 296,	(14,909) : 719,	(15,909) : 852,	(16,909) : 340,	(17,909) : 910,	(18,909) : 689,	(19,909) : 738,	(20,909) : 870,
(1,910) : 283,	(2,910) : 721,	(3,910) : 211,	(4,910) : 258,	(5,910) : 184,	(6,910) : 310,	(7,910) : 697,	(8,910) : 475,	(9,910) : 494,	(10,910) : 816,	(11,910) : 382,	(12,910) : 101,	(13,910) : 309,	(14,910) : 784,	(15,910) : 825,	(16,910) : 276,	(17,910) : 682,	(18,910) : 131,	(19,910) : 538,	(20,910) : 594,
(1,911) : 404,	(2,911) : 348,	(3,911) : 215,	(4,911) : 397,	(5,911) : 226,	(6,911) : 502,	(7,911) : 412,	(8,911) : 406,	(9,911) : 142,	(10,911) : 427,	(11,911) : 160,	(12,911) : 447,	(13,911) : 442,	(14,911) : 427,	(15,911) : 438,	(16,911) : 381,	(17,911) : 305,	(18,911) : 318,	(19,911) : 145,	(20,911) : 233,
(1,912) : 210,	(2,912) : 430,	(3,912) : 401,	(4,912) : 225,	(5,912) : 382,	(6,912) : 299,	(7,912) : 305,	(8,912) : 86,	(9,912) : 268,	(10,912) : 647,	(11,912) : 161,	(12,912) : 404,	(13,912) : 228,	(14,912) : 451,	(15,912) : 556,	(16,912) : 195,	(17,912) : 573,	(18,912) : 451,	(19,912) : 399,	(20,912) : 530,
(1,913) : 95,	(2,913) : 716,	(3,913) : 417,	(4,913) : 93,	(5,913) : 386,	(6,913) : 24,	(7,913) : 605,	(8,913) : 295,	(9,913) : 515,	(10,913) : 896,	(11,913) : 376,	(12,913) : 199,	(13,913) : 74,	(14,913) : 748,	(15,913) : 839,	(16,913) : 117,	(17,913) : 791,	(18,913) : 392,	(19,913) : 618,	(20,913) : 722,
(1,914) : 534,	(2,914) : 302,	(3,914) : 312,	(4,914) : 528,	(5,914) : 331,	(6,914) : 633,	(7,914) : 436,	(8,914) : 512,	(9,914) : 181,	(10,914) : 306,	(11,914) : 272,	(12,914) : 573,	(13,914) : 572,	(14,914) : 393,	(15,914) : 359,	(16,914) : 511,	(17,914) : 175,	(18,914) : 417,	(19,914) : 65,	(20,914) : 104,
(1,915) : 372,	(2,915) : 767,	(3,915) : 727,	(4,915) : 395,	(5,915) : 700,	(6,915) : 345,	(7,915) : 570,	(8,915) : 304,	(9,915) : 650,	(10,915) : 1015,	(11,915) : 545,	(12,915) : 568,	(13,915) : 340,	(14,915) : 753,	(15,915) : 890,	(16,915) : 386,	(17,915) : 955,	(18,915) : 735,	(19,915) : 783,	(20,915) : 916,
(1,916) : 181,	(2,916) : 703,	(3,916) : 544,	(4,916) : 201,	(5,916) : 515,	(6,916) : 148,	(7,916) : 546,	(8,916) : 222,	(9,916) : 538,	(10,916) : 922,	(11,916) : 412,	(12,916) : 369,	(13,916) : 145,	(14,916) : 714,	(15,916) : 830,	(16,916) : 198,	(17,916) : 838,	(18,916) : 542,	(19,916) : 662,	(20,916) : 785,
(1,917) : 297,	(2,917) : 552,	(3,917) : 591,	(4,917) : 321,	(5,917) : 569,	(6,917) : 334,	(7,917) : 359,	(8,917) : 116,	(9,917) : 446,	(10,917) : 801,	(11,917) : 359,	(12,917) : 520,	(13,917) : 286,	(14,917) : 540,	(15,917) : 675,	(16,917) : 298,	(17,917) : 749,	(18,917) : 626,	(19,917) : 582,	(20,917) : 717,
(1,918) : 103,	(2,918) : 555,	(3,918) : 267,	(4,918) : 92,	(5,918) : 239,	(6,918) : 197,	(7,918) : 483,	(8,918) : 242,	(9,918) : 341,	(10,918) : 715,	(11,918) : 203,	(12,918) : 213,	(13,918) : 143,	(14,918) : 599,	(15,918) : 673,	(16,918) : 82,	(17,918) : 607,	(18,918) : 283,	(19,918) : 435,	(20,918) : 538,
(1,919) : 139,	(2,919) : 629,	(3,919) : 491,	(4,919) : 163,	(5,919) : 463,	(6,919) : 154,	(7,919) : 479,	(8,919) : 153,	(9,919) : 463,	(10,919) : 846,	(11,919) : 338,	(12,919) : 354,	(13,919) : 115,	(14,919) : 642,	(15,919) : 755,	(16,919) : 150,	(17,919) : 764,	(18,919) : 500,	(19,919) : 588,	(20,919) : 712,
(1,920) : 187,	(2,920) : 706,	(3,920) : 270,	(4,920) : 162,	(5,920) : 238,	(6,920) : 205,	(7,920) : 650,	(8,920) : 395,	(9,920) : 482,	(10,920) : 834,	(11,920) : 354,	(12,920) : 46,	(13,920) : 209,	(14,920) : 758,	(15,920) : 819,	(16,920) : 185,	(17,920) : 710,	(18,920) : 220,	(19,920) : 551,	(20,920) : 629,
(1,921) : 207,	(2,921) : 541,	(3,921) : 505,	(4,921) : 230,	(5,921) : 481,	(6,921) : 255,	(7,921) : 375,	(8,921) : 57,	(9,921) : 400,	(10,921) : 774,	(11,921) : 294,	(12,921) : 429,	(13,921) : 199,	(14,921) : 545,	(15,921) : 667,	(16,921) : 206,	(17,921) : 706,	(18,921) : 535,	(19,921) : 533,	(20,921) : 665,
(1,922) : 547,	(2,922) : 524,	(3,922) : 201,	(4,922) : 532,	(5,922) : 233,	(6,922) : 630,	(7,922) : 631,	(8,922) : 611,	(9,922) : 359,	(10,922) : 488,	(11,922) : 373,	(12,922) : 495,	(13,922) : 587,	(14,922) : 614,	(15,922) : 579,	(16,922) : 526,	(17,922) : 341,	(18,922) : 280,	(19,922) : 286,	(20,922) : 247,
(1,923) : 395,	(2,923) : 353,	(3,923) : 561,	(4,923) : 412,	(5,923) : 549,	(6,923) : 470,	(7,923) : 152,	(8,923) : 185,	(9,923) : 304,	(10,923) : 614,	(11,923) : 282,	(12,923) : 597,	(13,923) : 404,	(14,923) : 333,	(15,923) : 472,	(16,923) : 383,	(17,923) : 583,	(18,923) : 628,	(19,923) : 434,	(20,923) : 571,
(1,924) : 203,	(2,924) : 438,	(3,924) : 400,	(4,924) : 217,	(5,924) : 381,	(6,924) : 291,	(7,924) : 313,	(8,924) : 83,	(9,924) : 274,	(10,924) : 654,	(11,924) : 164,	(12,924) : 398,	(13,924) : 220,	(14,924) : 459,	(15,924) : 564,	(16,924) : 188,	(17,924) : 578,	(18,924) : 448,	(19,924) : 404,	(20,924) : 535,
(1,925) : 334,	(2,925) : 479,	(3,925) : 89,	(4,925) : 320,	(5,925) : 92,	(6,925) : 421,	(7,925) : 508,	(8,925) : 409,	(9,925) : 258,	(10,925) : 555,	(11,925) : 196,	(12,925) : 324,	(13,925) : 374,	(14,925) : 552,	(15,925) : 573,	(16,925) : 313,	(17,925) : 424,	(18,925) : 185,	(19,925) : 278,	(20,925) : 340,
(1,926) : 340,	(2,926) : 674,	(3,926) : 125,	(4,926) : 317,	(5,926) : 106,	(6,926) : 389,	(7,926) : 681,	(8,926) : 503,	(9,926) : 451,	(10,926) : 741,	(11,926) : 359,	(12,926) : 199,	(13,926) : 373,	(14,926) : 745,	(15,926) : 770,	(16,926) : 328,	(17,926) : 603,	(18,926) : 31,	(19,926) : 472,	(20,926) : 512,
(1,927) : 412,	(2,927) : 330,	(3,927) : 563,	(4,927) : 429,	(5,927) : 552,	(6,927) : 489,	(7,927) : 128,	(8,927) : 206,	(9,927) : 293,	(10,927) : 593,	(11,927) : 282,	(12,927) : 610,	(13,927) : 422,	(14,927) : 310,	(15,927) : 449,	(16,927) : 399,	(17,927) : 566,	(18,927) : 633,	(19,927) : 421,	(20,927) : 558,
(1,928) : 292,	(2,928) : 483,	(3,928) : 548,	(4,928) : 314,	(5,928) : 528,	(6,928) : 348,	(7,928) : 296,	(8,928) : 77,	(9,928) : 378,	(10,928) : 731,	(11,928) : 300,	(12,928) : 511,	(13,928) : 290,	(14,928) : 475,	(15,928) : 606,	(16,928) : 287,	(17,928) : 679,	(18,928) : 592,	(19,928) : 514,	(20,928) : 650,
(1,929) : 149,	(2,929) : 770,	(3,929) : 453,	(4,929) : 146,	(5,929) : 421,	(6,929) : 56,	(7,929) : 657,	(8,929) : 342,	(9,929) : 568,	(10,929) : 948,	(11,929) : 429,	(12,929) : 205,	(13,929) : 125,	(14,929) : 801,	(15,929) : 893,	(16,929) : 170,	(17,929) : 841,	(18,929) : 418,	(19,929) : 669,	(20,929) : 770,
(1,930) : 681,	(2,930) : 518,	(3,930) : 349,	(4,930) : 668,	(5,930) : 380,	(6,930) : 769,	(7,930) : 671,	(8,930) : 715,	(9,930) : 412,	(10,930) : 405,	(11,930) : 469,	(12,930) : 643,	(13,930) : 720,	(14,930) : 612,	(15,930) : 540,	(16,930) : 659,	(17,930) : 266,	(18,930) : 428,	(19,930) : 297,	(20,930) : 191,
(1,931) : 309,	(2,931) : 629,	(3,931) : 97,	(4,931) : 287,	(5,931) : 69,	(6,931) : 367,	(7,931) : 632,	(8,931) : 460,	(9,931) : 404,	(10,931) : 706,	(11,931) : 311,	(12,931) : 201,	(13,931) : 344,	(14,931) : 698,	(15,931) : 726,	(16,931) : 294,	(17,931) : 570,	(18,931) : 57,	(19,931) : 432,	(20,931) : 481,
(1,932) : 348,	(2,932) : 307,	(3,932) : 462,	(4,932) : 361,	(5,932) : 451,	(6,932) : 437,	(7,932) : 169,	(8,932) : 179,	(9,932) : 206,	(10,932) : 548,	(11,932) : 180,	(12,932) : 528,	(13,932) : 366,	(14,932) : 316,	(15,932) : 433,	(16,932) : 331,	(17,932) : 499,	(18,932) : 534,	(19,932) : 340,	(20,932) : 478,
(1,933) : 735,	(2,933) : 124,	(3,933) : 685,	(4,933) : 743,	(5,933) : 694,	(6,933) : 831,	(7,933) : 277,	(8,933) : 581,	(9,933) : 351,	(10,933) : 282,	(11,933) : 474,	(12,933) : 873,	(13,933) : 760,	(14,933) : 93,	(15,933) : 85,	(16,933) : 715,	(17,933) : 357,	(18,933) : 787,	(19,933) : 351,	(20,933) : 419,
(1,934) : 249,	(2,934) : 657,	(3,934) : 169,	(4,934) : 225,	(5,934) : 138,	(6,934) : 296,	(7,934) : 635,	(8,934) : 425,	(9,934) : 431,	(10,934) : 759,	(11,934) : 318,	(12,934) : 126,	(13,934) : 281,	(14,934) : 720,	(15,934) : 763,	(16,934) : 238,	(17,934) : 628,	(18,934) : 120,	(19,934) : 479,	(20,934) : 542,
(1,935) : 409,	(2,935) : 365,	(3,935) : 583,	(4,935) : 427,	(5,935) : 571,	(6,935) : 481,	(7,935) : 156,	(8,935) : 195,	(9,935) : 325,	(10,935) : 628,	(11,935) : 305,	(12,935) : 614,	(13,935) : 416,	(14,935) : 340,	(15,935) : 482,	(16,935) : 398,	(17,935) : 602,	(18,935) : 649,	(19,935) : 454,	(20,935) : 592,
(1,936) : 338,	(2,936) : 840,	(3,936) : 702,	(4,936) : 356,	(5,936) : 672,	(6,936) : 276,	(7,936) : 661,	(8,936) : 356,	(9,936) : 692,	(10,936) : 1071,	(11,936) : 570,	(12,936) : 495,	(13,936) : 299,	(14,936) : 839,	(15,936) : 966,	(16,936) : 357,	(17,936) : 995,	(18,936) : 690,	(19,936) : 819,	(20,936) : 944,
(1,937) : 597,	(2,937) : 206,	(3,937) : 421,	(4,937) : 595,	(5,937) : 438,	(6,937) : 698,	(7,937) : 382,	(8,937) : 533,	(9,937) : 188,	(10,937) : 209,	(11,937) : 317,	(12,937) : 666,	(13,937) : 632,	(14,937) : 299,	(15,937) : 245,	(16,937) : 574,	(17,937) : 120,	(18,937) : 526,	(19,937) : 75,	(20,937) : 126,
(1,938) : 587,	(2,938) : 111,	(3,938) : 472,	(4,938) : 589,	(5,938) : 484,	(6,938) : 688,	(7,938) : 294,	(8,938) : 489,	(9,938) : 167,	(10,938) : 229,	(11,938) : 306,	(12,938) : 688,	(13,938) : 619,	(14,938) : 204,	(15,938) : 180,	(16,938) : 565,	(17,938) : 198,	(18,938) : 576,	(19,938) : 131,	(20,938) : 222,
(1,939) : 577,	(2,939) : 427,	(3,939) : 275,	(4,939) : 566,	(5,939) : 303,	(6,939) : 669,	(7,939) : 561,	(8,939) : 598,	(9,939) : 297,	(10,939) : 374,	(11,939) : 352,	(12,939) : 566,	(13,939) : 616,	(14,939) : 519,	(15,939) : 470,	(16,939) : 554,	(17,939) : 227,	(18,939) : 371,	(19,939) : 193,	(20,939) : 132,
(1,940) : 314,	(2,940) : 827,	(3,940) : 328,	(4,940) : 291,	(5,940) : 301,	(6,940) : 303,	(7,940) : 784,	(8,940) : 528,	(9,940) : 601,	(10,940) : 932,	(11,940) : 479,	(12,940) : 93,	(13,940) : 328,	(14,940) : 884,	(15,940) : 935,	(16,940) : 316,	(17,940) : 799,	(18,940) : 239,	(19,940) : 653,	(20,940) : 712,
(1,941) : 348,	(2,941) : 712,	(3,941) : 164,	(4,941) : 324,	(5,941) : 145,	(6,941) : 387,	(7,941) : 714,	(8,941) : 523,	(9,941) : 488,	(10,941) : 780,	(11,941) : 392,	(12,941) : 183,	(13,941) : 379,	(14,941) : 782,	(15,941) : 808,	(16,941) : 338,	(17,941) : 641,	(18,941) : 62,	(19,941) : 511,	(20,941) : 550,
(1,942) : 334,	(2,942) : 571,	(3,942) : 32,	(4,942) : 316,	(5,942) : 0,	(6,942) : 407,	(7,942) : 592,	(8,942) : 455,	(9,942) : 349,	(10,942) : 637,	(11,942) : 273,	(12,942) : 264,	(13,942) : 373,	(14,942) : 644,	(15,942) : 664,	(16,942) : 317,	(17,942) : 501,	(18,942) : 93,	(19,942) : 366,	(20,942) : 411,
(1,943) : 672,	(2,943) : 335,	(3,943) : 421,	(4,943) : 665,	(5,943) : 446,	(6,943) : 770,	(7,943) : 519,	(8,943) : 643,	(9,943) : 300,	(10,943) : 216,	(11,943) : 407,	(12,943) : 700,	(13,943) : 710,	(14,943) : 428,	(15,943) : 342,	(16,943) : 648,	(17,943) : 70,	(18,943) : 522,	(19,943) : 164,	(20,943) : 36,
(1,944) : 556,	(2,944) : 521,	(3,944) : 211,	(4,944) : 541,	(5,944) : 243,	(6,944) : 640,	(7,944) : 631,	(8,944) : 617,	(9,944) : 359,	(10,944) : 480,	(11,944) : 377,	(12,944) : 506,	(13,944) : 596,	(14,944) : 611,	(15,944) : 573,	(16,944) : 535,	(17,944) : 333,	(18,944) : 291,	(19,944) : 282,	(20,944) : 239,
(1,945) : 623,	(2,945) : 525,	(3,945) : 282,	(4,945) : 609,	(5,945) : 314,	(6,945) : 709,	(7,945) : 658,	(8,945) : 672,	(9,945) : 390,	(10,945) : 446,	(11,945) : 428,	(12,945) : 577,	(13,945) : 663,	(14,945) : 617,	(15,945) : 562,	(16,945) : 602,	(17,945) : 301,	(18,945) : 360,	(19,945) : 291,	(20,945) : 213,
(1,946) : 513,	(2,946) : 552,	(3,946) : 156,	(4,946) : 497,	(5,946) : 188,	(6,946) : 592,	(7,946) : 642,	(8,946) : 594,	(9,946) : 371,	(10,946) : 535,	(11,946) : 364,	(12,946) : 447,	(13,946) : 553,	(14,946) : 640,	(15,946) : 615,	(16,946) : 493,	(17,946) : 388,	(18,946) : 228,	(19,946) : 315,	(20,946) : 293,
(1,947) : 369,	(2,947) : 888,	(3,947) : 378,	(4,947) : 346,	(5,947) : 353,	(6,947) : 346,	(7,947) : 846,	(8,947) : 585,	(9,947) : 662,	(10,947) : 988,	(11,947) : 541,	(12,947) : 152,	(13,947) : 379,	(14,947) : 946,	(15,947) : 995,	(16,947) : 373,	(17,947) : 854,	(18,947) : 282,	(19,947) : 711,	(20,947) : 764,
(1,948) : 780,	(2,948) : 164,	(3,948) : 720,	(4,948) : 787,	(5,948) : 730,	(6,948) : 876,	(7,948) : 321,	(8,948) : 627,	(9,948) : 390,	(10,948) : 273,	(11,948) : 517,	(12,948) : 914,	(13,948) : 805,	(14,948) : 137,	(15,948) : 85,	(16,948) : 759,	(17,948) : 366,	(18,948) : 823,	(19,948) : 380,	(20,948) : 435,
(1,949) : 709,	(2,949) : 106,	(3,949) : 669,	(4,949) : 718,	(5,949) : 676,	(6,949) : 805,	(7,949) : 250,	(8,949) : 554,	(9,949) : 330,	(10,949) : 295,	(11,949) : 451,	(12,949) : 850,	(13,949) : 734,	(14,949) : 65,	(15,949) : 99,	(16,949) : 690,	(17,949) : 359,	(18,949) : 769,	(19,949) : 340,	(20,949) : 415,
(1,950) : 95,	(2,950) : 637,	(3,950) : 454,	(4,950) : 118,	(5,950) : 425,	(6,950) : 112,	(7,950) : 500,	(8,950) : 178,	(9,950) : 458,	(10,950) : 843,	(11,950) : 326,	(12,950) : 306,	(13,950) : 67,	(14,950) : 656,	(15,950) : 762,	(16,950) : 109,	(17,950) : 753,	(18,950) : 457,	(19,950) : 577,	(20,950) : 696,
(1,951) : 218,	(2,951) : 651,	(3,951) : 566,	(4,951) : 242,	(5,951) : 539,	(6,951) : 219,	(7,951) : 479,	(8,951) : 167,	(9,951) : 506,	(10,951) : 883,	(11,951) : 391,	(12,951) : 430,	(13,951) : 192,	(14,951) : 653,	(15,951) : 777,	(16,951) : 229,	(17,951) : 811,	(18,951) : 578,	(19,951) : 636,	(20,951) : 765,
(1,952) : 277,	(2,952) : 894,	(3,952) : 576,	(4,952) : 276,	(5,952) : 543,	(6,952) : 176,	(7,952) : 766,	(8,952) : 443,	(9,952) : 697,	(10,952) : 1079,	(11,952) : 559,	(12,952) : 301,	(13,952) : 246,	(14,952) : 920,	(15,952) : 1018,	(16,952) : 299,	(17,952) : 973,	(18,952) : 528,	(19,952) : 800,	(20,952) : 902,
(1,953) : 640,	(2,953) : 35,	(3,953) : 561,	(4,953) : 645,	(5,953) : 571,	(6,953) : 739,	(7,953) : 257,	(8,953) : 512,	(9,953) : 234,	(10,953) : 236,	(11,953) : 366,	(12,953) : 760,	(13,953) : 668,	(14,953) : 120,	(15,953) : 104,	(16,953) : 618,	(17,953) : 261,	(18,953) : 663,	(19,953) : 226,	(20,953) : 306,
(1,954) : 206,	(2,954) : 528,	(3,954) : 495,	(4,954) : 229,	(5,954) : 472,	(6,954) : 259,	(7,954) : 365,	(8,954) : 43,	(9,954) : 387,	(10,954) : 760,	(11,954) : 281,	(12,954) : 427,	(13,954) : 201,	(14,954) : 533,	(15,954) : 654,	(16,954) : 204,	(17,954) : 692,	(18,954) : 528,	(19,954) : 519,	(20,954) : 651,
(1,955) : 538,	(2,955) : 120,	(3,955) : 549,	(4,955) : 547,	(5,955) : 550,	(6,955) : 632,	(7,955) : 107,	(8,955) : 379,	(9,955) : 205,	(10,955) : 388,	(11,955) : 297,	(12,955) : 693,	(13,955) : 561,	(14,955) : 116,	(15,955) : 239,	(16,955) : 519,	(17,955) : 386,	(18,955) : 642,	(19,955) : 282,	(20,955) : 403,
(1,956) : 344,	(2,956) : 335,	(3,956) : 487,	(4,956) : 359,	(5,956) : 475,	(6,956) : 428,	(7,956) : 174,	(8,956) : 156,	(9,956) : 243,	(10,956) : 582,	(11,956) : 208,	(12,956) : 535,	(13,956) : 358,	(14,956) : 336,	(15,956) : 459,	(16,956) : 329,	(17,956) : 536,	(18,956) : 555,	(19,956) : 377,	(20,956) : 515,
(1,957) : 569,	(2,957) : 53,	(3,957) : 518,	(4,957) : 575,	(5,957) : 524,	(6,957) : 667,	(7,957) : 200,	(8,957) : 438,	(9,957) : 177,	(10,957) : 299,	(11,957) : 301,	(12,957) : 699,	(13,957) : 596,	(14,957) : 124,	(15,957) : 176,	(16,957) : 548,	(17,957) : 294,	(18,957) : 617,	(19,957) : 209,	(20,957) : 318,
(1,958) : 632,	(2,958) : 94,	(3,958) : 518,	(4,958) : 635,	(5,958) : 530,	(6,958) : 733,	(7,958) : 305,	(8,958) : 526,	(9,958) : 214,	(10,958) : 196,	(11,958) : 352,	(12,958) : 735,	(13,958) : 664,	(14,958) : 185,	(15,958) : 134,	(16,958) : 610,	(17,958) : 197,	(18,958) : 622,	(19,958) : 172,	(20,958) : 241,
(1,959) : 608,	(2,959) : 130,	(3,959) : 476,	(4,959) : 609,	(5,959) : 490,	(6,959) : 709,	(7,959) : 322,	(8,959) : 516,	(9,959) : 187,	(10,959) : 202,	(11,959) : 326,	(12,959) : 701,	(13,959) : 641,	(14,959) : 223,	(15,959) : 177,	(16,959) : 586,	(17,959) : 170,	(18,959) : 581,	(19,959) : 128,	(20,959) : 201,
(1,960) : 360,	(2,960) : 427,	(3,960) : 576,	(4,960) : 381,	(5,960) : 560,	(6,960) : 424,	(7,960) : 225,	(8,960) : 141,	(9,960) : 357,	(10,960) : 685,	(11,960) : 308,	(12,960) : 574,	(13,960) : 363,	(14,960) : 408,	(15,960) : 547,	(16,960) : 353,	(17,960) : 648,	(18,960) : 633,	(19,960) : 491,	(20,960) : 629,
(1,961) : 740,	(2,961) : 125,	(3,961) : 642,	(4,961) : 744,	(5,961) : 654,	(6,961) : 839,	(7,961) : 335,	(8,961) : 610,	(9,961) : 330,	(10,961) : 185,	(11,961) : 465,	(12,961) : 855,	(13,961) : 768,	(14,961) : 161,	(15,961) : 13,	(16,961) : 718,	(17,961) : 267,	(18,961) : 746,	(19,961) : 294,	(20,961) : 337,
(1,962) : 196,	(2,962) : 686,	(3,962) : 555,	(4,962) : 218,	(5,962) : 527,	(6,962) : 176,	(7,962) : 523,	(8,962) : 202,	(9,962) : 529,	(10,962) : 910,	(11,962) : 406,	(12,962) : 394,	(13,962) : 163,	(14,962) : 693,	(15,962) : 812,	(16,962) : 211,	(17,962) : 831,	(18,962) : 558,	(19,962) : 655,	(20,962) : 780,
(1,963) : 473,	(2,963) : 280,	(3,963) : 293,	(4,963) : 469,	(5,963) : 307,	(6,963) : 573,	(7,963) : 383,	(8,963) : 443,	(9,963) : 117,	(10,963) : 344,	(11,963) : 204,	(12,963) : 531,	(13,963) : 510,	(14,963) : 366,	(15,963) : 359,	(16,963) : 450,	(17,963) : 229,	(18,963) : 397,	(19,963) : 61,	(20,963) : 171,
(1,964) : 279,	(2,964) : 715,	(3,964) : 206,	(4,964) : 255,	(5,964) : 178,	(6,964) : 309,	(7,964) : 691,	(8,964) : 470,	(9,964) : 488,	(10,964) : 810,	(11,964) : 376,	(12,964) : 103,	(13,964) : 306,	(14,964) : 777,	(15,964) : 819,	(16,964) : 273,	(17,964) : 677,	(18,964) : 128,	(19,964) : 532,	(20,964) : 589,
(1,965) : 803,	(2,965) : 271,	(3,965) : 617,	(4,965) : 802,	(5,965) : 637,	(6,965) : 904,	(7,965) : 493,	(8,965) : 717,	(9,965) : 385,	(10,965) : 0,	(11,965) : 521,	(12,965) : 874,	(13,965) : 837,	(14,965) : 339,	(15,965) : 197,	(16,965) : 780,	(17,965) : 147,	(18,965) : 721,	(19,965) : 283,	(20,965) : 242,
(1,966) : 775,	(2,966) : 557,	(3,966) : 444,	(4,966) : 763,	(5,966) : 475,	(6,966) : 864,	(7,966) : 730,	(8,966) : 801,	(9,966) : 484,	(10,966) : 397,	(11,966) : 554,	(12,966) : 738,	(13,966) : 815,	(14,966) : 650,	(15,966) : 558,	(16,966) : 753,	(17,966) : 278,	(18,966) : 520,	(19,966) : 358,	(20,966) : 229,
(1,967) : 116,	(2,967) : 512,	(3,967) : 366,	(4,967) : 130,	(5,967) : 342,	(6,967) : 208,	(7,967) : 400,	(8,967) : 125,	(9,967) : 326,	(10,967) : 710,	(11,967) : 194,	(12,967) : 316,	(13,967) : 137,	(14,967) : 540,	(15,967) : 637,	(16,967) : 101,	(17,967) : 622,	(18,967) : 396,	(19,967) : 445,	(20,967) : 567,
(1,968) : 558,	(2,968) : 434,	(3,968) : 252,	(4,968) : 546,	(5,968) : 281,	(6,968) : 649,	(7,968) : 560,	(8,968) : 585,	(9,968) : 292,	(10,968) : 393,	(11,968) : 339,	(12,968) : 543,	(13,968) : 597,	(14,968) : 526,	(15,968) : 483,	(16,968) : 535,	(17,968) : 246,	(18,968) : 348,	(19,968) : 197,	(20,968) : 151,
(1,969) : 592,	(2,969) : 413,	(3,969) : 297,	(4,969) : 582,	(5,969) : 325,	(6,969) : 685,	(7,969) : 555,	(8,969) : 606,	(9,969) : 295,	(10,969) : 351,	(11,969) : 359,	(12,969) : 587,	(13,969) : 631,	(14,969) : 506,	(15,969) : 452,	(16,969) : 569,	(17,969) : 204,	(18,969) : 394,	(19,969) : 183,	(20,969) : 110,
(1,970) : 441,	(2,970) : 304,	(3,970) : 575,	(4,970) : 457,	(5,970) : 565,	(6,970) : 521,	(7,970) : 95,	(8,970) : 238,	(9,970) : 288,	(10,970) : 570,	(11,970) : 293,	(12,970) : 635,	(13,970) : 453,	(14,970) : 278,	(15,970) : 420,	(16,970) : 428,	(17,970) : 550,	(18,970) : 649,	(19,970) : 410,	(20,970) : 546,
(1,971) : 328,	(2,971) : 541,	(3,971) : 611,	(4,971) : 351,	(5,971) : 590,	(6,971) : 366,	(7,971) : 341,	(8,971) : 135,	(9,971) : 448,	(10,971) : 795,	(11,971) : 371,	(12,971) : 550,	(13,971) : 317,	(14,971) : 524,	(15,971) : 662,	(16,971) : 327,	(17,971) : 748,	(18,971) : 650,	(19,971) : 584,	(20,971) : 721,
(1,972) : 639,	(2,972) : 287,	(3,972) : 413,	(4,972) : 634,	(5,972) : 435,	(6,972) : 739,	(7,972) : 467,	(8,972) : 599,	(9,972) : 254,	(10,972) : 206,	(11,972) : 368,	(12,972) : 682,	(13,972) : 676,	(14,972) : 380,	(15,972) : 304,	(16,972) : 616,	(17,972) : 66,	(18,972) : 517,	(19,972) : 119,	(20,972) : 46,
(1,973) : 794,	(2,973) : 247,	(3,973) : 620,	(4,973) : 794,	(5,973) : 640,	(6,973) : 895,	(7,973) : 469,	(8,973) : 702,	(9,973) : 374,	(10,973) : 28,	(11,973) : 511,	(12,973) : 872,	(13,973) : 827,	(14,973) : 312,	(15,973) : 169,	(16,973) : 771,	(17,973) : 160,	(18,973) : 725,	(19,973) : 280,	(20,973) : 253,
(1,974) : 674,	(2,974) : 342,	(3,974) : 420,	(4,974) : 667,	(5,974) : 445,	(6,974) : 772,	(7,974) : 525,	(8,974) : 647,	(9,974) : 305,	(10,974) : 220,	(11,974) : 411,	(12,974) : 700,	(13,974) : 712,	(14,974) : 434,	(15,974) : 349,	(16,974) : 651,	(17,974) : 75,	(18,974) : 521,	(19,974) : 169,	(20,974) : 37,
(1,975) : 625,	(2,975) : 51,	(3,975) : 537,	(4,975) : 629,	(5,975) : 547,	(6,975) : 725,	(7,975) : 266,	(8,975) : 505,	(9,975) : 214,	(10,975) : 229,	(11,975) : 349,	(12,975) : 740,	(13,975) : 654,	(14,975) : 143,	(15,975) : 122,	(16,975) : 603,	(17,975) : 241,	(18,975) : 639,	(19,975) : 200,	(20,975) : 282,
(1,976) : 29,	(2,976) : 634,	(3,976) : 394,	(4,976) : 51,	(5,976) : 364,	(6,976) : 85,	(7,976) : 519,	(8,976) : 212,	(9,976) : 439,	(10,976) : 823,	(11,976) : 302,	(12,976) : 241,	(13,976) : 14,	(14,976) : 663,	(15,976) : 758,	(16,976) : 50,	(17,976) : 725,	(18,976) : 391,	(19,976) : 549,	(20,976) : 661,
(1,977) : 443,	(2,977) : 637,	(3,977) : 95,	(4,977) : 422,	(5,977) : 115,	(6,977) : 506,	(7,977) : 687,	(8,977) : 570,	(9,977) : 428,	(10,977) : 658,	(11,977) : 376,	(12,977) : 331,	(13,977) : 479,	(14,977) : 718,	(15,977) : 716,	(16,977) : 426,	(17,977) : 513,	(18,977) : 101,	(19,977) : 411,	(20,977) : 419,
(1,978) : 227,	(2,978) : 643,	(3,978) : 570,	(4,978) : 251,	(5,978) : 543,	(6,978) : 231,	(7,978) : 468,	(8,978) : 159,	(9,978) : 502,	(10,978) : 877,	(11,978) : 389,	(12,978) : 440,	(13,978) : 202,	(14,978) : 643,	(15,978) : 768,	(16,978) : 236,	(17,978) : 807,	(18,978) : 585,	(19,978) : 633,	(20,978) : 763,
(1,979) : 676,	(2,979) : 536,	(3,979) : 337,	(4,979) : 662,	(5,979) : 369,	(6,979) : 762,	(7,979) : 684,	(8,979) : 717,	(9,979) : 422,	(10,979) : 429,	(11,979) : 472,	(12,979) : 631,	(13,979) : 716,	(14,979) : 629,	(15,979) : 561,	(16,979) : 654,	(17,979) : 289,	(18,979) : 413,	(19,979) : 311,	(20,979) : 211,
(1,980) : 277,	(2,980) : 363,	(3,980) : 271,	(4,980) : 277,	(5,980) : 262,	(6,980) : 378,	(7,980) : 333,	(8,980) : 250,	(9,980) : 146,	(10,980) : 526,	(11,980) : 12,	(12,980) : 389,	(13,980) : 312,	(14,980) : 416,	(15,980) : 479,	(16,980) : 254,	(17,980) : 430,	(18,980) : 348,	(19,980) : 254,	(20,980) : 375,
(1,981) : 696,	(2,981) : 125,	(3,981) : 680,	(4,981) : 706,	(5,981) : 686,	(6,981) : 790,	(7,981) : 215,	(8,981) : 530,	(9,981) : 337,	(10,981) : 341,	(11,981) : 448,	(12,981) : 847,	(13,981) : 719,	(14,981) : 43,	(15,981) : 147,	(16,981) : 677,	(17,981) : 399,	(18,981) : 779,	(19,981) : 364,	(20,981) : 450,
(1,982) : 355,	(2,982) : 424,	(3,982) : 140,	(4,982) : 345,	(5,982) : 147,	(6,982) : 448,	(7,982) : 464,	(8,982) : 399,	(9,982) : 206,	(10,982) : 504,	(11,982) : 166,	(12,982) : 372,	(13,982) : 395,	(14,982) : 500,	(15,982) : 518,	(16,982) : 333,	(17,982) : 376,	(18,982) : 240,	(19,982) : 224,	(20,982) : 296,
(1,983) : 78,	(2,983) : 699,	(3,983) : 399,	(4,983) : 74,	(5,983) : 367,	(6,983) : 40,	(7,983) : 592,	(8,983) : 285,	(9,983) : 496,	(10,983) : 876,	(11,983) : 357,	(12,983) : 191,	(13,983) : 63,	(14,983) : 732,	(15,983) : 821,	(16,983) : 99,	(17,983) : 770,	(18,983) : 376,	(19,983) : 598,	(20,983) : 701,
(1,984) : 316,	(2,984) : 333,	(3,984) : 443,	(4,984) : 329,	(5,984) : 431,	(6,984) : 405,	(7,984) : 201,	(8,984) : 153,	(9,984) : 212,	(10,984) : 568,	(11,984) : 164,	(12,984) : 498,	(13,984) : 334,	(14,984) : 346,	(15,984) : 459,	(16,984) : 299,	(17,984) : 512,	(18,984) : 511,	(19,984) : 347,	(20,984) : 485,
(1,985) : 502,	(2,985) : 402,	(3,985) : 221,	(4,985) : 492,	(5,985) : 246,	(6,985) : 595,	(7,985) : 510,	(8,985) : 524,	(9,985) : 239,	(10,985) : 399,	(11,985) : 278,	(12,985) : 503,	(13,985) : 541,	(14,985) : 491,	(15,985) : 464,	(16,985) : 479,	(17,985) : 257,	(18,985) : 324,	(19,985) : 164,	(20,985) : 166,
(1,986) : 622,	(2,986) : 99,	(3,986) : 506,	(4,986) : 624,	(5,986) : 518,	(6,986) : 723,	(7,986) : 304,	(8,986) : 518,	(9,986) : 203,	(10,986) : 202,	(11,986) : 341,	(12,986) : 724,	(13,986) : 653,	(14,986) : 191,	(15,986) : 146,	(16,986) : 600,	(17,986) : 194,	(18,986) : 610,	(19,986) : 160,	(20,986) : 233,
(1,987) : 23,	(2,987) : 602,	(3,987) : 345,	(4,987) : 28,	(5,987) : 316,	(6,987) : 123,	(7,987) : 501,	(8,987) : 214,	(9,987) : 399,	(10,987) : 781,	(11,987) : 261,	(12,987) : 222,	(13,987) : 62,	(14,987) : 636,	(15,987) : 724,	(16,987) : 2,	(17,987) : 679,	(18,987) : 348,	(19,987) : 505,	(20,987) : 614,
(1,988) : 342,	(2,988) : 521,	(3,988) : 613,	(4,988) : 365,	(5,988) : 593,	(6,988) : 385,	(7,988) : 318,	(8,988) : 139,	(9,988) : 436,	(10,988) : 777,	(11,988) : 365,	(12,988) : 563,	(13,988) : 334,	(14,988) : 502,	(15,988) : 641,	(16,988) : 340,	(17,988) : 734,	(18,988) : 656,	(19,988) : 572,	(20,988) : 709,
(1,989) : 161,	(2,989) : 697,	(3,989) : 284,	(4,989) : 137,	(5,989) : 251,	(6,989) : 180,	(7,989) : 634,	(8,989) : 372,	(9,989) : 475,	(10,989) : 834,	(11,989) : 344,	(12,989) : 64,	(13,989) : 182,	(14,989) : 747,	(15,989) : 812,	(16,989) : 161,	(17,989) : 712,	(18,989) : 241,	(19,989) : 550,	(20,989) : 633,
(1,990) : 410,	(2,990) : 325,	(3,990) : 558,	(4,990) : 427,	(5,990) : 547,	(6,990) : 489,	(7,990) : 125,	(8,990) : 206,	(9,990) : 287,	(10,990) : 588,	(11,990) : 277,	(12,990) : 608,	(13,990) : 422,	(14,990) : 306,	(15,990) : 444,	(16,990) : 397,	(17,990) : 560,	(18,990) : 629,	(19,990) : 414,	(20,990) : 551,
(1,991) : 141,	(2,991) : 685,	(3,991) : 289,	(4,991) : 117,	(5,991) : 256,	(6,991) : 165,	(7,991) : 618,	(8,991) : 352,	(9,991) : 465,	(10,991) : 827,	(11,991) : 332,	(12,991) : 83,	(13,991) : 163,	(14,991) : 733,	(15,991) : 801,	(16,991) : 141,	(17,991) : 708,	(18,991) : 252,	(19,991) : 544,	(20,991) : 630,
(1,992) : 561,	(2,992) : 61,	(3,992) : 518,	(4,992) : 567,	(5,992) : 524,	(6,992) : 659,	(7,992) : 187,	(8,992) : 426,	(9,992) : 175,	(10,992) : 312,	(11,992) : 295,	(12,992) : 694,	(13,992) : 588,	(14,992) : 121,	(15,992) : 186,	(16,992) : 540,	(17,992) : 306,	(18,992) : 617,	(19,992) : 216,	(20,992) : 328,
(1,993) : 388,	(2,993) : 271,	(3,993) : 483,	(4,993) : 401,	(5,993) : 475,	(6,993) : 478,	(7,993) : 130,	(8,993) : 217,	(9,993) : 199,	(10,993) : 520,	(11,993) : 202,	(12,993) : 566,	(13,993) : 407,	(14,993) : 276,	(15,993) : 396,	(16,993) : 371,	(17,993) : 480,	(18,993) : 560,	(19,993) : 329,	(20,993) : 466,
(1,994) : 158,	(2,994) : 763,	(3,994) : 392,	(4,994) : 142,	(5,994) : 359,	(6,994) : 110,	(7,994) : 672,	(8,994) : 374,	(9,994) : 550,	(10,994) : 921,	(11,994) : 413,	(12,994) : 127,	(13,994) : 153,	(14,994) : 803,	(15,994) : 883,	(16,994) : 172,	(17,994) : 807,	(18,994) : 346,	(19,994) : 639,	(20,994) : 731,
(1,995) : 114,	(2,995) : 731,	(3,995) : 450,	(4,995) : 118,	(5,995) : 419,	(6,995) : 13,	(7,995) : 611,	(8,995) : 294,	(9,995) : 535,	(10,995) : 917,	(11,995) : 397,	(12,995) : 230,	(13,995) : 83,	(14,995) : 760,	(15,995) : 855,	(16,995) : 138,	(17,995) : 815,	(18,995) : 426,	(19,995) : 641,	(20,995) : 749,
(1,996) : 692,	(2,996) : 293,	(3,996) : 466,	(4,996) : 687,	(5,996) : 488,	(6,996) : 792,	(7,996) : 490,	(8,996) : 644,	(9,996) : 298,	(10,996) : 159,	(11,996) : 418,	(12,996) : 736,	(13,996) : 729,	(14,996) : 383,	(15,996) : 287,	(16,996) : 668,	(17,996) : 12,	(18,996) : 569,	(19,996) : 167,	(20,996) : 83,
(1,997) : 718,	(2,997) : 536,	(3,997) : 385,	(4,997) : 705,	(5,997) : 416,	(6,997) : 806,	(7,997) : 697,	(8,997) : 750,	(9,997) : 442,	(10,997) : 404,	(11,997) : 504,	(12,997) : 679,	(13,997) : 758,	(14,997) : 629,	(15,997) : 550,	(16,997) : 696,	(17,997) : 271,	(18,997) : 462,	(19,997) : 322,	(20,997) : 206,
(1,998) : 204,	(2,998) : 663,	(3,998) : 211,	(4,998) : 180,	(5,998) : 179,	(6,998) : 246,	(7,998) : 623,	(8,998) : 393,	(9,998) : 438,	(10,998) : 781,	(11,998) : 315,	(12,998) : 93,	(13,998) : 234,	(14,998) : 720,	(15,998) : 773,	(16,998) : 196,	(17,998) : 654,	(18,998) : 169,	(19,998) : 499,	(20,998) : 572,
(1,999) : 801,	(2,999) : 335,	(3,999) : 575,	(4,999) : 798,	(5,999) : 599,	(6,999) : 902,	(7,999) : 552,	(8,999) : 741,	(9,999) : 397,	(10,999) : 95,	(11,999) : 524,	(12,999) : 848,	(13,999) : 838,	(14,999) : 414,	(15,999) : 284,	(16,999) : 778,	(17,999) : 101,	(18,999) : 677,	(19,999) : 274,	(20,999) : 188,
(1,1000) : 189,	(2,1000) : 458,	(3,1000) : 408,	(4,1000) : 205,	(5,1000) : 387,	(6,1000) : 274,	(7,1000) : 328,	(8,1000) : 70,	(9,1000) : 294,	(10,1000) : 674,	(11,1000) : 182,	(12,1000) : 390,	(13,1000) : 204,	(14,1000) : 478,	(15,1000) : 584,	(16,1000) : 175,	(17,1000) : 598,	(18,1000) : 452,	(19,1000) : 424,	(20,1000) : 554,
(1,1001) : 441,	(2,1001) : 559,	(3,1001) : 79,	(4,1001) : 423,	(5,1001) : 111,	(6,1001) : 517,	(7,1001) : 622,	(8,1001) : 538,	(9,1001) : 357,	(10,1001) : 577,	(11,1001) : 322,	(12,1001) : 370,	(13,1001) : 480,	(14,1001) : 642,	(15,1001) : 635,	(16,1001) : 422,	(17,1001) : 433,	(18,1001) : 156,	(19,1001) : 330,	(20,1001) : 339,
(1,1002) : 112,	(2,1002) : 671,	(3,1002) : 301,	(4,1002) : 88,	(5,1002) : 269,	(6,1002) : 142,	(7,1002) : 596,	(8,1002) : 325,	(9,1002) : 454,	(10,1002) : 821,	(11,1002) : 318,	(12,1002) : 111,	(13,1002) : 135,	(14,1002) : 716,	(15,1002) : 788,	(16,1002) : 113,	(17,1002) : 705,	(18,1002) : 273,	(19,1002) : 538,	(20,1002) : 630,
(1,1003) : 239,	(2,1003) : 508,	(3,1003) : 513,	(4,1003) : 261,	(5,1003) : 492,	(6,1003) : 294,	(7,1003) : 336,	(8,1003) : 38,	(9,1003) : 380,	(10,1003) : 747,	(11,1003) : 284,	(12,1003) : 459,	(13,1003) : 236,	(14,1003) : 509,	(15,1003) : 633,	(16,1003) : 235,	(17,1003) : 685,	(18,1003) : 551,	(19,1003) : 514,	(20,1003) : 648,
(1,1004) : 486,	(2,1004) : 135,	(3,1004) : 459,	(4,1004) : 492,	(5,1004) : 461,	(6,1004) : 584,	(7,1004) : 177,	(8,1004) : 360,	(9,1004) : 113,	(10,1004) : 363,	(11,1004) : 222,	(12,1004) : 621,	(13,1004) : 514,	(14,1004) : 186,	(15,1004) : 259,	(16,1004) : 465,	(17,1004) : 327,	(18,1004) : 554,	(19,1004) : 197,	(20,1004) : 327,
(1,1005) : 132,	(2,1005) : 596,	(3,1005) : 469,	(4,1005) : 156,	(5,1005) : 442,	(6,1005) : 169,	(7,1005) : 448,	(8,1005) : 123,	(9,1005) : 429,	(10,1005) : 812,	(11,1005) : 305,	(12,1005) : 353,	(13,1005) : 117,	(14,1005) : 610,	(15,1005) : 722,	(16,1005) : 138,	(17,1005) : 731,	(18,1005) : 484,	(19,1005) : 555,	(20,1005) : 679,
(1,1006) : 473,	(2,1006) : 482,	(3,1006) : 146,	(4,1006) : 459,	(5,1006) : 175,	(6,1006) : 559,	(7,1006) : 568,	(8,1006) : 533,	(9,1006) : 297,	(10,1006) : 487,	(11,1006) : 297,	(12,1006) : 439,	(13,1006) : 513,	(14,1006) : 569,	(15,1006) : 551,	(16,1006) : 452,	(17,1006) : 343,	(18,1006) : 242,	(19,1006) : 248,	(20,1006) : 249,
(1,1007) : 292,	(2,1007) : 341,	(3,1007) : 293,	(4,1007) : 294,	(5,1007) : 285,	(6,1007) : 393,	(7,1007) : 309,	(8,1007) : 245,	(9,1007) : 129,	(10,1007) : 512,	(11,1007) : 14,	(12,1007) : 412,	(13,1007) : 325,	(14,1007) : 392,	(15,1007) : 459,	(16,1007) : 269,	(17,1007) : 422,	(18,1007) : 372,	(19,1007) : 246,	(20,1007) : 372,
(1,1008) : 629,	(2,1008) : 45,	(3,1008) : 595,	(4,1008) : 637,	(5,1008) : 601,	(6,1008) : 726,	(7,1008) : 194,	(8,1008) : 480,	(9,1008) : 253,	(10,1008) : 306,	(11,1008) : 371,	(12,1008) : 769,	(13,1008) : 654,	(14,1008) : 50,	(15,1008) : 138,	(16,1008) : 609,	(17,1008) : 335,	(18,1008) : 694,	(19,1008) : 280,	(20,1008) : 375,
(1,1009) : 221,	(2,1009) : 632,	(3,1009) : 562,	(4,1009) : 245,	(5,1009) : 535,	(6,1009) : 230,	(7,1009) : 459,	(8,1009) : 149,	(9,1009) : 491,	(10,1009) : 866,	(11,1009) : 378,	(12,1009) : 436,	(13,1009) : 197,	(14,1009) : 633,	(15,1009) : 758,	(16,1009) : 230,	(17,1009) : 796,	(18,1009) : 577,	(19,1009) : 622,	(20,1009) : 752,
(1,1010) : 722,	(2,1010) : 121,	(3,1010) : 611,	(4,1010) : 726,	(5,1010) : 625,	(6,1010) : 822,	(7,1010) : 340,	(8,1010) : 601,	(9,1010) : 307,	(10,1010) : 163,	(11,1010) : 444,	(12,1010) : 831,	(13,1010) : 752,	(14,1010) : 176,	(15,1010) : 48,	(16,1010) : 700,	(17,1010) : 233,	(18,1010) : 716,	(19,1010) : 262,	(20,1010) : 302,
(1,1011) : 477,	(2,1011) : 242,	(3,1011) : 325,	(4,1011) : 474,	(5,1011) : 337,	(6,1011) : 578,	(7,1011) : 346,	(8,1011) : 428,	(9,1011) : 88,	(10,1011) : 329,	(11,1011) : 199,	(12,1011) : 550,	(13,1011) : 512,	(14,1011) : 327,	(15,1011) : 327,	(16,1011) : 454,	(17,1011) : 229,	(18,1011) : 429,	(19,1011) : 52,	(20,1011) : 186,
(1,1012) : 134,	(2,1012) : 697,	(3,1012) : 498,	(4,1012) : 152,	(5,1012) : 468,	(6,1012) : 95,	(7,1012) : 554,	(8,1012) : 228,	(9,1012) : 519,	(10,1012) : 904,	(11,1012) : 387,	(12,1012) : 316,	(13,1012) : 95,	(14,1012) : 714,	(15,1012) : 823,	(16,1012) : 154,	(17,1012) : 814,	(18,1012) : 491,	(19,1012) : 638,	(20,1012) : 756,
(1,1013) : 425,	(2,1013) : 292,	(3,1013) : 271,	(4,1013) : 421,	(5,1013) : 280,	(6,1013) : 525,	(7,1013) : 364,	(8,1013) : 397,	(9,1013) : 92,	(10,1013) : 387,	(11,1013) : 155,	(12,1013) : 490,	(13,1013) : 461,	(14,1013) : 372,	(15,1013) : 385,	(16,1013) : 401,	(17,1013) : 277,	(18,1013) : 373,	(19,1013) : 104,	(20,1013) : 219,
(1,1014) : 666,	(2,1014) : 428,	(3,1014) : 369,	(4,1014) : 656,	(5,1014) : 398,	(6,1014) : 760,	(7,1014) : 593,	(8,1014) : 671,	(9,1014) : 347,	(10,1014) : 315,	(11,1014) : 426,	(12,1014) : 661,	(13,1014) : 705,	(14,1014) : 522,	(15,1014) : 445,	(16,1014) : 643,	(17,1014) : 172,	(18,1014) : 462,	(19,1014) : 220,	(20,1014) : 98,
(1,1015) : 691,	(2,1015) : 143,	(3,1015) : 551,	(4,1015) : 692,	(5,1015) : 567,	(6,1015) : 792,	(7,1015) : 363,	(8,1015) : 590,	(9,1015) : 270,	(10,1015) : 132,	(11,1015) : 409,	(12,1015) : 784,	(13,1015) : 723,	(14,1015) : 223,	(15,1015) : 120,	(16,1015) : 668,	(17,1015) : 164,	(18,1015) : 656,	(19,1015) : 201,	(20,1015) : 231,
(1,1016) : 691,	(2,1016) : 428,	(3,1016) : 396,	(4,1016) : 681,	(5,1016) : 425,	(6,1016) : 785,	(7,1016) : 601,	(8,1016) : 691,	(9,1016) : 361,	(10,1016) : 297,	(11,1016) : 447,	(12,1016) : 688,	(13,1016) : 730,	(14,1016) : 521,	(15,1016) : 438,	(16,1016) : 668,	(17,1016) : 160,	(18,1016) : 489,	(19,1016) : 231,	(20,1016) : 99,

    }
    return warehouses, customers, customer_demands, distance



## In this function, we are testing the input paramaters that is correct or not

In [10]:

def test_input(warehouses, customers, customer_demands, distance, distance_band, wh_status):
    for c in customers:
        for w in warehouses:
            if(distance[w,c]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')
    
    wh_counter = 0
    for w in wh_status:
        if wh_status[w][1] == 1:
            if wh_status[w][2]==1:
                wh_counter = wh_counter+1
            else:
                print("In wh_status, if the first value is 1, the second value can't be 0.  The problem is WH: ", w)
                file.write(f'\nwh_status for warehouse {w} cannot have a 1 and 0 for status')
    if wh_counter <= number_of_whs:
        pass
    else: 
        print("Your fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run")
        file.write(f'\nYour fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run')                   
        


    for c in customers.keys():
        if(customer_demands[c]>=0):
            pass
        else:
            file.write(f'\nDemand for Customer {c} is not available')
        
    
    if ((distance_band[0] < distance_band[1]) and (distance_band[1]  < distance_band[2]) and (distance_band[2]  < distance_band[3]) ):
        pass
    else:
        file.write(f'\nDistance bands are not in ascending order. You must fix and re-run')
        
    if isinstance(number_of_whs, int)==False:
        file.write(f'\nnumber_of_whs = {number_of_whs} is not integer.  You must fix and re-run')
              
    
        

# Create Map for Potantial Warehouse and Customers

In [11]:
# In this function, we are visualizaing the input data-- the data before the optimization has run
def input_visual(warehouses, customers,supplier):
    warehouse_list = []
    for w in warehouses.keys():
        wh = {
                    'text':'Warehouse-'+warehouses[w][1],
                    'State':warehouses[w][2],
                    'ZipCode':warehouses[w][3],
                    'lat':warehouses[w][4],
                    'long':warehouses[w][5],
                    'cnt':10000000,
                    'size' : 20,
                    'color' : 'rgba(0, 100, 0)'
                    }
        warehouse_list.append(wh)    
    # map location for customer
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)     
    supplier_list = []
    for s in supplier.keys():
        supp = {
            'text' : 'Supplier-'+supplier[s][0],
            'lat' : supplier[s][2],
            'long' : supplier[s][3],
            'cnt':1,
            'size' : 20,
            'color' : 'rgb(150,200,100)'
        }
        supplier_list.append(supp)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 
    df_supp = pd.DataFrame.from_records(supplier_list)
    df_supp['shape'] =  "triangle-up"
    df = df.append(df_supp, ignore_index = True) 
    
    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),

            ))]

    layout = dict(
            title = 'location of potential warehouse and customers',
            showlegend = False, 
            geo = dict(
                scope='asia',
                projection=dict( type='miller' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    py.offline.plot({ "data":locations, "layout":layout}, filename = scenario_name+'_input.html')  

    

# Generate map for Optimal Location of Warehouse and its connectivity

In [12]:
# In this function, we are visualizaing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, df_cu, wh_loc, scenario_name,supplier,df_su ):   
    
    warehouse_list = []
    for w in wh_loc:
            wh = {
                        'text':'Warehouse-'+warehouses[w][1],
                        'State':warehouses[w][2],
                        'ZipCode':warehouses[w][3],
                        'lat':warehouses[w][4],
                        'long':warehouses[w][5],
                        'cnt':10000000,
                        'size' : 30,
                        'color' : 'rgba(0, 100, 0)'
                        }
            warehouse_list.append(wh)    
            
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][0],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)
        
    supplier_list = []
    for s in supplier.keys():
        supp = {
            'text' : 'Supplier-'+supplier[s][0],
            'lat' : supplier[s][2],
            'long' : supplier[s][3],
            'cnt':1000,
            'size' : 30,
            'color' : 'rgb(100,200,50)'
        }
        supplier_list.append(supp)
        
    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 
    df_supp = pd.DataFrame.from_records(supplier_list)
    df_supp['shape'] =  "triangle-up"
    df = df.append(df_supp, ignore_index = True) 

    paths = []        
    for i in range( len( df_cu) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i] ],
                    lat = [ df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'red',
                    ),
                    opacity = 0.8,
                )
            )  
    
    for j in range( len( df_su) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_su['Warehouse Longitude'][j], df_su['Supplier Longitude'][j] ],
                    lat = [ df_su['Warehouse Latitude'][j], df_su['Supplier Latitude'][j] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'blue',
                    ),
                    opacity = 0.8,
                )
            )  
    

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),
            ))]

    layout = dict(
            title = 'Optimal location of warehouses',
            showlegend = False, 
            geo = dict(
                scope='asia',
                projection=dict( type='miller' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )

    py.offline.plot({"data":locations+paths, "layout":layout},filename=scenario_name+'_output.html')         

## This is the block of code that runs all the functions (including the optimization)

In [13]:
#getting input data
warehouses, customers, customer_demands, distance = get_data()

#print("loaded input data")
distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

#opening text file in write mode
file = open(scenario_name+'details about our analysis' + '.txt',"w")
file.write('loaded input data')
print("loaded input data")  #Also print to Jupyter

#testing input data
file.write('\nTesting Input Data')
test_input(warehouses, customers, customer_demands, distance, distance_band,wh_status)

#RunCode = ()  #We need some function in here that runs the whole thing.
file.write('\nBuilding the model')
df_assign_vars, list_facility_vars, df_supply_var = optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name,supplier,dist_supply,demand_supp)

#printing the maps
if input_map == True:
    input_visual(warehouses, customers,supplier)

if output_map == True:
    output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name,supplier,df_supply_var)

#closing the file
file.close()

loaded input data


c:\users\ssshi\appdata\local\programs\python\python38-32\lib\site-packages\pulp\apis\coin_api.py:71: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")
c:\users\ssshi\appdata\local\programs\python\python38-32\lib\site-packages\pulp\apis\coin_api.py:77: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Optimization Status Optimal
Weighted Average Distance: 194.32 km
Run Time in seconds 31.4
